In [113]:
# загрузка библиотек
from pydantic import BaseModel, field_validator
import requests
import re
import csv
import pandas as pd

In [191]:
# models

# информация, которую хотим вытащить из продукта (название: тип данных)
# 'id', 'название', 'цена', 'бренд', 'рейтинг покупателей', 'сколько в наличии'

# класс Item наследуется от BaseModel
class Item(BaseModel):
    id: int
    name: str
    salePriceU: float
    brand: str
    reviewRating: float
    volume: int
    # root - что-то вроде уникального id для каждого товара (дает связь - общая инфа каталога и инфа на конкретный товар)
    root: int
    feedback_count: int = None
    feedback_count_with_photo: int = None
    feedback_count_with_text: int = None
    feedbacks_all: list[Comment] = None
 

    # перевод стоимости - из копеек в рубли    
    @field_validator('salePriceU')
    def convert_sale_price(cls, sale_price: int) -> float:
        if sale_price is not None:
            return sale_price / 100   

    
# список продуктов, по которым будем собирать информацию
class Items(BaseModel):
    products: list[Item]

        
# информация, которую нужно достать из отзывов (кол-во отзывов, кол-во отзывов с фото, кол-во отзывов с текстом)
class Feedback(BaseModel):
    feedbackCount: int
    feedbackCountWithPhoto: int
    feedbackCountWithText: int
    feedback: list[Comment]

        
# что нужно достать из каждого отзыва
class Comment(BaseModel):
    createdDate: str
    globalUserId: str
    text: str
    productValuation: int        

In [194]:
class ParseWB:
    def __init__(self, url:str):
        self.brand_id = self.__get_brand_id(url)

    
    @staticmethod
    def __get_brand_id(url:str):
        regex = "(?<=seller/).+(?=)"
        brand_id = re.search(regex, url)[0]
        return brand_id
    
    
    def parse(self):
        # _page - номер страницы с товарами
        _page = 1       
        # вызов метода __create_csv()
        self.__create_csv()
        while True:
            # GET-запрос к сайту через cURL 
            response = requests.get(
                f'https://catalog.wb.ru/sellers/catalog?appType=1&curr=rub&dest=-1257786&regions=80,38,83,4,64,33,68,70,30,40,86,75,69,1,31,66,110,48,22,71,114&sort=popular&spp=33&supplier={self.brand_id}&page={_page}',    
            )
            # page={_page} - движемся по страницам магазина(продавца)
            _page += 1
            
            # вызываем модель Items(список продуктов), передаем ссылку 
            # + передаем data, т.к. в ней находится products (а в Items мы описывали products)
            items_info = Items.parse_obj(response.json()['data'])
            # перед сохранением проверяем, не закончились ли страницы; если закончились - break
            if not items_info.products:
                break  
                
            # перед сохранением вызываем метод для работы с отзывами
            self.__feedback(items_info)
            # сохраняем в csv то, что получили
            self.__save_csv(items_info)
    
    
    # создание csv файла, в котором будет храниться данные + задаем названия признакам
    def __create_csv(self):
        with open("wb_data_my_example.csv", mode="w", newline="") as file:
            writer = csv.writer(file)
            # название признаков(колонок) в датафрейме
            writer.writerow(['id', 'название', 'цена', 'бренд', 'рейтинг покупателей', 'сколько в наличии', 
                             'количество отзывов', 'количество отзывов с фото', 'количество отзывов с текстом', 
                             'текст отзыва' 
                             ])
    
    
    # запись информации в csv файл
    def __save_csv(self, items):
        with open("wb_data_my_example.csv", mode="a", newline="") as file:
            writer = csv.writer(file)        
            # проход по модели Items (items.products - достаем наш список products и далее идем по его элементам - product)
            for product in items.products:
                # наполнение датафрейма
                writer.writerow([product.id,
                                 product.name,
                                 product.salePriceU,
                                 product.brand,
                                 product.reviewRating,
                                 product.volume,
                                 product.feedback_count,
                                 product.feedback_count_with_photo,
                                 product.feedback_count_with_text,
                                 product.feedbacks_text
                                ])
    
    
    # метод, который занимается отзывами
    @staticmethod
    def __feedback(item_model: Items):
        # проход по каждому элементу из нашей модели Items
        for product in item_model.products:
            # url, по которому мы будем отправлять дальнейший запрос - подставляем root, чтобы выйти на конкретный товар
            url = f"https://feedbacks2.wb.ru/feedbacks/v1/{product.root}"
            # res - отправить запрос
            res = requests.get(url=url)
            # проверяем, что "есть контакт"
            if res.status_code == 200:
                # связываем с моделью Feedback и передаем json() нашего последнего запроса
                feedback = Feedback.model_validate(res.json())
                
                # достаем из полученного json() переменные, которые нам нужны:
                # 1 способ - через модель(через pydantic) - делает сразу валидацию (перевод типа данных) - ЛУЧШЕ
                product.feedback_count = feedback.feedbackCount
                # 2 способ - из json() запроса по ключу
                # product.feedback_count = res.json()["feedbackCount"]
                
                product.feedback_count_with_photo = feedback.feedbackCountWithPhoto
                product.feedback_count_with_text = feedback.feedbackCountWithText
                
                product.feedbacks_text = feedback.feedbacks.text
                #product.feedbacks_text = res.json()["feedbacks"][text]
                
                
                
    
if __name__ == "__main__":
    # ссылка на продавца(магазин) в wildberries
    ParseWB("https://www.wildberries.ru/seller/237660").parse() 

           

/var/folders/1m/3j9zld5j7194r5pn5j_47kdw0000gn/T/ipykernel_4622/3454888621.py:28: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  items_info = Items.parse_obj(response.json()['data'])


ValidationError: 1 validation error for Feedback
feedback
  Field required [type=missing, input_value={'photosUris': [['feedbac...ses': 0}, 'rank': 1.0}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.3/v/missing

-- ссылка на продавца
https://catalog.wb.ru/sellers/catalog?appType=1&curr=rub&dest=-1257786&regions=80,38,83,4,64,33,68,70,30,40,86,75,69,1,31,66,110,48,22,71,114&sort=popular&spp=33&supplier=237660   

всё, что после "https://catalog.wb.ru/sellers/catalog?" - это параметры, которые передаются.

- curr=rub - валюта
- sort=popular - сортировка по популярности
....

In [158]:
df_fitparad = pd.read_csv('wb_data_my_example.csv')

In [159]:
df_fitparad.head(20)

,id,название,цена,бренд,рейтинг покупателей,сколько в наличии,количество отзывов,количество отзывов с фото,количество отзывов с текстом,All
0,138785478,ФитПарад №7 саше 90 шт,266.0,FitParad,4.9,23,14788,695,8451,"[{'id': 'rzm9qooBqdFzcemq6NOs', 'globalUserId'..."
1,33482365,"ФитПарад №7 (стевиозид, сукралоза, эритрит) ба...",394.0,FitParad,4.9,22,7383,257,4194,"[{'id': 'juBctooBKNxVyjcCoX5_', 'globalUserId'..."
2,37021249,"ФитПарад № 10 (стевиозид, сукралоза, эритрит) ...",234.0,FitParad,4.9,24,11219,476,6986,"[{'id': 'vm-xrooBpT2rITcGG3wk', 'globalUserId'..."
3,37116681,"Эритрит (эритритол) 70% от сладости сахара, до...",425.0,FitParad,4.6,26,0,0,0,NaN
4,37030771,ФитПарад №17 со вкусом Карамели стики 100 шт,272.0,FitParad,4.8,20,0,0,0,NaN
5,37030770,ФитПарад №22 Ваниль стики 100 штук,272.0,FitParad,4.8,20,0,0,0,NaN
6,33484246,"ФитПарад №7 (стевиозид, сукралоза, эритрит) до...",396.0,FitParad,4.9,10,14309,380,8954,"[{'id': 'qdbpp4oBCaW1RxrSRuZt', 'globalUserId'..."
7,33480746,"ФитПарад №7 (стевиозид, сукралоза, эритрит) ба...",379.0,FitParad,4.9,22,7383,257,4194,"[{'id': 'juBctooBKNxVyjcCoX5_', 'globalUserId'..."
8,53904227,ФитПарад №21 со вкусом Ирландского Ликёра стик...,288.0,FitParad,4.8,22,0,0,0,NaN
9,53904228,ФитПарад №19 со вкусом Шоколада стики 100 шт,270.0,FitParad,4.8,22,0,0,0,NaN


In [160]:
df_fitparad.shape

(172, 10)

In [162]:
df_fitparad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            172 non-null    int64  
 1   название                      172 non-null    object 
 2   цена                          172 non-null    float64
 3   бренд                         172 non-null    object 
 4   рейтинг покупателей           172 non-null    float64
 5   сколько в наличии             172 non-null    int64  
 6   количество отзывов            172 non-null    int64  
 7   количество отзывов с фото     172 non-null    int64  
 8   количество отзывов с текстом  172 non-null    int64  
 9   All                           66 non-null     object 
dtypes: float64(2), int64(5), object(3)
memory usage: 13.6+ KB


In [163]:
df_fitparad.nunique()

id                              172
название                        172
цена                             78
бренд                             3
рейтинг покупателей              15
сколько в наличии                26
количество отзывов               27
количество отзывов с фото        20
количество отзывов с текстом     28
All                              28
dtype: int64

In [164]:
df_fitparad.describe()

,id,цена,рейтинг покупателей,сколько в наличии,количество отзывов,количество отзывов с фото,количество отзывов с текстом
count,1.720000e+02,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000
mean,6.663636e+07,343.715116,4.575000,14.627907,564.720930,23.406977,348.918605
std,3.806704e+07,141.060085,0.579082,14.388548,2259.795168,90.147102,1369.864715
min,3.346770e+07,171.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,4.323378e+07,257.000000,4.400000,6.000000,0.000000,0.000000,0.000000
50%,5.177748e+07,329.000000,4.700000,13.000000,0.000000,0.000000,0.000000
75%,6.722449e+07,396.000000,4.800000,22.000000,71.000000,4.000000,48.000000
max,1.633321e+08,1152.000000,5.000000,152.000000,14788.000000,695.000000,8954.000000


### Вытащить для каждого отзыва

    4. text: "3 упаковки с Фитпарадом пришли супер упакованными в коробку из плотного картона в идеальном состоянии. Срок изготовления середина августа 2023 года и 90 саше в каждой упаковке, итого 270. Нравится, что удобно брать в поездки и  добавлять в напитки, для меня разовая порция, самое то. Пока не открывала, тк еще есть остатки предыдущей покупки. Надеюсь , что будет,  как и всегда, качественный продукт. Спасибо)"
    
    5. createdDate: "2023-09-19T00:01:27Z" (дата написания отзыва)
    
    6. globalUserId: "15529005"
    
    7. wbUserDetails: {hasPhoto: false, country: "RU", name: "Элеонора"} - пользователь, который оставил отзыв


### feedbacks - все отзывы по одному товару

In [ ]:
[
    {
        "id": "rzm9qooBqdFzcemq6NOs",
        "globalUserId": "15529005",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Элеонора",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "3 упаковки с Фитпарадом пришли супер упакованными в коробку из плотного картона в идеальном состоянии. Срок изготовления середина августа 2023 года и 90 саше в каждой упаковке, итого 270. Нравится, что удобно брать в поездки и  добавлять в напитки, для меня разовая порция, самое то. Пока не открывала, тк еще есть остатки предыдущей покупки. Надеюсь , что будет,  как и всегда, качественный продукт. Спасибо)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T00:01:27Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/3146/31468103/9e69dfa1-9f2b-4b97-b410-4a897b0c1845_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/9e69dfa1-9f2b-4b97-b410-4a897b0c1845_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/9733058d-b395-4396-8798-6e1a232ead7b_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/9733058d-b395-4396-8798-6e1a232ead7b_ms.jpg"
            }
        ],
        "photo": [
            127532368,
            127532369
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 1000
    },
    {
        "id": "ERU-rooBBgFQoSo0he6C",
        "globalUserId": "39347781",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Заказала заменитель сахара, пришел вместо 270 шт, 180 шт - упаковки мятые, такое впечатление, что такое упаковали, так как все были упакованы в пупырчатую плёнку и коробку ,коробка снаружи была не мятая.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T16:20:48Z",
        "updatedDate": "2023-09-21T23:41:13Z",
        "answer": null,
        "metadata": {},
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/3146/31468103/4a787755-82e6-41ce-b610-354e13eb052d_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/4a787755-82e6-41ce-b610-354e13eb052d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/c119657f-f1c3-433f-8710-9483d51f383f_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/c119657f-f1c3-433f-8710-9483d51f383f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/96469b50-0ff8-4f3e-8635-9d1e1a8302b0_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/96469b50-0ff8-4f3e-8635-9d1e1a8302b0_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/139f4606-41a4-4d5f-9d5b-122b93f6a84f_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/139f4606-41a4-4d5f-9d5b-122b93f6a84f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/fdee95e6-b937-4ec5-808e-48dd7ee687e2_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/fdee95e6-b937-4ec5-808e-48dd7ee687e2_ms.jpg"
            }
        ],
        "photo": [
            127719752,
            127719753,
            127719754,
            127719755,
            127719756
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 999
    },
    {
        "id": "Jy5ej4oBqdFzcemq16Ky",
        "globalUserId": "31573621",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Айгуль"
        },
        "nmId": 138785478,
        "text": "Много что пробывала,но это прям находка! Нет никаких привкусов,просто сладость.Спасибо за возможность ,пить кофе по утрам)))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T16:27:52Z",
        "updatedDate": "2023-09-22T03:14:39Z",
        "answer": null,
        "metadata": {},
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/c8c7dc44-3e3b-4313-87ea-8f8bb5e40dcd_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/c8c7dc44-3e3b-4313-87ea-8f8bb5e40dcd_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/13878/138785478/40d0951d-fbb8-47ee-b0e5-1e863a485d84_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/40d0951d-fbb8-47ee-b0e5-1e863a485d84_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/13878/138785478/d79d0ae5-7ec6-4251-95ee-281ecfb0577f_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/d79d0ae5-7ec6-4251-95ee-281ecfb0577f_ms.jpg"
            }
        ],
        "photo": [
            126247493,
            126247494,
            126247495
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 998
    },
    {
        "id": "HL1oaooBCaW1RxrSFb1_",
        "globalUserId": "70697615",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Каша"
        },
        "nmId": 138785478,
        "text": "Мне понравилось!!! На вкус как обычный сахар, вообще не чувствую постороннего привкуса. Формат и правда удобный. Мне одного пакетика хватает на 2 чашки кофе👍🏻 Огонь, когда закончится буду еще покупать.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T12:12:01Z",
        "updatedDate": "2023-09-06T12:35:05Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T12:35:05Z",
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/0356c78b-8ea9-40ff-a62a-59c6687da3e8_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/0356c78b-8ea9-40ff-a62a-59c6687da3e8_ms.jpg"
            }
        ],
        "photo": [
            124347388
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 997
    },
    {
        "id": "2XJihYoBYE-HFOZ5pe8X",
        "globalUserId": "7308012",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "country": "by",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Коробки смяты, что требует дополнительной емкости для хранения. Не очень приятный момент. \nВкус сладкий, без посторонних привкусов. С кофе вкусно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T17:55:49Z",
        "updatedDate": "2023-09-22T04:46:33Z",
        "answer": null,
        "metadata": {},
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/3146/31468103/6f4a69a6-d78c-4f66-a209-2102eacd7cf6_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/6f4a69a6-d78c-4f66-a209-2102eacd7cf6_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/7956fe66-ad7f-49b1-b6b7-41735c93649c_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/7956fe66-ad7f-49b1-b6b7-41735c93649c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/d6d1fabb-8c30-4c46-9961-1236559fa9f7_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/d6d1fabb-8c30-4c46-9961-1236559fa9f7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/2e72d5e0-73c6-453e-bd1f-2a619ccb702d_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/2e72d5e0-73c6-453e-bd1f-2a619ccb702d_ms.jpg"
            }
        ],
        "photo": [
            125719089,
            125719090,
            125719091,
            125719092
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 996
    },
    {
        "id": "sxn1WooBqdFzcemqVY7b",
        "globalUserId": "84397408",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Малика",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень слишком сладко даже от одной пачки 🤭 одну пачку делю на два раза вкус как Сахар \nСамое главное не вредит фигуре 👍🏽👍🏽👍🏽☺️☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T12:12:22Z",
        "updatedDate": "2023-09-07T16:16:47Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T12:50:48Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "92492618",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-05T15:10:19Z"
            },
            {
                "globalUserId": "43631141",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-07T16:16:47Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/b4d41f1d-8f96-49fe-ab46-ae715afc0ee1_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/b4d41f1d-8f96-49fe-ab46-ae715afc0ee1_ms.jpg"
            }
        ],
        "photo": [
            123653527
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 995
    },
    {
        "id": "NMfRTIoBAtKOitoBE_i4",
        "globalUserId": "33570483",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Упаковка пришла вся мятая, не приятно в руках держать, сотрудник пункта выдачи какая-то несдержанная, злая, растрепаная, возмущенная. Сахоразаменитель хороший, вместо сахара отлично использовать.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T18:18:05Z",
        "updatedDate": "2023-09-06T08:26:22Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:26:22Z",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Равно как и не имеем отношение к сотруднице) Этими процессами занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/5b83ab35-d837-4cbb-99cd-13b7e1297848_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/5b83ab35-d837-4cbb-99cd-13b7e1297848_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/13878/138785478/454864da-db52-48fd-8bb5-e56bccda664e_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/454864da-db52-48fd-8bb5-e56bccda664e_ms.jpg"
            }
        ],
        "photo": [
            123086262,
            123086263
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 994
    },
    {
        "id": "r3bCxooBUYwa0QEfP8OJ",
        "globalUserId": "19729864",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Давно пользуюсь данным сахарозаменителем,отличная замена сахару.Смотрите фото",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T10:35:34Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/8e2c791b-cfbd-4975-9dfd-3c645bae368f_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/8e2c791b-cfbd-4975-9dfd-3c645bae368f_ms.jpg"
            }
        ],
        "photo": [
            128777015
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 993
    },
    {
        "id": "BSv2iIoBqdFzcemqJV-d",
        "globalUserId": "41816762",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ирина"
        },
        "nmId": 31468103,
        "text": "Хороший товар,вполне сладенько! Упакован хорошо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T10:35:47Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/3146/31468103/7f9a1250-6374-4fd3-9fe7-892bc6f91185_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/7f9a1250-6374-4fd3-9fe7-892bc6f91185_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/3146/31468103/10ff2132-3d78-4010-94af-370dd2e7c59e_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/10ff2132-3d78-4010-94af-370dd2e7c59e_ms.jpg"
            }
        ],
        "photo": [
            125858844,
            125858845
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 992
    },
    {
        "id": "Po-jy4oBYE-HFOZ5nYlZ",
        "globalUserId": "55967102",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Классная штука. Взяла варить варенье своим диабетикам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T09:20:12Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/5f633264-3324-4bad-ab1d-dd68a26a4925_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/5f633264-3324-4bad-ab1d-dd68a26a4925_ms.jpg"
            }
        ],
        "photo": [
            128995791
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 991
    },
    {
        "id": "vYkFqIoBmorVKbOQitH1",
        "globalUserId": "8949097",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "В"
        },
        "nmId": 138785478,
        "text": "Все отлично. Пересчитала 90шт ровно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T11:20:50Z",
        "updatedDate": "2023-09-22T00:22:32Z",
        "answer": null,
        "metadata": {},
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/ab0394be-5e5e-40ef-9a7f-d5de5df4e93b_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/ab0394be-5e5e-40ef-9a7f-d5de5df4e93b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/13878/138785478/0ff18f16-111f-4ee4-b43a-ff460dd3ba04_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/0ff18f16-111f-4ee4-b43a-ff460dd3ba04_ms.jpg"
            }
        ],
        "photo": [
            127370484,
            127370485
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 990
    },
    {
        "id": "iCYMwYoB0v3RIh2JVdD0",
        "globalUserId": "45363401",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Анастасия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Пришел в хорошей упаковке без повреждений. Сладкий.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T07:58:46Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/32b3ba20-05c6-48a0-adb7-29a34d5a8a65_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/32b3ba20-05c6-48a0-adb7-29a34d5a8a65_ms.jpg"
            }
        ],
        "photo": [
            128526231
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 989
    },
    {
        "id": "9XCvmIoBUvtK502_4SPX",
        "globalUserId": "18810090",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ксения",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Пришел товар целым не помятым . Хорошо упакованным .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T11:52:58Z",
        "updatedDate": "2023-09-15T20:45:37Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "8077553",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-15T20:45:37Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/3146/31468103/f67b8cae-4b47-4938-9430-550871a2b344_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/f67b8cae-4b47-4938-9430-550871a2b344_ms.jpg"
            }
        ],
        "photo": [
            126672577
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 988
    },
    {
        "id": "kxnqwIkBfuChzMpwogh0",
        "globalUserId": "46075279",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ксения",
            "country": "kz",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Супер. Заказываю уже не первый раз. Все как всегда отлично. Правда раньше я могла заказать и оплатить при получении, а теперь как объяснили в пункте выдаче, что продукты питания и тд оплата сразу. Не очень удобно если честно. И в этот раз без подарка пришло... Но это не критично. Заказывать все равно буду еще",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-04T14:19:10Z",
        "updatedDate": "2023-08-16T10:09:52.015967209Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "45079291",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-05T17:14:56Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3b95813f-d226-45a2-be62-fe2ef7a4450f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3b95813f-d226-45a2-be62-fe2ef7a4450f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/99a33032-bb05-479b-9f31-d3d071ba4663_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/99a33032-bb05-479b-9f31-d3d071ba4663_ms.jpg"
            }
        ],
        "photo": [
            115926670,
            115926672
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 987
    },
    {
        "id": "dgk9g4oBl47NwnZ1IsQK",
        "globalUserId": "85139338",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Маша"
        },
        "nmId": 138785478,
        "text": "Пришло быстро. Но, очень расстроена! Упаковка мятая и вместо 90 саше 86...",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T07:55:36Z",
        "updatedDate": "2023-09-22T05:14:54Z",
        "answer": null,
        "metadata": {},
        "feedbackHelpfulness": [
            {
                "globalUserId": "14774013",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-12T10:13:16Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/4db7635b-2fdc-4dd7-9bec-aa02dedf25ae_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/4db7635b-2fdc-4dd7-9bec-aa02dedf25ae_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/13878/138785478/b00ab29b-0fe9-4200-9dda-d84aa59d4ffd_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/b00ab29b-0fe9-4200-9dda-d84aa59d4ffd_ms.jpg"
            }
        ],
        "photo": [
            125535123,
            125535124
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 986
    },
    {
        "id": "VvDVrIoBZx8DR9RAG6Yj",
        "globalUserId": "94340508",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Джамалудинова  Айша Магомедовна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Упаковка было ужасном состоянии ,а так все пришло во время , из за упаковки ставлю 4",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T09:46:02Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/0c67e55e-1915-4d4f-ae04-5d7ab25b2286_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/0c67e55e-1915-4d4f-ae04-5d7ab25b2286_ms.jpg"
            }
        ],
        "photo": [
            127610764
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 985
    },
    {
        "id": "R_B8UYoBBgFQoSo0_rBE",
        "globalUserId": "13780875",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Для диабета самое то сладко и не вредно 👧🏼",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T16:04:21Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/6379352d-f8f0-4445-8f78-4d79b67278ee_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/6379352d-f8f0-4445-8f78-4d79b67278ee_ms.jpg"
            }
        ],
        "photo": [
            123264841
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 984
    },
    {
        "id": "Wp9IfIkBUdF-4nlzO7G9",
        "globalUserId": "32571644",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "наконец то приобрела нашумевший сах.зам Фитпарад #7, пользовалась другими, а тут вдруг скидка и взяла. посмотрим, спасибо....",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-22T06:27:36Z",
        "updatedDate": "2023-08-16T10:09:51.312243122Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T06:49:49Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/98d83258-f576-42d8-8140-5dd5007f50ee_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/98d83258-f576-42d8-8140-5dd5007f50ee_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/7958f03c-0a14-4631-8173-ba226111e36e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7958f03c-0a14-4631-8173-ba226111e36e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a7e200ab-bf5e-47eb-bee2-6e965889b87c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a7e200ab-bf5e-47eb-bee2-6e965889b87c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/84380c47-90f6-4fb6-8557-dffcb3e40680_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/84380c47-90f6-4fb6-8557-dffcb3e40680_ms.jpg"
            }
        ],
        "photo": [
            113355475,
            113355476,
            113355477,
            113355478
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 983
    },
    {
        "id": "pCD824kBfuChzMpwxkJj",
        "globalUserId": "29624227",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Патимат",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший сахар, удобно для пользования особенно для приготовления еды. В кофе или чае пить очень приятно нет никакого привкуса. Рекомендую к покупке.  Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-09T20:28:43Z",
        "updatedDate": "2023-08-16T10:09:52.742135852Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-10T05:57:53Z",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e5351e1b-773e-466e-9948-d9eb4b48d92b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e5351e1b-773e-466e-9948-d9eb4b48d92b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c771d15d-d71d-41f8-a3be-4fd698d8ddc0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c771d15d-d71d-41f8-a3be-4fd698d8ddc0_ms.jpg"
            }
        ],
        "photo": [
            117634493,
            117634494
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 982
    },
    {
        "id": "REs80IkBXuPSsPE16L1U",
        "globalUserId": "22659403",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Вот в таком позорном виде упаковывают продукты питания. Саше пришли в мятой коробке, перемотаны в рваный картон. Заявку на брак отклонили. Уважаемые сотрудники Вы поставляете продукты людям! Против фирмы ничего против не имею, доставка отвратительна. Подумайте, прежде чем заказывать!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-07T13:43:20Z",
        "updatedDate": "2023-08-16T10:09:52.278707309Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/901f169a-1650-4587-8f7f-5004edd8a543_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/901f169a-1650-4587-8f7f-5004edd8a543_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0b17a933-8635-4ef2-9383-d6c93079cbd5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0b17a933-8635-4ef2-9383-d6c93079cbd5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2f9670e2-d328-4c66-9b22-a9ff4a8f4c5d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2f9670e2-d328-4c66-9b22-a9ff4a8f4c5d_ms.jpg"
            }
        ],
        "photo": [
            116726887,
            116726885,
            116726890
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 981
    },
    {
        "id": "W55grYoBqVKcxP1mNYQS",
        "globalUserId": "5113135",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Игорь",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень плохо упаковано. Вот в таком измятом виде пришло.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T12:17:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/194bdaf9-ae53-4089-b007-ed81b367acff_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/194bdaf9-ae53-4089-b007-ed81b367acff_ms.jpg"
            }
        ],
        "photo": [
            127653359
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 980
    },
    {
        "id": "qBCLdIkBfuChzMpwq-tT",
        "globalUserId": "32361443",
        "wbUserId": 32361443,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Юлия"
        },
        "nmId": 62622901,
        "text": "товар пришёл быстро. Много, не считала,но надеюсь как и заявлено 1000 штук, хватит на долго..... Спасибо продавцу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-20T18:24:18Z",
        "updatedDate": "2023-08-16T10:09:51.264022778Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T10:56:44Z",
            "editable": false,
            "metadata": null,
            "text": "Добрый день! Спасибо за высокую оценку. Оставайтесь с нами :)",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/df2ae1c7-1bfb-4142-b80a-458ac8f19bfa_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/df2ae1c7-1bfb-4142-b80a-458ac8f19bfa_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d4cdad59-6dbf-4316-beee-bdfea709436d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d4cdad59-6dbf-4316-beee-bdfea709436d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4e7e403d-494a-439d-82a7-ee5666e38496_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4e7e403d-494a-439d-82a7-ee5666e38496_ms.jpg"
            }
        ],
        "photo": [
            113123732,
            113123733,
            113123734
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 979
    },
    {
        "id": "pUJ5_IkBCYLDkq7Nh184",
        "globalUserId": "25611337",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ирина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Купила по рекомендации подруги диабетика. Очень сладкий на вкус, без запаха химии, понравился. Одного пакетика зватает на два раза. Рекомендую для тех, кто считает калории",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-16T03:52:50Z",
        "updatedDate": "2023-08-16T10:09:54.493643066Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-16T08:53:14Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6e1ed93c-741d-4cb1-ae92-140b619542a2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6e1ed93c-741d-4cb1-ae92-140b619542a2_ms.jpg"
            }
        ],
        "photo": [
            119048248
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 978
    },
    {
        "id": "-bUHPIoBqd2abVcyyZ9E",
        "globalUserId": "2752948",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Светлана"
        },
        "nmId": 31468103,
        "text": "Пришёл весь помятый. А сахзам хороший, несколько лет уже покупаю",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T12:04:18Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/3146/31468103/281b751e-7695-42fe-b225-ec654b7d3606_fs.jpg",
                "minSizeUri": "feedbacks/3146/31468103/281b751e-7695-42fe-b225-ec654b7d3606_ms.jpg"
            }
        ],
        "photo": [
            122183262
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 977
    },
    {
        "id": "BQSXIIkBpT2rITcGBdbO",
        "globalUserId": "53012301",
        "wbUserId": 53012301,
        "wbUserDetails": {
            "name": "Евгения",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Товаром довольна. Упаковано хорошо, в картонную коробку, ничего не помялось. Качество сахарозаменителя отличное, хватило 1 пакетика на кружку 250мл, сладко. Буду использовать при изготовлении ПП блюд.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-04T11:08:36Z",
        "updatedDate": "2023-08-16T10:09:50.608540123Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-05T10:41:37Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6f5b835b-d3a1-4799-b4cb-e6d25dfa7338_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6f5b835b-d3a1-4799-b4cb-e6d25dfa7338_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/af686a5c-68e1-4a26-aa07-dafc4c48dcab_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/af686a5c-68e1-4a26-aa07-dafc4c48dcab_ms.jpg"
            }
        ],
        "photo": [
            110041831,
            110041832
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 976
    },
    {
        "id": "3kE8HYoBUvtK502_9EEf",
        "globalUserId": "66517423",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алёна"
        },
        "nmId": 138785478,
        "text": "Заменитель сахара пришёл в целости и сахранности. Сладкий, на одну кружку одного пакетика будет достаточно👍. \nСпасибо за товар🙂",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-22T12:34:09Z",
        "updatedDate": "2023-08-22T12:46:00Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-22T12:46:00Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a6e9aa82-2e06-4ea3-b384-e1f4e0c5d78e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a6e9aa82-2e06-4ea3-b384-e1f4e0c5d78e_ms.jpg"
            }
        ],
        "photo": [
            120734958
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 975
    },
    {
        "id": "CPOLU4kBJytgTHhBLi8W",
        "globalUserId": "104799911",
        "wbUserId": 104799911,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Валентина",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Упаковано все отлично.Удобные пакетики.Мне не понравился вкус.Уж как по мне то лучше чай без сахара.С кофе ещё не пробовала.С сестрой решили исключить сахар.Ей сладко.Мне не вкусно.Посоветовала знакомая которая села на пп.Говорит хорошая вещь от сахара не отличается.Ну не проверишь не узнаешь.Не призываю не заказывать.В любом случае кому то нравится ко муторно нет.Нужно пробовать😉",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-14T08:36:18Z",
        "updatedDate": "2023-08-16T10:09:51.018408487Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:28:18Z",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "text": "Вероятно у вас повышенная чувствительность к компонентам сахарозаменителя."
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "5915839",
                "wbUserId": 5915839,
                "helpfulness": "minus",
                "time": "2023-07-14T11:59:37Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ccb6cbbf-5a35-40ce-8bbb-a7403d36bc9c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ccb6cbbf-5a35-40ce-8bbb-a7403d36bc9c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/527f4a7f-f962-40f1-88bf-e4cbad4cb45a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/527f4a7f-f962-40f1-88bf-e4cbad4cb45a_ms.jpg"
            }
        ],
        "photo": [
            111849421,
            111849422
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 974
    },
    {
        "id": "0-ieaYcBheedNkNuCTy5",
        "globalUserId": "57011147",
        "wbUserId": 57011147,
        "wbUserDetails": {
            "country": "RU",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Мне безумно понравился!! Наконец-то я нашла чёткий сахарозаменитель, ни какого привкуса. Как сахар. Я их столько перепробовала и всё одно г...., попадалась с приторным вкусом. А тут всё в тик токе его используют. Нашла решила попробовать и очень довольна! Рекомендую всем.  Ещё и подарочек внутри 3 пробничка с виноградным вкусом положили.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-10T05:22:58Z",
        "updatedDate": "2023-08-16T10:09:47.967068523Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-10T06:58:21Z",
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "20405000",
                "wbUserId": 20405000,
                "helpfulness": "plus",
                "time": "2023-04-10T10:25:01Z"
            },
            {
                "globalUserId": "16037898",
                "wbUserId": 16037898,
                "helpfulness": "plus",
                "time": "2023-04-11T16:54:15Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c3b81503-99f0-45c7-82b9-35db7867679c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c3b81503-99f0-45c7-82b9-35db7867679c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/3a6124e8-d09f-418f-868c-5f4b677eeb8f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3a6124e8-d09f-418f-868c-5f4b677eeb8f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d4e6a764-c97a-4eb3-9959-3b9f193bebaf_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d4e6a764-c97a-4eb3-9959-3b9f193bebaf_ms.jpg"
            }
        ],
        "photo": [
            95694270,
            95694271,
            95694272
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 973
    },
    {
        "id": "lBvLw4cBG4PPbu1t6odP",
        "globalUserId": "14304147",
        "wbUserId": 14304147,
        "wbUserDetails": {
            "name": "Яна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все целое. Товаром довольна. Единственный Fit parad, которой с приятным вкусом, не кислый и не соленый. 1 пакета достаточно на кружку 180 мл. Рекомендую к покупке.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-27T17:38:54Z",
        "updatedDate": "2023-08-16T10:09:48.627482882Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-28T05:36:59Z",
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "20405000",
                "wbUserId": 20405000,
                "helpfulness": "plus",
                "time": "2023-04-28T16:38:29Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9d890297-fa40-45f9-a83b-0a0dad88ea0b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9d890297-fa40-45f9-a83b-0a0dad88ea0b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/3956623a-742c-4d61-b968-1aa94788e11f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3956623a-742c-4d61-b968-1aa94788e11f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/43c68644-f47e-4b17-bd42-67b060f755ee_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/43c68644-f47e-4b17-bd42-67b060f755ee_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/685cad46-75d4-44ed-b3ca-bf7db688af0a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/685cad46-75d4-44ed-b3ca-bf7db688af0a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4af0de01-8a7a-4f98-b425-4b3e3f1e747c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4af0de01-8a7a-4f98-b425-4b3e3f1e747c_ms.jpg"
            }
        ],
        "photo": [
            98615745,
            98615748,
            98615749,
            98615750,
            98615751
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 972
    },
    {
        "id": "zqVv04gBG4PPbu1tDNhp",
        "globalUserId": "12443034",
        "wbUserId": 12443034,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ирина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Заказ пришел быстро. Первый раз попробовала у дочки. Очень понравился сахарозаменитель, без постороннего привкуса. Добавляем в чай, кофе, кашу. Вкусно и полезно. К покупке рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-19T11:34:10Z",
        "updatedDate": "2023-08-16T10:09:50.142514335Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-19T12:45:22Z",
            "createDate": "2023-06-19T12:45:22Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9b8aa82f-4852-4173-a20c-a081b2b133b9_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9b8aa82f-4852-4173-a20c-a081b2b133b9_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0fa98062-7fa5-4fae-9875-66135d8db7db_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0fa98062-7fa5-4fae-9875-66135d8db7db_ms.jpg"
            }
        ],
        "photo": [
            107365694,
            107365695
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 971
    },
    {
        "id": "SwjdAIgBZx8DR9RAxC5F",
        "globalUserId": "9355705",
        "wbUserId": 9355705,
        "wbUserDetails": {
            "name": "Анна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Товар пришёл в целости и сохранности, хорошо упакован в дополнительную коробку. Свежайший. \nСах.зам. хороший. Мне хватает половины мереной ложечки, вложенной в баночку с этим же сахарозаменителем.\nСпасибо большое!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-09T14:15:14Z",
        "updatedDate": "2023-08-16T10:09:48.965902600Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-17T06:14:51Z",
            "createDate": "2023-05-17T06:14:51Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:43:25Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/69f24629-38b8-4763-b474-e12a22665925_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/69f24629-38b8-4763-b474-e12a22665925_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/61e3b61d-df8e-43b2-8cd9-efd0d54b5a90_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/61e3b61d-df8e-43b2-8cd9-efd0d54b5a90_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/36740130-597c-4589-830e-19117998cc97_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/36740130-597c-4589-830e-19117998cc97_ms.jpg"
            }
        ],
        "photo": [
            100604452,
            100604453,
            100604454
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 970
    },
    {
        "id": "dnMzTIkBCaW1RxrSVhvy",
        "globalUserId": "24987647",
        "wbUserId": 24987647,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Полина"
        },
        "nmId": 138785478,
        "text": "ге считала, но пакетиков, вроде, 90. Всё хорошо, вкус приятный, сладкий. Единственное, что смутило, это то, что пакетики безобразно лежали в коробке. не знаю, вина продавца или вб, но всё же. Спасибо♡",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T22:23:00Z",
        "updatedDate": "2023-08-16T10:09:50.964209929Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T06:50:21Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "24310227",
                "wbUserId": 24310227,
                "helpfulness": "minus",
                "time": "2023-07-13T06:11:54Z"
            },
            {
                "globalUserId": "5915839",
                "wbUserId": 5915839,
                "helpfulness": "minus",
                "time": "2023-07-14T12:00:09Z"
            },
            {
                "globalUserId": "16104734",
                "wbUserId": 16104734,
                "helpfulness": "minus",
                "time": "2023-07-14T12:08:53Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5adf907a-65e5-438a-8803-8ffbc4611ecc_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5adf907a-65e5-438a-8803-8ffbc4611ecc_ms.jpg"
            }
        ],
        "photo": [
            111615769
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 3
        },
        "rank": 969
    },
    {
        "id": "3eLPXIgBuQ90tE6DcRlz",
        "globalUserId": "30490060",
        "wbUserId": 30490060,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Товар пришёл быстро. Упакован идеально, в две пупырчатые плёнки, чтобы не помялись коробки. Дата выпуска апрель 2023. Срок годности большой. Рекомендую продавца.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-27T10:44:39Z",
        "updatedDate": "2023-08-16T10:09:49.509512566Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-29T11:19:39Z",
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "metadata": null,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bc4d3d25-ef11-4997-8f3a-270e34e9e995_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bc4d3d25-ef11-4997-8f3a-270e34e9e995_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/9224cc19-ae2a-4449-b853-4372c9ecb4a8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9224cc19-ae2a-4449-b853-4372c9ecb4a8_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/9933980c-ba79-4b25-a367-50a6dfb45620_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9933980c-ba79-4b25-a367-50a6dfb45620_ms.jpg"
            }
        ],
        "photo": [
            103552344,
            103552345,
            103552346
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 968
    },
    {
        "id": "D27YEIkBCaW1RxrS_wnG",
        "globalUserId": "15142457",
        "wbUserId": 15142457,
        "wbUserDetails": {
            "name": "Юлия",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Спасибо огромное продавцу за хорошо упакованный товар, за качество товара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-01T09:46:44Z",
        "updatedDate": "2023-08-16T10:09:50.520556915Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-07T06:25:23Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9cdc7e2b-4f53-4cc6-a7a5-7802f7756dc2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9cdc7e2b-4f53-4cc6-a7a5-7802f7756dc2_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8dac0ad0-7bce-4be5-b1c4-80ddb664a07b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8dac0ad0-7bce-4be5-b1c4-80ddb664a07b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/70304089-3468-4a68-9337-1716de4f09fc_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/70304089-3468-4a68-9337-1716de4f09fc_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/137e04f0-c968-40d2-b737-979120a9862f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/137e04f0-c968-40d2-b737-979120a9862f_ms.jpg"
            }
        ],
        "photo": [
            109489075,
            109489076,
            109489077,
            109489078
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 967
    },
    {
        "id": "TUhmwIkBXuPSsPE16rfL",
        "globalUserId": "22071636",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Яна"
        },
        "nmId": 138785478,
        "text": "Упаковано отлично!Коробка не помята!Качественный товар!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-04T11:55:18Z",
        "updatedDate": "2023-08-16T10:09:52.006912108Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T12:42:00Z",
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0748741e-1644-4f6d-b34b-c80cae17a990_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0748741e-1644-4f6d-b34b-c80cae17a990_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0bc039af-4046-481c-bca6-9d2afd103e32_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0bc039af-4046-481c-bca6-9d2afd103e32_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/478fbef5-a3ca-406e-bb66-93a46cc06a1e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/478fbef5-a3ca-406e-bb66-93a46cc06a1e_ms.jpg"
            }
        ],
        "photo": [
            115895112,
            115895113,
            115895114
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 966
    },
    {
        "id": "9Ar8KIkBfuChzMpwAhDk",
        "globalUserId": "42982759",
        "wbUserId": 42982759,
        "wbUserDetails": {
            "name": "Екатерина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все пришло в отличном состоянии, упаковано было в доп.коробку, что очень порадовало! Спасибо продавцу! Рекомендую к покупке!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-06T02:15:52Z",
        "updatedDate": "2023-08-16T10:09:50.671998357Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-06T05:26:48Z",
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3c6c108c-0d87-4d89-8aec-02a1889f7c2f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3c6c108c-0d87-4d89-8aec-02a1889f7c2f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/6f761228-8086-4480-b1d8-9e1d05bd6692_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6f761228-8086-4480-b1d8-9e1d05bd6692_ms.jpg"
            }
        ],
        "photo": [
            110344383,
            110344384
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 965
    },
    {
        "id": "0tPKB4oBoNtFBUnP-moU",
        "globalUserId": "37583493",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Пришло всё целое, упаковано в отдельной упаковке. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-18T08:37:37Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0630fafc-3784-4da3-b271-0181c66dfcaa_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0630fafc-3784-4da3-b271-0181c66dfcaa_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c866124a-0acb-4e41-887a-560511cc6ae5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c866124a-0acb-4e41-887a-560511cc6ae5_ms.jpg"
            }
        ],
        "photo": [
            119552193,
            119552194
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 964
    },
    {
        "id": "dZOL74kBKNxVyjcCo6SJ",
        "globalUserId": "43990074",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Дарья",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "10/10 даже не ожидала, нет никакого противного привкуса, на целую кружку прям одного пакетика с головой",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-13T15:37:33Z",
        "updatedDate": "2023-08-16T10:09:53.844107134Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T07:10:45Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "6751895",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-13T17:42:14Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/94d5c143-2b49-405f-9d62-0ca3d587ef6f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/94d5c143-2b49-405f-9d62-0ca3d587ef6f_ms.jpg"
            }
        ],
        "photo": [
            118483421
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 963
    },
    {
        "id": "n4-B4IkBuX2t2Z2-rwvG",
        "globalUserId": "22239414",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Заказала первый раз, упаковка пришла целая, заказ пришёл быстро, мне пока нравится, буду готовить десертики",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-10T17:32:23Z",
        "updatedDate": "2023-08-16T10:09:53.031308973Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/62323490-7245-4a3c-847a-433969f221cf_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/62323490-7245-4a3c-847a-433969f221cf_ms.jpg"
            }
        ],
        "photo": [
            117845056
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 962
    },
    {
        "id": "EEc6SogBBgFQoSo0DNS0",
        "globalUserId": "29059617",
        "wbUserId": 29059617,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Евгения"
        },
        "nmId": 33478818,
        "text": "Сняла звезды за товарный вид. Пришла коробка вся мятая, соответственно и пакетики внутри тоже помяты. \nНа вкус нормально, я пью кофе с половинкой ложки сахара, тут пакетик, по сладости почти так же. Есть какой-то привкус, но думаю привыкну к нему. Мне понравился , рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-23T20:08:18Z",
        "updatedDate": "2023-08-16T10:09:49.410412586Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-24T08:32:02Z",
            "createDate": "2023-05-24T08:32:02Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1b3f094c-e3e4-4dbc-9638-91ddd146d18a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1b3f094c-e3e4-4dbc-9638-91ddd146d18a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4c54175b-2427-4bce-8209-dc3e5b16a7ac_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4c54175b-2427-4bce-8209-dc3e5b16a7ac_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/dceac992-2e3b-43cc-848e-77dcf6dd28c7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dceac992-2e3b-43cc-848e-77dcf6dd28c7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/22f0b7db-2e4e-4c94-b50d-672939969842_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/22f0b7db-2e4e-4c94-b50d-672939969842_ms.jpg"
            }
        ],
        "photo": [
            103024353,
            103024354,
            103024355,
            103024356
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 961
    },
    {
        "id": "6r3tl4kBl47NwnZ1Tx8x",
        "globalUserId": "34080185",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру не в первый раз. Отличная замена рафинированному сахару, использую при выпечке и в десертах, если необходим сладкий вкус",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T15:17:39Z",
        "updatedDate": "2023-08-16T10:09:51.590676465Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T10:27:08Z",
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/fd5d5bba-2178-4ea5-9a65-2f1407bce60c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/fd5d5bba-2178-4ea5-9a65-2f1407bce60c_ms.jpg"
            }
        ],
        "photo": [
            114442351
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 960
    },
    {
        "id": "klpf3IYBMJfQQSHDul-k",
        "globalUserId": "31034907",
        "wbUserId": 31034907,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Надежда"
        },
        "nmId": 138785478,
        "text": "Товар пришёл быстро. Упаковали в картон, но пришёл с помятой коробкой. При вскрытии лежал маленький подарок,  fit parad чёрная смородина, что подняло настроение после первого впечатления (испорченная упаковка). Пакетиков 90 шт, на долго хватит.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-13T19:08:27Z",
        "updatedDate": "2023-08-16T10:09:46.644544447Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-14T05:39:35Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5a459650-045e-4202-a192-19584c9ae7f3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5a459650-045e-4202-a192-19584c9ae7f3_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/635b6c59-b8b3-4e93-88e0-5ae680fb4cf4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/635b6c59-b8b3-4e93-88e0-5ae680fb4cf4_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2092543b-3005-4b6f-883b-64246bc9cefa_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2092543b-3005-4b6f-883b-64246bc9cefa_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/5821ab33-7e31-4d9c-8066-8300a5591ced_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5821ab33-7e31-4d9c-8066-8300a5591ced_ms.jpg"
            }
        ],
        "photo": [
            90898107,
            90898108,
            90898109,
            90898110
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 959
    },
    {
        "id": "gGNnoYgBBgFQoSo0-7Xj",
        "globalUserId": "44434613",
        "wbUserId": 44434613,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хороший заменитель. Пришёл в дополнительной картонной коробке без повреждений. Внутри сама коробка с сахаром была помята, но не вскрыта. Всë на месте.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-09T18:25:26Z",
        "updatedDate": "2023-08-16T10:09:49.871857116Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T09:10:05Z",
            "createDate": "2023-06-20T09:10:05Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/05a55b53-160e-4b10-b07f-5a05c903164f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/05a55b53-160e-4b10-b07f-5a05c903164f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/00b49ab0-bd67-4bb7-849e-e94e7091cd07_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/00b49ab0-bd67-4bb7-849e-e94e7091cd07_ms.jpg"
            }
        ],
        "photo": [
            105733652,
            105733653
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 958
    },
    {
        "id": "mWzrmIcB3ieoZoqxl4eu",
        "globalUserId": "30273091",
        "wbUserId": 30273091,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Отличный заменитель сахара! Реально 1 грамм заменяет 2 чайные ложки сахара. По КЖБУ соотношение калорий, белков, жиров и углеводов, всё по нулям. Идеально для тех кто не может без сладкого совсем. Попробовала на вкус, посторонних привкусов нет, только сладость 😁 Благодарю, покупкой довольна!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-19T09:49:49Z",
        "updatedDate": "2023-08-16T10:09:48.315324223Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-19T09:58:37Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2559fd75-9704-4aba-88a2-6b48cf034c61_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2559fd75-9704-4aba-88a2-6b48cf034c61_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/14114fb8-cca5-4425-b4ac-d5f599fde5df_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/14114fb8-cca5-4425-b4ac-d5f599fde5df_ms.jpg"
            }
        ],
        "photo": [
            97172410,
            97172411
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 957
    },
    {
        "id": "hX6PvIkBCaW1RxrS9Uxy",
        "globalUserId": "119360920",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "by",
            "hasPhoto": false,
            "name": ""
        },
        "nmId": 138785478,
        "text": "Лучший подсластитель, удобные пакетики, хорошая цена. Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-03T18:01:38Z",
        "updatedDate": "2023-08-16T10:09:51.957569559Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T07:36:41Z",
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "38976941",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-10T00:52:40Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/01f4fee2-a2e4-4d21-ab7e-003c1cfb58ad_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/01f4fee2-a2e4-4d21-ab7e-003c1cfb58ad_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c1786614-4954-4c42-ae02-61cfad6af9ff_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c1786614-4954-4c42-ae02-61cfad6af9ff_ms.jpg"
            }
        ],
        "photo": [
            115773266,
            115773267
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 956
    },
    {
        "id": "aHpEc4kBKNxVyjcCOo7U",
        "globalUserId": "49370193",
        "wbUserId": 49370193,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анастасия"
        },
        "nmId": 31468103,
        "text": "Все целенькое!!! Срок годности от 15 06.2023г. - свежее. Продавца рекомендую!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-20T12:26:39Z",
        "updatedDate": "2023-08-16T10:09:51.250407096Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:30:40Z",
            "text": "Благодарим за покупку!",
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54611991",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-07-24T01:56:22Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bbba6778-fc85-4fe4-a3c0-ef56ed5540a4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bbba6778-fc85-4fe4-a3c0-ef56ed5540a4_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/b3b905a6-6879-4ce5-b2e8-20aa41aa8093_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b3b905a6-6879-4ce5-b2e8-20aa41aa8093_ms.jpg"
            }
        ],
        "photo": [
            113044484,
            113044485
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 955
    },
    {
        "id": "T-U18IcBlM6cTg_RdyF9",
        "globalUserId": "99661618",
        "wbUserId": 99661618,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Надежда"
        },
        "nmId": 138785478,
        "text": "Теперь и я смогу попить кофе с таким заменителем,не боясь,что поднимется сахар в крови. Можно разделить саше(пакетик) еще пополам,сладкий.Огромное спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-06T08:37:29Z",
        "updatedDate": "2023-08-16T10:09:48.853772464Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-19T08:00:36Z",
            "createDate": "2023-05-19T08:00:36Z",
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1b5dd874-0fcf-42a5-bdef-23e4e70db738_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1b5dd874-0fcf-42a5-bdef-23e4e70db738_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/576faeef-4307-439a-a8fd-bd8cffc35068_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/576faeef-4307-439a-a8fd-bd8cffc35068_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/56a421a5-0ba5-4d5f-af46-669c303bd17c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/56a421a5-0ba5-4d5f-af46-669c303bd17c_ms.jpg"
            }
        ],
        "photo": [
            100054015,
            100054016,
            100054017
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 954
    },
    {
        "id": "TQn0HIkBfuChzMpw-RpE",
        "globalUserId": "9239815",
        "wbUserId": 9239815,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Виктор"
        },
        "nmId": 138785478,
        "text": "Упаковано на отлично. Заменитель хороший. Со вкусами также рекомендую. Саше - это удобно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-03T18:12:44Z",
        "updatedDate": "2023-08-16T10:09:50.584165775Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-04T05:11:02Z",
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3aa06a07-a945-42b1-a674-3e5b0f1e7bf1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3aa06a07-a945-42b1-a674-3e5b0f1e7bf1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4bacc4b4-3264-4f97-be0c-0f3441705744_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4bacc4b4-3264-4f97-be0c-0f3441705744_ms.jpg"
            }
        ],
        "photo": [
            109942357,
            109942359
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 953
    },
    {
        "id": "zvmwkYkBJytgTHhBCQza",
        "globalUserId": "31659993",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ангелина"
        },
        "nmId": 138785478,
        "text": "Нет неприятного привкуса, сладко, но есть и другие добавки помимо ставии, если кому это принципиально",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-26T10:13:00Z",
        "updatedDate": "2023-08-16T10:09:51.513192967Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T07:08:29Z",
            "employeeId": 0,
            "text": "Добрый день, спасибо что уделили время и написали отзыв.",
            "supplierId": 237660,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/dbbbd21f-5149-40b5-81fa-e10567f3fc2f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dbbbd21f-5149-40b5-81fa-e10567f3fc2f_ms.jpg"
            }
        ],
        "photo": [
            114179822
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 952
    },
    {
        "id": "23PlIYkBKNxVyjcCeVfR",
        "globalUserId": "33599041",
        "wbUserId": 33599041,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Сах.зам просто супер ! Беру уже много раз и себе и всему коллективу на работе ! Заказываем номер 7 и номер 10 , самые лучшие ! Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-04T17:13:54Z",
        "updatedDate": "2023-08-16T10:09:50.620592125Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-05T10:37:44Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e1d775f9-6359-4763-9a6e-f61800585daf_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e1d775f9-6359-4763-9a6e-f61800585daf_ms.jpg"
            }
        ],
        "photo": [
            110117847
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 951
    },
    {
        "id": "XFu9j4cBUYwa0QEf-2J9",
        "globalUserId": "50538805",
        "wbUserId": 50538805,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Всё отлично, в меру сладкий, как и хотела. Рекомендую тем, кто не любит приторно-сладкий вкус.  Отдельное спасибо за вкладыш внутри коробки с информацией о новых продуктах и за пробники ароматизированных стиков с чёрной смородиной!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-17T15:03:25Z",
        "updatedDate": "2023-08-16T10:09:48.255092694Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-18T05:09:56Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за доверие нашему бренду!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c82df09c-e3bb-405a-a060-7b8c147ae712_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c82df09c-e3bb-405a-a060-7b8c147ae712_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/1d9957de-1b66-4f5b-8101-c778f5ccc6ff_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1d9957de-1b66-4f5b-8101-c778f5ccc6ff_ms.jpg"
            }
        ],
        "photo": [
            96888317,
            96888320
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 950
    },
    {
        "id": "Vy5jookBmorVKbOQtTgg",
        "globalUserId": "38412940",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ольга"
        },
        "nmId": 138785478,
        "text": "заменитель сахара, был упакован в маленькую коробку,очень аккуратно.Поставщика рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-29T16:03:11Z",
        "updatedDate": "2023-08-16T10:09:51.702854507Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-31T09:04:22Z",
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2737b8b1-3d39-46c0-8349-908d83d98c9f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2737b8b1-3d39-46c0-8349-908d83d98c9f_ms.jpg"
            }
        ],
        "photo": [
            114812497
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 949
    },
    {
        "id": "xGoF54gBCaW1RxrSo5D3",
        "globalUserId": "41396466",
        "wbUserId": 41396466,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Наталья"
        },
        "nmId": 33478818,
        "text": "Отличный продукт, пришло все целое , стики удобно брать с собой 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-23T06:51:27Z",
        "updatedDate": "2023-08-16T10:09:50.249607269Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-23T08:26:33Z",
            "createDate": "2023-06-23T08:26:33Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d99179be-1a77-44f3-88b9-668f7b5d3545_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d99179be-1a77-44f3-88b9-668f7b5d3545_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/824b38af-8e1c-4a16-b7e1-168fda9fc82a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/824b38af-8e1c-4a16-b7e1-168fda9fc82a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3e9b8ca-fde6-4313-a23d-125456f89996_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3e9b8ca-fde6-4313-a23d-125456f89996_ms.jpg"
            }
        ],
        "photo": [
            108065061,
            108065064,
            108065065
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 948
    },
    {
        "id": "hdieeIkBqdFzcemqJOAZ",
        "globalUserId": "38479747",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "kz",
            "hasPhoto": false,
            "name": "Дилара"
        },
        "nmId": 138785478,
        "text": "Доставка быстрая , пришло в запечатанном виде, спасибо 👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-21T13:22:57Z",
        "updatedDate": "2023-08-16T10:09:51.292731833Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T07:01:57Z",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за ваш отзыв!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/be16dcea-9a4e-44cb-b4c4-8575785007fb_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/be16dcea-9a4e-44cb-b4c4-8575785007fb_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/b0e8e5bd-e505-4208-bd92-7649ec714906_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b0e8e5bd-e505-4208-bd92-7649ec714906_ms.jpg"
            }
        ],
        "photo": [
            113251664,
            113251668
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 947
    },
    {
        "id": "Obc-UIoBKNxVyjcCiNuS",
        "globalUserId": "6635288",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Галина",
            "hasPhoto": false,
            "country": "kz"
        },
        "nmId": 138785478,
        "text": "Хороший!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T10:16:30Z",
        "updatedDate": "2023-09-06T08:15:49Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:15:49Z",
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/87ef1a99-facd-4e37-bbb1-4b93581e11af_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/87ef1a99-facd-4e37-bbb1-4b93581e11af_ms.jpg"
            }
        ],
        "photo": [
            123185832
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 946
    },
    {
        "id": "tnDaK4kBCaW1RxrS8UdJ",
        "globalUserId": "88768823",
        "wbUserId": 88768823,
        "wbUserDetails": {
            "country": "ru",
            "name": "Алёна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Заказываю второй раз. Все надёжно упаковано в коробку. Товаром довольна.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-06T15:38:36Z",
        "updatedDate": "2023-08-16T10:09:50.699043702Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-07T06:04:58Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/36b601ae-fe80-476c-ac58-98dc0a7b0797_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/36b601ae-fe80-476c-ac58-98dc0a7b0797_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/f1c2de06-9ec0-463a-b575-7b09b046a0c3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f1c2de06-9ec0-463a-b575-7b09b046a0c3_ms.jpg"
            }
        ],
        "photo": [
            110465968,
            110465969
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 945
    },
    {
        "id": "tLzebYkB0v3RIh2JEwyY",
        "globalUserId": "109992414",
        "wbUserId": 109992414,
        "wbUserDetails": {
            "name": "Залина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хорошо упакован, коробка не мятая, всем довольна спасибо WB",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-19T11:16:58Z",
        "updatedDate": "2023-08-16T10:09:51.199364541Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T08:54:02Z",
            "metadata": null,
            "text": "Благодарим за высокую оценку нашего продукта!",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/19026bff-1f58-4172-9cec-004541cea59b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/19026bff-1f58-4172-9cec-004541cea59b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/164e8781-e5f2-4e5a-8ec4-0e01ebf85f2e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/164e8781-e5f2-4e5a-8ec4-0e01ebf85f2e_ms.jpg"
            }
        ],
        "photo": [
            112831949,
            112831950
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 944
    },
    {
        "id": "tiofyogBuQ90tE6DlUad",
        "globalUserId": "27604694",
        "wbUserId": 27604694,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Лия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все хорошо. Пришло дополнительно в картонной упаковке. Целостность коробочки не нарушена.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-17T16:10:48Z",
        "updatedDate": "2023-08-16T10:09:50.102671013Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T07:29:10Z",
            "createDate": "2023-06-20T07:29:10Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ab5920a4-a6d2-4c46-a91c-0438fe242098_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ab5920a4-a6d2-4c46-a91c-0438fe242098_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0278e1c9-a834-45ca-81f7-fd0c4b9a6309_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0278e1c9-a834-45ca-81f7-fd0c4b9a6309_ms.jpg"
            }
        ],
        "photo": [
            107075960,
            107075961
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 943
    },
    {
        "id": "Zp1z5oYBbskpjQp26sjY",
        "globalUserId": "16759339",
        "wbUserId": 16759339,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Наталья"
        },
        "nmId": 31468103,
        "text": "Купила упаковку из 3 коробок, получилось очень выгодно, в каждой коробочке лежал джем в подарок - очень приятно ☺️ 👍 я покупала зелёные и синие саше, синие мне показались менее сладкие и пыльные, высыпаешь пакетик и по рту от этой пыли сразу сладкий вкус. Остановилась на зелёных)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-15T18:06:42Z",
        "updatedDate": "2023-08-16T10:09:46.726722964Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-16T11:33:34Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5df2a5fe-0988-4c8e-bed0-7454cf823029_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5df2a5fe-0988-4c8e-bed0-7454cf823029_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/5461f7cf-15d4-4745-93f1-dbaf41232c89_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5461f7cf-15d4-4745-93f1-dbaf41232c89_ms.jpg"
            }
        ],
        "photo": [
            91271267,
            91271268
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 942
    },
    {
        "id": "7WDFsYkBnnhI58-2I31d",
        "globalUserId": "28150735",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ангелина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все хорошо👍сладенький,добавляла в выпечку",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-01T15:43:54Z",
        "updatedDate": "2023-08-16T10:09:51.858798667Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T08:06:15Z",
            "employeeId": 0,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "editable": true,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4973aa76-698b-4adf-94a0-1da992873e8c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4973aa76-698b-4adf-94a0-1da992873e8c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/60d03a60-759c-4367-b2cf-d33e4f691039_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/60d03a60-759c-4367-b2cf-d33e4f691039_ms.jpg"
            }
        ],
        "photo": [
            115358858,
            115358859
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 941
    },
    {
        "id": "sUluDYoBCYLDkq7NCUxD",
        "globalUserId": "85318520",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Мадина",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Товар пришел в целости и сохранности, рекомендую 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-19T10:53:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9534545d-beba-4a0a-9c7f-e23fbaf08878_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9534545d-beba-4a0a-9c7f-e23fbaf08878_ms.jpg"
            }
        ],
        "photo": [
            119821709
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 940
    },
    {
        "id": "lYmOQYkBZx8DR9RAKsIb",
        "globalUserId": "93436014",
        "wbUserId": 93436014,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Яна",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Все пришло как и заявлено. Пользуюсь давно, но в магазинах на порядок дороже. Покупкой довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-10T20:46:23Z",
        "updatedDate": "2023-08-16T10:09:50.877041511Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:41:43Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Добрый день! Спасибо за высокую оценку. Оставайтесь с нами :)",
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "21900803",
                "wbUserId": 21900803,
                "helpfulness": "plus",
                "time": "2023-07-11T06:55:06Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e6c75299-247d-4b44-b5f2-29d0ee78c4c3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e6c75299-247d-4b44-b5f2-29d0ee78c4c3_ms.jpg"
            }
        ],
        "photo": [
            111223763
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 939
    },
    {
        "id": "BtCkFokBqdFzcemqmSsi",
        "globalUserId": "59789000",
        "wbUserId": 59789000,
        "wbUserDetails": {
            "name": "Екатерина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Товар был хорошо упакован, коробка без повреждений. Отличный продукт за небольшие деньги. Вкусно и полезно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-02T12:47:13Z",
        "updatedDate": "2023-08-16T10:09:50.544640759Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-06T08:38:40Z",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2a3c8eba-cc8c-4739-96d5-fd9e569edc7e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2a3c8eba-cc8c-4739-96d5-fd9e569edc7e_ms.jpg"
            }
        ],
        "photo": [
            109691236
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 938
    },
    {
        "id": "r4QgRokBAtKOitoBhHlF",
        "globalUserId": "88916850",
        "wbUserId": 88916850,
        "wbUserDetails": {
            "name": "Алексей",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный заменитель! Беру постоянно! Упакован дополнительно в коробку. Респект поставщику!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-11T18:04:43Z",
        "updatedDate": "2023-08-16T10:09:50.917725992Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-12T06:29:58Z",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "100416396",
                "wbUserId": 100416396,
                "helpfulness": "plus",
                "time": "2023-07-12T04:31:11Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/051693f1-2821-446e-98e8-478b9ad9e28f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/051693f1-2821-446e-98e8-478b9ad9e28f_ms.jpg"
            }
        ],
        "photo": [
            111392102
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 937
    },
    {
        "id": "mkIJDYcBfnpWqlBeqcK-",
        "globalUserId": "48027464",
        "wbUserId": 48027464,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анастасия",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сахарозаменитель отличный. До этого пробовала ради интереса заменить сахар на различные другие таблетки, но мне не нравился вкус. А в этот раз решила попробовать еще раз и мне очень даже понравилось, сах. зам. сладкий и практичная упаковка. Брала 60 пакетов за 199 рублей и думала что это самая низкая цена. Но увидела в Ашане за 149 рублей и немножко огорчилась. Буду теперь брать там.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-23T05:55:50Z",
        "updatedDate": "2023-08-16T10:09:47.072540953Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T05:56:43Z",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/8f6632f6-7cd5-4105-ab1d-f170d7fc392c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8f6632f6-7cd5-4105-ab1d-f170d7fc392c_ms.jpg"
            }
        ],
        "photo": [
            92580205
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 936
    },
    {
        "id": "9lAxz4kBuWYO9ah2yBtH",
        "globalUserId": "44142489",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алена"
        },
        "nmId": 138785478,
        "text": "Спасибо за быструю доставку, рекомендую продавца и магазин.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-07T08:51:34Z",
        "updatedDate": "2023-08-16T10:09:52.251729057Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T09:01:14Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f757b9aa-eaae-4b27-967d-065279130832_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f757b9aa-eaae-4b27-967d-065279130832_ms.jpg"
            }
        ],
        "photo": [
            116605862
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 935
    },
    {
        "id": "YfkJAIgBqd2abVcyP4Hc",
        "globalUserId": "47975846",
        "wbUserId": 47975846,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Люблю этот подсластитель, в подарок на пробу положили три саше с экстрактом черной смородины ) очень необычный, и приятно! Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-09T10:23:06Z",
        "updatedDate": "2023-08-16T10:09:48.962871758Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-17T06:18:47Z",
            "createDate": "2023-05-17T06:18:47Z",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:43:31Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0de577ab-67e0-49c5-bc21-8ea0aa3fa84c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0de577ab-67e0-49c5-bc21-8ea0aa3fa84c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/bcd7abff-a806-4ac0-9074-b1adc1fb9baf_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bcd7abff-a806-4ac0-9074-b1adc1fb9baf_ms.jpg"
            }
        ],
        "photo": [
            100558073,
            100558076
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 934
    },
    {
        "id": "k2YrH4cB8a_hUs_odbbE",
        "globalUserId": "45137034",
        "wbUserId": 45137034,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Марина"
        },
        "nmId": 33478818,
        "text": "Без. Запаха. Мне подошёл отлично. 1 пакетика вполне хватает на чашку чая. Упаковка целая не мятая. Доставка быстрая. Спасибо продавцу за хороший товар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T18:25:55Z",
        "updatedDate": "2023-08-16T10:09:47.264717493Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T13:05:06Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b3ad7b30-f8e2-4cad-bc1a-07513a886aca_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b3ad7b30-f8e2-4cad-bc1a-07513a886aca_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/634d58cf-9cca-43d1-872a-ab9a308c13b0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/634d58cf-9cca-43d1-872a-ab9a308c13b0_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/14e4c977-227d-4603-a93f-765e3e2332d6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/14e4c977-227d-4603-a93f-765e3e2332d6_ms.jpg"
            }
        ],
        "photo": [
            93248119,
            93248120,
            93248122
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 933
    },
    {
        "id": "jjQGOocBQOOSTxXo6_NW",
        "globalUserId": "32841825",
        "wbUserId": 32841825,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Подделка на лицо? Очень интересно 🤔 Мы с молодым человеком пользовались до этого сах.замом «FitParad” покупая его оффлайн у магазинах Пятёрочка, Магнит и так далее. Приобрела сах.зам у этого продавца и мы оба заметили существенную разницу-сах.зам у данного продавцы с WB гораздо менее сладкий. О-оригинал слева и П-подделка справа на фото. Существенно отличается: качество печати (о. имеет более чёткий шрифт) качество бумаги для саше (о. более глянцевый) и конечно насыщенность сладости (о. более сладкий). Вопрос: как это возможно?",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-31T23:35:45Z",
        "updatedDate": "2023-08-28T21:43:06Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-07T12:07:57Z",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Рецептура сахарозаменителя и производственный процесс - не менялись. Рекомендуем встряхивать сахарозаменитель перед употреблением, это способствует равномерному распределению компонентов и достижению указанной сладости! \nКак возможно? Партии в типографии могут незначительно отличаться своим цветом) Мы - магазин, работающий напрямую от производителя, ни о какой подделке речь быть не может. "
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14244925",
                "wbUserId": 14244925,
                "helpfulness": "minus",
                "time": "2023-04-01T08:12:11Z"
            },
            {
                "globalUserId": "23287984",
                "wbUserId": 23287984,
                "helpfulness": "minus",
                "time": "2023-04-01T10:25:26Z"
            },
            {
                "globalUserId": "101396904",
                "wbUserId": 101396904,
                "helpfulness": "minus",
                "time": "2023-04-01T11:52:27Z"
            },
            {
                "globalUserId": "54699442",
                "wbUserId": 54699442,
                "helpfulness": "minus",
                "time": "2023-04-04T05:19:36Z"
            },
            {
                "globalUserId": "28190082",
                "wbUserId": 28190082,
                "helpfulness": "minus",
                "time": "2023-06-16T08:14:55Z"
            },
            {
                "globalUserId": "39343426",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-28T21:43:06Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9538bae1-2864-47bd-9e90-ccb21a313a35_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9538bae1-2864-47bd-9e90-ccb21a313a35_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/fe87864b-4ad0-4fcb-9940-44b8e6e4224a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/fe87864b-4ad0-4fcb-9940-44b8e6e4224a_ms.jpg"
            }
        ],
        "photo": [
            94145746,
            94145747
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 6
        },
        "rank": 932
    },
    {
        "id": "vGJjA4oBXuPSsPE1tCvb",
        "globalUserId": "82694652",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Софья",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Товар отличный, буду рекомендовать, смотри фото",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T12:06:21Z",
        "updatedDate": "2023-08-17T21:12:56Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/8bf1eddc-fdfe-4f2e-8e8c-cfbe8fcfa8e6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8bf1eddc-fdfe-4f2e-8e8c-cfbe8fcfa8e6_ms.jpg"
            }
        ],
        "photo": [
            119374848
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 931
    },
    {
        "id": "BQ0h64kBJytgTHhBikV2",
        "globalUserId": "59994151",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Инна"
        },
        "nmId": 138785478,
        "text": "Всё хорошо, одного пакетика мне хватает попить чай.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-12T19:03:11Z",
        "updatedDate": "2023-08-16T10:09:53.621993783Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/fdbcbb8a-627e-4f2d-917d-4578835044da_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/fdbcbb8a-627e-4f2d-917d-4578835044da_ms.jpg"
            }
        ],
        "photo": [
            118296456
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 930
    },
    {
        "id": "RyQyI4cBxpMXtsmOAgTQ",
        "globalUserId": "5252969",
        "wbUserId": 5252969,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "BY"
        },
        "nmId": 138785478,
        "text": "Беру не в первый раз. Всегда упаковано хорошо. В этот раз положили подарочек: заменитель со вкусом смородины, запах от него на всю кухню). Приятно!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T13:11:33Z",
        "updatedDate": "2023-08-16T10:09:47.297763195Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T12:31:44Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/79052880-de1c-42b6-8e16-813a8ffdcbfa_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/79052880-de1c-42b6-8e16-813a8ffdcbfa_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/b3298fa8-8fda-4041-b750-c3bcac8d0dde_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b3298fa8-8fda-4041-b750-c3bcac8d0dde_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2788fde8-8991-41ab-bf06-d4e8666d5e06_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2788fde8-8991-41ab-bf06-d4e8666d5e06_ms.jpg"
            }
        ],
        "photo": [
            93364997,
            93364998,
            93364999
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 929
    },
    {
        "id": "mhR8SYgBqd2abVcyfayx",
        "globalUserId": "44918098",
        "wbUserId": 44918098,
        "wbUserDetails": {
            "name": "Виктория",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Беру в первый раз мне понравилось.  Один пакетик = 2 ч. л мне этого много я просто делю по полам. На вкус сладкий без всяких посторонних привкусов. Однозначно рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-23T16:41:15Z",
        "updatedDate": "2023-08-16T10:09:49.408871815Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-24T08:33:34Z",
            "createDate": "2023-05-24T08:33:34Z",
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "43410268",
                "wbUserId": 43410268,
                "helpfulness": "plus",
                "time": "2023-05-24T05:37:08Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a7bef42c-4d30-4625-8585-66b6fe8387f3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a7bef42c-4d30-4625-8585-66b6fe8387f3_ms.jpg"
            }
        ],
        "photo": [
            102991132
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 928
    },
    {
        "id": "h3kR64YB7nVCbHkCyc2z",
        "globalUserId": "47125410",
        "wbUserId": 47125410,
        "wbUserDetails": {
            "country": "RU",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Получила всё вовремя, упаковка не нарушена, внутри понравилась  рекламная листовка с видами продукции. Спасибо за приятный бонус 3 стика с вкусом чёрной смородины.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T15:37:37Z",
        "updatedDate": "2023-08-16T10:09:46.778784567Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-17T11:33:26Z",
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3e006f25-c554-4ff2-aa36-895be77cd23b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3e006f25-c554-4ff2-aa36-895be77cd23b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/44ee7256-294d-4cae-9835-ce57bd91531e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/44ee7256-294d-4cae-9835-ce57bd91531e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d587e12e-5448-4283-b2a7-df10934d2ff1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d587e12e-5448-4283-b2a7-df10934d2ff1_ms.jpg"
            }
        ],
        "photo": [
            91426221,
            91426222,
            91426224
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 927
    },
    {
        "id": "-dYFlIgBWg4f_fch8tis",
        "globalUserId": "8394515",
        "wbUserId": 8394515,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель. Пришёл хорошо упакован. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-07T04:03:18Z",
        "updatedDate": "2023-08-16T10:09:49.792705999Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-07T06:03:56Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/fe1e03ad-e10a-460e-8bde-968d1e6eff6a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/fe1e03ad-e10a-460e-8bde-968d1e6eff6a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/fd2345dc-3a2b-4a2c-9dda-175ea62062cd_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/fd2345dc-3a2b-4a2c-9dda-175ea62062cd_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2d0fc5e9-8f05-4c45-b9d5-02cb91bd660c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2d0fc5e9-8f05-4c45-b9d5-02cb91bd660c_ms.jpg"
            }
        ],
        "photo": [
            105257791,
            105257792,
            105257793
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 926
    },
    {
        "id": "UwHX0IkBJytgTHhB7I65",
        "globalUserId": "91579513",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алёна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Доставка быстрая, соответствует описанию. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-07T16:32:39Z",
        "updatedDate": "2023-08-16T10:09:52.296790627Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-08T05:22:27Z",
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "57303961",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-08T05:24:56Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7f22a456-b151-4ffa-ab4d-d754a4d59834_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7f22a456-b151-4ffa-ab4d-d754a4d59834_ms.jpg"
            }
        ],
        "photo": [
            116791046
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 925
    },
    {
        "id": "lV32I4cBmBixw9tzsI7M",
        "globalUserId": "5845779",
        "wbUserId": 5845779,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Альбина"
        },
        "nmId": 138785478,
        "text": "Все хорошо. Приехал Фит хорошо упакован. Коробочка целая. Внутри так же все отлично. До этого брала в пачке, сейчас в стиках. Очень удобно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T16:46:22Z",
        "updatedDate": "2023-08-16T10:09:47.308319636Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T12:25:35Z",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4055fe43-7826-4176-a152-5ecf1af12679_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4055fe43-7826-4176-a152-5ecf1af12679_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d2c49bb8-fe72-4642-aba5-a425ff6ea4c1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d2c49bb8-fe72-4642-aba5-a425ff6ea4c1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0feff5c9-a2a8-4feb-b8b0-2a93b733b7a9_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0feff5c9-a2a8-4feb-b8b0-2a93b733b7a9_ms.jpg"
            }
        ],
        "photo": [
            93413792,
            93413793,
            93413794
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 924
    },
    {
        "id": "_w5GoIkBUYwa0QEf8cTM",
        "globalUserId": "39516308",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "беру не в первый раз, мне понравился, удобно брать на работу)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-29T06:12:31Z",
        "updatedDate": "2023-08-16T10:09:51.673104413Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:42:39Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9cf5f94b-47cd-468f-b000-45458b864288_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9cf5f94b-47cd-468f-b000-45458b864288_ms.jpg"
            }
        ],
        "photo": [
            114702736
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 923
    },
    {
        "id": "oiHDFogBUdF-4nlz8iM0",
        "globalUserId": "15947776",
        "wbUserId": 15947776,
        "wbUserDetails": {
            "country": "ru",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Доставка быстрая, упаковано хорошо. Срок годности 2 года. Изготовление 03.2023. Мне нравится этот продукт. Без запаха , в меру сладкий. Одного пакетика хватает на большой  стакан чая.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-13T20:18:40Z",
        "updatedDate": "2023-08-16T10:09:49.088034721Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-16T12:43:56Z",
            "createDate": "2023-05-16T12:43:56Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/fdcb8b24-d99d-43fb-84f7-616a635f0396_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/fdcb8b24-d99d-43fb-84f7-616a635f0396_ms.jpg"
            }
        ],
        "photo": [
            101363867
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 922
    },
    {
        "id": "E6PDMogB3ieoZoqxO6lF",
        "globalUserId": "2820250",
        "wbUserId": 2820250,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Людмила"
        },
        "nmId": 33478818,
        "text": "Удобно брать с собой в машину, на заправке в кофе добавлять) очень нравился сахзам этой фирмы, так как нет посторонних запахов, просто сладость+очень экономичен",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-19T06:47:16Z",
        "updatedDate": "2023-08-16T10:09:49.259709440Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-19T06:48:22Z",
            "createDate": "2023-05-19T06:48:22Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e90f82cd-2c28-42c7-a82d-fe2e9d4053e7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e90f82cd-2c28-42c7-a82d-fe2e9d4053e7_ms.jpg"
            }
        ],
        "photo": [
            102253223
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 921
    },
    {
        "id": "6O126oYB5HDhYKpHIAAt",
        "globalUserId": "2527811",
        "wbUserId": 2527811,
        "wbUserDetails": {
            "name": "Олеся",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Упаковано отлично, всё в защитной пленке. В каждой упаковке по джему в подарок, очень приятно. На фит параде уже 6 лет, без никого никак)) \nДата изготовления вообще свежая, 5 марта 2023👍 благодарю продавца!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T12:47:35Z",
        "updatedDate": "2023-08-16T10:09:46.768088079Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-16T13:08:20Z",
            "metadata": null,
            "editable": false,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "2527811",
                "wbUserId": 2527811,
                "helpfulness": "plus",
                "time": "2023-03-16T13:21:03Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5f12f2a0-d21e-48cf-aa3d-a0c088fb64ed_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5f12f2a0-d21e-48cf-aa3d-a0c088fb64ed_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/601a7038-9d09-4c67-ae00-feed0bc38943_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/601a7038-9d09-4c67-ae00-feed0bc38943_ms.jpg"
            }
        ],
        "photo": [
            91388556,
            91388557
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 920
    },
    {
        "id": "f395b4kBqd2abVcye6w_",
        "globalUserId": "25747568",
        "wbUserId": 25747568,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Надежда",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Упаковано ужасно, коробки помяты. Минус звезда за это. Внутри все целое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-19T18:46:20Z",
        "updatedDate": "2023-08-16T10:09:51.227865668Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T08:46:11Z",
            "supplierId": 237660,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "22659403",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-07T18:24:53Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/01633442-5adb-4355-b2b0-ffdb7abc74ee_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/01633442-5adb-4355-b2b0-ffdb7abc74ee_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2bdc6a62-9ba8-4f72-8343-800df0a20d78_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2bdc6a62-9ba8-4f72-8343-800df0a20d78_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/07a04bc4-8fa9-4573-9156-a421abd813a6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/07a04bc4-8fa9-4573-9156-a421abd813a6_ms.jpg"
            }
        ],
        "photo": [
            112931965,
            112931968,
            112931969
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 919
    },
    {
        "id": "3NZFqogBCYLDkq7NZdN_",
        "globalUserId": "10680221",
        "wbUserId": 10680221,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алла"
        },
        "nmId": 138785478,
        "text": "Товар пришел быстро . По качеству и вкусу мне нравится . Привычки вроде пока не вызвал . Отличная замена сахару",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-11T11:44:15Z",
        "updatedDate": "2023-08-16T10:09:49.926449251Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-13T12:56:50Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b532bc2a-bc38-4f9f-87c7-b78a407a14e4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b532bc2a-bc38-4f9f-87c7-b78a407a14e4_ms.jpg"
            }
        ],
        "photo": [
            105977878
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 918
    },
    {
        "id": "84YG0YkBKNxVyjcCFQph",
        "globalUserId": "18403681",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ханукаханат",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все пришло в целости и сохранности , спасибо 🌹",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-07T17:23:04Z",
        "updatedDate": "2023-08-16T10:09:52.304242616Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-08T05:21:41Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/dc063f7d-9215-412d-891e-055b0026e306_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dc063f7d-9215-412d-891e-055b0026e306_ms.jpg"
            }
        ],
        "photo": [
            116808907
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 917
    },
    {
        "id": "QeD7PIcBTZEOO9f5-HPk",
        "globalUserId": "33548378",
        "wbUserId": 33548378,
        "wbUserDetails": {
            "name": "Екатерина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Берём не первый раз ,но в этот раз ждал приятный сюрприз виде джема в каждой коробке. Отличный сахара заменитель!\nДжем отлично подойдёт к чаю ,к блинам и сырникам)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-01T13:22:39Z",
        "updatedDate": "2023-08-16T10:09:47.529141871Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T11:54:29Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "11185984",
                "wbUserId": 11185984,
                "helpfulness": "plus",
                "time": "2023-04-01T14:21:03Z"
            },
            {
                "globalUserId": "33548378",
                "wbUserId": 33548378,
                "helpfulness": "plus",
                "time": "2023-04-06T18:50:58Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/34288683-1f2f-4c8c-b5c7-a9652141bba4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/34288683-1f2f-4c8c-b5c7-a9652141bba4_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/22d2ea6f-191a-4ca0-af2f-75332e9ae918_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/22d2ea6f-191a-4ca0-af2f-75332e9ae918_ms.jpg"
            }
        ],
        "photo": [
            94234261,
            94234262
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 916
    },
    {
        "id": "rTW994gBXuPSsPE1AN7M",
        "globalUserId": "18190847",
        "wbUserId": 18190847,
        "wbUserDetails": {
            "name": "Ксения",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Даже одну звезду не за что ставить. Пришел абсолютно другой товар. За 574 р вместо трех упаковок сахарозаменителя прислали суп из перловки!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-26T12:45:39Z",
        "updatedDate": "2023-08-16T10:09:50.352650614Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-26T13:13:10Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Мы не занимаемся доставкой. Товар одобрен к возврату. ",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "18190847",
                "wbUserId": 18190847,
                "helpfulness": "plus",
                "time": "2023-06-26T12:47:47Z"
            },
            {
                "globalUserId": "58853655",
                "wbUserId": 58853655,
                "helpfulness": "plus",
                "time": "2023-06-26T13:36:32Z"
            },
            {
                "globalUserId": "9474901",
                "wbUserId": 9474901,
                "helpfulness": "plus",
                "time": "2023-06-29T08:23:57Z"
            },
            {
                "globalUserId": "98564982",
                "wbUserId": 98564982,
                "helpfulness": "plus",
                "time": "2023-07-12T22:40:32Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/248b18c1-044a-4e6b-8a57-bbcfbbac92d5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/248b18c1-044a-4e6b-8a57-bbcfbbac92d5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2fa0b1bf-5f7a-4970-addd-df151935dbab_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2fa0b1bf-5f7a-4970-addd-df151935dbab_ms.jpg"
            }
        ],
        "photo": [
            108626614,
            108626615
        ],
        "video": null,
        "votes": {
            "pluses": 4,
            "minuses": 0
        },
        "rank": 915
    },
    {
        "id": "ZbrqOIcBJ99b87NGvyAG",
        "globalUserId": "17704122",
        "wbUserId": 17704122,
        "wbUserDetails": {
            "name": "Дарина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Заказывала для мамы, ей очень понравился 😻 Сказала что нет привкуса, этого послевкусия….Как обычно бывает у сахарозаменителей! Будем заказывать еще!!!!))) было два подарочных стика со вкусом черники, приятно 👌🏼☺️ Продолжайте радовать своих клиентов и дальше !!!))))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-31T18:25:21Z",
        "updatedDate": "2023-08-16T10:09:47.514106152Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-07T12:29:25Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/96815d89-675e-46b4-bd52-67e1576a0fb6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/96815d89-675e-46b4-bd52-67e1576a0fb6_ms.jpg"
            }
        ],
        "photo": [
            94127047
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 914
    },
    {
        "id": "AHm6UIcB8a_hUs_oJfeU",
        "globalUserId": "85704051",
        "wbUserId": 85704051,
        "wbUserDetails": {
            "country": "BY",
            "name": "Нина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Добрый день, товар пришел без повреждений, отдельное спасибо за подарочек, а также когда я просматривала отзывы у меня был шок, люди вы в первую очередь заказываете сам товар ,а не  подарок и я не вижу смысла снижать звёзды за то что вам его не положили",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-05T09:23:09Z",
        "updatedDate": "2023-08-16T10:09:47.735923637Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-05T09:28:24Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "13742342",
                "wbUserId": 13742342,
                "helpfulness": "plus",
                "time": "2023-04-05T10:19:35Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4c3a87e2-b4db-414b-86be-f210bc588b4f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4c3a87e2-b4db-414b-86be-f210bc588b4f_ms.jpg"
            }
        ],
        "photo": [
            94879114
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 913
    },
    {
        "id": "IfsyQoYB8a_hUs_ok1u_",
        "globalUserId": "47767742",
        "wbUserId": 47767742,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ангелина"
        },
        "nmId": 33478818,
        "text": "отличный заменитель сладкому сахару ! беру уже второй раз у данного производителя . доставка быстрая , товар упакован в небольшую коробку, благодаря которой все доходит в целости и сохранности ! добавляю сахарозаменитель в основном в чай и его не отличить привкусу от обычного сахара ! так же пробовала добавлять в выпечку и она получилась абсолютно такой же , как на сахаре . поэтому настоятельно рекомендую к приобретению ! не пожалеете 💗",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-11T20:37:36Z",
        "updatedDate": "2023-08-16T10:09:45.580653688Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-13T08:56:12Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/290fd29c-5cd3-4685-bb3e-9b274a3cec65_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/290fd29c-5cd3-4685-bb3e-9b274a3cec65_ms.jpg"
            }
        ],
        "photo": [
            85509604
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 912
    },
    {
        "id": "03dgHYcBMJfQQSHDXF_6",
        "globalUserId": "14252930",
        "wbUserId": 14252930,
        "wbUserDetails": {
            "name": "Марина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Товар пришёл упакован в коробку. Всё целое, без повреждений. Дома открыла, в каждой коробочке лежал подарочек- джем черника. Очень приятно. Спасибо. К покупке советую!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T10:04:27Z",
        "updatedDate": "2023-08-16T10:09:47.230241954Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:37:36Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a92ec5ec-6f6b-4ac4-8632-832c1b1217a5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a92ec5ec-6f6b-4ac4-8632-832c1b1217a5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/5dee5ac4-93ce-4110-b1cf-0dce8c6c6060_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5dee5ac4-93ce-4110-b1cf-0dce8c6c6060_ms.jpg"
            }
        ],
        "photo": [
            93143291,
            93143292
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 911
    },
    {
        "id": "57XpH4cBbskpjQp2NsZB",
        "globalUserId": "21815658",
        "wbUserId": 21815658,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Раксана"
        },
        "nmId": 31468103,
        "text": "Я очень приятно была удивлена когда увидела подарок внутри сколько не заказывала никогда не было ☺️спасибо большое ☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T21:53:10Z",
        "updatedDate": "2023-08-16T10:09:47.267680073Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T13:02:56Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "61830412",
                "wbUserId": 61830412,
                "helpfulness": "plus",
                "time": "2023-03-27T00:18:28Z"
            },
            {
                "globalUserId": "14174265",
                "wbUserId": 14174265,
                "helpfulness": "plus",
                "time": "2023-03-27T05:31:37Z"
            },
            {
                "globalUserId": "20405000",
                "wbUserId": 20405000,
                "helpfulness": "plus",
                "time": "2023-04-03T02:24:27Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/15721115-d38a-4b6f-aaff-b175020a310b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/15721115-d38a-4b6f-aaff-b175020a310b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/bd2648a0-d2da-4fac-bd20-2bea98e6b36c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bd2648a0-d2da-4fac-bd20-2bea98e6b36c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/dc5d7a02-5b9f-4a2f-8492-d1196dd5ac01_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dc5d7a02-5b9f-4a2f-8492-d1196dd5ac01_ms.jpg"
            }
        ],
        "photo": [
            93267005,
            93267006,
            93267007
        ],
        "video": null,
        "votes": {
            "pluses": 3,
            "minuses": 0
        },
        "rank": 910
    },
    {
        "id": "q7MhKYcBJ99b87NG-enY",
        "globalUserId": "36206881",
        "wbUserId": 36206881,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Светлана"
        },
        "nmId": 31468103,
        "text": "Беру не первый раз. Да, упаковка порой мятая(  это, конечно, зависит от транспортировки) , но упаковано всё очень хорошо! Мне нравится! Буду заказывать и дальше. В подарок ( в каждую упаковку) положили сюрприз- черничный джем)))Неожиданно и ооочень приятно. Благодарю",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-28T16:51:45Z",
        "updatedDate": "2023-08-16T10:09:47.358104982Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:18:50Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bb7a4056-5e8d-4952-913f-0cc45ab4ec81_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bb7a4056-5e8d-4952-913f-0cc45ab4ec81_ms.jpg"
            }
        ],
        "photo": [
            93591318
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 909
    },
    {
        "id": "_FcwcogBG4PPbu1thotg",
        "globalUserId": "29709114",
        "wbUserId": 29709114,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Дарья"
        },
        "nmId": 31468103,
        "text": "Прислали 60 саше, а не 90 как заявлено и заплатила я соответственно, как за 90!Взяла и не посмотрела,так бы отказалась.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-31T14:22:43Z",
        "updatedDate": "2023-08-16T10:09:49.634394101Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-01T10:49:45Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Очень жаль, что товар был перепутан, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "18550757",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-07-16T07:24:00Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/06a8347d-a705-4b4d-abd4-b22b0fba29da_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/06a8347d-a705-4b4d-abd4-b22b0fba29da_ms.jpg"
            }
        ],
        "photo": [
            104227402
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 908
    },
    {
        "id": "194PsocBqd2abVcysuBk",
        "globalUserId": "11195753",
        "wbUserId": 11195753,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Светлана"
        },
        "nmId": 138785478,
        "text": "Не люблю сахарозаиенитель. Но этот очень понравился. У него нет неприятного послевкусия и искажения вкуса, как после некоторых заменителей. Этот буду заказывать ещё. Спасибо продавцу и  wb.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-24T06:59:46Z",
        "updatedDate": "2023-08-16T10:09:48.491345748Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-24T08:57:25Z",
            "createDate": "2023-04-24T08:57:25Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3c5fc2bd-3085-4e03-8aca-736f5aa18a3b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3c5fc2bd-3085-4e03-8aca-736f5aa18a3b_ms.jpg"
            }
        ],
        "photo": [
            97985793
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 907
    },
    {
        "id": "cAjpLYcB5HDhYKpHiemc",
        "globalUserId": "25349435",
        "wbUserId": 25349435,
        "wbUserDetails": {
            "name": "Алина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всё пришло в идеальном состоянии, коробочка не помятая. А ещё в подарок положили 3 стика сахзама со вкусом чёрной смородины🥰 Очень обрадовалась, потому что любитель подобных штучек! Я уже пробовала в стиках с ванилью и карамелью, тоже очень классные)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T15:08:13Z",
        "updatedDate": "2023-08-16T10:09:47.419816912Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:44:18Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/007a8320-1ab6-4592-a29e-5cd565c25693_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/007a8320-1ab6-4592-a29e-5cd565c25693_ms.jpg"
            }
        ],
        "photo": [
            93744739
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 906
    },
    {
        "id": "KApge4oB0v3RIh2JzQW4",
        "globalUserId": "55373318",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Не думала, что смогу найти достойный сахарозаменитель. Но это чудо. Я очень довольна товаром. 1 пакетик на чашку кофе или чая - очень сладко и вкусно. Я брала для похудения. Не могу кофе пить без сахара. Брала другие сахарозаменители, какой то не приятный привкус, а этот, просто прелесть. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T19:17:36Z",
        "updatedDate": "2023-09-15T20:48:38Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "8077553",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-15T20:48:38Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 905
    },
    {
        "id": "i8uWOokBU2re2XP7wHp3",
        "globalUserId": "22914016",
        "wbUserId": 22914016,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Виктория"
        },
        "nmId": 138785478,
        "text": "Беру всегда именно этот заменитель, всё хорошо, доставка быстрая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-09T12:18:26Z",
        "updatedDate": "2023-08-16T10:09:50.811963081Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-10T13:25:23Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/8f248703-bff1-44ef-a583-2decb89b47da_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8f248703-bff1-44ef-a583-2decb89b47da_ms.jpg"
            }
        ],
        "photo": [
            110940174
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 904
    },
    {
        "id": "dl8QKYcBmBixw9tzlo5O",
        "globalUserId": "58396864",
        "wbUserId": 58396864,
        "wbUserDetails": {
            "name": "Вера",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Пришло хорошо упаковано. Для меня это важно. За подарочек отдельное спасибо! Доставили быстро. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-28T16:32:46Z",
        "updatedDate": "2023-08-16T10:09:47.355067505Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:18:55Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bd864a4f-0af2-4512-bf11-c4fa61222cf2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bd864a4f-0af2-4512-bf11-c4fa61222cf2_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c56bf72d-1155-4ab2-a59f-6232cd461997_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c56bf72d-1155-4ab2-a59f-6232cd461997_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0b51a401-4320-412e-8165-c417dc351f97_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0b51a401-4320-412e-8165-c417dc351f97_ms.jpg"
            }
        ],
        "photo": [
            93587041,
            93587042,
            93587043
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 903
    },
    {
        "id": "lgIWhocBG4PPbu1tw5Jl",
        "globalUserId": "100958020",
        "wbUserId": 100958020,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Всё супер ! И товар , и упаковка , и быстрая доставка , и обслуживание , и неожиданный , но оооочень приятный подарок 🎁 Спасибо Вам ! Буду заказывать только у Вас и рекомендовать всем знакомым ! ❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-15T18:04:11Z",
        "updatedDate": "2023-08-16T10:09:48.184703041Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:28:50Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "100958020",
                "wbUserId": 100958020,
                "helpfulness": "plus",
                "time": "2023-04-18T09:37:59Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/671c4b5e-181a-45d6-9618-94d85c3e58de_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/671c4b5e-181a-45d6-9618-94d85c3e58de_ms.jpg"
            }
        ],
        "photo": [
            96597251
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 902
    },
    {
        "id": "8CC7o4gBlM6cTg_RJONf",
        "globalUserId": "97030051",
        "wbUserId": 97030051,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Спасибо продавцу, товар пришёл в целости и сохранности, но доставка это что то.... Логистика🤦‍♀️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-10T05:15:31Z",
        "updatedDate": "2023-08-16T10:09:49.882585321Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T09:08:07Z",
            "createDate": "2023-06-20T09:08:07Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c364119e-94ee-4929-ba78-269e0bd6c427_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c364119e-94ee-4929-ba78-269e0bd6c427_ms.jpg"
            }
        ],
        "photo": [
            105763501
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 901
    },
    {
        "id": "e1xeSIcB6z8QKrUHRW0T",
        "globalUserId": "59087668",
        "wbUserId": 59087668,
        "wbUserDetails": {
            "country": "RU",
            "name": "Лариса",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришёл упакован в коробку. В упаковке все 90 шт. Плюс бонус, подсластить фитпарад со вкусом черной смородины. Огромное спасибо за бонус.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-03T18:25:50Z",
        "updatedDate": "2023-08-16T10:09:47.657817289Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T05:20:00Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/08f89f74-2a81-4f14-9aa8-27859d390ed1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/08f89f74-2a81-4f14-9aa8-27859d390ed1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2bcf02ee-9efe-468f-b347-fc14fb0a0372_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2bcf02ee-9efe-468f-b347-fc14fb0a0372_ms.jpg"
            }
        ],
        "photo": [
            94632966,
            94632967
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 900
    },
    {
        "id": "9-rsIooB0v3RIh2JfjLs",
        "globalUserId": "19215440",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Инна"
        },
        "nmId": 138785478,
        "text": "Вот в таком виде получила сегодня товар.обидно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-23T15:03:59Z",
        "updatedDate": "2023-08-24T06:28:33Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-24T06:28:33Z",
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9701c164-61d4-4324-b975-18d36074b956_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9701c164-61d4-4324-b975-18d36074b956_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d345d140-a2a0-481a-b52f-6d4ba3c5c3f5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d345d140-a2a0-481a-b52f-6d4ba3c5c3f5_ms.jpg"
            }
        ],
        "photo": [
            121029920,
            121029922
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 899
    },
    {
        "id": "zxJnrYkBpT2rITcGWyZ0",
        "globalUserId": "52476497",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Очень хороший заменитель сахара.Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-31T19:22:59Z",
        "updatedDate": "2023-08-16T10:09:51.824282098Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T08:30:01Z",
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/413a9a1b-fe50-4d55-afae-7229cc3a4219_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/413a9a1b-fe50-4d55-afae-7229cc3a4219_ms.jpg"
            }
        ],
        "photo": [
            115215499
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 898
    },
    {
        "id": "HnhjW4cBIq9DQExlUlKT",
        "globalUserId": "60590208",
        "wbUserId": 60590208,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ирина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо за пробник со смородиной, аромат шикарный, будто свежие листочки смородины.\nХочу порадовать с ванилью, рекомендовали его.\nВсе супер.\nТолько понял не могу, если мне для стрепни надо 100гр, сахара, это 10саше?",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-07T11:04:09Z",
        "updatedDate": "2023-08-16T10:09:47.832286007Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-07T11:19:57Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Да, всё верно ",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "60590208",
                "wbUserId": 60590208,
                "helpfulness": "minus",
                "time": "2023-04-07T11:41:55Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/db2dc092-56e3-4763-9ce7-6d4777c23eea_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/db2dc092-56e3-4763-9ce7-6d4777c23eea_ms.jpg"
            }
        ],
        "photo": [
            95248681
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 897
    },
    {
        "id": "hoUceocBqVKcxP1mIW6-",
        "globalUserId": "16037605",
        "wbUserId": 16037605,
        "wbUserDetails": {
            "country": "ru",
            "name": "Вера",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "К товару нет претензии.  Упаковано хорошо . Доставка задержалась . Понимаю ,что это не вина продовца .Спасибо за бонус .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-13T10:14:37Z",
        "updatedDate": "2023-08-16T10:09:48.080416325Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-13T10:28:31Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0fed4541-44fa-4ea9-b523-c6a315cd79ce_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0fed4541-44fa-4ea9-b523-c6a315cd79ce_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4011f94b-ec97-4268-bc5d-be96f67e6bf5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4011f94b-ec97-4268-bc5d-be96f67e6bf5_ms.jpg"
            }
        ],
        "photo": [
            96187174,
            96187175
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 896
    },
    {
        "id": "2VB5YogBBgFQoSo033AI",
        "globalUserId": "27256159",
        "wbUserId": 27256159,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Товар пришел упакован в картонную коробку. Товар пришёл целым. Продавца советую. Заказала две упаковки сразу.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-28T13:08:54Z",
        "updatedDate": "2023-08-16T10:09:49.542906888Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-29T10:29:50Z",
            "createDate": "2023-05-29T10:29:50Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b617f927-3a93-433b-aeda-d574943b20a6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b617f927-3a93-433b-aeda-d574943b20a6_ms.jpg"
            }
        ],
        "photo": [
            103722738
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 895
    },
    {
        "id": "wUNLP4gBBgFQoSo0BE1O",
        "globalUserId": "7812360",
        "wbUserId": 7812360,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Александра"
        },
        "nmId": 33478818,
        "text": "Купленный на ВБ справа. \nОн сильно отличается по текстуре от заказанного на официальном. \nвкус тоже разный \nПро пакетики вообще молчу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-21T17:11:01Z",
        "updatedDate": "2023-08-16T10:09:49.326603563Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-24T08:42:20Z",
            "text": "Здравствуйте, у Вас фото не приложилось. Отмечаем, что мы единственный официальный магазин от производителя на маркетплейсе и поставляем товар напрямую с производства. ",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "33002849",
                "wbUserId": 33002849,
                "helpfulness": "plus",
                "time": "2023-05-23T07:59:21Z"
            },
            {
                "globalUserId": "112048132",
                "wbUserId": 112048132,
                "helpfulness": "plus",
                "time": "2023-05-25T10:38:51Z"
            },
            {
                "globalUserId": "21685125",
                "wbUserId": 21685125,
                "helpfulness": "plus",
                "time": "2023-06-04T11:38:03Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/97642390-372f-4f71-a61a-7ccea0c6c6d7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/97642390-372f-4f71-a61a-7ccea0c6c6d7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/55204ea8-4bc0-48ef-99c3-04bebc1959f1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/55204ea8-4bc0-48ef-99c3-04bebc1959f1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a73c40c0-5697-4ead-b903-820199813867_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a73c40c0-5697-4ead-b903-820199813867_ms.jpg"
            }
        ],
        "photo": [
            102669402,
            102669403,
            102669404
        ],
        "video": null,
        "votes": {
            "pluses": 3,
            "minuses": 0
        },
        "rank": 894
    },
    {
        "id": "sdqPy4kBU2re2XP7bdTt",
        "globalUserId": "1250966",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Упаковка не пострадала",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-06T15:55:22Z",
        "updatedDate": "2023-08-16T10:09:52.182781089Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T05:45:25Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ec47c7aa-ddcb-44c6-8952-3e88077de634_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ec47c7aa-ddcb-44c6-8952-3e88077de634_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/e51abf76-8a0f-4b53-8e35-6f856779a61b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e51abf76-8a0f-4b53-8e35-6f856779a61b_ms.jpg"
            }
        ],
        "photo": [
            116426399,
            116426400
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 893
    },
    {
        "id": "yAl_JIkBfuChzMpwqKVJ",
        "globalUserId": "86076769",
        "wbUserId": 86076769,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Всё отлично запаковано.Хороший заменитель. Вкус не приторный.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-05T05:21:33Z",
        "updatedDate": "2023-08-16T10:09:50.638849059Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-05T10:36:31Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d80fa1cc-7efe-4ca5-b4ce-423671e07b42_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d80fa1cc-7efe-4ca5-b4ce-423671e07b42_ms.jpg"
            }
        ],
        "photo": [
            110168965
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 892
    },
    {
        "id": "49jHRYoBUdF-4nlzZBQU",
        "globalUserId": "57074128",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Валентина"
        },
        "nmId": 138785478,
        "text": "Сахарозаменитель ФитПарад № 7 СУПЕР. Мне достаточно 1 пакета 1 гр на кружку 180 мл. В составе есть стивия, которую я не переношу. Думала, что все заказ пойдет на выброс, но стивия не чувствуется на вкус в этом заменителе. Я перепробовала уже много сахарозаменителей и все были на выброс. Этот подошел отлично. Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T09:30:10Z",
        "updatedDate": "2023-08-30T21:33:56Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-30T10:58:35Z",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "8159439",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-30T21:33:56Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 891
    },
    {
        "id": "zwbpYYcBBhsOOdMGspvM",
        "globalUserId": "12041562",
        "wbUserId": 12041562,
        "wbUserDetails": {
            "name": "Анастасия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень удобный формат заменителя сахара, удобно брать с собой. В подарок положили 2 саше сах.зама со вкусом смородины",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-08T17:28:38Z",
        "updatedDate": "2023-08-16T10:09:47.907894960Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T06:12:17Z",
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ac0cb7bd-2dce-41ce-bc37-809b61bcce52_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ac0cb7bd-2dce-41ce-bc37-809b61bcce52_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/bb880e3a-9a71-4caf-8ece-27ccf7709ad4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bb880e3a-9a71-4caf-8ece-27ccf7709ad4_ms.jpg"
            }
        ],
        "photo": [
            95489140,
            95489141
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 890
    },
    {
        "id": "iPjypYgBuQ90tE6Dnjra",
        "globalUserId": "62861135",
        "wbUserId": 62861135,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Владислав",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Мошенничество! В коробке оказалось не 90 шт, как заявлено, а 60. Что будем делать, граждане продавцы? Будете досылать ещё одну коробку?",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-10T15:35:21Z",
        "updatedDate": "2023-08-16T10:09:49.902174592Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-20T08:59:12Z",
            "text": "Здравствуйте, просьба написать нам по адресу, указанному на упаковке, приложив фотографии. ",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "17504743",
                "wbUserId": 17504743,
                "helpfulness": "plus",
                "time": "2023-06-11T15:26:01Z"
            },
            {
                "globalUserId": "62861135",
                "wbUserId": 62861135,
                "helpfulness": "plus",
                "time": "2023-06-12T12:09:06Z"
            },
            {
                "globalUserId": "58823141",
                "wbUserId": 58823141,
                "helpfulness": "plus",
                "time": "2023-06-13T07:11:20Z"
            },
            {
                "globalUserId": "27604694",
                "wbUserId": 27604694,
                "helpfulness": "plus",
                "time": "2023-06-14T11:01:43Z"
            },
            {
                "globalUserId": "30268521",
                "wbUserId": 30268521,
                "helpfulness": "plus",
                "time": "2023-06-15T11:45:19Z"
            },
            {
                "globalUserId": "9194838",
                "wbUserId": 9194838,
                "helpfulness": "plus",
                "time": "2023-07-10T17:38:17Z"
            },
            {
                "globalUserId": "98564982",
                "wbUserId": 98564982,
                "helpfulness": "plus",
                "time": "2023-07-12T22:40:13Z"
            },
            {
                "globalUserId": "18550757",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-07-16T07:22:55Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/86d0aa3e-b61f-46c7-ad64-a499e47c9225_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/86d0aa3e-b61f-46c7-ad64-a499e47c9225_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/38396b6b-f466-4bd2-8557-a5c46da7f631_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/38396b6b-f466-4bd2-8557-a5c46da7f631_ms.jpg"
            }
        ],
        "photo": [
            105865811,
            105865812
        ],
        "video": null,
        "votes": {
            "pluses": 8,
            "minuses": 0
        },
        "rank": 889
    },
    {
        "id": "kGG1LYcBmBixw9tzX0Bh",
        "globalUserId": "26022269",
        "wbUserId": 26022269,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анна"
        },
        "nmId": 31468103,
        "text": "Редко пишу отзывы, но тут была очень приятно удивлена, заказала набор, из трёх коробочек, и в каждой коробочке нашла сладкий сюрприз, спасибо большое было очень приятно получить такой бонус, особенно черничный🤣😘😘😘",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T14:11:14Z",
        "updatedDate": "2023-08-16T10:09:47.415342984Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:12:22Z",
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/62eae26a-e41a-4123-ab0e-86be77b7c92e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/62eae26a-e41a-4123-ab0e-86be77b7c92e_ms.jpg"
            }
        ],
        "photo": [
            93733421
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 888
    },
    {
        "id": "WBOpsocBG4PPbu1tRPvb",
        "globalUserId": "12747052",
        "wbUserId": 12747052,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Мария"
        },
        "nmId": 33478818,
        "text": "Все отлично) этот пока не пробовала, купила такой же, но ванильный и начала с него, он супер) так что этот, уверена, такой же, просто без ванильного привкуса)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-24T09:47:30Z",
        "updatedDate": "2023-08-16T10:09:48.498989157Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-24T10:19:45Z",
            "createDate": "2023-04-24T10:19:45Z",
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/de228a44-b443-486b-85fc-a627d540d7c6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/de228a44-b443-486b-85fc-a627d540d7c6_ms.jpg"
            }
        ],
        "photo": [
            98013818
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 887
    },
    {
        "id": "-BAEQocB5HDhYKpH7Fvn",
        "globalUserId": "20542153",
        "wbUserId": 20542153,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Юлия",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Хороший сахзам, не один год его покупаю. А в этот раз взяла вложение на 90 Саше и внутри был презент из трёх пакетиков черносмородинного вкуса. В след раз закажу такой, очень ароматный. Спасибо, приятно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T12:50:32Z",
        "updatedDate": "2023-08-16T10:09:47.580455342Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T10:02:08Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/503f448c-fe31-430c-84ef-6129eba39385_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/503f448c-fe31-430c-84ef-6129eba39385_ms.jpg"
            }
        ],
        "photo": [
            94390741
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 886
    },
    {
        "id": "qhQZqIcBl47NwnZ1CDMB",
        "globalUserId": "46091931",
        "wbUserId": 46091931,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Дилара"
        },
        "nmId": 33478818,
        "text": "Коробка пришла немного порвана, но не критично. Сахзам очень сладкий",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-22T08:33:45Z",
        "updatedDate": "2023-08-16T10:09:48.426011113Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-26T08:19:01Z",
            "createDate": "2023-04-26T08:19:01Z",
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/db1aec3b-5282-48bd-833c-c9da6a0086b0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/db1aec3b-5282-48bd-833c-c9da6a0086b0_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/60405e92-07db-4b47-9ec0-65eff810156c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/60405e92-07db-4b47-9ec0-65eff810156c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c36d20df-d345-43d2-a2ca-fe4632b3dbd4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c36d20df-d345-43d2-a2ca-fe4632b3dbd4_ms.jpg"
            }
        ],
        "photo": [
            97674851,
            97674855,
            97674857
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 885
    },
    {
        "id": "DEqL7IYBIq9DQExlYfOT",
        "globalUserId": "87749467",
        "wbUserId": 87749467,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ислам"
        },
        "nmId": 33478818,
        "text": "У меня к вам два вопроса ! \nНе знаю может это из-за этого , но хочу уточнить ! Я месяц как не ем сахара содержащий продукты и сам сахар , но как я выпил чай с пакетиком , у меня заболел живот , что может стать причиной этому ? \nВторой вопрос! \nМожно ли например версию 7 и 10 добавить в чай смещая их по одному пакетику (1гр саше и 0.5гр стик) в кружку чая ? \nСпасибо за ответ ♥️🌹",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T22:30:03Z",
        "updatedDate": "2023-08-16T10:09:46.790964600Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-17T11:29:16Z",
            "supplierId": 237660,
            "text": "Здравствуйте, компоненты сахарозаменителя могут оказывать послабляющее действие! Смешивать конечно можете, главное соблюдать суточную норму. ",
            "metadata": null,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/58ae2210-9c27-4692-9cfb-f434fa04359e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/58ae2210-9c27-4692-9cfb-f434fa04359e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c2fd28be-916d-4ce4-9475-72d1c4db858c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c2fd28be-916d-4ce4-9475-72d1c4db858c_ms.jpg"
            }
        ],
        "photo": [
            91483872,
            91483873
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 884
    },
    {
        "id": "uO069ocBCaW1RxrSW6lv",
        "globalUserId": "24103956",
        "wbUserId": 24103956,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анна"
        },
        "nmId": 138785478,
        "text": "Отличный товар! В подарок 3 пакетика с каким то вкусом.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-07T12:40:32Z",
        "updatedDate": "2023-08-16T10:09:48.899733306Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-18T09:01:03Z",
            "createDate": "2023-05-18T09:01:03Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:44:21Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0af3d175-45e8-4638-8c59-df02b8524e81_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0af3d175-45e8-4638-8c59-df02b8524e81_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/db3a4903-3e5d-4c04-9e4e-116d31a6a71b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/db3a4903-3e5d-4c04-9e4e-116d31a6a71b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4abd60ff-707c-4e14-a16b-7a96c2061341_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4abd60ff-707c-4e14-a16b-7a96c2061341_ms.jpg"
            }
        ],
        "photo": [
            100258469,
            100258471,
            100258472
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 883
    },
    {
        "id": "WglHwYoBUdF-4nlzPUAk",
        "globalUserId": "48138831",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Виктория",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Высыпала в кружку с кофе (кружка примерно 350 мл) целый пакетик, никогда еще такой компот не пила, ну уж очень сладко 😁. А вообще хороший подсластитель, без химозного привкуса, но надо не весь пакет",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T09:03:06Z",
        "updatedDate": "2023-09-23T10:00:48Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "111614915",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-23T10:00:48Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 882
    },
    {
        "id": "OAmonYcBBgFQoSo05T0h",
        "globalUserId": "20787241",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все целое, в коробочке подарок 🎁 ,  фит парад  со вкусом смородины , пахнет приятно на вкус  ещё не пробовали , . заказала 1000 штук  пакет,  продавцу спасибо!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-20T07:55:04Z",
        "updatedDate": "2023-08-16T10:09:48.352115Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-20T08:05:31Z",
            "createDate": "2023-04-20T08:05:31Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9f1768c0-fd95-4c45-8a87-1324117b48f7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9f1768c0-fd95-4c45-8a87-1324117b48f7_ms.jpg"
            }
        ],
        "photo": [
            97323867
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 881
    },
    {
        "id": "FmBFUocB6z8QKrUHtqMS",
        "globalUserId": "29771807",
        "wbUserId": 29771807,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Давно хотела отказаться от сахара, и нашла очень хорошую ему замену в 0 каллорий, также он очень хорош для выпечки!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-05T16:35:13Z",
        "updatedDate": "2023-08-16T10:09:47.760108133Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T10:54:33Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9b363f28-4630-475a-84e8-d508161aac8d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9b363f28-4630-475a-84e8-d508161aac8d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0d696c31-b751-4201-88b6-e248d75ad482_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0d696c31-b751-4201-88b6-e248d75ad482_ms.jpg"
            }
        ],
        "photo": [
            94966438,
            94966440
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 880
    },
    {
        "id": "_-E24YgBU_f9vkXPRTL4",
        "globalUserId": "44067668",
        "wbUserId": 44067668,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Fit parad пришёл, все во время,отлично упаковано, продавца рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-22T03:46:50Z",
        "updatedDate": "2023-08-16T10:09:50.212573466Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-22T06:09:43Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/66db1fff-d49d-4857-9970-f9cd6ee57762_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/66db1fff-d49d-4857-9970-f9cd6ee57762_ms.jpg"
            }
        ],
        "photo": [
            107863329
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 879
    },
    {
        "id": "seSnvYcBqd2abVcyOXbl",
        "globalUserId": "61916888",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Прекрасный сахзам, все целое пришло .доставка в указанный срок",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-26T13:01:06Z",
        "updatedDate": "2023-08-16T10:09:48.585219664Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-28T08:32:51Z",
            "editable": false,
            "metadata": null,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e4886ae9-ae73-42b5-aaeb-2a8497c3dd11_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e4886ae9-ae73-42b5-aaeb-2a8497c3dd11_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/ed45b46f-adb1-4fa9-abd4-6889e3dfadff_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ed45b46f-adb1-4fa9-abd4-6889e3dfadff_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/19c08ff7-7707-46e5-82a1-885c5a5a2758_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/19c08ff7-7707-46e5-82a1-885c5a5a2758_ms.jpg"
            }
        ],
        "photo": [
            98401231,
            98401232,
            98401235
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 878
    },
    {
        "id": "nCo-HocBMBFuPskoO4Mt",
        "globalUserId": "95246951",
        "wbUserId": 95246951,
        "wbUserDetails": {
            "name": "Мария",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Все хорошо! Сначала подумала, что плохо, коробки мятые все. А открыла, все пакетики на месте, еще и 3 джема положили, по одной в каждую коробку! Очень приятно, попробую джем, если понравится - закажу еще ❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T14:06:48Z",
        "updatedDate": "2023-08-16T10:09:47.254155264Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:24:01Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9f1bdbec-042c-4a87-9e94-50da1dcc78cc_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9f1bdbec-042c-4a87-9e94-50da1dcc78cc_ms.jpg"
            }
        ],
        "photo": [
            93196205
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 877
    },
    {
        "id": "DGHQ6YYBMJfQQSHDF9D9",
        "globalUserId": "47188273",
        "wbUserId": 47188273,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Товар отличный, открываю коробку а там….ещё баночка джема в подарок, это было так неожиданно,что вскрыла все три коробочки!))))) и вуаля…..👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T09:46:14Z",
        "updatedDate": "2023-08-16T10:09:46.755924332Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-16T10:52:41Z",
            "supplierId": 237660,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "47188273",
                "wbUserId": 47188273,
                "helpfulness": "plus",
                "time": "2023-03-16T11:23:11Z"
            },
            {
                "globalUserId": "21369821",
                "wbUserId": 21369821,
                "helpfulness": "plus",
                "time": "2023-03-16T23:17:19Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c4fe3479-2ac6-47f3-8c48-064b102999b1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c4fe3479-2ac6-47f3-8c48-064b102999b1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/e12b62d6-28cf-4eb2-b098-3da3c56dd154_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e12b62d6-28cf-4eb2-b098-3da3c56dd154_ms.jpg"
            }
        ],
        "photo": [
            91349780,
            91349781
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 876
    },
    {
        "id": "2Y5HHYcB7nVCbHkCBXXs",
        "globalUserId": "62634544",
        "wbUserId": 62634544,
        "wbUserDetails": {
            "country": "RU",
            "name": "Виктория",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Комплектация соответствует, подсластитель замечательный, положили ещё попробовать ароматизированные стики с черной смородиной",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T09:36:47Z",
        "updatedDate": "2023-08-16T10:09:47.228729900Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:38:17Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f630392e-d9c0-4229-824d-e91c7603c31a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f630392e-d9c0-4229-824d-e91c7603c31a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/227d0ca7-04ad-4dbd-8ae4-113fd1f5c63f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/227d0ca7-04ad-4dbd-8ae4-113fd1f5c63f_ms.jpg"
            }
        ],
        "photo": [
            93137419,
            93137421
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 875
    },
    {
        "id": "0Ox-KIoB0v3RIh2JhmMO",
        "globalUserId": "25066854",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анюта"
        },
        "nmId": 138785478,
        "text": "Спасибо, все доставили🌺",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-24T17:01:35Z",
        "updatedDate": "2023-08-26T17:17:00Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "23125206",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-25T19:06:17Z"
            },
            {
                "globalUserId": "63445371",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-25T20:41:08Z"
            },
            {
                "globalUserId": "72204271",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-26T17:17:00Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/069f8136-4582-44d2-a882-2fb431c6529b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/069f8136-4582-44d2-a882-2fb431c6529b_ms.jpg"
            }
        ],
        "photo": [
            121296723
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 3
        },
        "rank": 874
    },
    {
        "id": "M55QT4YB5HDhYKpHhhvE",
        "globalUserId": "87522698",
        "wbUserId": 87522698,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Алина"
        },
        "nmId": 138785478,
        "text": "Упаковка чуть помята, но это не так страшно, положили для пробы 3 шт другого сахара, этой же фирмы)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-14T09:45:23Z",
        "updatedDate": "2023-08-16T10:09:45.668077909Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-15T09:53:47Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5a6c9164-dce6-4486-9caf-d484662722b5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5a6c9164-dce6-4486-9caf-d484662722b5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/58d3003f-9cb7-42f4-857c-b53617526637_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/58d3003f-9cb7-42f4-857c-b53617526637_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c02b6244-f954-468a-bad5-49b4b32c859d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c02b6244-f954-468a-bad5-49b4b32c859d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/63b1ccbd-f36c-4de2-86bc-ca7ec4a84f85_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/63b1ccbd-f36c-4de2-86bc-ca7ec4a84f85_ms.jpg"
            }
        ],
        "photo": [
            85900995,
            85900996,
            85900997,
            85900998
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 873
    },
    {
        "id": "_FnGOYgB4LCMYDfDQtyb",
        "globalUserId": "33252857",
        "wbUserId": 33252857,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Пришел товар упакован. Всё целое. И очень выгодная покупка за три коробочки. Всем большое спасибо 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-20T15:27:54Z",
        "updatedDate": "2023-08-16T10:09:49.300708191Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-22T10:25:21Z",
            "createDate": "2023-05-22T10:25:21Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0c7542b8-6419-4fb0-af16-023616610583_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0c7542b8-6419-4fb0-af16-023616610583_ms.jpg"
            }
        ],
        "photo": [
            102496705
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 872
    },
    {
        "id": "BJ5RmoUBfnpWqlBe-X6-",
        "globalUserId": "",
        "wbUserId": 10695900,
        "wbUserDetails": {
            "country": "BY",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 33478818,
        "text": "Упаковано очень хорошо. По вкусу не химозно, как сахар обычный, по мне так одного пакетика достаточно на кружку чая. Однозначно рекомендую к покупке. В будущем буду покупать ещё 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-10T06:15:42Z",
        "updatedDate": "2023-08-16T10:09:44.616185183Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-10T08:21:32Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Благодарим вас за рекомендацию нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 10695900,
                "helpfulness": "plus",
                "time": "2023-01-10T08:39:17Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c9e3e437-d3f6-4b8b-871d-e3bfe258d5bc_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c9e3e437-d3f6-4b8b-871d-e3bfe258d5bc_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/aeb63b8b-3f9d-48f3-b9e3-daedcb1795ce_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/aeb63b8b-3f9d-48f3-b9e3-daedcb1795ce_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2f894db9-790d-4328-babc-8799424ff6f4_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2f894db9-790d-4328-babc-8799424ff6f4_ms.jpg"
            }
        ],
        "photo": [
            78181695,
            78181696,
            78181697
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 871
    },
    {
        "id": "drkEUogBYE-HFOZ5oA_E",
        "globalUserId": "112048132",
        "wbUserId": 112048132,
        "wbUserDetails": {
            "country": "ru",
            "name": "",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все доставлено. Срок хороший. Минус звезда за повреждение товара. Хотела в подарок, не выйдет теперь.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-25T08:26:55Z",
        "updatedDate": "2023-08-16T10:09:49.451461039Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-25T09:14:45Z",
            "createDate": "2023-05-25T09:14:45Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/177a97d5-eeee-433b-b156-f5464331ebd7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/177a97d5-eeee-433b-b156-f5464331ebd7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/e35eeba3-b046-4cad-814d-a25bded5f9ca_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e35eeba3-b046-4cad-814d-a25bded5f9ca_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3cad16e-e041-4609-9d24-445198a52234_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3cad16e-e041-4609-9d24-445198a52234_ms.jpg"
            }
        ],
        "photo": [
            103223032,
            103223033,
            103223034
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 870
    },
    {
        "id": "bHAGKYoBXuPSsPE1Z43U",
        "globalUserId": "33818801",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Маргарита",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо, все доставили",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-24T19:30:00Z",
        "updatedDate": "2023-08-26T17:16:56Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "23125206",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-25T19:06:16Z"
            },
            {
                "globalUserId": "63445371",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-25T20:41:03Z"
            },
            {
                "globalUserId": "72204271",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-26T17:16:56Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/477e544d-f23a-4594-99f4-03aa022b3bda_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/477e544d-f23a-4594-99f4-03aa022b3bda_ms.jpg"
            }
        ],
        "photo": [
            121331096
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 3
        },
        "rank": 869
    },
    {
        "id": "TelCw4cBN75J-nvtviOu",
        "globalUserId": "10953493",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Хорошая замена сахару, сроки годности в порядке, спасибо большое производителю",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-27T15:09:04Z",
        "updatedDate": "2023-08-16T10:09:48.619881440Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-28T06:13:01Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за доверие нашему бренду!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/425a4a85-f306-45f5-a20c-784ff2dbf577_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/425a4a85-f306-45f5-a20c-784ff2dbf577_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/6752f805-280d-4926-9286-b0d053e31bf9_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6752f805-280d-4926-9286-b0d053e31bf9_ms.jpg"
            }
        ],
        "photo": [
            98584932,
            98584933
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 868
    },
    {
        "id": "9JfWSocBk9xzjhYZ7dny",
        "globalUserId": "2371062",
        "wbUserId": 2371062,
        "wbUserDetails": {
            "name": "Анна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру не первый раз, удобно на работе и в дороге! Спасибо за подарочек,  очень вкусный сахзам со вкусом чёрной смородины! Попробовала на кофе с заправки, прямо кайфанула!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-04T05:56:52Z",
        "updatedDate": "2023-08-16T10:09:47.672891657Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T06:57:59Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b884318b-353d-412b-a319-0e3d89299dfd_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b884318b-353d-412b-a319-0e3d89299dfd_ms.jpg"
            }
        ],
        "photo": [
            94670916
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 867
    },
    {
        "id": "7HsnoYkBCaW1RxrSeDdJ",
        "globalUserId": "13916925",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Юлия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Очень нравится эта марка. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-29T10:17:46Z",
        "updatedDate": "2023-08-16T10:09:51.687905975Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-31T10:47:12Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/422cda3f-5924-42f3-aeca-42ec75f0726a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/422cda3f-5924-42f3-aeca-42ec75f0726a_ms.jpg"
            }
        ],
        "photo": [
            114744511
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 866
    },
    {
        "id": "g3x7A4cBk9xzjhYZ9yIH",
        "globalUserId": "69014908",
        "wbUserId": 69014908,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Тимко"
        },
        "nmId": 31468103,
        "text": "Сладко как сахар. Без постороннего привкуса. Быстрая доставка. Упаковка целая. Спасибо за подарочки в каждой упаковке.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-21T09:24:29Z",
        "updatedDate": "2023-08-16T10:09:46.982692957Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:14:10Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4b838336-e4af-4ffa-9ee7-c5550054d870_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4b838336-e4af-4ffa-9ee7-c5550054d870_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/1f15e20d-3e3e-4529-a3f0-d4dbb04bfc4f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1f15e20d-3e3e-4529-a3f0-d4dbb04bfc4f_ms.jpg"
            }
        ],
        "photo": [
            92256664,
            92256665
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 865
    },
    {
        "id": "swfvd4YBtPujtG5seRUA",
        "globalUserId": "26777111",
        "wbUserId": 26777111,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Екатерина"
        },
        "nmId": 33478818,
        "text": "Заказала первый раз. Очень довольна. Продукт хороший. Удобен тем, что в пакетиках. Можно в тесто добавлять. Буду ещё заказывать. Рекомендую своим знакомым🤗",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-22T07:03:48Z",
        "updatedDate": "2023-08-16T10:09:45.972962600Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-22T07:05:09Z",
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "26777111",
                "wbUserId": 26777111,
                "helpfulness": "plus",
                "time": "2023-02-22T10:21:59Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/74786226-7e0e-4144-9663-2a028610790a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/74786226-7e0e-4144-9663-2a028610790a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8f2bfc50-d8cb-4792-a272-380d1b5a8a93_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8f2bfc50-d8cb-4792-a272-380d1b5a8a93_ms.jpg"
            }
        ],
        "photo": [
            87279993,
            87279994
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 864
    },
    {
        "id": "zSCORIgBZx8DR9RAn_1N",
        "globalUserId": "83285059",
        "wbUserId": 83285059,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": ""
        },
        "nmId": 31468103,
        "text": "Доставлено и упакована запечатлено 😌👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-22T17:42:58Z",
        "updatedDate": "2023-08-16T10:09:49.360152791Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-23T05:58:17Z",
            "createDate": "2023-05-23T05:58:17Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/96fcdd4e-d368-4952-8ec6-d707f73cecbf_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/96fcdd4e-d368-4952-8ec6-d707f73cecbf_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d4864582-75c5-422f-ba8a-45decda91d85_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d4864582-75c5-422f-ba8a-45decda91d85_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2f192013-0325-437c-9c6f-eed4e6ab8076_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2f192013-0325-437c-9c6f-eed4e6ab8076_ms.jpg"
            }
        ],
        "photo": [
            102838083,
            102838084,
            102838085
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 863
    },
    {
        "id": "Y491LIcBc4LImbbGz4qA",
        "globalUserId": "37829642",
        "wbUserId": 37829642,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё вроде отлично, упаковка хорошая, ровно 60 шт. Вкус немного отличается от сахара, но такой же сладкий(разницы особо не заметила) , скорее всего ещё возьму если понравится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T08:22:11Z",
        "updatedDate": "2023-08-16T10:09:47.398508115Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-29T10:48:29Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f519afad-1355-44fb-b491-e490c9f303ac_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f519afad-1355-44fb-b491-e490c9f303ac_ms.jpg"
            }
        ],
        "photo": [
            93665918
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 862
    },
    {
        "id": "SkIyCocBfnpWqlBeAANc",
        "globalUserId": "81597132",
        "wbUserId": 81597132,
        "wbUserDetails": {
            "country": "RU",
            "name": "Вера",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Доставка в указанный срок.Заменитель достаточно сладкий, одного пакетика хватает на кружку кофе.Буду брать ещё.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-22T16:41:02Z",
        "updatedDate": "2023-08-16T10:09:47.065040879Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T05:34:29Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/42a122d2-75f6-4827-94a0-e9a236485fa4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/42a122d2-75f6-4827-94a0-e9a236485fa4_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/18958588-780f-47f2-986c-97c3c25ff633_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/18958588-780f-47f2-986c-97c3c25ff633_ms.jpg"
            }
        ],
        "photo": [
            92523605,
            92523606
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 861
    },
    {
        "id": "b4qGJ4cBv9_TjX_mIg5e",
        "globalUserId": "47397626",
        "wbUserId": 47397626,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель👍беру не в первый раз. В этом раз в подарок положили на пробу пакетики со вкусом чёрная смородина . Спасибо продавцу за подарочек ❤️ очень приятно!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-28T09:21:55Z",
        "updatedDate": "2023-08-16T10:09:47.332547188Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T10:02:32Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "31963462",
                "wbUserId": 31963462,
                "helpfulness": "plus",
                "time": "2023-03-28T13:26:00Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ed030804-d55d-4362-ac72-1b76090afcea_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ed030804-d55d-4362-ac72-1b76090afcea_ms.jpg"
            }
        ],
        "photo": [
            93500403
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 860
    },
    {
        "id": "wjMYy4kBmorVKbOQwYs0",
        "globalUserId": "83917438",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Людмила"
        },
        "nmId": 138785478,
        "text": "Кайф. Вам чëтко.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-06T13:45:44Z",
        "updatedDate": "2023-08-16T10:09:52.173766459Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T05:47:53Z",
            "editable": false,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7cd19b00-c4af-4796-88c3-1ff241d49656_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7cd19b00-c4af-4796-88c3-1ff241d49656_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d52d2343-4cef-4113-81ac-d2d2c3671a65_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d52d2343-4cef-4113-81ac-d2d2c3671a65_ms.jpg"
            }
        ],
        "photo": [
            116387206,
            116387207
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 859
    },
    {
        "id": "nfA58YYB5HDhYKpHN02e",
        "globalUserId": "39068422",
        "wbUserId": 39068422,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ася"
        },
        "nmId": 31468103,
        "text": "Отличный сах.зам. пользуюсь давно. Нравится😊🩵👍 еще очень приятно, спасибо поставщику, заказывала 3 уп., положили в каждую попробовать черничный джем без сах.😊👍❤️очень понравился спасибо.❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-17T20:18:24Z",
        "updatedDate": "2023-08-16T10:09:46.850201415Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:40:29Z",
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e0b677ae-9e39-455a-ba69-37496eeaa22e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e0b677ae-9e39-455a-ba69-37496eeaa22e_ms.jpg"
            }
        ],
        "photo": [
            91657763
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 858
    },
    {
        "id": "LlA1pIgBUdF-4nlzCyxi",
        "globalUserId": "12843312",
        "wbUserId": 12843312,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Как и написано в описании, один пакетик саше заменяет 2ч.л сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-10T07:28:39Z",
        "updatedDate": "2023-08-16T10:09:49.888629398Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T09:05:50Z",
            "createDate": "2023-06-20T09:05:50Z",
            "metadata": null,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bb3eaefa-ef27-4b1a-99b1-9af53ba74b3e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bb3eaefa-ef27-4b1a-99b1-9af53ba74b3e_ms.jpg"
            }
        ],
        "photo": [
            105777537
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 857
    },
    {
        "id": "4RrNWYcB5HDhYKpHWaAE",
        "globalUserId": "63184234",
        "wbUserId": 63184234,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Елена"
        },
        "nmId": 31468103,
        "text": "Товар отличный, хорошо упакован, в каждой коробочке был подарок, спасибо большое продавцу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-07T03:40:43Z",
        "updatedDate": "2023-08-16T10:09:47.814047266Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-07T07:04:22Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4fb84877-187c-41c3-97c9-a7d5acd57bae_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4fb84877-187c-41c3-97c9-a7d5acd57bae_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/55361f51-33b7-4e04-8506-b19ff840be02_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/55361f51-33b7-4e04-8506-b19ff840be02_ms.jpg"
            }
        ],
        "photo": [
            95189230,
            95189229
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 856
    },
    {
        "id": "iXYMC4kBqd2abVcyj-UI",
        "globalUserId": "111196104",
        "wbUserId": 111196104,
        "wbUserDetails": {
            "name": "",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Все пришло, все целое, не помято, не подмочено))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-30T06:45:20Z",
        "updatedDate": "2023-08-16T10:09:50.480741187Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T08:03:08Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/49a9f6b4-1114-44f5-b932-c66c20d7cf2b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/49a9f6b4-1114-44f5-b932-c66c20d7cf2b_ms.jpg"
            }
        ],
        "photo": [
            109287376
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 855
    },
    {
        "id": "VNL-6IkB0v3RIh2J23tJ",
        "globalUserId": "19967895",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Олеся",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "хороший сахарозаменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-12T09:06:04Z",
        "updatedDate": "2023-08-16T10:09:53.482635331Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/dd30ed15-e7f3-4264-9227-af100ab6e9e4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dd30ed15-e7f3-4264-9227-af100ab6e9e4_ms.jpg"
            }
        ],
        "photo": [
            118163649
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 854
    },
    {
        "id": "l4_eLIcBc4LImbbGrMPD",
        "globalUserId": "32784428",
        "wbUserId": 32784428,
        "wbUserDetails": {
            "name": "Диана",
            "hasPhoto": false,
            "country": "BY"
        },
        "nmId": 138785478,
        "text": "Все супер! Спасибо за маленький подарок) \nОчень приятно и сладко!!\nВ чай добавила один пакетик сах.зам 0.5 и чай очень сладкий 😋 😋  как будто 2 ложки сахар а не 1",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T10:16:43Z",
        "updatedDate": "2023-08-16T10:09:47.406493882Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-29T10:42:53Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4c1bde0a-1b5a-49a9-94c9-208f2e53be71_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4c1bde0a-1b5a-49a9-94c9-208f2e53be71_ms.jpg"
            }
        ],
        "photo": [
            93686334
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 853
    },
    {
        "id": "HwYANYkBpT2rITcGr5WF",
        "globalUserId": "81128553",
        "wbUserId": 81128553,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Нина"
        },
        "nmId": 31468103,
        "text": "Хороший заменитель сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-08T10:16:25Z",
        "updatedDate": "2023-08-16T10:09:50.771276424Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T11:27:13Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "text": "Добрый день! Спасибо за высокую оценку. Оставайтесь с нами :)",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6b109e50-663e-44fd-8638-58275a0f584a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6b109e50-663e-44fd-8638-58275a0f584a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/03ec085d-7b27-4d59-9abd-848a759696d1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/03ec085d-7b27-4d59-9abd-848a759696d1_ms.jpg"
            }
        ],
        "photo": [
            110752152,
            110752153
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 852
    },
    {
        "id": "8LKFJIcBJ99b87NGNCd8",
        "globalUserId": "15979677",
        "wbUserId": 15979677,
        "wbUserDetails": {
            "country": "RU",
            "name": "Виктория",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Мой тренер очень любит этот сахзам. Ей очень нравится состав и качество. Посоветовала. Я довольна. Отдельное спасибо за подарочек в виде джема в каждой пачке.🔥))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T19:22:02Z",
        "updatedDate": "2023-08-16T10:09:47.312896711Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T10:03:44Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a0f6b935-634a-4895-9bd5-cd0860d3de89_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a0f6b935-634a-4895-9bd5-cd0860d3de89_ms.jpg"
            }
        ],
        "photo": [
            93443635
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 851
    },
    {
        "id": "48_djYcBqd2abVcySPz1",
        "globalUserId": "60953065",
        "wbUserId": 60953065,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Здравствуйте,  заказ пришёл быстро, хорошо упакован, всем спасибо ,рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-17T06:18:22Z",
        "updatedDate": "2023-08-16T10:09:48.230729703Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T07:39:33Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bb5e645a-3443-4d64-8c3c-ca278d3c2a9d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bb5e645a-3443-4d64-8c3c-ca278d3c2a9d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/13c16cad-950b-4fe9-adce-ca380b795855_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/13c16cad-950b-4fe9-adce-ca380b795855_ms.jpg"
            }
        ],
        "photo": [
            96786906,
            96786907
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 850
    },
    {
        "id": "PnzkjIkBKNxVyjcCMfj0",
        "globalUserId": "60293914",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Все отлично 👍, спасибо продавцу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-25T11:51:53Z",
        "updatedDate": "2023-08-16T10:09:51.459499619Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T07:15:20Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за доверие нашему бренду!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ead2e41a-5c24-4ccd-a96b-ad23ea54ff44_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ead2e41a-5c24-4ccd-a96b-ad23ea54ff44_ms.jpg"
            }
        ],
        "photo": [
            113999361
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 849
    },
    {
        "id": "D7khb4kBl47NwnZ1NCrD",
        "globalUserId": "60654171",
        "wbUserId": 60654171,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Эля"
        },
        "nmId": 31468103,
        "text": "получила заказ,пришло в своё время",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-19T17:09:54Z",
        "updatedDate": "2023-08-16T10:09:51.223394006Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T08:45:15Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/22e8aef4-8510-44a4-80be-ff6a4e721fef_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/22e8aef4-8510-44a4-80be-ff6a4e721fef_ms.jpg"
            }
        ],
        "photo": [
            112911903
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 848
    },
    {
        "id": "P2rxcYcBCYLDkq7N_Zwz",
        "globalUserId": "90416194",
        "wbUserId": 90416194,
        "wbUserDetails": {
            "name": "А",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный сахар , беру не первый раз,  привкуса нет , просто супер 👍 в подарок положили на пробу виноградный сахар ❤️ это очень мило 🍬",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-11T20:11:37Z",
        "updatedDate": "2023-08-16T10:09:48.014417268Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T05:16:54Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "90416194",
                "wbUserId": 90416194,
                "helpfulness": "plus",
                "time": "2023-04-12T06:31:24Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/616e772c-82fa-4f37-970a-c72ad778622c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/616e772c-82fa-4f37-970a-c72ad778622c_ms.jpg"
            }
        ],
        "photo": [
            95960554
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 847
    },
    {
        "id": "8kaaQ4gBG4PPbu1tbN6a",
        "globalUserId": "43603672",
        "wbUserId": 43603672,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 33478818,
        "text": "Все как всегда отлично а можно спросить где можно приобрести вот такой сах зам?",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-22T13:16:14Z",
        "updatedDate": "2023-08-16T10:09:49.350972253Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-23T06:06:33Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Теперь саше будет доступно только в фасовке по 90 шт. Артикул: 138785478",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "42804208",
                "wbUserId": 42804208,
                "helpfulness": "minus",
                "time": "2023-05-22T18:31:08Z"
            },
            {
                "globalUserId": "16192051",
                "wbUserId": 16192051,
                "helpfulness": "minus",
                "time": "2023-05-23T02:27:12Z"
            },
            {
                "globalUserId": "39343426",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-28T09:29:21Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ff4b52b9-aa17-45e2-9355-99f9c9fd9ea4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ff4b52b9-aa17-45e2-9355-99f9c9fd9ea4_ms.jpg"
            }
        ],
        "photo": [
            102786685
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 3
        },
        "rank": 846
    },
    {
        "id": "muW0UIYBfnpWqlBe17AQ",
        "globalUserId": "33709658",
        "wbUserId": 33709658,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Надежда",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Очень классный сахзам!!! Мне очень нравится 👍 👍 👍  Ещё в подарок на пробу положили три пакетика со смородиновым вкусом, спасибо, очень понравился))))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-14T16:14:34Z",
        "updatedDate": "2023-08-16T10:09:45.695221146Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-15T08:57:43Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/98f5fc08-764d-41fa-861d-94807dd70d87_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/98f5fc08-764d-41fa-861d-94807dd70d87_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8bb93c16-95bf-4d1f-abe5-461175ff616e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8bb93c16-95bf-4d1f-abe5-461175ff616e_ms.jpg"
            }
        ],
        "photo": [
            85977195,
            85977196
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 845
    },
    {
        "id": "uYHE3YgBZx8DR9RALRgX",
        "globalUserId": "28081171",
        "wbUserId": 28081171,
        "wbUserDetails": {
            "name": "Валентина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Соответствует описанию, доставка быстрая, всё целое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-21T11:43:21Z",
        "updatedDate": "2023-08-16T10:09:50.200470823Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-21T12:09:47Z",
            "createDate": "2023-06-21T12:09:47Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cad70e0d-5753-46a5-92df-a472b84bc169_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cad70e0d-5753-46a5-92df-a472b84bc169_ms.jpg"
            }
        ],
        "photo": [
            107748230
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 844
    },
    {
        "id": "4zZXhogBZx8DR9RA38cl",
        "globalUserId": "17448067",
        "wbUserId": 17448067,
        "wbUserDetails": {
            "country": "ru",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всё как в описании. Вкус бесподобен, в меру сладко 👌 Закажу ещё )",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-04T12:18:06Z",
        "updatedDate": "2023-08-16T10:09:49.718296295Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-07T06:19:55Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14479763",
                "wbUserId": 14479763,
                "helpfulness": "plus",
                "time": "2023-06-06T06:20:04Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3e45273-7462-420b-80a0-5cde28d9bfca_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3e45273-7462-420b-80a0-5cde28d9bfca_ms.jpg"
            }
        ],
        "photo": [
            104825947
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 843
    },
    {
        "id": "OguznIUBbskpjQp2I3pu",
        "globalUserId": "",
        "wbUserId": 47258070,
        "wbUserDetails": {
            "name": "Ирина",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Доставлен вовремя,очень надежная упаковка, дополнительно упаковали в коробку. Качественный продукт, сроки хорошие. Спасибо магазину и валдбериз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-10T17:21:04Z",
        "updatedDate": "2023-08-16T10:09:44.626810654Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-11T05:26:44Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/4e7af497-37f5-4bf1-8e31-35f64a40d198_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/4e7af497-37f5-4bf1-8e31-35f64a40d198_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/0bf41735-ce84-4704-9deb-2270a8369442_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0bf41735-ce84-4704-9deb-2270a8369442_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/040acaea-9dd4-48cb-a0a0-e3cc5481a6c6_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/040acaea-9dd4-48cb-a0a0-e3cc5481a6c6_ms.jpg"
            }
        ],
        "photo": [
            78304305,
            78304307,
            78304308
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 842
    },
    {
        "id": "5mB9gYgBl47NwnZ15df7",
        "globalUserId": "63756115",
        "wbUserId": 63756115,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Заказываю не в первый раз. В этот раз пришёл в смятой коробке\nНе критично, но звезду сняла за упаковку",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-03T13:41:32Z",
        "updatedDate": "2023-08-16T10:09:49.687894682Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T10:25:30Z",
            "createDate": "2023-06-20T10:25:30Z",
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cb0bae63-3216-4285-946c-65cfa28bc56d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cb0bae63-3216-4285-946c-65cfa28bc56d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4347599f-1160-484d-9c81-c53a1eb0b14d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4347599f-1160-484d-9c81-c53a1eb0b14d_ms.jpg"
            }
        ],
        "photo": [
            104690988,
            104690989
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 841
    },
    {
        "id": "lW4XuIcBpT2rITcGNDZb",
        "globalUserId": "22286126",
        "wbUserId": 22286126,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Виолетта"
        },
        "nmId": 138785478,
        "text": "Доставка быстрая,упаковка не повреждена,беру много раз,вкуса нет,послевкусие отсутствует. К покупке рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-25T11:05:41Z",
        "updatedDate": "2023-08-16T10:09:48.549238783Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-25T11:39:37Z",
            "createDate": "2023-04-25T11:39:37Z",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5eb336aa-07b5-4cf8-a1be-4f910f3d4722_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5eb336aa-07b5-4cf8-a1be-4f910f3d4722_ms.jpg"
            }
        ],
        "photo": [
            98205268
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 840
    },
    {
        "id": "jeylNYoBl47NwnZ1W8-6",
        "globalUserId": "30177094",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Екатерина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-27T06:19:04Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/13878/138785478/4c4ef2ab-779c-4102-85c4-c5000a28452b_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/4c4ef2ab-779c-4102-85c4-c5000a28452b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/13878/138785478/5ff0b0f6-39cf-4c3a-87e2-8be73a987eb7_fs.jpg",
                "minSizeUri": "feedbacks/13878/138785478/5ff0b0f6-39cf-4c3a-87e2-8be73a987eb7_ms.jpg"
            }
        ],
        "photo": [
            121833935,
            121833936
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 839
    },
    {
        "id": "hStSIocBQOOSTxXohTJX",
        "globalUserId": "32264359",
        "wbUserId": 32264359,
        "wbUserDetails": {
            "country": "RU",
            "name": "Людмила",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Фит Парад взяла по рекомендации фитнес тренера. Пью с напитками уже несколько дней и добавляю в пищу. Всё нравится. Вкус фактически не отличим от сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T09:07:26Z",
        "updatedDate": "2023-08-16T10:09:47.278113574Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T12:55:33Z",
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "metadata": null,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b66303b4-0fb1-4f37-953d-df24c2ccf5d8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b66303b4-0fb1-4f37-953d-df24c2ccf5d8_ms.jpg"
            }
        ],
        "photo": [
            93311597
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 838
    },
    {
        "id": "EcuMgYgBCYLDkq7NGVwM",
        "globalUserId": "49140840",
        "wbUserId": 49140840,
        "wbUserDetails": {
            "name": "Елизавета",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Товар пришёл в несоответствии с заявленным параметрами. В описании 3 по 90гр. Пришёл 3 по 60гр. Жаль что поверила а не проверила сразу. Получилось дороже чем в магазине.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-03T13:57:02Z",
        "updatedDate": "2023-08-16T10:09:49.689428851Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T10:25:19Z",
            "createDate": "2023-06-20T10:25:19Z",
            "editable": false,
            "text": "Здравствуйте, просьба написать нам по адресу указанному на упаковке с товаром! ",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "44434613",
                "wbUserId": 44434613,
                "helpfulness": "minus",
                "time": "2023-06-09T18:26:20Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/892af251-7c47-4784-887e-775f3a009296_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/892af251-7c47-4784-887e-775f3a009296_ms.jpg"
            }
        ],
        "photo": [
            104693507
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 837
    },
    {
        "id": "onYZAYkBqd2abVcyfSks",
        "globalUserId": "20969237",
        "wbUserId": 20969237,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Валерия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хорошая замена обычному сахару, пришел быстро",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-28T08:23:15Z",
        "updatedDate": "2023-08-16T10:09:50.424852592Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-28T13:25:52Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e9e1fb75-e2e0-4ffb-8668-b7601dff27aa_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e9e1fb75-e2e0-4ffb-8668-b7601dff27aa_ms.jpg"
            }
        ],
        "photo": [
            108942934
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 836
    },
    {
        "id": "ntxVM4cBTZEOO9f5Pa6h",
        "globalUserId": "26702475",
        "wbUserId": 26702475,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Хорошо было упаковано, положили подарочек, подсластитель со вкусом «чёрная смородина» нужно будет взять обязательно попробовать! Буду брать еще.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-30T16:23:57Z",
        "updatedDate": "2023-08-16T10:09:47.463500192Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:15:16Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/21acc901-9577-447c-badc-46fc9ccc858b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/21acc901-9577-447c-badc-46fc9ccc858b_ms.jpg"
            }
        ],
        "photo": [
            93937197
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 835
    },
    {
        "id": "IwRiZYYBIq9DQExlNgha",
        "globalUserId": "36726072",
        "wbUserId": 36726072,
        "wbUserDetails": {
            "name": "Марина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё отлично!!! Одного пакетика  хватает на 250  грамовый бокал чая ...Советую к покупке однозначно! Спасибо За честную работу!!! 👍👍👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-18T16:36:21Z",
        "updatedDate": "2023-08-16T10:09:45.846632414Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-20T11:06:03Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14852945",
                "wbUserId": 14852945,
                "helpfulness": "plus",
                "time": "2023-02-19T06:50:04Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2e20a7f0-e18b-40b8-b977-2cc20bc73414_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2e20a7f0-e18b-40b8-b977-2cc20bc73414_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/53e14ffb-828a-4b76-b37e-71141020aa7b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/53e14ffb-828a-4b76-b37e-71141020aa7b_ms.jpg"
            }
        ],
        "photo": [
            86689348,
            86689349
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 834
    },
    {
        "id": "DkrJ6oYBIq9DQExl2BOL",
        "globalUserId": "41247582",
        "wbUserId": 41247582,
        "wbUserDetails": {
            "name": "Олеся",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Доставка быстрая, упаковка целая, все 60 пакетиков. На вкус ещё не пробовала. Рекомендую к покупке 😉",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T14:19:02Z",
        "updatedDate": "2023-08-16T10:09:46.775741240Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-17T11:38:35Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "41247582",
                "wbUserId": 41247582,
                "helpfulness": "plus",
                "time": "2023-03-17T15:30:29Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ba771bdf-2c35-44fa-aca1-74003e41ef60_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ba771bdf-2c35-44fa-aca1-74003e41ef60_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/57578b4a-9af4-4e70-9bda-5f0af7d6b155_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/57578b4a-9af4-4e70-9bda-5f0af7d6b155_ms.jpg"
            }
        ],
        "photo": [
            91408537,
            91408538
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 833
    },
    {
        "id": "iTZ58IYBfnpWqlBeCLDt",
        "globalUserId": "33419305",
        "wbUserId": 33419305,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Дарья"
        },
        "nmId": 31468103,
        "text": "Заказываю не первый раз и как всегда все супер! Спасибо продавцу за подарок. Попробую джем и скорее всего он мне понравится и я в дальнейшем буду его заказывать :)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-17T16:48:29Z",
        "updatedDate": "2023-08-16T10:09:46.836603263Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:44:59Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за доверие нашему бренду!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f312bb74-47be-4753-b6e7-e50da7f6a245_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f312bb74-47be-4753-b6e7-e50da7f6a245_ms.jpg"
            }
        ],
        "photo": [
            91625588
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 832
    },
    {
        "id": "gqDOXocBk9xzjhYZ2tab",
        "globalUserId": "19925261",
        "wbUserId": 19925261,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Спасибо за товар. Пришёл в хорошем состоянии. Спасибо продавцу за небольшой подарочек. Fit Parad N 27. Попробую   удовольствием.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-08T03:00:27Z",
        "updatedDate": "2023-08-16T10:09:47.868406671Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T06:53:52Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cac11813-a6da-4a03-a398-2c32cbb86081_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cac11813-a6da-4a03-a398-2c32cbb86081_ms.jpg"
            }
        ],
        "photo": [
            95358157
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 831
    },
    {
        "id": "N2aLAooBuWYO9ah2E2NC",
        "globalUserId": "29377904",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Заказала Сахар, прислали жемчуг… интересное явление😂",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T08:09:43Z",
        "updatedDate": "2023-09-10T04:13:00Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "56975684",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-10T04:13:00Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/550aa362-b3d2-4840-a3c2-1efb23e02697_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/550aa362-b3d2-4840-a3c2-1efb23e02697_ms.jpg"
            }
        ],
        "photo": [
            119315652
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 830
    },
    {
        "id": "y_NguYoBAtKOitoBgDfH",
        "globalUserId": "29323012",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Максим",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Сахарозаменитель супер! Но...\nКоробка вся стая как из жопы, доставка безуууумно долгая(не 2 дня как я привык), но это уже претензии к wb.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T20:13:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 829
    },
    {
        "id": "q-FwX4oBUdF-4nlzIblq",
        "globalUserId": "23667938",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Отличное саше. Привкус стевии не так выражен, как в 10. Послевкусье остается не ярко выраженное. Обучаюсь на нутрициолога, нам рекомендовали именно это соотношение сахзама и подсластителей.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-04T09:04:59Z",
        "updatedDate": "2023-09-04T12:39:00Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T12:39:00Z",
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 828
    },
    {
        "id": "zEoFGYcB6z8QKrUH7wrb",
        "globalUserId": "66314818",
        "wbUserId": 66314818,
        "wbUserDetails": {
            "name": "Мария",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Вместо сахарозаменителя ФитПарад 7 выслали какую  то запчасть.   Мало того , что выслали  непонятно что, ещё деньги не вернули. Как это вообще возможно? Беспредел .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T13:47:12Z",
        "updatedDate": "2023-08-16T10:09:47.190959127Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T09:49:02Z",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Очень жаль, что товар был перепутан, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов."
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "37121870",
                "wbUserId": 37121870,
                "helpfulness": "minus",
                "time": "2023-03-30T20:10:27Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/25fa4e23-cd50-434d-b3b9-2c557d7e414e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/25fa4e23-cd50-434d-b3b9-2c557d7e414e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/f8244294-0b9b-4b6b-9dc4-63b09d3f0555_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f8244294-0b9b-4b6b-9dc4-63b09d3f0555_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a3d5e05e-51fc-4283-b1b8-1bb8e621e6fc_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a3d5e05e-51fc-4283-b1b8-1bb8e621e6fc_ms.jpg"
            }
        ],
        "photo": [
            93020199,
            93020200,
            93020202
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 827
    },
    {
        "id": "e5SKQocBv9_TjX_mxW9m",
        "globalUserId": "16021607",
        "wbUserId": 16021607,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Беру не в первый раз.Очень нравится ,нет никакого послевкусия: хоть в чай клади,хоть в кофе или в любые десерты. Рекомендую. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T15:16:43Z",
        "updatedDate": "2023-08-16T10:09:47.591017370Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T10:00:07Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "24338790",
                "wbUserId": 24338790,
                "helpfulness": "minus",
                "time": "2023-04-04T21:48:45Z"
            },
            {
                "globalUserId": "16021607",
                "wbUserId": 16021607,
                "helpfulness": "plus",
                "time": "2023-04-05T11:45:04Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2530591b-c529-4171-baf0-5251fd302cfb_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2530591b-c529-4171-baf0-5251fd302cfb_ms.jpg"
            }
        ],
        "photo": [
            94420664
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 1
        },
        "rank": 826
    },
    {
        "id": "OTeY1ocB4LCMYDfD9tGB",
        "globalUserId": "90051091",
        "wbUserId": 90051091,
        "wbUserDetails": {
            "country": "KZ",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Молодцы хорошо упаковали, впервые пришел товар в коробке.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-01T09:16:02Z",
        "updatedDate": "2023-08-16T10:09:48.728844815Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-04T06:39:56Z",
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/34165a46-a18b-432e-b375-ab92b059a20f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/34165a46-a18b-432e-b375-ab92b059a20f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4fa99298-ca33-4800-a96d-8c3ed349896e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4fa99298-ca33-4800-a96d-8c3ed349896e_ms.jpg"
            }
        ],
        "photo": [
            99174826,
            99174827
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 825
    },
    {
        "id": "2UcvooYBk9xzjhYZ3nAY",
        "globalUserId": "27779138",
        "wbUserId": 27779138,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Валентина Паршунина"
        },
        "nmId": 33478818,
        "text": "Беру его не первый раз, очень нравится. Пришёл в коробочке, ничего не помялось👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-02T11:58:11Z",
        "updatedDate": "2023-08-16T10:09:46.245900560Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-02T12:08:11Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/8eb5f91a-7343-4d8d-9f1c-822f9293cab8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8eb5f91a-7343-4d8d-9f1c-822f9293cab8_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/5160750d-9916-4a16-b9a1-48c6ae1a4613_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5160750d-9916-4a16-b9a1-48c6ae1a4613_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0fb768a5-1e7e-4c36-a319-3e2d40240671_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0fb768a5-1e7e-4c36-a319-3e2d40240671_ms.jpg"
            }
        ],
        "photo": [
            88736038,
            88736039,
            88736040
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 824
    },
    {
        "id": "osO69YYBTZEOO9f53N7y",
        "globalUserId": "53650410",
        "wbUserId": 53650410,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Отличный сахарозаменитель. В каждой упаковке ещё и подарочек- Черничный джем. Упаковано надёжно в пакет и в коробку. Доставка быстрая. Всё отлично. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-18T17:18:30Z",
        "updatedDate": "2023-08-16T10:09:46.888121657Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T10:48:22Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/29c0fb04-877d-48cd-b6c5-d95f2d0048b3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/29c0fb04-877d-48cd-b6c5-d95f2d0048b3_ms.jpg"
            }
        ],
        "photo": [
            91810628
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 823
    },
    {
        "id": "8w1Rp4cBBgFQoSo0pS--",
        "globalUserId": "24420732",
        "wbUserId": 24420732,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Беру не первый раз. Заменитель понравился. Спасибо за подарочек - ароматические стики чёрная смородина!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-22T04:55:59Z",
        "updatedDate": "2023-08-16T10:09:48.420050647Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-26T08:21:46Z",
            "createDate": "2023-04-26T08:21:46Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7b5c0550-a953-43df-9ec2-4c05f2d5bde6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7b5c0550-a953-43df-9ec2-4c05f2d5bde6_ms.jpg"
            }
        ],
        "photo": [
            97646695
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 822
    },
    {
        "id": "QdYSj4oBqd2abVcy3BHY",
        "globalUserId": "23849223",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Беру каждый раз как заканчивается, я заменила им обычный сахар, ни чем мне он не отдает, вкус нравиться, один пакетик как две ложки сахара, очень сладко",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T15:04:53Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 821
    },
    {
        "id": "SWgVQYcBmBixw9tzb1_F",
        "globalUserId": "69002666",
        "wbUserId": 69002666,
        "wbUserDetails": {
            "country": "BY",
            "name": "Синяк",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо большое за быструю доставку и за подарочек. Очень хороший сахзам. Мне понравился больше чем стевия.  К покупке рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T08:28:57Z",
        "updatedDate": "2023-08-16T10:09:47.565259669Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T10:44:47Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/24f854ac-a677-4dc8-9d4b-3bc7be2ba5a1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/24f854ac-a677-4dc8-9d4b-3bc7be2ba5a1_ms.jpg"
            }
        ],
        "photo": [
            94337559
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 820
    },
    {
        "id": "ndefW4gBqVKcxP1mETMz",
        "globalUserId": "9413183",
        "wbUserId": 9413183,
        "wbUserDetails": {
            "name": "Ксения",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Товар пришел весь помятый, упакован в простую пленку и немного попурки! Хотя бы в коробку бы положили",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-27T05:12:11Z",
        "updatedDate": "2023-08-16T10:09:49.503425128Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-29T11:34:27Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "10898754",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-05-27T09:57:50Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/32dcf820-8a19-40a6-b14b-80c534ed980a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/32dcf820-8a19-40a6-b14b-80c534ed980a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c92c31e9-9890-4418-8abe-8c8fc999f9af_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c92c31e9-9890-4418-8abe-8c8fc999f9af_ms.jpg"
            }
        ],
        "photo": [
            103506826,
            103506827
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 819
    },
    {
        "id": "0B7XNIkBCYLDkq7NjlA0",
        "globalUserId": "87610480",
        "wbUserId": 87610480,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Всё отлично) вкус хороший не горчит",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-08T09:31:29Z",
        "updatedDate": "2023-08-16T10:09:50.766674491Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T11:26:41Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Добрый день! Спасибо за высокую оценку. Оставайтесь с нами :)",
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ba8589c6-34a7-4fe4-b98e-b1562310d2d6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ba8589c6-34a7-4fe4-b98e-b1562310d2d6_ms.jpg"
            }
        ],
        "photo": [
            110742860
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 818
    },
    {
        "id": "JIrtJocBk9xzjhYZyQbn",
        "globalUserId": "33501813",
        "wbUserId": 33501813,
        "wbUserDetails": {
            "country": "RU",
            "name": "Алена",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Отличный заменитель сазана, на стевии, натуральный, пользуюсь очень давно и он лучший, еще и в каждой коробочке джем, спасибо приятно💪😊",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-28T06:35:31Z",
        "updatedDate": "2023-08-16T10:09:47.321961034Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T08:14:11Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1bb1d0ff-18df-4df2-9458-8c18b4411c92_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1bb1d0ff-18df-4df2-9458-8c18b4411c92_ms.jpg"
            }
        ],
        "photo": [
            93474982
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 817
    },
    {
        "id": "kcNnb4oBKNxVyjcC2ntE",
        "globalUserId": "32173077",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Впервые ставлю такую оценку, очень сложно,но ,я за правду.\nНЕ СЛАДКО!\nДумала,что это мне кажется,поскольку очень много препаратов принимаю .Но,нет!\nДала попробовать подругам,мужу,у всех кофе и чай не сладкие.\nСразу  же сравнили с другим ,который купили в супермаркете ,все ок,и достаточно половины упаковки.\nПонимаю,что это производитель сэкономил,но отзывы для этого и есть.\nВажно всем,кто контролит сахара.\n\nP.S.Я покупатель,не конкурент.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T11:29:52Z",
        "updatedDate": "2023-09-07T16:14:44Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "2760768",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-09-07T15:48:53Z"
            },
            {
                "globalUserId": "43631141",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-09-07T16:14:44Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 2
        },
        "rank": 816
    },
    {
        "id": "Hs-YX4cBbskpjQp2hWDa",
        "globalUserId": "20405000",
        "wbUserId": 20405000,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 33478818,
        "text": "В магните дешевле, сейчас ещё скидка, поэтому отказалась от фит парада с вб. Подсластитель мне понравился, 1г (пакетик) заменяет 10г сахара. Привкуса неприятного нет, на вкус как сахар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-08T06:40:44Z",
        "updatedDate": "2023-08-16T10:09:47.873384963Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T06:53:10Z",
            "supplierId": 237660,
            "text": "Ценообразование сторонних магазинов не связано с данным маркетплейсом. Сравнение некорректно, но Вы как покупатель вправе приобретать товар там, где Вам это удобнее :)",
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "16716723",
                "wbUserId": 16716723,
                "helpfulness": "plus",
                "time": "2023-04-13T12:13:27Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/67c53057-cc53-4352-a9ac-e1fcfea673ae_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/67c53057-cc53-4352-a9ac-e1fcfea673ae_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8f80c72f-8995-449f-aa2b-dc3c492ee638_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8f80c72f-8995-449f-aa2b-dc3c492ee638_ms.jpg"
            }
        ],
        "photo": [
            95374099,
            95374101
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 815
    },
    {
        "id": "QPRx1ocBAtKOitoBCDHo",
        "globalUserId": "6262179",
        "wbUserId": 6262179,
        "wbUserDetails": {
            "name": "Юлия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 62622901,
        "text": "Заказываю не в первый раз, очень нравится!!! Новая упаковка понравилась, удобно. Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-01T08:32:25Z",
        "updatedDate": "2023-08-16T10:09:48.727333769Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-04T06:40:12Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "21902285",
                "wbUserId": 21902285,
                "helpfulness": "plus",
                "time": "2023-05-01T11:37:59Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/88704b5f-4221-4d6d-a19b-545860195846_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/88704b5f-4221-4d6d-a19b-545860195846_ms.jpg"
            }
        ],
        "photo": [
            99166519
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 814
    },
    {
        "id": "5V25NYcBT-IFIyX_szfc",
        "globalUserId": "38832306",
        "wbUserId": 38832306,
        "wbUserDetails": {
            "name": "Юлия",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Пришло все хорошо упаковано. Открыла одну пачку там сюрприз. Первый раз за все время.. Причем в каждой пачке. Приятно. Благодарю.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-31T03:32:55Z",
        "updatedDate": "2023-08-16T10:09:47.477009241Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T09:59:02Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/86a59232-0314-4bf7-8bcd-e83b725a77a5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/86a59232-0314-4bf7-8bcd-e83b725a77a5_ms.jpg"
            }
        ],
        "photo": [
            93985943
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 813
    },
    {
        "id": "i3vX7oYB7nVCbHkCglTq",
        "globalUserId": "41678467",
        "wbUserId": 41678467,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Алёна"
        },
        "nmId": 31468103,
        "text": "Отличный сах.зам.,заказываю только его,в этот раз обрадовали подарочки в каждой из 3х коробок,жаль что одинаковые,а так оч вкусные,для тех кто на пп 👍🏼",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-17T09:12:27Z",
        "updatedDate": "2023-08-16T10:09:46.809104985Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-17T11:17:56Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5bb1bb6b-879e-4d90-92dc-46ab815ba456_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5bb1bb6b-879e-4d90-92dc-46ab815ba456_ms.jpg"
            }
        ],
        "photo": [
            91528976
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 812
    },
    {
        "id": "O1XTHYoBmorVKbOQihhX",
        "globalUserId": "94871974",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Валентина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Коробка вся мятая и рваная. Товар шёл долго",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-22T15:18:37Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0e440101-4c9b-4a93-973c-b31f44cb96c5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0e440101-4c9b-4a93-973c-b31f44cb96c5_ms.jpg"
            }
        ],
        "photo": [
            120782200
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 811
    },
    {
        "id": "RQXxb4oB0v3RIh2JEdmQ",
        "globalUserId": "21524431",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ирина Черемисина"
        },
        "nmId": 138785478,
        "text": "Я его обажаю👍1пакетик соответствует 3 ч.л сахара , никого постороннего вкуса. Удобно брать с сабой. У меня эти пакетики везде во всех карманах. Это очень удобно!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T13:59:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 810
    },
    {
        "id": "RlXCPYcBfnpWqlBezNVV",
        "globalUserId": "5473863",
        "wbUserId": 5473863,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "наталья"
        },
        "nmId": 31468103,
        "text": "Сегодня забрала свои любимый фит парад... Открываю коробку а там сюрприз.. Подарочек в ввиде джема. Очень приятно. Спасибо 😊",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-01T16:59:49Z",
        "updatedDate": "2023-08-16T10:09:47.541178094Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T11:36:20Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4a0a4e38-1bab-4dcb-9530-beb3cb48726b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4a0a4e38-1bab-4dcb-9530-beb3cb48726b_ms.jpg"
            }
        ],
        "photo": [
            94273344
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 809
    },
    {
        "id": "O-nNGYcBBhsOOdMGDdWE",
        "globalUserId": "37981624",
        "wbUserId": 37981624,
        "wbUserDetails": {
            "name": "Карина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличная штука. Вкус практически как у сахара. Один пакетик мне заменяет три ложки сахара в кружке чая. Доставка в норме. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T17:24:41Z",
        "updatedDate": "2023-08-16T10:09:47.201360231Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:54:16Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "52866731",
                "wbUserId": 52866731,
                "helpfulness": "plus",
                "time": "2023-03-26T04:19:42Z"
            },
            {
                "globalUserId": "13651148",
                "wbUserId": 13651148,
                "helpfulness": "plus",
                "time": "2023-03-26T05:44:51Z"
            },
            {
                "globalUserId": "20405000",
                "wbUserId": 20405000,
                "helpfulness": "plus",
                "time": "2023-04-03T02:24:41Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c45069fd-b90a-4b46-894e-46aa099a0314_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c45069fd-b90a-4b46-894e-46aa099a0314_ms.jpg"
            }
        ],
        "photo": [
            93060607
        ],
        "video": null,
        "votes": {
            "pluses": 3,
            "minuses": 0
        },
        "rank": 808
    },
    {
        "id": "nkrr6oYBIq9DQExl-Cjw",
        "globalUserId": "64359745",
        "wbUserId": 64359745,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Всё хорошо, упаковка хорошая в пупырке и пакете . Срок изготовления хороший рекомендую ❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T14:56:19Z",
        "updatedDate": "2023-08-16T10:09:46.777266212Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-17T11:35:57Z",
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/af45e077-109f-40e8-a1cd-bdf5c3fa4c79_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/af45e077-109f-40e8-a1cd-bdf5c3fa4c79_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/ec3c8720-5aef-408a-a1ca-9a42c9045ef9_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ec3c8720-5aef-408a-a1ca-9a42c9045ef9_ms.jpg"
            }
        ],
        "photo": [
            91416500,
            91416502
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 807
    },
    {
        "id": "Ne1P9YcBCaW1RxrSaVHp",
        "globalUserId": "32587870",
        "wbUserId": 32587870,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Софья",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Фит номер 7 как фит номер 7 . Беру часто когда худею. Но подорожал . Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-07T08:23:55Z",
        "updatedDate": "2023-08-16T10:09:48.889059146Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-18T09:07:17Z",
            "createDate": "2023-05-18T09:07:17Z",
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:44:32Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/79b81310-b422-4856-81b0-220bf5d7f82b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/79b81310-b422-4856-81b0-220bf5d7f82b_ms.jpg"
            }
        ],
        "photo": [
            100208641
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 806
    },
    {
        "id": "zw7gXYkBfuChzMpwS6i-",
        "globalUserId": "45175213",
        "wbUserId": 45175213,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все хорошо. Еще не пробовала.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-16T08:45:28Z",
        "updatedDate": "2023-08-16T10:09:51.085972012Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-18T11:00:31Z",
            "editable": true,
            "text": "Благодарим за высокую оценку нашего продукта!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3ba7ec34-6e93-43aa-a805-21ca06779f95_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3ba7ec34-6e93-43aa-a805-21ca06779f95_ms.jpg"
            }
        ],
        "photo": [
            112215287
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 805
    },
    {
        "id": "kbG9PooBKNxVyjcCUQmx",
        "globalUserId": "59981936",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Уже 2 пачка. От вкуса сахара не отличить! Пакетика на кружку чая хватает с лихвой!Спасибо за качественный товар.Пока нареканий нет. Диабет 2 типа,для меня это находка.Благодарность вам!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-29T00:41:49Z",
        "updatedDate": "2023-08-31T10:05:42Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "37222790",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-31T10:05:42Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 804
    },
    {
        "id": "GfIGF4gBlM6cTg_RLAJR",
        "globalUserId": "43748941",
        "wbUserId": 43748941,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Юра"
        },
        "nmId": 33478818,
        "text": "Прислали пустую коробку, это конечно, полный ужас. Прошу вернуть деньги, возврат оформила, видео с камер имеются.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-13T21:31:01Z",
        "updatedDate": "2023-09-10T04:13:57Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-16T12:43:43Z",
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "18364301",
                "wbUserId": 18364301,
                "helpfulness": "plus",
                "time": "2023-05-14T06:00:24Z"
            },
            {
                "globalUserId": "60008609",
                "wbUserId": 60008609,
                "helpfulness": "plus",
                "time": "2023-05-14T07:19:17Z"
            },
            {
                "globalUserId": "55507412",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-05-15T08:34:03Z"
            },
            {
                "globalUserId": "27191078",
                "wbUserId": 27191078,
                "helpfulness": "plus",
                "time": "2023-05-19T01:54:36Z"
            },
            {
                "globalUserId": "54441439",
                "wbUserId": 54441439,
                "helpfulness": "plus",
                "time": "2023-07-11T05:41:45Z"
            },
            {
                "globalUserId": "98564982",
                "wbUserId": 98564982,
                "helpfulness": "plus",
                "time": "2023-07-12T22:40:40Z"
            },
            {
                "globalUserId": "56975684",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-10T04:13:57Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5034baf5-5a5d-4bc0-9cec-0a89a9071995_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5034baf5-5a5d-4bc0-9cec-0a89a9071995_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/de9a33d6-408f-4512-a784-38b0b04a637f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/de9a33d6-408f-4512-a784-38b0b04a637f_ms.jpg"
            }
        ],
        "photo": [
            101368116,
            101368117
        ],
        "video": null,
        "votes": {
            "pluses": 7,
            "minuses": 0
        },
        "rank": 803
    },
    {
        "id": "C6Q8c4oBnnhI58-2CEQz",
        "globalUserId": "13300735",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ирина",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Очень хороший подсластитель. Покупала такой же в Магните- выбросила- никакой, а этот классный, действительно один пакетик на чашку достаточно- сладкий!!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T05:20:29Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 802
    },
    {
        "id": "PnZAqIoBUvtK502_D3Nz",
        "globalUserId": "29055298",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Заменитель этот раньше покупала в магазине, правда цена была вдвое дешевле, со своими функциями справляется вполне, рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T12:24:45Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 801
    },
    {
        "id": "jxFOtokBUYwa0QEf_yty",
        "globalUserId": "87077225",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Алина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Супер мне понравилось",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-02T12:52:58Z",
        "updatedDate": "2023-08-16T10:09:51.897896785Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-02T13:25:30Z",
            "editable": true,
            "employeeId": 0,
            "text": "Благодарим за отзыв, но боимся, что Вы прикрепили ошибочное фото) ",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "10262818",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-06T16:51:55Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/83d12c9a-537f-47b4-b446-b74113d08466_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/83d12c9a-537f-47b4-b446-b74113d08466_ms.jpg"
            }
        ],
        "photo": [
            115509898
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 800
    },
    {
        "id": "1LupUIgBCYLDkq7NwUNg",
        "globalUserId": "18613745",
        "wbUserId": 18613745,
        "wbUserDetails": {
            "name": "Виктория",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Заказала две коробки , всё пришло целое и хорошо упаковано .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-25T02:08:02Z",
        "updatedDate": "2023-08-16T10:09:49.434710363Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-25T08:22:35Z",
            "createDate": "2023-05-25T08:22:35Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/663c4412-8c97-44a5-9b58-124710275e4f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/663c4412-8c97-44a5-9b58-124710275e4f_ms.jpg"
            }
        ],
        "photo": [
            103191702
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 799
    },
    {
        "id": "SqTkyIcBqVKcxP1mWK5R",
        "globalUserId": "98022278",
        "wbUserId": 98022278,
        "wbUserDetails": {
            "name": "Оксана",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Всё хорошо, всё целое, 60 пакетиков",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-28T17:23:41Z",
        "updatedDate": "2023-08-16T10:09:48.663861448Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-04T08:03:07Z",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/67ed0c4c-5b3b-4c0e-bc1a-39b187447420_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/67ed0c4c-5b3b-4c0e-bc1a-39b187447420_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/30b39f79-c575-43ed-854c-fdad4b237ab4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/30b39f79-c575-43ed-854c-fdad4b237ab4_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/91456826-64ef-40bb-85b5-3bba0c0fb45d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/91456826-64ef-40bb-85b5-3bba0c0fb45d_ms.jpg"
            }
        ],
        "photo": [
            98784731,
            98784732,
            98784733
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 798
    },
    {
        "id": "PNQzbYcBbskpjQp2prrv",
        "globalUserId": "52054763",
        "wbUserId": 52054763,
        "wbUserDetails": {
            "name": "Анжела",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Понравился. Хороший вкус, без лишних запахов, привкусов. Цена очень хорошая. В магазине намного дороже",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-10T22:05:14Z",
        "updatedDate": "2023-08-16T10:09:47.988162893Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-11T05:45:06Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a072b167-8826-40fe-bd4e-b7a68f35d5f0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a072b167-8826-40fe-bd4e-b7a68f35d5f0_ms.jpg"
            }
        ],
        "photo": [
            95838175
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 797
    },
    {
        "id": "AUtP-YYBmBixw9tz3slO",
        "globalUserId": "28222381",
        "wbUserId": 28222381,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Наталия"
        },
        "nmId": 31468103,
        "text": "Упаковка отвратительная. Вроде не одна коробочка в заказе,  можно как то надежнее было упаковать, не экономить. В итоге все три полурваные, не приятно это мягко говоря. А откажешься еще и за возврат заплатишь.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-19T10:00:06Z",
        "updatedDate": "2023-08-16T10:09:46.901780018Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-20T12:20:30Z",
            "metadata": null,
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/636a4587-05aa-4260-98d7-95b9eabbe138_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/636a4587-05aa-4260-98d7-95b9eabbe138_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/259febe0-68a5-4e30-a459-050d2be4d458_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/259febe0-68a5-4e30-a459-050d2be4d458_ms.jpg"
            }
        ],
        "photo": [
            91895583,
            91895584
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 796
    },
    {
        "id": "MRme_YgBCYLDkq7NleYP",
        "globalUserId": "15437794",
        "wbUserId": 15437794,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Мария"
        },
        "nmId": 138785478,
        "text": "Доставка задержалась на несколько дней. Коробка была вся мятая, как будто её пинали по дороге.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 1,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-27T16:10:08Z",
        "updatedDate": "2023-08-16T10:09:50.399143538Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T06:04:09Z",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c95c899f-e854-48d1-86b2-b5bef2649367_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c95c899f-e854-48d1-86b2-b5bef2649367_ms.jpg"
            }
        ],
        "photo": [
            108856439
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 795
    },
    {
        "id": "YYnOz4YBJ99b87NGVvaA",
        "globalUserId": "34270437",
        "wbUserId": 34270437,
        "wbUserDetails": {
            "name": "Виктория",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень вкусный заменитель сахара, лучше ещё не пробовала, в подарок положили 3 стика с черничным вкусом, тоже классные! Спасибо, приятно очень 😘",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-11T08:34:12Z",
        "updatedDate": "2023-08-16T10:09:46.549604180Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T08:10:28Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/110953ca-b952-4ae9-af8c-a862114c4938_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/110953ca-b952-4ae9-af8c-a862114c4938_ms.jpg"
            }
        ],
        "photo": [
            90371280
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 794
    },
    {
        "id": "fqaDV4cB7nVCbHkC1SVR",
        "globalUserId": "11910582",
        "wbUserId": 11910582,
        "wbUserDetails": {
            "country": "KZ",
            "name": "Александра",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всегда беру только у этого продавца, упакован отлично, в подарок положили 6 саше со смородиной, спасибо)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-06T17:01:10Z",
        "updatedDate": "2023-08-16T10:09:47.803571082Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-07T07:07:15Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a414ca45-fe4f-4a6e-9ab8-78fdd1254538_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a414ca45-fe4f-4a6e-9ab8-78fdd1254538_ms.jpg"
            }
        ],
        "photo": [
            95146983
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 793
    },
    {
        "id": "wG-FfocBCYLDkq7NcCr2",
        "globalUserId": "9553513",
        "wbUserId": 9553513,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Наталия"
        },
        "nmId": 138785478,
        "text": "Спасибо за подарок! Аромат смородины сразу как открыла коробку, теперь и его хочется заказать 👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-14T06:48:07Z",
        "updatedDate": "2023-08-16T10:09:48.119532821Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-14T11:54:17Z",
            "editable": false,
            "employeeId": 0,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/599c5333-e09d-44f3-8c77-79099e3eab8e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/599c5333-e09d-44f3-8c77-79099e3eab8e_ms.jpg"
            }
        ],
        "photo": [
            96324815
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 792
    },
    {
        "id": "LjaB6oYB6z8QKrUH-FAS",
        "globalUserId": "55484081",
        "wbUserId": 55484081,
        "wbUserDetails": {
            "country": "KZ",
            "hasPhoto": false,
            "name": "Эльмира"
        },
        "nmId": 138785478,
        "text": "Спасибо,все пришло хорошо,упаковка целая ,приятно за бонус с  фруктовым вкусом👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T13:00:32Z",
        "updatedDate": "2023-08-16T10:09:46.769633428Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-16T13:08:10Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b02d54bc-7e7e-4160-b4c0-ca77b761ea7e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b02d54bc-7e7e-4160-b4c0-ca77b761ea7e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/150600f2-b448-48da-821d-2e3e94ba5d70_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/150600f2-b448-48da-821d-2e3e94ba5d70_ms.jpg"
            }
        ],
        "photo": [
            91391271,
            91391272
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 791
    },
    {
        "id": "e2WCkYgBl47NwnZ1S7k8",
        "globalUserId": "12694645",
        "wbUserId": 12694645,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 62622901,
        "text": "Самый вкусный из заменителей",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-06T16:20:15Z",
        "updatedDate": "2023-08-16T10:09:49.783539605Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-07T06:12:41Z",
            "createDate": "2023-06-07T06:12:41Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2bf4e572-2b2d-48a3-a039-b79debbe1724_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2bf4e572-2b2d-48a3-a039-b79debbe1724_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/7c5e3436-5dbb-47ea-bc96-b4980dc4c049_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7c5e3436-5dbb-47ea-bc96-b4980dc4c049_ms.jpg"
            }
        ],
        "photo": [
            105204580,
            105204582
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 790
    },
    {
        "id": "hsFGW4oBqd2abVcyd1t8",
        "globalUserId": "8159439",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Галина"
        },
        "nmId": 138785478,
        "text": "Классная штука ...1 пакетик на большой бокал самое то...если это чай кофе с сыром например....кто не любит очень сладко,можно и не полностью высыпать.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T13:40:59Z",
        "updatedDate": "2023-09-04T04:53:22Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T04:53:22Z",
            "employeeId": 0,
            "metadata": null,
            "editable": true,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 789
    },
    {
        "id": "LVFPFokBnnhI58-28dxz",
        "globalUserId": "5865189",
        "wbUserId": 5865189,
        "wbUserDetails": {
            "name": "Мария",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Порадовала привлекательная цена.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-02T11:14:45Z",
        "updatedDate": "2023-08-16T10:09:50.543133477Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-06T08:39:09Z",
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cba039db-5eb3-4c42-9e8f-86ac53fc468c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cba039db-5eb3-4c42-9e8f-86ac53fc468c_ms.jpg"
            }
        ],
        "photo": [
            109671622
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 788
    },
    {
        "id": "WS4x4IYBfnpWqlBeGTgw",
        "globalUserId": "7652612",
        "wbUserId": 7652612,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Виктория",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Покупаю данный сахзам для использования при приготовлении десертов. Супер! Еще положили в подарок пакетики с новым вкусом)спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-14T12:56:00Z",
        "updatedDate": "2023-08-16T10:09:46.673175051Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-14T13:01:40Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c6f836bc-bc23-4fad-ad36-bcdacca64787_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c6f836bc-bc23-4fad-ad36-bcdacca64787_ms.jpg"
            }
        ],
        "photo": [
            91005291
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 787
    },
    {
        "id": "PRB7c4YB8a_hUs_ofgqH",
        "globalUserId": "43308231",
        "wbUserId": 43308231,
        "wbUserDetails": {
            "country": "RU",
            "name": "Алла",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Открыла коробку, упаковочный пакет лежал отдельно, коробка не запакована, Саше все в целости 60 шт.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-21T10:18:39Z",
        "updatedDate": "2023-08-16T10:09:45.941081499Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-21T13:26:23Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b936a64e-a222-411b-84db-d66e701a6258_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b936a64e-a222-411b-84db-d66e701a6258_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/2a860ddc-c8d7-48d0-8fe7-8e98d21b5850_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2a860ddc-c8d7-48d0-8fe7-8e98d21b5850_ms.jpg"
            }
        ],
        "photo": [
            87136529,
            87136530
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 786
    },
    {
        "id": "8BfdKogBZx8DR9RAzdRD",
        "globalUserId": "59745266",
        "wbUserId": 59745266,
        "wbUserDetails": {
            "name": "Елена",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Хорошо упакован, без неприятного послевкусия, очень сладкий",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-17T17:59:19Z",
        "updatedDate": "2023-08-16T10:09:49.206075786Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-18T08:46:35Z",
            "createDate": "2023-05-18T08:46:35Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9234d0ba-1c0d-41b2-bc2b-324d29b37027_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9234d0ba-1c0d-41b2-bc2b-324d29b37027_ms.jpg"
            }
        ],
        "photo": [
            102040685
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 785
    },
    {
        "id": "o2lCGYQBOCC9xHZPVLDN",
        "globalUserId": "",
        "wbUserId": 30435997,
        "wbUserDetails": {
            "name": "Анна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сам то сах.заменитель нормальный 5💫,но блин...честно ни одной звезды не хотела ставить,пишут...пишут вам отзывы,что нужно изменить упаковку,но видимо поставщику не интересно мнение покупателя))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-27T11:44:48Z",
        "updatedDate": "2023-08-16T10:09:42.912814063Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-02T11:45:26Z",
            "employeeId": 0,
            "editable": false,
            "text": "Здравствуйте. Данная проблема в большей степени связана с доставкой самого маркетплейса. Мы поставляем товар во множество торговых точек и только WB славится своим \"уничтожением\" товара в попытках доставить его покупателю. Увы, мы не можем со своей стороны повлиять на курьеров.  ",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 20429417,
                "helpfulness": "plus",
                "time": "2022-11-10T06:06:19.614759474Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 7860283,
                "helpfulness": "plus",
                "time": "2022-12-02T15:14:46.294184528Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/72832c60-7801-4721-91d8-f3fa64455b70_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/72832c60-7801-4721-91d8-f3fa64455b70_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/4b61b277-c000-4990-bbb4-a09957847405_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/4b61b277-c000-4990-bbb4-a09957847405_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/0c900da2-dc01-4117-9396-1486c1603bb7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0c900da2-dc01-4117-9396-1486c1603bb7_ms.jpg"
            }
        ],
        "photo": [
            12450997,
            12450998,
            12450999
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 784
    },
    {
        "id": "DiKTT4YBv9_TjX_mYyg8",
        "globalUserId": "27698187",
        "wbUserId": 27698187,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Доставка быстрая, упаковка прочная. Беру не в первый раз, сахарозаменитель нравится. Положили ещё и подарочек- 3 стика СЗ со вкусом \"чёрная смородина\".Очень приятно. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-14T10:58:25Z",
        "updatedDate": "2023-08-16T10:09:45.674156869Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-14T11:55:10Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/19ba9dfe-880b-46c3-972d-e9b88cacf5bc_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/19ba9dfe-880b-46c3-972d-e9b88cacf5bc_ms.jpg"
            }
        ],
        "photo": [
            85915057
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 783
    },
    {
        "id": "ouszrYcBZx8DR9RAEwZ8",
        "globalUserId": "87000122",
        "wbUserId": 87000122,
        "wbUserDetails": {
            "name": "Дарья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "всё супер, ещё положили 3 пакетика сахарозаменителя со вкусом смородины) спасибо🤍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-23T08:20:18Z",
        "updatedDate": "2023-08-16T10:09:48.463924410Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-25T13:32:06Z",
            "createDate": "2023-04-25T13:32:06Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bb5a3d28-25c9-4a36-af2a-ee96e9ba779a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bb5a3d28-25c9-4a36-af2a-ee96e9ba779a_ms.jpg"
            }
        ],
        "photo": [
            97829786
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 782
    },
    {
        "id": "PXnx9IYBc4LImbbGjiU2",
        "globalUserId": "61987890",
        "wbUserId": 61987890,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Светлана"
        },
        "nmId": 31468103,
        "text": "Давно уже заменяю сахар, очень нравится именно фитпарад, и выпечка с ним вкусная, в этот раз был внутри подарочек) Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-18T13:38:37Z",
        "updatedDate": "2023-08-16T10:09:46.880531287Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T11:57:44Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/18096bb2-d83d-461a-924b-57f3be986289_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/18096bb2-d83d-461a-924b-57f3be986289_ms.jpg"
            }
        ],
        "photo": [
            91767496
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 781
    },
    {
        "id": "eoZ6vYMBwEHULCRVwmGW",
        "globalUserId": "",
        "wbUserId": 92634622,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ирина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Пользуюсь уже пару лет. Очень довольна. Привкуса никакого нет.  На 200 граммовую кружку достаточно 1 саше. Пришел упакованный в прозрачную пленку неповрежденную. Сама коробочка немного помятая. Но это не принципиально. На качество товара это не влияет. Срок изготовления 07.09.2022. срок годности 2 года.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-09T16:01:22Z",
        "updatedDate": "2023-08-16T10:09:42.499542787Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-12T06:41:33Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/fcc4fad0-a33a-4222-bb9c-ed64a0415dde_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/fcc4fad0-a33a-4222-bb9c-ed64a0415dde_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/efc3cb54-3bee-401b-8c4b-00d293021c86_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/efc3cb54-3bee-401b-8c4b-00d293021c86_ms.jpg"
            }
        ],
        "photo": [
            68693582,
            68693600
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 780
    },
    {
        "id": "5NGSAooBoNtFBUnP7z26",
        "globalUserId": "15912130",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Всё отлично👍👏😆",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T08:18:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/215f459a-2532-47ca-99fe-298ef2ac41fc_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/215f459a-2532-47ca-99fe-298ef2ac41fc_ms.jpg"
            }
        ],
        "photo": [
            119317698
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 779
    },
    {
        "id": "ojDuLYcBMBFuPskoFNqn",
        "globalUserId": "85127489",
        "wbUserId": 85127489,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Марина"
        },
        "nmId": 31468103,
        "text": "Спасибо за быструю доставку, всё хорошо упаковано. И отдельное спасибо за подарочки! Очень неожиданно))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T15:13:10Z",
        "updatedDate": "2023-08-16T10:09:47.421295806Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:43:43Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/20f59480-379f-43f1-95c8-6fe3f06656e4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/20f59480-379f-43f1-95c8-6fe3f06656e4_ms.jpg"
            }
        ],
        "photo": [
            93745880
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 778
    },
    {
        "id": "oVwVIocBmBixw9tzqVTU",
        "globalUserId": "17319422",
        "wbUserId": 17319422,
        "wbUserDetails": {
            "name": "Екатерина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Самый любимый сахзам!Ещё и подарочек положили в каждую коробку)))) Большое спасибо, очень приятно!!!❤️❤️❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T08:00:58Z",
        "updatedDate": "2023-08-16T10:09:47.275110645Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T12:58:59Z",
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0b1b2f4e-758f-4eb3-9327-43ef2e1fcf9b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0b1b2f4e-758f-4eb3-9327-43ef2e1fcf9b_ms.jpg"
            }
        ],
        "photo": [
            93299262
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 777
    },
    {
        "id": "KhaVyoUBJ99b87NGCgeq",
        "globalUserId": "62058215",
        "wbUserId": 62058215,
        "wbUserDetails": {
            "name": "Мария",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "сначала не поняла вкуса, не понравилось, но распробовала и полностью довольна, рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-19T15:10:43Z",
        "updatedDate": "2023-08-16T10:09:44.874293295Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-25T06:52:11Z",
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "62058215",
                "wbUserId": 62058215,
                "helpfulness": "plus",
                "time": "2023-01-25T09:58:59Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/71760351-a47f-4d22-bc9b-38882fbafc47_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/71760351-a47f-4d22-bc9b-38882fbafc47_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d6c1d945-257e-4d86-a6bf-7161f72ebbf8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d6c1d945-257e-4d86-a6bf-7161f72ebbf8_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/ede476e0-0cf5-4616-b16f-dff8efcf01a0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ede476e0-0cf5-4616-b16f-dff8efcf01a0_ms.jpg"
            }
        ],
        "photo": [
            79832134,
            79832135,
            79832136
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 776
    },
    {
        "id": "ouXOf4oBwM8MKVou_AYQ",
        "globalUserId": "92492618",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Юлия"
        },
        "nmId": 31468103,
        "text": "взяла сразу 3 коробки и не ошиблась\nу них вообще нет типичного сахзам привкуса, это просто как обычный сахар!! суперский",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T15:56:26Z",
        "updatedDate": "2023-09-13T00:46:23Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "42743833",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-11T10:15:33Z"
            },
            {
                "globalUserId": "118154029",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-11T11:31:52Z"
            },
            {
                "globalUserId": "46787135",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-13T00:46:23Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 3,
            "minuses": 0
        },
        "rank": 775
    },
    {
        "id": "uvCd84YB5HDhYKpHvuIV",
        "globalUserId": "42574413",
        "wbUserId": 42574413,
        "wbUserDetails": {
            "country": "RU",
            "name": "Сергей",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хороший заменитель. Не портит вкус. Заказывал первый раз, заказал ещё.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-18T07:27:27Z",
        "updatedDate": "2023-08-16T10:09:46.862189385Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:16:46Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0ed96b72-5051-4800-8b98-616e761a7d3f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0ed96b72-5051-4800-8b98-616e761a7d3f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/0537ac0d-430e-436d-87cf-82b3769bcd6a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0537ac0d-430e-436d-87cf-82b3769bcd6a_ms.jpg"
            }
        ],
        "photo": [
            91690045,
            91690046
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 774
    },
    {
        "id": "ZsoAU4cBbskpjQp2qmzn",
        "globalUserId": "42356414",
        "wbUserId": 42356414,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Дочька дала попробовать. Понравился. Заказала в саше потому что удобно и можно взять с собой.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-05T19:59:25Z",
        "updatedDate": "2023-08-16T10:09:47.772196857Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T06:13:47Z",
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6b209f63-f065-4b00-a4fe-70d08f303108_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6b209f63-f065-4b00-a4fe-70d08f303108_ms.jpg"
            }
        ],
        "photo": [
            95002891
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 773
    },
    {
        "id": "MQbwuIoBUdF-4nlzr1Pi",
        "globalUserId": "53789295",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всё отлично. Помогает снизить общее потребление калорий. По вкусу не отличить от обычного сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T18:11:36Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 772
    },
    {
        "id": "NLJlyIoBuWYO9ah2EHGG",
        "globalUserId": "23844337",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Нина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все хорошо, получила товар хорошо упакованный,быстрая доставка.спасибо продавцу и валберису",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T18:13:01Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 771
    },
    {
        "id": "609fUIcBQBfka8TNHudC",
        "globalUserId": "17568111",
        "wbUserId": 17568111,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Эльвира"
        },
        "nmId": 138785478,
        "text": "Спасибо за подарок, очень приятный бонус)) за такую цену 218р- 90шт очень выгодная покупка)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-05T07:43:44Z",
        "updatedDate": "2023-08-16T10:09:47.726834523Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-05T07:48:49Z",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1b6c4d80-c707-4d94-bf7c-9a6cc3c1dff8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1b6c4d80-c707-4d94-bf7c-9a6cc3c1dff8_ms.jpg"
            }
        ],
        "photo": [
            94862297
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 770
    },
    {
        "id": "ICJgCYcBMBFuPskoj7yA",
        "globalUserId": "17883394",
        "wbUserId": 17883394,
        "wbUserDetails": {
            "name": "Елена",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Отличный товар. Внутри подарок - джем черничный. Была приятно удивлена. Кстати, джем вкусненький. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-22T12:52:16Z",
        "updatedDate": "2023-08-16T10:09:47.057530927Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-22T12:54:39Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/dfd16320-801e-42af-9e4a-15184cb02cc7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dfd16320-801e-42af-9e4a-15184cb02cc7_ms.jpg"
            }
        ],
        "photo": [
            92475349
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 769
    },
    {
        "id": "K0GP8IYBT-IFIyX_a6yJ",
        "globalUserId": "10702306",
        "wbUserId": 10702306,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Екатерина"
        },
        "nmId": 31468103,
        "text": "Уже не первый год пользуюсь данным сах.замом.\nВ каждой коробке был подарочек в виде джема без сахара. Приятно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-17T17:12:56Z",
        "updatedDate": "2023-08-16T10:09:46.839620210Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:44:49Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a8f2fcfc-fb60-4b97-94e3-bac5e23904ab_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a8f2fcfc-fb60-4b97-94e3-bac5e23904ab_ms.jpg"
            }
        ],
        "photo": [
            91630714
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 768
    },
    {
        "id": "pDQdnYoBqdFzcemqHyS8",
        "globalUserId": "52183077",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Наталья"
        },
        "nmId": 31468103,
        "text": "Товар хороший! Беру у вас постоянно! Главное успеть, когда есть в наличии. Взяла три упаковки, пока был!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T08:30:46Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 767
    },
    {
        "id": "kF2UGYcBIq9DQExliuMo",
        "globalUserId": "10832748",
        "wbUserId": 10832748,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Элвира"
        },
        "nmId": 138785478,
        "text": "Спасибо за товар и доставку. Будем всегда брать вашу продукцию.  Положили подарок. Приятно. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T16:22:58Z",
        "updatedDate": "2023-08-16T10:09:47.196896292Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:58:03Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d61f54ee-0dd9-4de6-8e58-bbe8fb548909_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d61f54ee-0dd9-4de6-8e58-bbe8fb548909_ms.jpg"
            }
        ],
        "photo": [
            93049656
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 766
    },
    {
        "id": "9WxKQIcBIq9DQExlb4OF",
        "globalUserId": "96296284",
        "wbUserId": 96296284,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Получила, всё целое коробка в коробке.Ещё не пробовала.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T04:47:13Z",
        "updatedDate": "2023-08-16T10:09:47.553287730Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T11:03:38Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4354081f-c459-49b5-9558-81691bb9d2d8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4354081f-c459-49b5-9558-81691bb9d2d8_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/33a26162-6a05-4cfd-af48-688e8a65e501_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/33a26162-6a05-4cfd-af48-688e8a65e501_ms.jpg"
            }
        ],
        "photo": [
            94312308,
            94312309
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 765
    },
    {
        "id": "MTTFCYcBQBfka8TNcdfW",
        "globalUserId": "37004633",
        "wbUserId": 37004633,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Марина"
        },
        "nmId": 138785478,
        "text": "Хорошо упаковано, спасибо за пробнички другого фит парада. Рекомендую! Товар пришёл на следующий день.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-22T14:42:27Z",
        "updatedDate": "2023-08-16T10:09:47.062014703Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T05:49:18Z",
            "editable": false,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b0f1f96a-eeb8-4b8c-b3c3-68b44dcab3a6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b0f1f96a-eeb8-4b8c-b3c3-68b44dcab3a6_ms.jpg"
            }
        ],
        "photo": [
            92497822
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 764
    },
    {
        "id": "EfIVMIYB8a_hUs_osiFl",
        "globalUserId": "10923104",
        "wbUserId": 10923104,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Надежда"
        },
        "nmId": 31468103,
        "text": "Сахаразаменитель супер, самое то для человека, который на ПП))Веду бесплатный чат по правильному питанию все вопросы по телефону 89двадцать три326девятнадцать65",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-08T08:12:54Z",
        "updatedDate": "2023-08-16T10:09:45.442417892Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-08T08:17:08Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "10923104",
                "wbUserId": 10923104,
                "helpfulness": "plus",
                "time": "2023-02-08T08:14:31Z"
            },
            {
                "globalUserId": "25893489",
                "wbUserId": 25893489,
                "helpfulness": "plus",
                "time": "2023-02-08T13:10:17Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1500dcc5-510f-4dae-88ac-617ebb051684_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1500dcc5-510f-4dae-88ac-617ebb051684_ms.jpg"
            }
        ],
        "photo": [
            84899521
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 763
    },
    {
        "id": "fVlaSIcBfnpWqlBens9S",
        "globalUserId": "81497180",
        "wbUserId": 81497180,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Галина"
        },
        "nmId": 138785478,
        "text": "Очень понравился сахарозаменитель буду заказывать ещё",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-03T18:21:51Z",
        "updatedDate": "2023-08-16T10:09:47.654766169Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T05:20:05Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/31546ecf-b910-4604-a1fe-ca94681e054a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/31546ecf-b910-4604-a1fe-ca94681e054a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/1d5ee2e7-9c8c-44d3-896c-2050de054bb2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1d5ee2e7-9c8c-44d3-896c-2050de054bb2_ms.jpg"
            }
        ],
        "photo": [
            94632248,
            94632249
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 762
    },
    {
        "id": "oa4kp4YBBhsOOdMGhWp1",
        "globalUserId": "19899037",
        "wbUserId": 19899037,
        "wbUserDetails": {
            "country": "KZ",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "В заказе было написано, что один пакетик содержит 1 чайную ложку. А пришли пакетики, которые содержат по две чайные ложки в одном. И как я должна теперь 1 чайную ложку из этого пакетика отмерять????",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-03T11:03:54Z",
        "updatedDate": "2023-08-16T10:09:46.285185060Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-03T11:27:32Z",
            "text": "Благодарим Вас за информацию, обновили данные в карточке! ",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "12392270",
                "wbUserId": 12392270,
                "helpfulness": "plus",
                "time": "2023-03-05T19:18:50Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/841802ae-c5d1-41a3-abe7-a4aaab715a13_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/841802ae-c5d1-41a3-abe7-a4aaab715a13_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/9494cbec-5307-4100-a2f2-60c49d34befd_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9494cbec-5307-4100-a2f2-60c49d34befd_ms.jpg"
            }
        ],
        "photo": [
            88900240,
            88900241
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 761
    },
    {
        "id": "Gd1_aooBwM8MKVouVAtU",
        "globalUserId": "33587974",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Жанна"
        },
        "nmId": 138785478,
        "text": "Хороший товар, никакого постороннего привкуса нет, на кружку жидкости один пакет мне можно даже половину, очень сладко",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T12:37:24Z",
        "updatedDate": "2023-09-06T12:37:43Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T12:37:43Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 760
    },
    {
        "id": "wguw1YcBUdF-4nlzfCfe",
        "globalUserId": "110162664",
        "wbUserId": 110162664,
        "wbUserDetails": {
            "country": "ru",
            "name": "",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Брала для мамы. Она сказала что 1 Саше хватает на 2 кружки чая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-01T05:02:06Z",
        "updatedDate": "2023-08-16T10:09:48.721152076Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-04T06:42:08Z",
            "createDate": "2023-05-04T06:42:08Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3cb3a40-833a-4d90-924a-460aae46c470_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3cb3a40-833a-4d90-924a-460aae46c470_ms.jpg"
            }
        ],
        "photo": [
            99141690
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 759
    },
    {
        "id": "woyS7IcB3ieoZoqxj38f",
        "globalUserId": "93518933",
        "wbUserId": 93518933,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Оксана"
        },
        "nmId": 138785478,
        "text": "Покупкой довольна.Очень хорошо для тех кто сидит на диете.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-05T15:40:41Z",
        "updatedDate": "2023-08-16T10:09:48.843301243Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-19T08:55:09Z",
            "createDate": "2023-05-19T08:55:09Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54271706",
                "wbUserId": 54271706,
                "helpfulness": "minus",
                "time": "2023-05-06T03:25:10Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e1a096a5-cf7d-4dce-8712-8e6d6ac06cbe_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e1a096a5-cf7d-4dce-8712-8e6d6ac06cbe_ms.jpg"
            }
        ],
        "photo": [
            99965971
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 758
    },
    {
        "id": "NacreooBnnhI58-2kQMS",
        "globalUserId": "17991893",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Очень люблю этот сахарозаменитель. Никакого привкуса, удобная форма выпуска, легко применять в еде и напитках!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T13:39:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 757
    },
    {
        "id": "RN0Hh4YBMBFuPskozK4z",
        "globalUserId": "96108240",
        "wbUserId": 96108240,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна Морозова",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Заменитель сахара мне понравился, беру первый раз, Саше заменяет две ложечки сахара он полезней чем дешманский сахар. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-25T05:24:41Z",
        "updatedDate": "2023-08-16T10:09:46.067295134Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-02T08:12:12Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/44bcf175-2f3b-4b7c-a0f8-552d98c98838_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/44bcf175-2f3b-4b7c-a0f8-552d98c98838_ms.jpg"
            }
        ],
        "photo": [
            87775387
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 756
    },
    {
        "id": "8SCz34YBQBfka8TNadJj",
        "globalUserId": "11339879",
        "wbUserId": 11339879,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Валентина",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Все отлично! Беру раз в 3-4 месяца, хватает на пол года. Спасибо за джем! Не ожидала, в каждой пачке лежал.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-14T10:38:42Z",
        "updatedDate": "2023-08-16T10:09:46.661083588Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-14T13:03:45Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ddcc1ef5-4e2b-4262-8d7c-c9051927e243_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ddcc1ef5-4e2b-4262-8d7c-c9051927e243_ms.jpg"
            }
        ],
        "photo": [
            90975274
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 755
    },
    {
        "id": "p_owcogBI9q8Yb8jvOZZ",
        "globalUserId": "16443532",
        "wbUserId": 16443532,
        "wbUserDetails": {
            "name": "Лариса",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришло хорошо упаковано.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-31T14:22:57Z",
        "updatedDate": "2023-08-16T10:09:49.635888397Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-01T10:08:39Z",
            "createDate": "2023-06-01T10:08:39Z",
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c88762e6-dbb0-4e77-b248-71cc4f438496_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c88762e6-dbb0-4e77-b248-71cc4f438496_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a1079795-fe8d-4244-8ac1-ddcaa8106123_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a1079795-fe8d-4244-8ac1-ddcaa8106123_ms.jpg"
            }
        ],
        "photo": [
            104227443,
            104227444
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 754
    },
    {
        "id": "fehu5YcBCaW1RxrSl611",
        "globalUserId": "40889461",
        "wbUserId": 40889461,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Светлана",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный продукт. Спасибо продавцу.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-04T06:24:03Z",
        "updatedDate": "2023-08-16T10:09:48.796493618Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-04T06:25:14Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f6b74c4d-73b5-4062-9e41-c040d9fea3c5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f6b74c4d-73b5-4062-9e41-c040d9fea3c5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/5e64fa24-4634-4996-a5de-64315a72cc03_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5e64fa24-4634-4996-a5de-64315a72cc03_ms.jpg"
            }
        ],
        "photo": [
            99691846,
            99691849
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 753
    },
    {
        "id": "KiXEi4oBU2re2XP7FQsT",
        "globalUserId": "56154126",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анна"
        },
        "nmId": 138785478,
        "text": "Хороший заменитель,брала до этого в другом месте 1 пакетика хватало,с этим надо 2 мне чтобы было сладко как я хотела.вроде выгодно а вроде и нет((90 шт а хватает так же как 60 к сожалению поэтому 3 звезды,привкус не имеет как другие много пробывала,а так рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T23:39:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 752
    },
    {
        "id": "UcKz84YBTZEOO9f58q1y",
        "globalUserId": "19711631",
        "wbUserId": 19711631,
        "wbUserDetails": {
            "name": "Виктория",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший продукт, давно им пользуюсь, спасибо большое за подарок) обязательно попробую все ваши вкусы)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-18T07:51:42Z",
        "updatedDate": "2023-08-16T10:09:46.866755014Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:16:24Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f1d4f8e1-bc4a-43f8-9124-acc0fdfce539_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f1d4f8e1-bc4a-43f8-9124-acc0fdfce539_ms.jpg"
            }
        ],
        "photo": [
            91693984
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 751
    },
    {
        "id": "xem7fYcBUdF-4nlziLkf",
        "globalUserId": "50357950",
        "wbUserId": 50357950,
        "wbUserDetails": {
            "name": "Мария",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Очень хороший заменитель, беру не первый раз. А за джем отдельное спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-14T03:07:35Z",
        "updatedDate": "2023-08-16T10:09:48.115039485Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-14T11:55:11Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a828235e-8595-4c15-9d50-9218905f3668_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a828235e-8595-4c15-9d50-9218905f3668_ms.jpg"
            }
        ],
        "photo": [
            96307470
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 750
    },
    {
        "id": "qdubo4oBuX2t2Z2-qH-R",
        "globalUserId": "26814618",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Мария"
        },
        "nmId": 138785478,
        "text": "Мне очень понравился, добавляю один пакетик в кофе и получается даже вкуснее, чем с сахаром",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T14:46:42Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 749
    },
    {
        "id": "DAGzHIcB5HDhYKpHIch0",
        "globalUserId": "4221610",
        "wbUserId": 4221610,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Всё пришло быстро. Отдельное спасибо за подарочек, в каждой упаковке лежал черничный джем 🤤",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T06:55:14Z",
        "updatedDate": "2023-08-16T10:09:47.219611964Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-12T08:42:16Z",
            "createDate": "2023-04-12T08:41:51Z",
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6a4a28c6-1abf-4d49-82b1-053b2e2b68de_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6a4a28c6-1abf-4d49-82b1-053b2e2b68de_ms.jpg"
            }
        ],
        "photo": [
            93109339
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 748
    },
    {
        "id": "IssofooBuX2t2Z2-dxjk",
        "globalUserId": "14410339",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Татьяна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "В Магните по акции вышел дешевле, ну что ж буду знать на будущее. \nСам продукт хорошо упакован, всë норм.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T08:14:56Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 747
    },
    {
        "id": "7O-hPIoBl47NwnZ1oGdt",
        "globalUserId": "47033758",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Юлия",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "На вкус как сахар, без всяких посторонних привкусов. Сахар в крови не поднимает, а это главное. Для диабетика 1 типа в самый раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T14:52:20Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 746
    },
    {
        "id": "nGFcI4cBIq9DQExlzpkj",
        "globalUserId": "66395820",
        "wbUserId": 66395820,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Товар пришёл в целой упаковке, сроки свежие! Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T13:58:17Z",
        "updatedDate": "2023-08-16T10:09:47.299251644Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T12:27:54Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6887dfc0-9e7a-49b6-9e7f-455951ed4381_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6887dfc0-9e7a-49b6-9e7f-455951ed4381_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3327068-8b74-4867-a14c-d8e5a1f36600_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3327068-8b74-4867-a14c-d8e5a1f36600_ms.jpg"
            }
        ],
        "photo": [
            93375325,
            93375326
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 745
    },
    {
        "id": "HN9AhYoBZx8DR9RAOumu",
        "globalUserId": "14264257",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру не первый раз. Хороший,без постороннего привкуса. На чашку для меня хватает одного пакетика.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T17:18:14Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 744
    },
    {
        "id": "QX3GnooBYE-HFOZ5Ju-W",
        "globalUserId": "15945862",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Удобно в поездках , в кафе. Всегда со мной в сумке и нет проблем с использованием сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T16:15:01Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 743
    },
    {
        "id": "NW0SgYcBYE-HFOZ5uJXo",
        "globalUserId": "42984637",
        "wbUserId": 42984637,
        "wbUserDetails": {
            "country": "RU",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришло целое, в слюде и коробке. Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-14T18:41:41Z",
        "updatedDate": "2023-08-16T10:09:48.140721491Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:57:45Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/8e3cb656-772e-40a3-93bf-7ab36c4e4a6b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8e3cb656-772e-40a3-93bf-7ab36c4e4a6b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/77a295a6-2e57-4331-95d9-c5cdf41ab2e7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/77a295a6-2e57-4331-95d9-c5cdf41ab2e7_ms.jpg"
            }
        ],
        "photo": [
            96438590,
            96438591
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 742
    },
    {
        "id": "G182PYoBmorVKbOQuqv2",
        "globalUserId": "12169780",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ирина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Покупаю не в первый раз, очень нравится. Сахар стараюсь не употреблять. Такого пакетика на утреннюю чашку кофе мне хватает",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T17:35:12Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 741
    },
    {
        "id": "rg32DogBZx8DR9RAb3xk",
        "globalUserId": "43681749",
        "wbUserId": 43681749,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Карина",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Все целое,спасибо продавцу за подарок😉🥰Джем 🫦🔥❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-12T07:56:51Z",
        "updatedDate": "2023-08-16T10:09:49.039555537Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-16T13:00:36Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b971f437-5d43-40f9-a889-17511527d01a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b971f437-5d43-40f9-a889-17511527d01a_ms.jpg"
            }
        ],
        "photo": [
            101065794
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 740
    },
    {
        "id": "P23lMYcB8a_hUs_ohOTh",
        "globalUserId": "26606737",
        "wbUserId": 26606737,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Евгения",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Самый лучший сахзам, не изменяет вкус и достаточно сладкий. Спасибо за подарок)))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-30T09:42:18Z",
        "updatedDate": "2023-08-16T10:09:47.445327672Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:28:06Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c92d3867-1be4-4e38-8201-6cf347d164d5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c92d3867-1be4-4e38-8201-6cf347d164d5_ms.jpg"
            }
        ],
        "photo": [
            93856245
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 739
    },
    {
        "id": "xZDjLYcBc4LImbbGy1TQ",
        "globalUserId": "13280617",
        "wbUserId": 13280617,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Мария",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Дешевле, чем в магазине,спасибо за маленький презент,понравился смородиновый вкус",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T15:01:56Z",
        "updatedDate": "2023-08-16T10:09:47.418316359Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:44:22Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6097ae7a-d413-4412-8d93-7efdbe5752f7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6097ae7a-d413-4412-8d93-7efdbe5752f7_ms.jpg"
            }
        ],
        "photo": [
            93743445
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 738
    },
    {
        "id": "OKsmdYMBE-Uw035e70ro",
        "globalUserId": "",
        "wbUserId": 29058495,
        "wbUserDetails": {
            "name": "Анастасия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Коробка немного помята, но для меня не страшно. Заявлено, что 1 пакетик равен 10 гр сахара, т.е. 1 ч.л. Я добавила 1 п. в чай и мне хватило.  Буду брать ещё, советую к приобретению👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-25T14:57:09Z",
        "updatedDate": "2023-08-16T10:09:42.249642821Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T06:04:53Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/31cef143-c64f-4361-9fdf-3d7bb1b42341_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/31cef143-c64f-4361-9fdf-3d7bb1b42341_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/235b1ee0-d0c7-41f0-8d70-12a7bc42d621_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/235b1ee0-d0c7-41f0-8d70-12a7bc42d621_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/b17312f0-1e94-4bf4-8374-10db1058eeb7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b17312f0-1e94-4bf4-8374-10db1058eeb7_ms.jpg"
            }
        ],
        "photo": [
            68296054,
            68296072,
            68296091
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 737
    },
    {
        "id": "5amivYoBXuPSsPE1HqVz",
        "globalUserId": "122997846",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Залина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Нормальн все, пришли в отличом состоянии,на один день раньше пришел товар.👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T16:03:53Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 736
    },
    {
        "id": "VOlqsYYBxpMXtsmOaNe6",
        "globalUserId": "21280508",
        "wbUserId": 21280508,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "BY"
        },
        "nmId": 33478818,
        "text": "Пользуюсь не первый год этим сахарозаменителем. При доставке был упакован в картонную коробку. Благодарю.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-05T10:56:26Z",
        "updatedDate": "2023-08-16T10:09:46.358915853Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-06T08:15:46Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c7b52399-25cb-4c67-92d0-b0ea2f575554_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c7b52399-25cb-4c67-92d0-b0ea2f575554_ms.jpg"
            }
        ],
        "photo": [
            89265631
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 735
    },
    {
        "id": "PLrpqIoBnnhI58-2sv2A",
        "globalUserId": "17504491",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Эльмира"
        },
        "nmId": 31468103,
        "text": "Всё целое. Единственное, абсолютно не отслеживается товар,но пришло всё во время.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T15:30:02Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 734
    },
    {
        "id": "k4zndIcBuWYO9ah2B7ZD",
        "globalUserId": "44145562",
        "wbUserId": 44145562,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Как всегда всё отлично. Положили подарок, нужно будет попробовать 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-12T09:58:30Z",
        "updatedDate": "2023-08-16T10:09:48.030819641Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T10:03:54Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/39610a67-04b2-4494-a216-42588d99f105_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/39610a67-04b2-4494-a216-42588d99f105_ms.jpg"
            }
        ],
        "photo": [
            96021652
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 733
    },
    {
        "id": "q-qJHYcBBhsOOdMGxudm",
        "globalUserId": "7640985",
        "wbUserId": 7640985,
        "wbUserDetails": {
            "name": "Карина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Спасибо! В каждой коробочке лежал подарок в виде джема с черникой! Приятненько🤗🤗🤗",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-26T10:49:41Z",
        "updatedDate": "2023-08-16T10:09:47.234761836Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:36:10Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14174265",
                "wbUserId": 14174265,
                "helpfulness": "plus",
                "time": "2023-03-27T05:33:12Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3030e09-d557-46a1-a259-b21a6f132121_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3030e09-d557-46a1-a259-b21a6f132121_ms.jpg"
            }
        ],
        "photo": [
            93152751
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 732
    },
    {
        "id": "kxVu_4YBxpMXtsmO58Cs",
        "globalUserId": "60594431",
        "wbUserId": 60594431,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Всегда беру такой . Взяла 3 шт в каждой оказался маленький подарочек 🙂🤓 Очень приятно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-20T14:31:44Z",
        "updatedDate": "2023-08-16T10:09:46.956735818Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:29:23Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "60594431",
                "wbUserId": 60594431,
                "helpfulness": "plus",
                "time": "2023-03-23T15:49:37Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9785b4f4-7133-4598-b229-6e6e1d34c800_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9785b4f4-7133-4598-b229-6e6e1d34c800_ms.jpg"
            }
        ],
        "photo": [
            92137242
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 731
    },
    {
        "id": "UZKfvIoBmorVKbOQ-DRo",
        "globalUserId": "28303219",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный товар, без всякой  химической отдушки. Несомненно буду брать ещё.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T11:21:55Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 730
    },
    {
        "id": "xv8jiYcBNO444I63lLe1",
        "globalUserId": "100505258",
        "wbUserId": 100505258,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Любовь"
        },
        "nmId": 33478818,
        "text": "Спасибо продавцу,всё упаковано отлично ,ничего не просыпалось🙂",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-16T08:17:03Z",
        "updatedDate": "2023-08-16T10:09:48.191396621Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:25:07Z",
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c7e99d8b-3767-4f5c-8a15-413fadce1983_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c7e99d8b-3767-4f5c-8a15-413fadce1983_ms.jpg"
            }
        ],
        "photo": [
            96646626
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 729
    },
    {
        "id": "-PWEN4cBBhsOOdMGrBbk",
        "globalUserId": "21747112",
        "wbUserId": 21747112,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Нелли"
        },
        "nmId": 31468103,
        "text": "Отличный сахзам, прекрасная доставка! Спасибо за подарочек в виде джема ❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-31T11:54:15Z",
        "updatedDate": "2023-08-16T10:09:47.495894561Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:11:00Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/34fdfc82-bbae-4749-867e-a9258cbff726_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/34fdfc82-bbae-4749-867e-a9258cbff726_ms.jpg"
            }
        ],
        "photo": [
            94053647
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 728
    },
    {
        "id": "ROO8vYoBKNxVyjcCwb9B",
        "globalUserId": "28514754",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Людмила",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Хороший, маме нравится заказываю не первый раз и дальше будем заказывать",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T16:32:59Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 727
    },
    {
        "id": "e6AfXYcBv9_TjX_mcy2_",
        "globalUserId": "25124457",
        "wbUserId": 25124457,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Василина"
        },
        "nmId": 33478818,
        "text": "Взяла по рекомендациям , упаковано хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-07T19:09:15Z",
        "updatedDate": "2023-08-16T10:09:47.860870397Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T07:07:53Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/bd142dcd-07b9-4640-b511-0fede45934ee_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/bd142dcd-07b9-4640-b511-0fede45934ee_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/7f1036bd-543b-49ac-a4d0-f6439cfb6c0f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7f1036bd-543b-49ac-a4d0-f6439cfb6c0f_ms.jpg"
            }
        ],
        "photo": [
            95341336,
            95341337
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 726
    },
    {
        "id": "536mmIkBKNxVyjcCyk03",
        "globalUserId": "43762793",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Мариана"
        },
        "nmId": 138785478,
        "text": "Все отлично 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T18:40:15Z",
        "updatedDate": "2023-08-16T10:09:51.599644555Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T10:25:56Z",
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e3e71d65-31a1-4754-a8dd-f1ece4caddb0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e3e71d65-31a1-4754-a8dd-f1ece4caddb0_ms.jpg"
            }
        ],
        "photo": [
            114484137
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 725
    },
    {
        "id": "xXfkj4oBYE-HFOZ5cdXC",
        "globalUserId": "36095903",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Мне подощел заменитель в чае кампоте ничем не отличается как сахар советую к покупке",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T18:53:48Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 724
    },
    {
        "id": "xCGjy4oBBgFQoSo0QROC",
        "globalUserId": "96414582",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Получила, хорошо упакован. Отзыв о продукте позже. Будем пробовать)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T09:19:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 723
    },
    {
        "id": "Lc7_4IIBTTPU1bwQUFzU",
        "globalUserId": "",
        "wbUserId": 22641285,
        "wbUserDetails": {
            "name": "Ксения",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Коробка пришла целое и невредимая. К продавцу никаких замечаний нет. Я брала для готовки десертов. К сожалению, чувствую горьковатый привкус сахарозаменителя в печенье. Так что тут всё индивидуально.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-27T20:30:04Z",
        "updatedDate": "2023-08-16T10:09:41.846490230Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-30T12:23:16Z",
            "metadata": null,
            "editable": false,
            "text": "Вероятно у вас повышенная чувствительность к компонентам в составе. ",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/352424bd-b342-42fc-ab9e-88a786e195f6_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/352424bd-b342-42fc-ab9e-88a786e195f6_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6e17a850-8f59-4d27-8a88-021f938894ad_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6e17a850-8f59-4d27-8a88-021f938894ad_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/3bc01ee0-51cd-440d-b926-0bf9d363dfa9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3bc01ee0-51cd-440d-b926-0bf9d363dfa9_ms.jpg"
            }
        ],
        "photo": [
            75292967,
            75292977,
            75292987
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 722
    },
    {
        "id": "MNztx4QBfdxlb2FDddVx",
        "globalUserId": "",
        "wbUserId": 12464559,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ирина"
        },
        "nmId": 33478818,
        "text": "Ну очень сладкий, мне целого пакетика на чашку кофе прям много. Буду уменьшать дозу) Коробка немного помялась в пути, ну это не важно. Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-30T09:45:38Z",
        "updatedDate": "2023-08-16T10:09:43.723236554Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-05T11:55:11Z",
            "metadata": null,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/02143a32-b22b-4c03-a22f-c583ab6a0c72_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/02143a32-b22b-4c03-a22f-c583ab6a0c72_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/316c2a25-db6e-4d98-997b-928d991b0adf_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/316c2a25-db6e-4d98-997b-928d991b0adf_ms.jpg"
            }
        ],
        "photo": [
            48874918,
            48874919
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 721
    },
    {
        "id": "1bSLhIcB5g4sVs4DAjVg",
        "globalUserId": "62692024",
        "wbUserId": 62692024,
        "wbUserDetails": {
            "country": "RU",
            "name": "Юлия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всё отлично, спасибо за подарок, неожиданно но очень приятно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-15T10:51:55Z",
        "updatedDate": "2023-08-16T10:09:48.161753987Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:44:24Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/baa22cfd-77c8-4ff2-8b8f-925fc03483ec_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/baa22cfd-77c8-4ff2-8b8f-925fc03483ec_ms.jpg"
            }
        ],
        "photo": [
            96519814
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 720
    },
    {
        "id": "T3TPsIoBfuChzMpw-oMV",
        "globalUserId": "16057908",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Надежда",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший сахорозаменитель. Беру уже не первый раз.нет неприятного привкуса",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-20T04:18:55Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 719
    },
    {
        "id": "dgq5MocB5HDhYKpHcK2A",
        "globalUserId": "40328362",
        "wbUserId": 40328362,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Кира"
        },
        "nmId": 138785478,
        "text": "в подарок положили сахар со вкусом чёрной смородины🫐 спасибо большое 🫶💕",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-30T13:33:46Z",
        "updatedDate": "2023-08-16T10:09:47.457434173Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:17:32Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f7cd75b4-8508-4d6a-96f0-dea8d36899a3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f7cd75b4-8508-4d6a-96f0-dea8d36899a3_ms.jpg"
            }
        ],
        "photo": [
            93902053
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 718
    },
    {
        "id": "EklqNooBUvtK502_1KV3",
        "globalUserId": "26933685",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ирина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Мне одно Саше на одну чашку кофе слишком сладко, делю на 2 раза, противного привкуса как и многих сах замов нет",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-27T09:54:45Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 717
    },
    {
        "id": "DJRQQocBk9xzjhYZIaq0",
        "globalUserId": "33511802",
        "wbUserId": 33511802,
        "wbUserDetails": {
            "name": "Евгения",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Доставка быстрая, все отлично 👍 отдельное спасибо за приятный бонус😻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T14:12:40Z",
        "updatedDate": "2023-08-16T10:09:47.584989687Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T10:01:03Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/314dbb4c-1d71-4b20-b35c-4d14f32339de_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/314dbb4c-1d71-4b20-b35c-4d14f32339de_ms.jpg"
            }
        ],
        "photo": [
            94407794
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 716
    },
    {
        "id": "g2WbjYoBfuChzMpwE-Bg",
        "globalUserId": "122120258",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Упаковано отлично на все 💯 и доставка пришла на день раньше к чему я очень рада 😍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T08:14:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 715
    },
    {
        "id": "v5LXeooBuWYO9ah2D2s_",
        "globalUserId": "19054410",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Исакова Вера"
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель. Без посторонних привкусов и запахов. Всегда его заказываю.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T16:47:09Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 714
    },
    {
        "id": "O8LOQYcBbskpjQp2F8FB",
        "globalUserId": "41657146",
        "wbUserId": 41657146,
        "wbUserDetails": {
            "name": "Дания",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Удивительно, в коробочках оказался сюрприз  джем черничный. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T11:50:38Z",
        "updatedDate": "2023-08-16T10:09:47.578939874Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T10:04:30Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/37b1d500-0fc8-41b6-8787-9baa7e121814_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/37b1d500-0fc8-41b6-8787-9baa7e121814_ms.jpg"
            }
        ],
        "photo": [
            94378358
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 713
    },
    {
        "id": "XZWi4YYBJ99b87NGAXnP",
        "globalUserId": "9289042",
        "wbUserId": 9289042,
        "wbUserDetails": {
            "country": "RU",
            "name": "Надежда",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "сахарозаменитель очень нравится. Спасибо продавцу.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-14T19:38:56Z",
        "updatedDate": "2023-08-16T10:09:46.691301607Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T06:10:41Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c768df3e-98d6-4adb-b46f-7f74f52d79a3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c768df3e-98d6-4adb-b46f-7f74f52d79a3_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4ccfd3d6-7f4d-4710-b17d-2f5254a3498f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4ccfd3d6-7f4d-4710-b17d-2f5254a3498f_ms.jpg"
            }
        ],
        "photo": [
            91096331,
            91096333
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 712
    },
    {
        "id": "qVk5_4YB8a_hUs_oANpC",
        "globalUserId": "17973490",
        "wbUserId": 17973490,
        "wbUserDetails": {
            "name": "Любовь",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Сахар перешёл да ещё с сюрпризом. Джем. Сахар нравится покупаю постоянно фит.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-20T13:32:51Z",
        "updatedDate": "2023-08-16T10:09:46.953625878Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:29:44Z",
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/64585252-d050-42df-b545-f6f62149db2c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/64585252-d050-42df-b545-f6f62149db2c_ms.jpg"
            }
        ],
        "photo": [
            92124054
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 711
    },
    {
        "id": "CHUnvooBpT2rITcGRJVo",
        "globalUserId": "57798884",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Светлана",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Товар соответствует описанию. Покупаем не первый раз. \nРекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T18:29:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 710
    },
    {
        "id": "cmMjk4oBpT2rITcG8eh_",
        "globalUserId": "17792856",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Упаковано хорошо, сам заменитель сладкий без постороннего привкуса, понравился",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T10:02:01Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 709
    },
    {
        "id": "IUpVGYcB6z8QKrUHRS95",
        "globalUserId": "39506896",
        "wbUserId": 39506896,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Все отлично! Купила 90 пакетиков по очень выгодной цене. Хватит надолго!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T15:13:51Z",
        "updatedDate": "2023-08-16T10:09:47.193940039Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:58:37Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/133d0871-55a2-47df-a811-c25b9f6f7287_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/133d0871-55a2-47df-a811-c25b9f6f7287_ms.jpg"
            }
        ],
        "photo": [
            93036632
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 708
    },
    {
        "id": "r1fjCYcBIq9DQExlFfPc",
        "globalUserId": "54659682",
        "wbUserId": 54659682,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Отличный товар, быстро пришло, огромное спасибо за подарок, очень приятно)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-22T15:14:50Z",
        "updatedDate": "2023-08-16T10:09:47.063549253Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T05:45:25Z",
            "metadata": null,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7a36c4a3-e551-47d4-b993-2ab2ea9c1d3f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7a36c4a3-e551-47d4-b993-2ab2ea9c1d3f_ms.jpg"
            }
        ],
        "photo": [
            92504764
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 707
    },
    {
        "id": "0ar4vooBXuPSsPE1vhIG",
        "globalUserId": "18842728",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Дмитрий",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "полное отсутствие контроля качества продукции - раздрожает разная сладость саше от пачки к пачке. иногда мало 1 саше на кружку, иногда аж притворно. форма саше неудобная.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T22:18:07Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 706
    },
    {
        "id": "tNBHSooBwM8MKVoumuKO",
        "globalUserId": "108739987",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Галина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все отлично, кто хочет заменить сахар то только его рекомендую, остальное как то таблетками отдает",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T06:28:41Z",
        "updatedDate": "2023-09-06T08:41:02Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:41:02Z",
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 705
    },
    {
        "id": "8-Z8dYcBUdF-4nlz77Xf",
        "globalUserId": "16649893",
        "wbUserId": 16649893,
        "wbUserDetails": {
            "name": "Серафима",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Люблю этот сах.зам. Очень приятно за небольшой сюрприз ☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-12T12:42:15Z",
        "updatedDate": "2023-08-16T10:09:48.035262727Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T13:19:41Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3605407c-235c-46ad-b3c0-6f5af0fc0b70_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3605407c-235c-46ad-b3c0-6f5af0fc0b70_ms.jpg"
            }
        ],
        "photo": [
            96052795
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 704
    },
    {
        "id": "4cIC9IYBTZEOO9f59tyd",
        "globalUserId": "7548076",
        "wbUserId": 7548076,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ольга"
        },
        "nmId": 31468103,
        "text": "Спасибо большое продавцу, положили подарок, так мило 🥹 доставка очень быстрая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-18T09:18:00Z",
        "updatedDate": "2023-08-16T10:09:46.871306477Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:11:29Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/15de206b-ee11-4349-a881-a7daf504d996_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/15de206b-ee11-4349-a881-a7daf504d996_ms.jpg"
            }
        ],
        "photo": [
            91710964
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 703
    },
    {
        "id": "TbhzsIUBtPujtG5sFBjF",
        "globalUserId": "51627791",
        "wbUserId": 51627791,
        "wbUserDetails": {
            "name": "Сашок",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Я бы поставила 100★, если бы это было возможно.. Прекрасная замена сахару!! На вкус просто шикарно, ещё и кк нет, мечта. Обязательно закажу ещё:)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-14T13:23:30Z",
        "updatedDate": "2023-08-16T10:09:44.735938955Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-26T07:54:33Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "92290869",
                "wbUserId": 92290869,
                "helpfulness": "plus",
                "time": "2023-01-20T19:31:24Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/4981b7b8-5a3f-4b18-a1a9-64f14b7db5b0_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/4981b7b8-5a3f-4b18-a1a9-64f14b7db5b0_ms.jpg"
            }
        ],
        "photo": [
            78939546
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 702
    },
    {
        "id": "_VgAPYcB6z8QKrUHPhO6",
        "globalUserId": "16257444",
        "wbUserId": 16257444,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 31468103,
        "text": "Беру постоянно,нравится!Упаковано в коробку блоком из трех штук.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-01T13:27:19Z",
        "updatedDate": "2023-08-16T10:09:47.530637678Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T11:54:22Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/96e117a9-a8f2-43a4-b844-dc4d95997c90_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/96e117a9-a8f2-43a4-b844-dc4d95997c90_ms.jpg"
            }
        ],
        "photo": [
            94235250
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 701
    },
    {
        "id": "VfEUXoUBJ99b87NGR2Ik",
        "globalUserId": "",
        "wbUserId": 85851405,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ангелина"
        },
        "nmId": 33478818,
        "text": "Был хорошо упакован в коробочку, пришёл без всяких повреждений, в деле не пробовала, но знаю, что он очень хороший будет, смотря на состав. Думаю, буду заказывать ещё",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-29T13:31:05Z",
        "updatedDate": "2023-08-16T10:09:44.450117925Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-11T06:26:06Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 33872616,
                "helpfulness": "plus",
                "time": "2022-12-30T21:30:56Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/e0192cac-3848-4d8f-b275-1dd1055d336f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e0192cac-3848-4d8f-b275-1dd1055d336f_ms.jpg"
            }
        ],
        "photo": [
            76500351
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 700
    },
    {
        "id": "7Mq7MIkBU2re2XP7QrP9",
        "globalUserId": "87573871",
        "wbUserId": 87573871,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "всё супер,спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-07T14:22:06Z",
        "updatedDate": "2023-08-16T10:09:50.733650354Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T11:01:06Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Добрый день, спасибо что уделили время и написали отзыв."
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4e0e9412-b54b-4459-be75-9ac5e36c776c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4e0e9412-b54b-4459-be75-9ac5e36c776c_ms.jpg"
            }
        ],
        "photo": [
            110631330
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 699
    },
    {
        "id": "8D7p6YYBT-IFIyX_wlGV",
        "globalUserId": "8516189",
        "wbUserId": 8516189,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Валерия"
        },
        "nmId": 138785478,
        "text": "Все хорошо. В подарок были 3 саше с ароматизатором чёрной смородины 🥰спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T10:14:16Z",
        "updatedDate": "2023-08-16T10:09:46.760478493Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-16T10:51:29Z",
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c7a1cc20-14f2-40dc-ace8-715deea8fa54_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c7a1cc20-14f2-40dc-ace8-715deea8fa54_ms.jpg"
            }
        ],
        "photo": [
            91355372
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 698
    },
    {
        "id": "VHK7RIkBCaW1RxrSrmIi",
        "globalUserId": "23975452",
        "wbUserId": 23975452,
        "wbUserDetails": {
            "name": "Екатерина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "коробка вся помята",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-11T11:34:58Z",
        "updatedDate": "2023-08-16T10:09:50.901193565Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-11T12:00:16Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a061c444-6836-4a5f-bcfd-07bbf9e55c6b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a061c444-6836-4a5f-bcfd-07bbf9e55c6b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d98c5a8e-dedd-4243-a14a-8c41e91d4ff7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d98c5a8e-dedd-4243-a14a-8c41e91d4ff7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/29c4f36b-59d2-4fb4-84c2-7e87a8316d0b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/29c4f36b-59d2-4fb4-84c2-7e87a8316d0b_ms.jpg"
            }
        ],
        "photo": [
            111306853,
            111306856,
            111306862
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 697
    },
    {
        "id": "wP9s84UBc4LImbbGXV5k",
        "globalUserId": "54796415",
        "wbUserId": 54796415,
        "wbUserDetails": {
            "country": "RU",
            "name": "Людмила",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Очень приятный на вкус, много пакетиков.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-27T13:30:43Z",
        "updatedDate": "2023-08-16T10:09:45.112651573Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:03:42Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d08c69d3-c12a-47d4-a5a7-08acb3130180_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d08c69d3-c12a-47d4-a5a7-08acb3130180_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/aa7b31f9-4731-4d44-b0e8-3f9a2728f289_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/aa7b31f9-4731-4d44-b0e8-3f9a2728f289_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8932c7ca-3a9c-4986-94f9-94ed6b8d9df6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8932c7ca-3a9c-4986-94f9-94ed6b8d9df6_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c82f87cd-9245-4cef-8ba4-1b586633cf35_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c82f87cd-9245-4cef-8ba4-1b586633cf35_ms.jpg"
            }
        ],
        "photo": [
            80995690,
            80995691,
            80995693,
            80995694
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 696
    },
    {
        "id": "m6rRg4oBnnhI58-2Q9FP",
        "globalUserId": "9017329",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришло быстро. Результат 0 ккал. Удобно добавлять в стряпню , чай кофе и т.п.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T10:37:24Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 695
    },
    {
        "id": "Z7a4mIkBBgFQoSo0LZIP",
        "globalUserId": "52959963",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ирина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "всё отлично!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T18:59:14Z",
        "updatedDate": "2023-08-16T10:09:51.601127783Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T10:25:44Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/420440ab-f91d-497b-89c7-fba8a8d8345c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/420440ab-f91d-497b-89c7-fba8a8d8345c_ms.jpg"
            }
        ],
        "photo": [
            114487216
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 694
    },
    {
        "id": "j_MxjYoBUdF-4nlzc8RA",
        "globalUserId": "58267406",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Надежда",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Выписываю уже не первый раз. Очень нравится. Хватает одного саше на чашку.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T06:19:03Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 693
    },
    {
        "id": "lmvfqIoBUYwa0QEf8jdD",
        "globalUserId": "32899352",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Галина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный постоянно беру в место сахара ,нет пивкуса , рекомендую .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T15:19:23Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 692
    },
    {
        "id": "hVvETIcBfnpWqlBeBpd1",
        "globalUserId": "33818956",
        "wbUserId": 33818956,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Пришло всё целое, спасибо за подарок",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-04T14:55:28Z",
        "updatedDate": "2023-08-16T10:09:47.701149331Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T11:22:53Z",
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5f38f7d4-8685-49aa-bba3-47a3755e6969_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5f38f7d4-8685-49aa-bba3-47a3755e6969_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4075ded7-f4f0-4cba-8c7b-b773169b382e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4075ded7-f4f0-4cba-8c7b-b773169b382e_ms.jpg"
            }
        ],
        "photo": [
            94767489,
            94767492
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 691
    },
    {
        "id": "4EBUP4oBUYwa0QEfIOr5",
        "globalUserId": "26382720",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ксения",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Рекомендовала диетолог. Отличный вкус, очень сладко ) можно один пакетик на пару раз растянуть )",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-29T03:26:33Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 690
    },
    {
        "id": "SWpi4ogBCaW1RxrSJw62",
        "globalUserId": "25476019",
        "wbUserId": 25476019,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Сусана",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Всё отлично упаковано",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-22T09:14:23Z",
        "updatedDate": "2023-08-16T10:09:50.221690972Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-22T13:16:28Z",
            "createDate": "2023-06-22T13:16:28Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/431f6069-7c3f-4327-ada5-d4f909558fab_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/431f6069-7c3f-4327-ada5-d4f909558fab_ms.jpg"
            }
        ],
        "photo": [
            107904511
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 689
    },
    {
        "id": "PFxeh4oBUYwa0QEf6obN",
        "globalUserId": "44476839",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хороший сахарозаменитель, беру не первый раз. Нет послевкусия, рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T03:10:59Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 688
    },
    {
        "id": "lPIc94EBT-hvrnNLqMmz",
        "globalUserId": "",
        "wbUserId": 29277307,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Лучший полезный сахарозаменитель. \nОдин пакетик по ощущениям заменяет 2 чайные ложки сахара. \nПриятный сладкий вкус, без открытой приторности в отличие от чистой стевии. \nДелаю с ним запеканки, сырники. Когда особо грустно, могу добавить в чай) Аппетит не повышает. \nРекомендую всем, кто следит за молодостью кожи и питанием!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-13T10:25:14Z",
        "updatedDate": "2023-08-16T10:09:41.369600534Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-07-13T10:49:58Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "text": "Благодарим вас за рекомендацию нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 15244338,
                "helpfulness": "plus",
                "time": "2022-07-13T16:48:26Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 46634894,
                "helpfulness": "plus",
                "time": "2022-07-18T10:17:36Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/b68179a7-44c6-4b74-869d-87311cdc4537_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b68179a7-44c6-4b74-869d-87311cdc4537_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/bdcee27e-ee8b-406c-8494-870e08ad9979_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/bdcee27e-ee8b-406c-8494-870e08ad9979_ms.jpg"
            }
        ],
        "photo": [
            60062905,
            60062921
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 687
    },
    {
        "id": "s-Ydt4UBk9xzjhYZEDvj",
        "globalUserId": "30390280",
        "wbUserId": 30390280,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный вкус, без привкуса и удобная порция. Закалываю не первый раз! Все супер.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-15T20:26:54Z",
        "updatedDate": "2023-08-16T10:09:44.764896651Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-18T10:11:38Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8ebe1912-be9c-4934-93bc-3636c186bea1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8ebe1912-be9c-4934-93bc-3636c186bea1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/74d6a13f-32bd-44e4-9275-160a7b2d8a9c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/74d6a13f-32bd-44e4-9275-160a7b2d8a9c_ms.jpg"
            }
        ],
        "photo": [
            79195684,
            79195685
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 686
    },
    {
        "id": "OvVTS4oBl47NwnZ1SBOm",
        "globalUserId": "12864254",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Все отлично! Заказываю уже третий раз, все приходит без нареканий. Рекомендую определенно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T11:21:04Z",
        "updatedDate": "2023-09-06T08:35:24Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:35:24Z",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 685
    },
    {
        "id": "j3VOGIcBMJfQQSHDvKqJ",
        "globalUserId": "27830549",
        "wbUserId": 27830549,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Спасибо продавцу! Всё пришло целое и в каждой пачке лежал джем👍👌🔥",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T10:27:06Z",
        "updatedDate": "2023-08-16T10:09:47.181752761Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T09:53:11Z",
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0bfc4aa3-c98a-487c-ba12-b71bd5170bae_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0bfc4aa3-c98a-487c-ba12-b71bd5170bae_ms.jpg"
            }
        ],
        "photo": [
            92977855
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 684
    },
    {
        "id": "uvGwBIgBCaW1RxrS4ZPh",
        "globalUserId": "86878909",
        "wbUserId": 86878909,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Пришло в коробке упаковочной, в срок.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-10T08:04:41Z",
        "updatedDate": "2023-08-16T10:09:48.978011180Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-10T10:51:18Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:43:08Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4a8b5787-0ef2-4e45-9e73-91b638a00185_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4a8b5787-0ef2-4e45-9e73-91b638a00185_ms.jpg"
            }
        ],
        "photo": [
            100697629
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 683
    },
    {
        "id": "i9J7jooBuX2t2Z2-waY1",
        "globalUserId": "9423891",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Покупкой удовлетворена. Мне хватает двух пакетиков на 300 гр. кружку.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T12:19:50Z",
        "updatedDate": "2023-09-13T12:44:12Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "2443293",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-13T12:44:12Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 682
    },
    {
        "id": "jtc9b4oBZx8DR9RA0yr7",
        "globalUserId": "49080212",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Понравился больше, чем 10. Вообще никакого послевкусия. Упаковано все отлично!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T10:43:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 681
    },
    {
        "id": "8jBWZIMBeiioYr22dqHq",
        "globalUserId": "",
        "wbUserId": 83513223,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Заказываю второй раз, нет постороннего вкуса. Количество соответствует. Даставка быстрая, качественная. Цена на 50р дешевле чем в магазинах.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-22T08:35:31Z",
        "updatedDate": "2023-08-16T10:09:42.198071133Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T06:35:29Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/f127d0b6-f2fe-4d7a-9f7b-950d3bf42af2_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f127d0b6-f2fe-4d7a-9f7b-950d3bf42af2_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/94799a35-e0e1-46c2-8983-1b41de73f531_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/94799a35-e0e1-46c2-8983-1b41de73f531_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/ad2ab9a1-8a11-4da9-bd90-08966575f9f1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ad2ab9a1-8a11-4da9-bd90-08966575f9f1_ms.jpg"
            }
        ],
        "photo": [
            75842098,
            75842107,
            75842118
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 680
    },
    {
        "id": "IPEAaoYB6z8QKrUH0uD2",
        "globalUserId": "63060597",
        "wbUserId": 63060597,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Евгения"
        },
        "nmId": 33478818,
        "text": "По сравнению с другими  заменителями вполне приличное послевкусие. Надеюсь к нему привыкну.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-19T14:08:04Z",
        "updatedDate": "2023-08-16T10:09:45.874543831Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-20T10:13:29Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d100b735-4f54-4251-9c29-bc535b2dc683_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d100b735-4f54-4251-9c29-bc535b2dc683_ms.jpg"
            }
        ],
        "photo": [
            86835255
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 679
    },
    {
        "id": "CT2CL4gBBgFQoSo0QOoL",
        "globalUserId": "6705783",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Оксана",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Упаковка супер, доставка быстрая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-18T15:37:25Z",
        "updatedDate": "2023-08-16T10:09:49.239404398Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-19T06:48:14Z",
            "createDate": "2023-05-19T06:48:14Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f1f2b8f2-bc33-4df4-b5dc-d6b5a974d3c3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f1f2b8f2-bc33-4df4-b5dc-d6b5a974d3c3_ms.jpg"
            }
        ],
        "photo": [
            102182079
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 678
    },
    {
        "id": "aFPFbooBUYwa0QEfqwDt",
        "globalUserId": "27441329",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Оксана",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Обожаю, самый приятный. Экономный, мнп одной пачки хватает на 3 кружечки чая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T08:32:43Z",
        "updatedDate": "2023-09-07T10:18:55Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-07T10:18:55Z",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 677
    },
    {
        "id": "l9O874MBE-Uw035erQa3",
        "globalUserId": "",
        "wbUserId": 9949183,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Галина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Пришло вовремя. Всё целое, упаковано в несколько слоёв пупырки. Товар свежий (июнь 2022), плюс два года гарантии! Всё отлично! Спасибо производителю и Вайлдбериз👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-19T10:14:23Z",
        "updatedDate": "2023-08-16T10:09:42.730142197Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-19T10:15:19Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/16c1fee1-f34a-4bda-96ba-aa1a677780f2_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/16c1fee1-f34a-4bda-96ba-aa1a677780f2_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/e382d6b7-e45b-4ae5-9b8c-9fe5b1ed13b7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e382d6b7-e45b-4ae5-9b8c-9fe5b1ed13b7_ms.jpg"
            }
        ],
        "photo": [
            6731249,
            6731250
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 676
    },
    {
        "id": "j4MyZ4cB8a_hUs_oOXNO",
        "globalUserId": "42028608",
        "wbUserId": 42028608,
        "wbUserDetails": {
            "name": "Назият",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный заменитель сахара без посторонних вкусов 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-09T18:05:57Z",
        "updatedDate": "2023-08-16T10:09:47.953420907Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-10T07:03:23Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/32fc7695-71dc-43db-a978-3d78342e9a46_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/32fc7695-71dc-43db-a978-3d78342e9a46_ms.jpg"
            }
        ],
        "photo": [
            95660746
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 675
    },
    {
        "id": "EJygQIcB7nVCbHkCQyFK",
        "globalUserId": "16192509",
        "wbUserId": 16192509,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Юлия"
        },
        "nmId": 31468103,
        "text": "Вау Вау! Мой любимый сахзам! Еще и подарочек. Спасибо;))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T06:20:58Z",
        "updatedDate": "2023-08-16T10:09:47.559225854Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T11:00:25Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/91f78e38-7ca1-43b0-8942-8cde14907a1a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/91f78e38-7ca1-43b0-8942-8cde14907a1a_ms.jpg"
            }
        ],
        "photo": [
            94319302
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 674
    },
    {
        "id": "f8lDiYcBKNxVyjcCVjOi",
        "globalUserId": "86603719",
        "wbUserId": 86603719,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Татьяна"
        },
        "nmId": 33478818,
        "text": "Всё отлично!👍Как заявлено. Буду заказывать ещё.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-16T08:51:44Z",
        "updatedDate": "2023-08-16T10:09:48.194417173Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:24:33Z",
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "108039099",
                "wbUserId": 108039099,
                "helpfulness": "minus",
                "time": "2023-04-19T12:16:11Z"
            },
            {
                "globalUserId": "102916631",
                "wbUserId": 102916631,
                "helpfulness": "plus",
                "time": "2023-05-10T20:26:33Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0d49291c-86f2-4781-a85b-e35ccc41f22b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0d49291c-86f2-4781-a85b-e35ccc41f22b_ms.jpg"
            }
        ],
        "photo": [
            96652598
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 1
        },
        "rank": 673
    },
    {
        "id": "5U5DLIcBfnpWqlBeRbYD",
        "globalUserId": "6208588",
        "wbUserId": 6208588,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Спасибо все очень хорошо, за подарок отдельное спасибо 👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T07:26:59Z",
        "updatedDate": "2023-08-16T10:09:47.394090119Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-29T11:55:20Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f0a2f6d8-f7ac-49cb-af84-7f3cbcc73a4f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f0a2f6d8-f7ac-49cb-af84-7f3cbcc73a4f_ms.jpg"
            }
        ],
        "photo": [
            93657162
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 672
    },
    {
        "id": "mMXddYoBKNxVyjcCcvs6",
        "globalUserId": "88129625",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Галина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру постоянно.Послевкусия нет.Иногда выручает когда хочется  сладкое.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T17:36:02Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 671
    },
    {
        "id": "8IMXzoQBTTPU1bwQAwSG",
        "globalUserId": "",
        "wbUserId": 16352802,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Людмила"
        },
        "nmId": 31468103,
        "text": "Быстро, качественно, вкусно и отлично упаковано. Спасибо большое! Пользуюсь давно! Рекомендую! Самый лучший сахзам без искажения вкусовых качеств! Буду заказывать еще однозначно. 🫶🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-01T14:28:45Z",
        "updatedDate": "2023-08-16T10:09:43.777817261Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-02T05:39:30Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ad8ad71e-4198-4782-bb2c-3561da187022_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ad8ad71e-4198-4782-bb2c-3561da187022_ms.jpg"
            }
        ],
        "photo": [
            50580252
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 670
    },
    {
        "id": "f9EVL4gBuQ90tE6DiEHI",
        "globalUserId": "19094769",
        "wbUserId": 19094769,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Елена"
        },
        "nmId": 33478818,
        "text": "Всегда его беру , мне нравится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-18T13:38:40Z",
        "updatedDate": "2023-08-16T10:09:49.236356484Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-19T06:48:54Z",
            "createDate": "2023-05-19T06:48:54Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/be20071f-0af3-400e-a0b2-bc543b292cac_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/be20071f-0af3-400e-a0b2-bc543b292cac_ms.jpg"
            }
        ],
        "photo": [
            102159534
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 669
    },
    {
        "id": "8-BhPYcBTZEOO9f5pKM4",
        "globalUserId": "55204253",
        "wbUserId": 55204253,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Галина",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Для меня это лучший сахарозаменитель. Спасибо за джем.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-01T15:13:42Z",
        "updatedDate": "2023-08-16T10:09:47.533638821Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T11:37:15Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/146e67d5-b55a-493d-9907-543b99c98d9a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/146e67d5-b55a-493d-9907-543b99c98d9a_ms.jpg"
            }
        ],
        "photo": [
            94254979
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 668
    },
    {
        "id": "f-5R1YcBN75J-nvtYtLk",
        "globalUserId": "36717901",
        "wbUserId": 36717901,
        "wbUserDetails": {
            "country": "ru",
            "name": "Лидия",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Покупала 60шт саше-3 коробки, а в моих заказах отображается что 90 Саше 3 коробки. Что то непонятно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-01T03:18:13Z",
        "updatedDate": "2023-08-16T10:09:48.719621407Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-04T07:40:02Z",
            "createDate": "2023-05-04T07:40:02Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Здравствуйте, Вы верно заказали. Происходит замена коробок 60 шт. на 90 шт. Данные изменились во время заказа. В остальном товар остаётся прежним.  "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5f467500-7319-4391-8947-e16d4618ed61_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5f467500-7319-4391-8947-e16d4618ed61_ms.jpg"
            }
        ],
        "photo": [
            99137137
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 667
    },
    {
        "id": "Q6Jnt4oBqVKcxP1mmngu",
        "globalUserId": "21206434",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Екатерина"
        },
        "nmId": 31468103,
        "text": "Самый хороший сахзам. Из всех единственный без привкуса",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T11:02:15Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 666
    },
    {
        "id": "4KjFrooBuWYO9ah2Q5Uc",
        "globalUserId": "92756921",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Поставила в ожидание, сейчас нет в наличии, очень люблю!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T18:47:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 665
    },
    {
        "id": "TBar7oYBMBFuPskoK8OH",
        "globalUserId": "82353284",
        "wbUserId": 82353284,
        "wbUserDetails": {
            "name": "Снежана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Взяла для выпечки и сырников. Все устраивает. Большое спасибо ☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-17T08:24:01Z",
        "updatedDate": "2023-08-16T10:09:46.804583064Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-17T11:18:27Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c29c7e6d-11c7-495d-812c-92303ff5cf08_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c29c7e6d-11c7-495d-812c-92303ff5cf08_ms.jpg"
            }
        ],
        "photo": [
            91520213
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 664
    },
    {
        "id": "ktLVjYoBuX2t2Z2-8Tpa",
        "globalUserId": "24577072",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Очень сладкий, одного пакетика на чашку кофе - много. Делю на 2",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T09:18:43Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 663
    },
    {
        "id": "l-iWFIcBBhsOOdMGQQzS",
        "globalUserId": "54795434",
        "wbUserId": 54795434,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Пришло быстро. Упаковка хорошая.Спасибо большое за подарок",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-24T17:06:44Z",
        "updatedDate": "2023-08-16T10:09:47.146305925Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T10:32:35Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/08501a65-3fef-4379-aa4a-a8f3d0d84c00_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/08501a65-3fef-4379-aa4a-a8f3d0d84c00_ms.jpg"
            }
        ],
        "photo": [
            92882149
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 662
    },
    {
        "id": "f15xJ4cBmBixw9tzUJ7N",
        "globalUserId": "19043695",
        "wbUserId": 19043695,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Юлия"
        },
        "nmId": 31468103,
        "text": "Спасибо продавцу, за милый презент внутри каждой коробки",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-28T08:59:10Z",
        "updatedDate": "2023-08-16T10:09:47.329534938Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T09:11:55Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/00c72eb2-6e68-40d2-a437-2da99fa74231_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/00c72eb2-6e68-40d2-a437-2da99fa74231_ms.jpg"
            }
        ],
        "photo": [
            93496503
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 661
    },
    {
        "id": "-CBot4oBl47NwnZ1BxOk",
        "globalUserId": "28035653",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Евгения"
        },
        "nmId": 138785478,
        "text": "Прям как сахар, только без калорий! Понравилось очень.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T11:02:43Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 660
    },
    {
        "id": "gt7hjocBZx8DR9RAsOzb",
        "globalUserId": "36248784",
        "wbUserId": 36248784,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анастасия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все целое, товар хороший. Спасибо за подарок",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-17T11:02:48Z",
        "updatedDate": "2023-08-16T10:09:48.249041016Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-20T07:46:20Z",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "94999485",
                "wbUserId": 94999485,
                "helpfulness": "plus",
                "time": "2023-04-19T08:16:00Z"
            },
            {
                "globalUserId": "38494908",
                "wbUserId": 38494908,
                "helpfulness": "minus",
                "time": "2023-06-19T14:31:30Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b8f27fa6-3b71-421d-9c42-1d19a1bc7786_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b8f27fa6-3b71-421d-9c42-1d19a1bc7786_ms.jpg"
            }
        ],
        "photo": [
            96836828
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 1
        },
        "rank": 659
    },
    {
        "id": "feVBaYgBuWYO9ah2Qlpa",
        "globalUserId": "55705112",
        "wbUserId": 55705112,
        "wbUserDetails": {
            "name": "Надежда",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Соответствует описанию 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-29T20:44:25Z",
        "updatedDate": "2023-08-16T10:09:49.579576148Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-30T08:31:16Z",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "31689063",
                "wbUserId": 31689063,
                "helpfulness": "minus",
                "time": "2023-05-31T14:15:29Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1776d4a8-3572-4c60-b11e-fd99d3bf5c71_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1776d4a8-3572-4c60-b11e-fd99d3bf5c71_ms.jpg"
            }
        ],
        "photo": [
            103960967
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 658
    },
    {
        "id": "BqkDM4oBCaW1RxrS-3Wn",
        "globalUserId": "114987187",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Заказ пришёл быстро. Аккуратно запакован. Прекрасный заменитель сахара. Пользуйтесь.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-26T18:03:34Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 657
    },
    {
        "id": "H9e8n4oBKNxVyjcCwIwj",
        "globalUserId": "42359091",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Беру этот товар втопой раз, теперь для мамы. Все нравится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T20:44:22Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 656
    },
    {
        "id": "Jrg4NoYBQOOSTxXoXe1H",
        "globalUserId": "26756690",
        "wbUserId": 26756690,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Люблю его ,отличная замена сахара.и спасибо за подарок ,положили 3 саше черная смородина 😍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-09T12:48:29Z",
        "updatedDate": "2023-08-16T10:09:45.482086992Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-14T06:27:00Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "50268419",
                "wbUserId": 50268419,
                "helpfulness": "plus",
                "time": "2023-02-10T13:41:25Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7f4f313a-c8bf-4897-a6ea-99ffd95fd8c3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7f4f313a-c8bf-4897-a6ea-99ffd95fd8c3_ms.jpg"
            }
        ],
        "photo": [
            85114821
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 655
    },
    {
        "id": "Rw6Ch4YBtPujtG5seDva",
        "globalUserId": "48117586",
        "wbUserId": 48117586,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Идеальный подсластитель, положили маленький подарок\nОчень приятно, спасибо 🥰",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-25T07:38:40Z",
        "updatedDate": "2023-08-16T10:09:46.073386101Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-02T08:11:27Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "48117586",
                "wbUserId": 48117586,
                "helpfulness": "plus",
                "time": "2023-03-02T08:20:54Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4b94156a-5b19-4964-9c03-18e60c288eba_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4b94156a-5b19-4964-9c03-18e60c288eba_ms.jpg"
            }
        ],
        "photo": [
            87789918
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 654
    },
    {
        "id": "JKEJ-4YBJ99b87NGCQQ-",
        "globalUserId": "12668612",
        "wbUserId": 12668612,
        "wbUserDetails": {
            "country": "RU",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Спасибо большое, пришло всё целое,в подарок  вот такой джем",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-19T18:01:59Z",
        "updatedDate": "2023-08-16T10:09:46.932330648Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-20T11:53:58Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cc3857a4-7c43-40a8-ac90-8dc5a50ef484_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cc3857a4-7c43-40a8-ac90-8dc5a50ef484_ms.jpg"
            }
        ],
        "photo": [
            92000123
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 653
    },
    {
        "id": "sUmwA4cBT-IFIyX_4tP8",
        "globalUserId": "68594878",
        "wbUserId": 68594878,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Джагарян",
            "country": "AM"
        },
        "nmId": 33478818,
        "text": "Все отлично!!!! Спасибо вам большое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-21T10:22:17Z",
        "updatedDate": "2023-08-16T10:09:46.990205562Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:11:55Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4813f418-561c-432e-8143-3d217a63c730_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4813f418-561c-432e-8143-3d217a63c730_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/ae579ae0-dc99-4c96-a612-125be5ff8ede_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ae579ae0-dc99-4c96-a612-125be5ff8ede_ms.jpg"
            }
        ],
        "photo": [
            92267875,
            92267876
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 652
    },
    {
        "id": "0n2g84YB7nVCbHkCwkhE",
        "globalUserId": "54986583",
        "wbUserId": 54986583,
        "wbUserDetails": {
            "country": "RU",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Всё хорошо упакована, спасибо за бонус. Маленькая приятность",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-18T07:30:44Z",
        "updatedDate": "2023-08-16T10:09:46.863692639Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:16:32Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0e2c18e2-9dfd-4be6-b248-54efa41ca973_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0e2c18e2-9dfd-4be6-b248-54efa41ca973_ms.jpg"
            }
        ],
        "photo": [
            91690592
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 651
    },
    {
        "id": "hwe8NoYBMJfQQSHDe1BH",
        "globalUserId": "99456899",
        "wbUserId": 99456899,
        "wbUserDetails": {
            "country": "RU",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё целое, хорошо упаковано. Отличный заменитель сахара, реально 1 пак и сладенький чай!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-09T15:12:47Z",
        "updatedDate": "2023-08-16T10:09:45.488238761Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-14T06:21:55Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/78c1c26d-024f-4ab4-9584-45bea88adf35_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/78c1c26d-024f-4ab4-9584-45bea88adf35_ms.jpg"
            }
        ],
        "photo": [
            85132679
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 650
    },
    {
        "id": "Jr6qiocBlM6cTg_RVNS7",
        "globalUserId": "27033681",
        "wbUserId": 27033681,
        "wbUserDetails": {
            "name": "Ландыш",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Подружки сказали хороший , будем пробовать",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-16T15:23:51Z",
        "updatedDate": "2023-08-16T10:09:48.211083557Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:04:35Z",
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "108039099",
                "wbUserId": 108039099,
                "helpfulness": "minus",
                "time": "2023-04-19T12:16:08Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5d06994a-b911-4091-94a2-63d77319f36e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5d06994a-b911-4091-94a2-63d77319f36e_ms.jpg"
            }
        ],
        "photo": [
            96722981
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 649
    },
    {
        "id": "AbaFMIYBMBFuPskoUbxb",
        "globalUserId": "43173612",
        "wbUserId": 43173612,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Софья"
        },
        "nmId": 33478818,
        "text": "Хорошо упаковано. Всё в целости и сохранности. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-08T10:14:49Z",
        "updatedDate": "2023-08-16T10:09:45.450065306Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-14T06:43:08Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2c088b36-5fde-4481-bf71-1419b0645c7f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2c088b36-5fde-4481-bf71-1419b0645c7f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/75cf1002-24e9-4955-a0c5-63996bd4eda4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/75cf1002-24e9-4955-a0c5-63996bd4eda4_ms.jpg"
            }
        ],
        "photo": [
            84921301,
            84921302
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 648
    },
    {
        "id": "NorffooBqVKcxP1mNHje",
        "globalUserId": "51385717",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Любовь",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Мне понравился. Чай пьешь как с сахаром. Не горчит. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T11:34:32Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 647
    },
    {
        "id": "WiZ_fooBqdFzcemqwbLo",
        "globalUserId": "60553586",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Эндокринолог дала добро на употребление этого заменителя сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T09:50:17Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 646
    },
    {
        "id": "usF-O4oBAtKOitoB5QcF",
        "globalUserId": "32949339",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Раиса",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хороший сахар, порционные и сладкий. Все пришли упаковано и в целости. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T09:34:47Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 645
    },
    {
        "id": "admon4oBuX2t2Z2-1PLv",
        "globalUserId": "58743403",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Екатерина"
        },
        "nmId": 138785478,
        "text": "Большая упаковка, фасовка по 1 гр, на два раза хватает.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T20:22:37Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 644
    },
    {
        "id": "b_sadIcBG4PPbu1t6c0i",
        "globalUserId": "73385892",
        "wbUserId": 73385892,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Мира"
        },
        "nmId": 33478818,
        "text": "Коробка помятая , остально все соответствует",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-12T06:15:33Z",
        "updatedDate": "2023-08-16T10:09:48.023348349Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T06:17:40Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/118e4c06-88db-4f6b-a880-d99ccac5129e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/118e4c06-88db-4f6b-a880-d99ccac5129e_ms.jpg"
            }
        ],
        "photo": [
            95986563
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 643
    },
    {
        "id": "qLd0T4oBKNxVyjcC9YCV",
        "globalUserId": "57421496",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Хороший сах/зам, буду ещё заказывать. Доставка быстрая. Советую к покупке",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T06:36:20Z",
        "updatedDate": "2023-09-06T08:22:02Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:22:02Z",
            "metadata": null,
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 642
    },
    {
        "id": "89QNPIoBUdF-4nlzW5Ap",
        "globalUserId": "7035095",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "by",
            "name": "Диана",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Я не переношу послевкусие от стевии, в описании написано, что его нет, но я все-равно его очень сильно чувствую, поэтому минус 1 звезду. А так хороший сладкий, пришло все быстро и хорошо упаковано.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T12:10:23Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 641
    },
    {
        "id": "otWIQIoBUdF-4nlzyPIk",
        "globalUserId": "38536041",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Фатима",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Обычно  фитпарад бывает очень мелкого помола, как пыль и когда просто открываешь пакетик во рту сразу чувствуется сладость, а этот как сахар обычный, никакого привкуса, странно, как будто подделка 🤔",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-29T09:03:41Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 640
    },
    {
        "id": "e5jekooBXuPSsPE1BhSx",
        "globalUserId": "36618012",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Анастасия",
            "hasPhoto": false,
            "country": "by"
        },
        "nmId": 138785478,
        "text": "Хороший заменитель, без горчинки. Прост в использовании.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T08:45:39Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 639
    },
    {
        "id": "myXHYn8BpGVGCfJ_rOiw",
        "globalUserId": "",
        "wbUserId": 24638696,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Мария"
        },
        "nmId": 33478818,
        "text": "Регулярно покупаю фитпарад 7, у этого продавца уже второй раз беру и сразу 2 пачки! Качеством очень довольна, а в этот раз внутри коробки меня ждал вот такой сюрпризик-минизефирка низкокалорийная! Очень приятно и вкусно!!! Рекомендую этого продавца!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-07T05:08:46Z",
        "updatedDate": "2023-08-16T10:09:39.658767224Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-09T08:41:33Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 23172247,
                "helpfulness": "plus",
                "time": "2022-03-07T08:00:50Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c34bf2b4-e47d-42f9-8e88-3089ac137676_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c34bf2b4-e47d-42f9-8e88-3089ac137676_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/94807d0f-163f-4cf0-84f8-88fefdad8ddc_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/94807d0f-163f-4cf0-84f8-88fefdad8ddc_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/bc8981ed-0fe0-4bed-ad85-8a31d3cc1688_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/bc8981ed-0fe0-4bed-ad85-8a31d3cc1688_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/59b4f35c-6698-445a-b461-7f41c4cadb26_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/59b4f35c-6698-445a-b461-7f41c4cadb26_ms.jpg"
            }
        ],
        "photo": [
            49384992,
            49385010,
            49385027,
            49385046
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 638
    },
    {
        "id": "Ws90mYcBKNxVyjcCEKB6",
        "globalUserId": "45485406",
        "wbUserId": 45485406,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Екатерина"
        },
        "nmId": 33478818,
        "text": "Все в хорошем состоянии",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-19T12:18:53Z",
        "updatedDate": "2023-08-16T10:09:48.324429548Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-19T12:57:31Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "45485406",
                "wbUserId": 45485406,
                "helpfulness": "plus",
                "time": "2023-04-19T14:42:18Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1cb24806-e400-43e5-b17e-42f7b7819382_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1cb24806-e400-43e5-b17e-42f7b7819382_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a98c70ea-fdec-4a46-be42-1fa8efc8aed5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a98c70ea-fdec-4a46-be42-1fa8efc8aed5_ms.jpg"
            }
        ],
        "photo": [
            97200810,
            97200811
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 637
    },
    {
        "id": "moC2VooBXuPSsPE1OUXx",
        "globalUserId": "59006912",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Лилия",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Хороший, не горчит. Вместо 100 г сахара кладу в выпечку 8 пакетиков.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T16:24:58Z",
        "updatedDate": "2023-09-05T08:16:14Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T08:16:14Z",
            "editable": true,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "118332889",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-09-02T19:10:34Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 636
    },
    {
        "id": "F7NMj4MBE-Uw035ehdXg",
        "globalUserId": "",
        "wbUserId": 29194209,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Маргарита",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Плохо упакован, весь измят, неприятно. Упаковывайте надежнее. Ставлю 5 звёзд ,что пострадала только коробка.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-30T16:48:20Z",
        "updatedDate": "2023-08-16T10:09:42.344658988Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-03T12:46:46Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9b43f0ce-15ce-4eef-8518-9f491bbd96e7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9b43f0ce-15ce-4eef-8518-9f491bbd96e7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4eda2720-ea41-48c4-9c5d-15e0b5f5f35f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4eda2720-ea41-48c4-9c5d-15e0b5f5f35f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/5ff59d52-a569-4213-af40-4fa4f4569905_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5ff59d52-a569-4213-af40-4fa4f4569905_ms.jpg"
            }
        ],
        "photo": [
            49479106,
            49479122,
            49479139
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 635
    },
    {
        "id": "Sgb4hIcBl47NwnZ1tPAM",
        "globalUserId": "9970641",
        "wbUserId": 9970641,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "ольга"
        },
        "nmId": 138785478,
        "text": "Пользуюсь давно. Дешевле чем в магазине!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-15T12:51:44Z",
        "updatedDate": "2023-08-16T10:09:48.167661400Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T08:32:22Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/95648263-fa96-41ec-a6d7-28b10ca541e3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/95648263-fa96-41ec-a6d7-28b10ca541e3_ms.jpg"
            }
        ],
        "photo": [
            96543352
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 634
    },
    {
        "id": "U6v4tooBuWYO9ah2eZOF",
        "globalUserId": "16930148",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Светлана"
        },
        "nmId": 31468103,
        "text": "Недорого, упаковано хорошо. Недорого.рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T09:00:52Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 633
    },
    {
        "id": "0e_ycYQBjTOCcEakYHYN",
        "globalUserId": "",
        "wbUserId": 46370562,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Доставка быстрая! Хорошая упаковка! Большой  объём. Хватит надолго 1010 саше! Посчитали просто из интереса 😄",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-13T17:03:40Z",
        "updatedDate": "2023-08-16T10:09:43.301322049Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-14T08:40:15Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/84f4ba60-dda2-4dd2-8024-a72d2555d934_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/84f4ba60-dda2-4dd2-8024-a72d2555d934_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/ac77f7e1-8c7d-408a-b9fe-a4fb3c5896a2_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/ac77f7e1-8c7d-408a-b9fe-a4fb3c5896a2_ms.jpg"
            }
        ],
        "photo": [
            34032336,
            34032340
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 632
    },
    {
        "id": "fhp_A4kBCYLDkq7NUW-Y",
        "globalUserId": "44332959",
        "wbUserId": 44332959,
        "wbUserDetails": {
            "name": "Алексей",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-28T19:33:43Z",
        "updatedDate": "2023-08-16T10:09:50.447621316Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T05:13:04Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c533328e-6b32-48fa-a79f-910128ad5452_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c533328e-6b32-48fa-a79f-910128ad5452_ms.jpg"
            }
        ],
        "photo": [
            109075431
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 631
    },
    {
        "id": "b9duQIcBheedNkNutYVb",
        "globalUserId": "55283011",
        "wbUserId": 55283011,
        "wbUserDetails": {
            "name": "Яна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо за подарок! Обязательно еще раз закажу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T05:26:50Z",
        "updatedDate": "2023-08-16T10:09:47.554773945Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T11:00:50Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/da7bd254-917f-43b7-bc7a-97fcfd0864e2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/da7bd254-917f-43b7-bc7a-97fcfd0864e2_ms.jpg"
            }
        ],
        "photo": [
            94314706
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 630
    },
    {
        "id": "z91Oy4oBlM6cTg_R5NDA",
        "globalUserId": "31384573",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "Карина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Понравился, нет посторонних привкусом и горечи, один минус что в супермаркетах такой-же заменитель на 40% дешевле",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T07:47:40Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 629
    },
    {
        "id": "6BHZUYIBwEHULCRVf8cu",
        "globalUserId": "",
        "wbUserId": 15514014,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 33478818,
        "text": "Всё хорошо пришёл в пленке немного помятый но это пофиг срок годности в норме в упаковке 60 штучек пересчитала как и заявлено на кружку 300 мл 2 пакетика сыплю и идеальная сладось для меня вкус приятный близок к сахару я люблю сладкий чай поэтому это идеальная замена сахару буду заказывать ещё думаю на месяц хватит но не факт 😅 спасибо продавцу🥰❤️ и вайлдберриз🔥",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-31T01:23:04Z",
        "updatedDate": "2023-08-16T10:09:41.545172035Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-03T10:11:55Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/faf87569-2f5e-4bf4-849e-2eaa21002ffd_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/faf87569-2f5e-4bf4-849e-2eaa21002ffd_ms.jpg"
            }
        ],
        "photo": [
            53091680
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 628
    },
    {
        "id": "19hZYYMBI2nds0aGOQ5-",
        "globalUserId": "",
        "wbUserId": 21709564,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Надежда"
        },
        "nmId": 33478818,
        "text": "Очень хороший сахарозаменитель. Для меня очень сладкий. Доставка быстрая и всё цело. Пришло упаковано в пакет.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-21T18:39:40Z",
        "updatedDate": "2023-08-16T10:09:42.193465970Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T06:44:32Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/e21c804a-32d3-4d77-8b09-a9a2381f7683_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e21c804a-32d3-4d77-8b09-a9a2381f7683_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/83426c09-dc1c-4d6a-9c5d-40999aecc160_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/83426c09-dc1c-4d6a-9c5d-40999aecc160_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/b26cae47-f448-440f-b525-02590177f998_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b26cae47-f448-440f-b525-02590177f998_ms.jpg"
            }
        ],
        "photo": [
            68298624,
            68298640,
            68298662
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 627
    },
    {
        "id": "bSYSjooBU2re2XP7qSEP",
        "globalUserId": "52685048",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Виктория",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Люблю семерочку,да и десяткой довольна. Беру постоянно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T10:25:02Z",
        "updatedDate": "2023-09-13T12:44:21Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "2443293",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-13T12:44:21Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 626
    },
    {
        "id": "4PZX_4YB5HDhYKpH6UDy",
        "globalUserId": "23411525",
        "wbUserId": 23411525,
        "wbUserDetails": {
            "name": "Елена",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Все пришло впорядке еще и плюсом подарочек спасибо 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-20T14:06:37Z",
        "updatedDate": "2023-08-16T10:09:46.955166862Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:29:29Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d0e1327d-7a5c-4636-a055-b97440308d1c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d0e1327d-7a5c-4636-a055-b97440308d1c_ms.jpg"
            }
        ],
        "photo": [
            92131522
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 625
    },
    {
        "id": "CWTEpYgBBgFQoSo0DrZv",
        "globalUserId": "28096191",
        "wbUserId": 28096191,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Мила"
        },
        "nmId": 138785478,
        "text": "Хороший заменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-10T14:44:29Z",
        "updatedDate": "2023-08-16T10:09:49.900656917Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T09:02:30Z",
            "createDate": "2023-06-20T09:02:30Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/53e48c00-ccc4-403a-a5cb-7992c43b241f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/53e48c00-ccc4-403a-a5cb-7992c43b241f_ms.jpg"
            }
        ],
        "photo": [
            105856947
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 624
    },
    {
        "id": "LSulmYoBU2re2XP7NlX8",
        "globalUserId": "49713070",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 31468103,
        "text": "Хороший сахарозаменитель. Вкус напитков не меняет.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T16:20:56Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 623
    },
    {
        "id": "CWlVsocBUYwa0QEf1llg",
        "globalUserId": "48229845",
        "wbUserId": 48229845,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Лейсан"
        },
        "nmId": 33478818,
        "text": "Попробовала, вкусно \nСроки хорошие",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-24T08:16:23Z",
        "updatedDate": "2023-08-16T10:09:48.495925890Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-24T08:54:33Z",
            "createDate": "2023-04-24T08:54:33Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/18a7e5c8-ab54-489e-988b-d58fdcc17067_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/18a7e5c8-ab54-489e-988b-d58fdcc17067_ms.jpg"
            }
        ],
        "photo": [
            97997768
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 622
    },
    {
        "id": "WoClC4YB5HDhYKpH8GVe",
        "globalUserId": "83046960",
        "wbUserId": 83046960,
        "wbUserDetails": {
            "name": "Варвара",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "не в первый раз беру сахзам, но очень довольна. Ничего не помято, приходит быстро ;₽",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-01T06:24:30Z",
        "updatedDate": "2023-08-16T10:09:45.240653913Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-01T06:46:57Z",
            "metadata": null,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "7255443",
                "wbUserId": 7255443,
                "helpfulness": "plus",
                "time": "2023-02-04T11:15:34Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/89a00e4f-ac3b-4068-8495-360eda39f555_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/89a00e4f-ac3b-4068-8495-360eda39f555_ms.jpg"
            }
        ],
        "photo": [
            81744672
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 621
    },
    {
        "id": "CUmKGIcB6z8QKrUHfM6j",
        "globalUserId": "25594931",
        "wbUserId": 25594931,
        "wbUserDetails": {
            "name": "Вера",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Отличный заменитель сахара, спасибо за подарок 🤩",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T11:32:22Z",
        "updatedDate": "2023-08-16T10:09:47.186329266Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T09:50:11Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4d40c602-de8a-4894-8c1a-3749d0b8bad1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4d40c602-de8a-4894-8c1a-3749d0b8bad1_ms.jpg"
            }
        ],
        "photo": [
            92991914
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 620
    },
    {
        "id": "4kSe6YYBtPujtG5sWbF4",
        "globalUserId": "4383325",
        "wbUserId": 4383325,
        "wbUserDetails": {
            "name": "Залина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Всё пришло целое, свежее😁 Спасибо за подарочки джем👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-16T08:51:54Z",
        "updatedDate": "2023-08-16T10:09:46.749412429Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-16T11:28:58Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54209907",
                "wbUserId": 54209907,
                "helpfulness": "plus",
                "time": "2023-03-16T09:28:54Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3e3d0bfb-e389-437a-b676-97d8819e7c00_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3e3d0bfb-e389-437a-b676-97d8819e7c00_ms.jpg"
            }
        ],
        "photo": [
            91339164
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 619
    },
    {
        "id": "6QuBxooBUdF-4nlzHinR",
        "globalUserId": "67567161",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алексей",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличная замена сахара, кто хочет похудеть",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T09:24:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 618
    },
    {
        "id": "XFvTgYoBpT2rITcGaNIO",
        "globalUserId": "59447039",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Классный сахарозаменитель!!! Всегда его заказываем!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T01:20:30Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 617
    },
    {
        "id": "NjNL84MBeiioYr22teMv",
        "globalUserId": "",
        "wbUserId": 34504118,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Все отлично! Беру не в первый раз! Не пересчитывала конечно, но на вид даже больше наверно. Хватает на долго. Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-20T02:49:28Z",
        "updatedDate": "2023-08-16T10:09:42.746760004Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-20T07:04:58Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/c7ccc812-6485-4e64-93bd-acc196eda375_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/c7ccc812-6485-4e64-93bd-acc196eda375_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/45a5c286-936d-4cc9-b7f8-ff5b39e47677_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/45a5c286-936d-4cc9-b7f8-ff5b39e47677_ms.jpg"
            }
        ],
        "photo": [
            6822246,
            6822247
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 616
    },
    {
        "id": "7r0w_IcBuWYO9ah2b-CP",
        "globalUserId": "16215560",
        "wbUserId": 16215560,
        "wbUserDetails": {
            "country": "ru",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все супер.быстрая доставка.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-08T16:27:25Z",
        "updatedDate": "2023-08-16T10:09:48.937036836Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-17T06:23:48Z",
            "createDate": "2023-05-17T06:23:48Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:43:52Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/57e223c4-60a5-42a7-b31e-175c2ee56451_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/57e223c4-60a5-42a7-b31e-175c2ee56451_ms.jpg"
            }
        ],
        "photo": [
            100468033
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 615
    },
    {
        "id": "tG_Rn4cB3ieoZoqxwk0M",
        "globalUserId": "40487868",
        "wbUserId": 40487868,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Продавец положил приятный бонус 👍🏼",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-20T17:58:57Z",
        "updatedDate": "2023-08-16T10:09:48.385398347Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-21T05:04:44Z",
            "createDate": "2023-04-21T05:04:44Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7be2926e-43c8-4b32-a140-9683ec300023_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7be2926e-43c8-4b32-a140-9683ec300023_ms.jpg"
            }
        ],
        "photo": [
            97442057
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 614
    },
    {
        "id": "X3ex-IYBk9xzjhYZSxTS",
        "globalUserId": "98009051",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "by",
            "name": "Элла"
        },
        "nmId": 31468103,
        "text": "Сюрприз внутри просто покорил моё сердце. СПАСИБО!",
        "pros": "",
        "cons": "",
        "matchingSize": "ok",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-19T07:06:54Z",
        "updatedDate": "2023-08-16T10:09:46.895702057Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:34:43Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "98009051",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-03-19T13:42:17Z"
            },
            {
                "globalUserId": "59731248",
                "wbUserId": 59731248,
                "helpfulness": "plus",
                "time": "2023-03-21T07:31:08Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b709aa25-8c1a-4fc5-a71b-405cc70a4999_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b709aa25-8c1a-4fc5-a71b-405cc70a4999_ms.jpg"
            }
        ],
        "photo": [
            91862355
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 613
    },
    {
        "id": "7CCUbooBqdFzcemqrN2b",
        "globalUserId": "73991697",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Светлана"
        },
        "nmId": 138785478,
        "text": "Отличный товар. На мой взгляд, лучший сахапрозаменитель.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T07:39:12Z",
        "updatedDate": "2023-09-07T09:51:53Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-07T09:51:53Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 612
    },
    {
        "id": "zszueIoBqd2abVcyKDmI",
        "globalUserId": "36255132",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Марина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Постоянно Беру только этот продукт, очень качественный",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T07:53:08Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 611
    },
    {
        "id": "A1WOQIkBnnhI58-2-3vV",
        "globalUserId": "94324088",
        "wbUserId": 94324088,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": ""
        },
        "nmId": 31468103,
        "text": "👍🏻👍🏻👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-10T16:07:40Z",
        "updatedDate": "2023-08-16T10:09:50.868118071Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:43:31Z",
            "employeeId": 0,
            "text": "Добрый день! Спасибо за высокую оценку. Оставайтесь с нами :)",
            "metadata": null,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6fbc1cc7-d249-4db0-95aa-6628640ffb33_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6fbc1cc7-d249-4db0-95aa-6628640ffb33_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/88f0f9d1-c3ad-4ce8-a8c9-b0ab4e7b8cd5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/88f0f9d1-c3ad-4ce8-a8c9-b0ab4e7b8cd5_ms.jpg"
            }
        ],
        "photo": [
            111173332,
            111173337
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 610
    },
    {
        "id": "62FmjYoBpT2rITcGwycs",
        "globalUserId": "26010147",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Вагаб",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Бегу регулярно и на вб и в магазинах все одинаково",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T07:17:17Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 609
    },
    {
        "id": "ZIfof4cBqVKcxP1mjI7t",
        "globalUserId": "39160106",
        "wbUserId": 39160106,
        "wbUserDetails": {
            "name": "Людмила",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Как всегда 👍 заказываю не первый раз",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-14T13:16:00Z",
        "updatedDate": "2023-08-16T10:09:48.128609240Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-17T10:01:09Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3365a76f-2a4e-4bee-bab5-beafeeb74a6b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3365a76f-2a4e-4bee-bab5-beafeeb74a6b_ms.jpg"
            }
        ],
        "photo": [
            96374663
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 608
    },
    {
        "id": "L6Jr_YUBQOOSTxXobFEn",
        "globalUserId": "18756414",
        "wbUserId": 18756414,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Всё понравилось\nДоставка быстрая, упаковано хорошо, коробка не помята. Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-29T12:05:54Z",
        "updatedDate": "2023-08-16T10:09:45.169129487Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-30T13:36:19Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "7255443",
                "wbUserId": 7255443,
                "helpfulness": "plus",
                "time": "2023-02-04T11:19:49Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3fbc1578-32e2-4221-be95-cac976df034a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3fbc1578-32e2-4221-be95-cac976df034a_ms.jpg"
            }
        ],
        "photo": [
            81305607
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 607
    },
    {
        "id": "JSheYYMBOCC9xHZP3bzq",
        "globalUserId": "",
        "wbUserId": 39535528,
        "wbUserDetails": {
            "name": "Марина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сколько раз брала, всё время приходил в мятой упаковке. Ну так то на качество не влияет. Сегодня первый раз пришёл в идеальной коробке. Приятно😊",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-21T18:45:50Z",
        "updatedDate": "2023-08-16T10:09:42.194994860Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T06:44:28Z",
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/e53c172c-a303-430d-80a3-64ed740e3a5c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e53c172c-a303-430d-80a3-64ed740e3a5c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/7ec630f3-9b8f-4a13-b83b-de75827682f1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/7ec630f3-9b8f-4a13-b83b-de75827682f1_ms.jpg"
            }
        ],
        "photo": [
            49629684,
            49629699
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 606
    },
    {
        "id": "kcfgTIoBAtKOitoBBv_o",
        "globalUserId": "47202750",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Отличный товар, большое спасибо! Смотри фото, оно того стоит!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T18:34:25Z",
        "updatedDate": "2023-09-06T08:26:31Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:26:31Z",
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 605
    },
    {
        "id": "xZG1HogBpT2rITcGHC1w",
        "globalUserId": "111916274",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Лариса"
        },
        "nmId": 33478818,
        "text": "Спасибо получила товар..",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-15T09:19:26Z",
        "updatedDate": "2023-08-16T10:09:49.133832410Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-16T12:33:19Z",
            "createDate": "2023-05-16T12:33:19Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7b9bf4dc-2b40-4e3c-9910-e70a34d6259e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7b9bf4dc-2b40-4e3c-9910-e70a34d6259e_ms.jpg"
            }
        ],
        "photo": [
            101584201
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 604
    },
    {
        "id": "u711-4cBuWYO9ah25JhB",
        "globalUserId": "48935198",
        "wbUserId": 48935198,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 33478818,
        "text": "Часто беру. Мне нравиться.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-08T13:03:40Z",
        "updatedDate": "2023-08-16T10:09:48.934014483Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-17T06:25:24Z",
            "createDate": "2023-05-17T06:25:24Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:44:00Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e2e00adb-c946-4bea-90c1-ec2e826e9083_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e2e00adb-c946-4bea-90c1-ec2e826e9083_ms.jpg"
            }
        ],
        "photo": [
            100427287
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 603
    },
    {
        "id": "ANdxeYcBZx8DR9RA9pyg",
        "globalUserId": "29717577",
        "wbUserId": 29717577,
        "wbUserDetails": {
            "name": "Анастасия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Быстрая доставка. Упаковано отлично!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-13T07:08:45Z",
        "updatedDate": "2023-08-16T10:09:48.068341920Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-13T08:09:42Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2f27a246-c993-4ba5-9111-b094998386db_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2f27a246-c993-4ba5-9111-b094998386db_ms.jpg"
            }
        ],
        "photo": [
            96164895
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 602
    },
    {
        "id": "cN0dNYcBTZEOO9f5_TBs",
        "globalUserId": "84092401",
        "wbUserId": 84092401,
        "wbUserDetails": {
            "name": "Людмила",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Все отлично все пришло целое как заявлено.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-31T00:42:51Z",
        "updatedDate": "2023-08-16T10:09:47.474032580Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:12:18Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/8e6c1b11-556c-47c7-ac93-1347c9db717e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8e6c1b11-556c-47c7-ac93-1347c9db717e_ms.jpg"
            }
        ],
        "photo": [
            93982544
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 601
    },
    {
        "id": "jmugmYoBfuChzMpw-WP2",
        "globalUserId": "24998942",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Алла",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший подсластитель без неприятного привкуса",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T16:16:18Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 600
    },
    {
        "id": "rDOzeIYBv9_TjX_mvj_6",
        "globalUserId": "37913574",
        "wbUserId": 37913574,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Марина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Срок годности в норме, упаковано было в дополнительную коробку.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-22T10:38:11Z",
        "updatedDate": "2023-08-16T10:09:45.982054836Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-02T12:42:37Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d521c4bd-0d41-493d-9852-5c4fe3909962_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d521c4bd-0d41-493d-9852-5c4fe3909962_ms.jpg"
            }
        ],
        "photo": [
            87318974
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 599
    },
    {
        "id": "foNc_4QB8a_hUs_oIigC",
        "globalUserId": "",
        "wbUserId": 5385556,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Упаковка не повреждена, спасибо за доставку. Отличная альтернатива сахару 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-11T04:05:38Z",
        "updatedDate": "2023-08-16T10:09:44.093607735Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-13T12:10:53Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/48d7caff-adaf-4c35-80be-0f12026f6b36_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/48d7caff-adaf-4c35-80be-0f12026f6b36_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/13c7d6f7-8eee-40ee-a414-c50c63842c46_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/13c7d6f7-8eee-40ee-a414-c50c63842c46_ms.jpg"
            }
        ],
        "photo": [
            61531106,
            61531107
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 598
    },
    {
        "id": "qFq7D4cBIq9DQExlcF8A",
        "globalUserId": "68495760",
        "wbUserId": 68495760,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Светлана",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Все пришло в срок, хорошо упаковано . Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-23T18:29:15Z",
        "updatedDate": "2023-08-16T10:09:47.107105663Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T05:36:33Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2f23cdaa-1559-461e-b2aa-eb6322e25027_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2f23cdaa-1559-461e-b2aa-eb6322e25027_ms.jpg"
            }
        ],
        "photo": [
            92721523
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 597
    },
    {
        "id": "FOg8vIUBk9xzjhYZvgIx",
        "globalUserId": "52513682",
        "wbUserId": 52513682,
        "wbUserDetails": {
            "name": "Кристина",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Все отлично. Упаковали в отдельную коробку. Все целое.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-16T20:19:36Z",
        "updatedDate": "2023-08-16T10:09:44.796744251Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-17T07:07:08Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "66180231",
                "wbUserId": 66180231,
                "helpfulness": "minus",
                "time": "2023-01-29T05:00:47Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d9488f6b-762d-449f-b9af-def1ad108936_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d9488f6b-762d-449f-b9af-def1ad108936_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/3328b7ae-3f6f-484b-9874-5a1bed6de841_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3328b7ae-3f6f-484b-9874-5a1bed6de841_ms.jpg"
            }
        ],
        "photo": [
            79381230,
            79381234
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 596
    },
    {
        "id": "-PFyOIoB0v3RIh2JUzUs",
        "globalUserId": "21726567",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Эльвина",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Заказываю не первый раз,очень нравится с/з. Рекомендую к покупки",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-27T19:22:11Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 595
    },
    {
        "id": "OYzqrYoBmorVKbOQGXJl",
        "globalUserId": "43920615",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Беру регулярно. 👍 Без послевкусия и запаха",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T14:48:35Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 594
    },
    {
        "id": "WrwpZ4UBMJfQQSHDHrGF",
        "globalUserId": "",
        "wbUserId": 21755673,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Александра",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Доставка конечно…судя по комментариям у большинства такая же проблема, упаковывайте хотя в доп. коробку/пакет, так как эта коробка даже без плёнки пришла,в итоге внутри 59/60 саше. Не критично, но не приятно. Отказываться уже не стали, Новый год на носу, нужен был срочно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-31T07:50:26Z",
        "updatedDate": "2023-08-16T10:09:44.474452140Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-11T06:25:59Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за отзыв, конечно же нам важно, чтобы товар доходил в своём лучшем виде до покупателя. Мы думаем над способом, который был бы компактен и существенно не отразился на цене товара. ",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/372e8ef5-85ee-4083-a7cb-2a65961af940_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/372e8ef5-85ee-4083-a7cb-2a65961af940_ms.jpg"
            }
        ],
        "photo": [
            76773697
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 593
    },
    {
        "id": "K-E2t4oBKNxVyjcCzwLG",
        "globalUserId": "34520444",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришел аккуратно упакован,вовремя.Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T10:08:57Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 592
    },
    {
        "id": "v2-FNocB8a_hUs_o04Ve",
        "globalUserId": "14244925",
        "wbUserId": 14244925,
        "wbUserDetails": {
            "name": "Марина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Приятный и очень вкусный бонус ,спасибо 😁",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-31T07:15:53Z",
        "updatedDate": "2023-08-16T10:09:47.484895872Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-31T10:07:25Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14244925",
                "wbUserId": 14244925,
                "helpfulness": "plus",
                "time": "2023-04-01T08:12:43Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/37f0aaa2-aaa8-4f32-9c78-16ff778cb221_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/37f0aaa2-aaa8-4f32-9c78-16ff778cb221_ms.jpg"
            }
        ],
        "photo": [
            94004186
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 591
    },
    {
        "id": "8HLdPIcB8a_hUs_oVmPt",
        "globalUserId": "28973642",
        "wbUserId": 28973642,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Оксана"
        },
        "nmId": 138785478,
        "text": "Пришло все быстро, спасибо за подарочек!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-01T12:49:11Z",
        "updatedDate": "2023-08-16T10:09:47.527631895Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T12:06:59Z",
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9a211e22-f5e3-42ee-83c0-5de03f0114f1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9a211e22-f5e3-42ee-83c0-5de03f0114f1_ms.jpg"
            }
        ],
        "photo": [
            94227796
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 590
    },
    {
        "id": "XFZYeIoBUYwa0QEfeFSf",
        "globalUserId": "6611185",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Марьяна"
        },
        "nmId": 33478818,
        "text": "Отличный заменитель сахара, без неприятной отдушки.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T05:09:39Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 589
    },
    {
        "id": "8SS9v4YBT-IFIyX_XLou",
        "globalUserId": "7361160",
        "wbUserId": 7361160,
        "wbUserDetails": {
            "country": "RU",
            "name": "виктория",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Рекомендую к покупке товаров соответствует описанию !",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-08T05:41:44Z",
        "updatedDate": "2023-08-16T10:09:46.441393102Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-09T08:05:09Z",
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/95b156af-b252-43f5-b967-4f2ed6f9349e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/95b156af-b252-43f5-b967-4f2ed6f9349e_ms.jpg"
            }
        ],
        "photo": [
            89794061
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 588
    },
    {
        "id": "eRDrRIoBqdFzcemqs_o5",
        "globalUserId": "45111851",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "очень мне нравится фит-парад - нет ни послевкусия, ни горечи.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T05:30:12Z",
        "updatedDate": "2023-09-06T08:58:55Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:58:55Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 587
    },
    {
        "id": "xJiVTIcBv9_TjX_mGpcU",
        "globalUserId": "22330704",
        "wbUserId": 22330704,
        "wbUserDetails": {
            "name": "Юлия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличная замена сахару👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-04T14:04:13Z",
        "updatedDate": "2023-08-16T10:09:47.699684565Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-06T11:23:11Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/54e6efc7-b947-4ee3-a4aa-d0b83ac29aca_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/54e6efc7-b947-4ee3-a4aa-d0b83ac29aca_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8e00a3bf-c360-43f5-897c-ba1c7ed13169_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8e00a3bf-c360-43f5-897c-ba1c7ed13169_ms.jpg"
            }
        ],
        "photo": [
            94756704,
            94756705
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 586
    },
    {
        "id": "pTiXuYoBU2re2XP7oUW0",
        "globalUserId": "28098433",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Валентина"
        },
        "nmId": 138785478,
        "text": "Хороший сахзам! Нет привкуса, не горчит!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T21:13:57Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 585
    },
    {
        "id": "49-f_oUB8a_hUs_o1ll4",
        "globalUserId": "22615629",
        "wbUserId": 22615629,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Дильдана"
        },
        "nmId": 31468103,
        "text": "Спасибо, все отлично. То, что надо. Упаковано супер в пупырку. Все ровненько!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-29T17:42:46Z",
        "updatedDate": "2023-08-16T10:09:45.179831654Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-30T12:12:34Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1dcd91a9-6572-4fd0-be8c-ab582ea24b30_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1dcd91a9-6572-4fd0-be8c-ab582ea24b30_ms.jpg"
            }
        ],
        "photo": [
            81373912
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 584
    },
    {
        "id": "7BWDJIQBJjG5B9ArjkOK",
        "globalUserId": "",
        "wbUserId": 33411645,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "Хорошо упакован, спасибо большое продавцу 🔥 покупаю не первый раз, всегда во время, продукт отличный! 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-29T16:11:52Z",
        "updatedDate": "2023-08-16T10:09:42.957066636Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-31T10:10:04Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/d950847e-e660-4b3c-9a48-72a193ef006d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d950847e-e660-4b3c-9a48-72a193ef006d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/e47ab98b-7906-436a-a51b-d6d077f08ca1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e47ab98b-7906-436a-a51b-d6d077f08ca1_ms.jpg"
            }
        ],
        "photo": [
            16464198,
            16464201
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 583
    },
    {
        "id": "K1w8x4YBk9xzjhYZm0OX",
        "globalUserId": "31743449",
        "wbUserId": 31743449,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ксения"
        },
        "nmId": 33478818,
        "text": "Отличная штука, вкус хороший от сахара не отличишь!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-09T16:38:03Z",
        "updatedDate": "2023-08-16T10:09:46.503932901Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T08:38:38Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/036a9a75-dc42-45fe-9e67-d62018cfbc45_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/036a9a75-dc42-45fe-9e67-d62018cfbc45_ms.jpg"
            }
        ],
        "photo": [
            90098115
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 582
    },
    {
        "id": "wUXHx4oBqdFzcemqK0aE",
        "globalUserId": "49461132",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Алла",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Давно использую ФитПарад. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T15:20:33Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 581
    },
    {
        "id": "qOvejYoBwM8MKVouvkCf",
        "globalUserId": "108538674",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ирина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру второй раз, все нормально, товар нравится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T09:28:20Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 580
    },
    {
        "id": "Iu3cqYoBAtKOitoBIAEk",
        "globalUserId": "28703207",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всё ок. Заказываю не первый раз. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T19:54:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 579
    },
    {
        "id": "4uoYp4MBJjG5B9ArBlBk",
        "globalUserId": "",
        "wbUserId": 37771755,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Коробка немного пришла помятая, но это не критично. Самая адекватная цена сах.зам., одного пакетика хватает на кружку чая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-05T07:41:53Z",
        "updatedDate": "2023-08-16T10:09:42.420685489Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T07:43:47Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/6cb4d9c5-24bd-499e-bdcf-a67df85bd442_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6cb4d9c5-24bd-499e-bdcf-a67df85bd442_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/e0dfbfdf-374e-4f44-9535-5c75662c1316_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e0dfbfdf-374e-4f44-9535-5c75662c1316_ms.jpg"
            }
        ],
        "photo": [
            59675358,
            59675374
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 578
    },
    {
        "id": "LuCQRoYBfnpWqlBe24r3",
        "globalUserId": "26712351",
        "wbUserId": 26712351,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Пришло всё целое, хорошо упакован)) спасибо! Буду заказывать ещё!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-12T16:59:04Z",
        "updatedDate": "2023-08-16T10:09:45.611099621Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-13T08:13:54Z",
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1b89aefa-5433-4f74-8099-92d79c250763_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1b89aefa-5433-4f74-8099-92d79c250763_ms.jpg"
            }
        ],
        "photo": [
            85647986
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 577
    },
    {
        "id": "PWB084UBBhsOOdMGF8q4",
        "globalUserId": "43798108",
        "wbUserId": 43798108,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Светлана"
        },
        "nmId": 33478818,
        "text": "Все пришло целое,упаковано в картонную коробку",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-27T13:39:10Z",
        "updatedDate": "2023-08-16T10:09:45.115683094Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:07:30Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/67f38c91-66cc-4beb-9fa8-1d9aa3319d4d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/67f38c91-66cc-4beb-9fa8-1d9aa3319d4d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c6e8ea34-debd-4ac5-977e-5bca54509826_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c6e8ea34-debd-4ac5-977e-5bca54509826_ms.jpg"
            }
        ],
        "photo": [
            80997380,
            80997381
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 576
    },
    {
        "id": "9lREA4cBIq9DQExlSbaT",
        "globalUserId": "14667773",
        "wbUserId": 14667773,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Катрин"
        },
        "nmId": 33478818,
        "text": "Быстрая доставка и отличная упаковка👍спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-21T08:23:40Z",
        "updatedDate": "2023-08-16T10:09:46.976628841Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:15:48Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/dc5d0aad-f491-48fe-925a-1a048570ec82_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dc5d0aad-f491-48fe-925a-1a048570ec82_ms.jpg"
            }
        ],
        "photo": [
            92246085
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 575
    },
    {
        "id": "Z-KrxYoBCaW1RxrSLWYY",
        "globalUserId": "51039921",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный продукт, все пришло вовремя.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T05:30:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 574
    },
    {
        "id": "ViNG1IQBMx-JN032OTxN",
        "globalUserId": "",
        "wbUserId": 1236146,
        "wbUserDetails": {
            "name": "Жанна",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Давно покупаю этот заменитель , для выпечки и для еды , нет постороннего вкуса , для меня это самый лучший заменитель сахара .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-02T19:18:02Z",
        "updatedDate": "2023-08-16T10:09:43.831157237Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-05T07:45:57Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/91877c74-75b4-441d-b113-21305375eab1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/91877c74-75b4-441d-b113-21305375eab1_ms.jpg"
            }
        ],
        "photo": [
            52299833
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 573
    },
    {
        "id": "kiZCz4YB6z8QKrUHeRsw",
        "globalUserId": "56775316",
        "wbUserId": 56775316,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Спасибо большое всё целое 👍🤗 за подарочек спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-11T06:01:26Z",
        "updatedDate": "2023-08-16T10:09:46.545014Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T08:14:06Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "63723358",
                "wbUserId": 63723358,
                "helpfulness": "minus",
                "time": "2023-03-12T06:41:05Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3b0590b7-6cb1-44d8-a3d9-4b9db741a83c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3b0590b7-6cb1-44d8-a3d9-4b9db741a83c_ms.jpg"
            }
        ],
        "photo": [
            90346580
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 572
    },
    {
        "id": "f1nQOoIBeiioYr22fn8F",
        "globalUserId": "",
        "wbUserId": 36921220,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Карина",
            "country": "KZ"
        },
        "nmId": 33478818,
        "text": "Первый раз заказала, пришла немного помятая коробка. А так все целое. Как указано 60 пакетиков, все точно. На вкус 1 пакетик в кофе очень сладко мне, использую половину пакетика. Спасибо! ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-26T14:01:58Z",
        "updatedDate": "2023-08-16T10:09:41.505675400Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-09T07:58:19Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/f14ea436-e5dc-4b02-b041-6d198b362efa_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f14ea436-e5dc-4b02-b041-6d198b362efa_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/bceef8fd-f0c4-41ca-afb4-824d19b1c60d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/bceef8fd-f0c4-41ca-afb4-824d19b1c60d_ms.jpg"
            }
        ],
        "photo": [
            52786831,
            52786846
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 571
    },
    {
        "id": "tDxZ24YBtPujtG5sR4eK",
        "globalUserId": "26849179",
        "wbUserId": 26849179,
        "wbUserDetails": {
            "name": "Елена",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё отлично упаковка картонная, большое спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-13T14:21:47Z",
        "updatedDate": "2023-08-16T10:09:46.626406036Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T06:32:08Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cc4b3ef6-6c93-46b5-9cfb-3f242ac4957b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cc4b3ef6-6c93-46b5-9cfb-3f242ac4957b_ms.jpg"
            }
        ],
        "photo": [
            90833846
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 570
    },
    {
        "id": "ZmoZqIoBUYwa0QEfHMQq",
        "globalUserId": "60619751",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Оксана",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "без слов, беру в кофейню ратается на ура",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T11:42:13Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 569
    },
    {
        "id": "pcIPeIoBCaW1RxrS8IFd",
        "globalUserId": "57253691",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Отличный фит парад, 1 саше мне хватает на 2 раза",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T03:50:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 568
    },
    {
        "id": "S_FcsIoBZx8DR9RAc_PC",
        "globalUserId": "92619031",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Шахноза",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Мои сладости пришли😍😍👍👍 спасибо большое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-20T02:12:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 567
    },
    {
        "id": "PwPgNIoBU2re2XP7qMMc",
        "globalUserId": "43326878",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Хороший сахарозаменитель, всегда его беру. Упаковано отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-27T02:44:13Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 566
    },
    {
        "id": "KvqusooBwM8MKVouTrvr",
        "globalUserId": "24799239",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Марина"
        },
        "nmId": 138785478,
        "text": "В Саше беру для поездок. Очень удобно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-20T13:01:23Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 565
    },
    {
        "id": "1rGWSooBCaW1RxrS6suU",
        "globalUserId": "85454868",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Беру уже не первый раз. Самый лучший заменитель сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T07:55:19Z",
        "updatedDate": "2023-09-06T08:38:29Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:38:29Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 564
    },
    {
        "id": "7vHTU4oBBgFQoSo0JjOt",
        "globalUserId": "8851424",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Еленка",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Самый хороший сахарозаменитель из тех что я пробовала",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T02:57:42Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 563
    },
    {
        "id": "5CZdx4oBl47NwnZ1dyst",
        "globalUserId": "66706893",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Все хорошо, цена приятная. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T13:25:06Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 562
    },
    {
        "id": "6GfrnIoBpT2rITcGy80c",
        "globalUserId": "50758043",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Хороший сахзам. Но для меня сладковатый.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T07:36:53Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 561
    },
    {
        "id": "HScGw4oB0v3RIh2JccrF",
        "globalUserId": "52986923",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Отличный, заказала  теперь  номер6",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T17:11:34Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 560
    },
    {
        "id": "AC5V3IQBBvkor_vvZ_0b",
        "globalUserId": "",
        "wbUserId": 33328474,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Анна"
        },
        "nmId": 31468103,
        "text": "Впервые заказала,действительно заменяет сахар,привкуса не имеет,вообщем рекомендую ,тем более 3 в 1 за 400р ,супер.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-04T08:51:35Z",
        "updatedDate": "2023-08-16T10:09:43.875103476Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-05T06:27:03Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d3a9ad8d-1015-42a7-8c10-c74c31a6c69d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d3a9ad8d-1015-42a7-8c10-c74c31a6c69d_ms.jpg"
            }
        ],
        "photo": [
            54474396
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 559
    },
    {
        "id": "wUEXyYcBqdFzcemqaD6P",
        "globalUserId": "45791672",
        "wbUserId": 45791672,
        "wbUserDetails": {
            "name": "Надежда",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Пришло всё в срок,  60 шт",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-28T18:19:27Z",
        "updatedDate": "2023-08-16T10:09:48.665362198Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-04T08:02:44Z",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f381778a-e0ff-4ced-ad0e-56df266afc8d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f381778a-e0ff-4ced-ad0e-56df266afc8d_ms.jpg"
            }
        ],
        "photo": [
            98794606
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 558
    },
    {
        "id": "ZVkkQooBYE-HFOZ5ogcZ",
        "globalUserId": "45419204",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "kz",
            "name": "Екатерина"
        },
        "nmId": 138785478,
        "text": "Отлично, товар пришёл целым хорошо упакован. Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-29T16:33:32Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 557
    },
    {
        "id": "M-F0dYoBwM8MKVouDjE_",
        "globalUserId": "48189045",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Марина"
        },
        "nmId": 138785478,
        "text": "Не похудела, но за здоровьем стала следить:)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T15:40:55Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 556
    },
    {
        "id": "ngyKmIoBBgFQoSo0Ouk_",
        "globalUserId": "19308657",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анна"
        },
        "nmId": 138785478,
        "text": "Беру постоянно, как всегда все отлично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T11:11:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 555
    },
    {
        "id": "tnhVu4oBfuChzMpwGq43",
        "globalUserId": "104559000",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алексей",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Пользуюсь регулярно. Хорошая штука",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T05:20:31Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 554
    },
    {
        "id": "JXp4VIoBqVKcxP1mSoYv",
        "globalUserId": "53526332",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Оксана",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "По вкусу ни чем не отличаться от сахара. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T05:58:04Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 553
    },
    {
        "id": "4MYQeYoBKNxVyjcC1ttk",
        "globalUserId": "32670036",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Яна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Постоянно покупаю, без привкуса неприятного",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T08:31:01Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 552
    },
    {
        "id": "jffpY4oBBgFQoSo08j2d",
        "globalUserId": "27624984",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анна"
        },
        "nmId": 138785478,
        "text": "Отличный,всегда пью цикорий и чай только с ним",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-05T05:56:31Z",
        "updatedDate": "2023-09-05T06:04:42Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T06:04:42Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 551
    },
    {
        "id": "b1w6Y4IBAN9IxNccqnRj",
        "globalUserId": "",
        "wbUserId": 60754997,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Кристина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Брала его для «бронекофе» с кокосовым маслом. Понравился! \n\nЭто мой первый сахзам, поэтому переживала, что будет, невкусно. Но, оказалось, что вполне себе неплохо :) Упаковка была запаяна в слюду. Пришла слегка помятая, но это некритично. Все 60 шт. на месте.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-03T10:22:40Z",
        "updatedDate": "2023-08-16T10:09:41.581574883Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-11T05:58:51Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a3ad9e07-31ae-4d7c-aa6f-5aefee30fc4f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a3ad9e07-31ae-4d7c-aa6f-5aefee30fc4f_ms.jpg"
            }
        ],
        "photo": [
            74894063
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 550
    },
    {
        "id": "pBqju4oBBgFQoSo02ug3",
        "globalUserId": "16678352",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ирина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Беру не впервой , хороший сахзам.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T06:46:32Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 549
    },
    {
        "id": "EaZR7okBZx8DR9RAr47Q",
        "globalUserId": "47989447",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Сергей"
        },
        "nmId": 31468103,
        "text": "Много лет пил чай и кофе без сахара, когда попробовал этот заменитель, был сильно удивлен. В кружку на 250-300 мл кофе высыпал пакетик и получил по вкусу эквивалент 3х чайных ложек сахара. Теперь засыпаю только половину. Так же нашел полезным использование пакетиков в поездках на фестивали, можно взять столько сколько потребуется, при этом они не занимают много места.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-13T09:54:38Z",
        "updatedDate": "2023-08-16T10:09:53.746575599Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 548
    },
    {
        "id": "gwwfg4oB0v3RIh2JG9HZ",
        "globalUserId": "52300374",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Людмила",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хороший продукт. Покупаю не впервые раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T07:22:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 547
    },
    {
        "id": "M-6klIoBwM8MKVou65N8",
        "globalUserId": "31806558",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Горьковатый привкус, но еле заметный.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T17:02:31Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 546
    },
    {
        "id": "VZEEQIUBtPujtG5so3XT",
        "globalUserId": "",
        "wbUserId": 43320530,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Евгения"
        },
        "nmId": 33478818,
        "text": "Упаковано хорошо, не помялось. Срок отличный. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-23T17:25:24Z",
        "updatedDate": "2023-08-16T10:09:44.357725240Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-29T06:30:57Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/6ca6c0fe-fdf8-482c-86f4-378df478f1a5_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6ca6c0fe-fdf8-482c-86f4-378df478f1a5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/c82db64f-25e7-4b4c-9fa8-cdeeee79d715_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c82db64f-25e7-4b4c-9fa8-cdeeee79d715_ms.jpg"
            }
        ],
        "photo": [
            73208434,
            73208435
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 545
    },
    {
        "id": "zPgjVYoBl47NwnZ1AXQW",
        "globalUserId": "36865307",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Марина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "ФитПарад добавляю в разную выпечка .рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T09:04:32Z",
        "updatedDate": "2023-09-06T08:05:39Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:05:39Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 544
    },
    {
        "id": "XfEnPIYBIq9DQExlNJ4g",
        "globalUserId": "65009359",
        "wbUserId": 65009359,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Виктория"
        },
        "nmId": 138785478,
        "text": "Спасибо за подарочек)))закажу еще",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-10T16:27:28Z",
        "updatedDate": "2023-08-16T10:09:45.521557983Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-14T05:51:11Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за доверие нашему бренду!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/94f400fb-25b9-47ea-866a-c0c4cd5a346b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/94f400fb-25b9-47ea-866a-c0c4cd5a346b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/d540e135-e8d7-4349-8d59-bcaf405f2afb_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d540e135-e8d7-4349-8d59-bcaf405f2afb_ms.jpg"
            }
        ],
        "photo": [
            85306884,
            85306885
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 543
    },
    {
        "id": "-wv3kocBl47NwnZ1iu1h",
        "globalUserId": "38725997",
        "wbUserId": 38725997,
        "wbUserDetails": {
            "name": "Наталия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Упаковка хорошая, спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-18T06:05:09Z",
        "updatedDate": "2023-08-16T10:09:48.271747757Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-18T12:21:49Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cb09c106-f1ca-450a-bb1c-2c73e59de3e9_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cb09c106-f1ca-450a-bb1c-2c73e59de3e9_ms.jpg"
            }
        ],
        "photo": [
            96966084
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 542
    },
    {
        "id": "eu9Cl4oBwM8MKVouqEz7",
        "globalUserId": "53103510",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Людмила"
        },
        "nmId": 62622901,
        "text": "Лучшая замена САХАРУ. Беру много лет",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T05:14:03Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 541
    },
    {
        "id": "ymJch4oBfuChzMpwXNxs",
        "globalUserId": "38348519",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель , рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T03:08:12Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 540
    },
    {
        "id": "biB_b4YBMJfQQSHDzznl",
        "globalUserId": "45979509",
        "wbUserId": 45979509,
        "wbUserDetails": {
            "name": "Ирина",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 62622901,
        "text": "Отличный товар , хорошая упаковка , пришёл быстро",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-20T15:44:53Z",
        "updatedDate": "2023-08-16T10:09:45.919847380Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-22T07:24:15Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/194c52c5-920a-457e-a08d-76e12b50c5b8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/194c52c5-920a-457e-a08d-76e12b50c5b8_ms.jpg"
            }
        ],
        "photo": [
            87025061
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 539
    },
    {
        "id": "ii_Nb4YBv9_TjX_maBtu",
        "globalUserId": "25600283",
        "wbUserId": 25600283,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Валентина"
        },
        "nmId": 33478818,
        "text": "Пришло все без повреждений, аккуратно упаковано..",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-20T17:09:38Z",
        "updatedDate": "2023-08-16T10:09:45.924411669Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-22T07:16:51Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/372fc803-2ea0-4ff9-aa7b-8f7c950ad70b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/372fc803-2ea0-4ff9-aa7b-8f7c950ad70b_ms.jpg"
            }
        ],
        "photo": [
            87044753
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 538
    },
    {
        "id": "7PJbMIYB8a_hUs_ogUcI",
        "globalUserId": "39997497",
        "wbUserId": 39997497,
        "wbUserDetails": {
            "country": "RU",
            "name": "Инна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличная замена сахару, и то что индивидуальные пакетики",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-08T09:29:09Z",
        "updatedDate": "2023-08-16T10:09:45.443995363Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-14T06:44:03Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за высокую оценку нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9f6fa956-5f3e-48a7-96a3-d351a63297a2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9f6fa956-5f3e-48a7-96a3-d351a63297a2_ms.jpg"
            }
        ],
        "photo": [
            84912782
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 537
    },
    {
        "id": "nTFZB4gBG4PPbu1tXQiz",
        "globalUserId": "21842348",
        "wbUserId": 21842348,
        "wbUserDetails": {
            "name": "Суайбат",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Самый вкусный сахар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-10T20:27:57Z",
        "updatedDate": "2023-08-16T10:09:49.007341172Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-11T05:05:15Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "33759585",
                "wbUserId": 33759585,
                "helpfulness": "plus",
                "time": "2023-05-13T07:11:36Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/a4913989-cbd6-45f4-8d5e-44c2fc8ac8a7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a4913989-cbd6-45f4-8d5e-44c2fc8ac8a7_ms.jpg"
            }
        ],
        "photo": [
            100851491
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 536
    },
    {
        "id": "1SlX2YYBfnpWqlBet0Ob",
        "globalUserId": "24821557",
        "wbUserId": 24821557,
        "wbUserDetails": {
            "name": "Гаухар",
            "hasPhoto": false,
            "country": "KZ"
        },
        "nmId": 33478818,
        "text": "Качество 👍. Давно пользуюсь. Сняла звезду так как было не 60 а 59 саше, мелочь конечно, но не приятно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-13T05:00:50Z",
        "updatedDate": "2023-08-16T10:09:46.606975058Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-13T11:27:22Z",
            "editable": false,
            "metadata": null,
            "text": "Благодарим за отзыв",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b5a1b294-d433-43db-9999-536f899ee2c4_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b5a1b294-d433-43db-9999-536f899ee2c4_ms.jpg"
            }
        ],
        "photo": [
            90727146
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 535
    },
    {
        "id": "RfU_U4QBBvkor_vvpO-L",
        "globalUserId": "",
        "wbUserId": 66708001,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Айгуль"
        },
        "nmId": 31468103,
        "text": "Не пожалела что заказала. Соответствует описанию 1пакетик заменяет 2 чайные ложки сахара. Будем пользоваться и заказывать ещё",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-07T17:59:50Z",
        "updatedDate": "2023-08-16T10:09:43.153014455Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-08T06:28:11Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/763b8aef-aea5-42df-9aca-2714904a26fb_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/763b8aef-aea5-42df-9aca-2714904a26fb_ms.jpg"
            }
        ],
        "photo": [
            27514390
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 534
    },
    {
        "id": "txA4QocB5HDhYKpH43VC",
        "globalUserId": "24086205",
        "wbUserId": 24086205,
        "wbUserDetails": {
            "country": "RU",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Всё супер,спасибо за подарочки",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-02T13:47:17Z",
        "updatedDate": "2023-08-16T10:09:47.581972093Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T10:01:37Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "107819954",
                "wbUserId": 107819954,
                "helpfulness": "plus",
                "time": "2023-04-03T12:29:18Z"
            },
            {
                "globalUserId": "24086205",
                "wbUserId": 24086205,
                "helpfulness": "plus",
                "time": "2023-04-04T13:46:34Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4cf674c3-17fb-4bc4-a76d-ca7e1ecc5763_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4cf674c3-17fb-4bc4-a76d-ca7e1ecc5763_ms.jpg"
            }
        ],
        "photo": [
            94402395
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 533
    },
    {
        "id": "qVuiOIEBZ4onbIiBfku-",
        "globalUserId": "",
        "wbUserId": 19537970,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анастасия",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Коробка пришла помятая \nА так ничего хороший заменитель один пакетик заменяет 2 ч.л сахара не пахнет ничем никакого привкуса. Буду брать еще",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-06T10:49:42Z",
        "updatedDate": "2023-08-16T10:09:40.890671814Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-06T11:20:08Z",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8c2939f6-7030-473f-bbbf-f14e032b8e5a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8c2939f6-7030-473f-bbbf-f14e032b8e5a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/3b81f5ef-e0e5-4f50-8ee9-c3f28af173d3_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3b81f5ef-e0e5-4f50-8ee9-c3f28af173d3_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/20b6eb21-2a18-4e54-a5a6-a542f69b23cd_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/20b6eb21-2a18-4e54-a5a6-a542f69b23cd_ms.jpg"
            }
        ],
        "photo": [
            62983980,
            62983998,
            62984013
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 532
    },
    {
        "id": "iN3YQIYBfnpWqlBeXLhd",
        "globalUserId": "59927217",
        "wbUserId": 59927217,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Зулхумар",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Все упакована хорошо взяли на пробу спосиба продавцу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-11T14:19:27Z",
        "updatedDate": "2023-08-16T10:09:45.568528168Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-13T11:35:22Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f654bf36-e14d-4d8e-b970-62bfe53a880f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f654bf36-e14d-4d8e-b970-62bfe53a880f_ms.jpg"
            }
        ],
        "photo": [
            85452933
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 531
    },
    {
        "id": "lL1-n4UB8a_hUs_oBzfv",
        "globalUserId": "",
        "wbUserId": 22516620,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Виктория",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Коробочка не мятая, подсластитель вкусный, брала первый раз) возьму ещё",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-11T06:21:55Z",
        "updatedDate": "2023-08-16T10:09:44.639414297Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-11T06:27:02Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 15216693,
                "helpfulness": "plus",
                "time": "2023-01-11T12:53:12Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a4fbb6af-c7e2-4840-b475-e8945dd2958f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a4fbb6af-c7e2-4840-b475-e8945dd2958f_ms.jpg"
            }
        ],
        "photo": [
            78351597
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 530
    },
    {
        "id": "ti5-ZIYBc4LImbbGcw0L",
        "globalUserId": "83178458",
        "wbUserId": 83178458,
        "wbUserDetails": {
            "name": "Разият Магомедова",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Мне понравился этот сах зам, буду заказывать ещё",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-18T12:27:34Z",
        "updatedDate": "2023-08-16T10:09:45.840505012Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-20T11:53:33Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14852945",
                "wbUserId": 14852945,
                "helpfulness": "plus",
                "time": "2023-02-19T06:50:09Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0193e534-9ead-4f02-ac7a-ef49dae9228a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0193e534-9ead-4f02-ac7a-ef49dae9228a_ms.jpg"
            }
        ],
        "photo": [
            86642626
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 529
    },
    {
        "id": "6VnLRIoBYE-HFOZ5vdQY",
        "globalUserId": "11784870",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анжела"
        },
        "nmId": 138785478,
        "text": "Беру не в первый раз. Всё отлично. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T04:55:18Z",
        "updatedDate": "2023-09-06T08:58:46Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:58:46Z",
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 528
    },
    {
        "id": "gZI6j4oBqVKcxP1mCBJW",
        "globalUserId": "72360647",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Зарина"
        },
        "nmId": 138785478,
        "text": "Все пришло целое. Спасибо продавцу😌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T15:47:40Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 527
    },
    {
        "id": "EtPvqoUBMJfQQSHDVw16",
        "globalUserId": "20622832",
        "wbUserId": 20622832,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 33478818,
        "text": "Пришёл упакованный в полиэтиленовый пакет",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-13T11:41:30Z",
        "updatedDate": "2023-08-16T10:09:44.693525414Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:50:22Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "44772209",
                "wbUserId": 44772209,
                "helpfulness": "plus",
                "time": "2023-01-15T04:53:43Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/775da4b5-0a2a-4b6a-8b8d-cec588859f28_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/775da4b5-0a2a-4b6a-8b8d-cec588859f28_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/a46fc901-7b09-42c2-8535-6ed1e16c12c1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a46fc901-7b09-42c2-8535-6ed1e16c12c1_ms.jpg"
            }
        ],
        "photo": [
            78745713,
            78745716
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 526
    },
    {
        "id": "nSYerIUBheedNkNuqpi0",
        "globalUserId": "29545775",
        "wbUserId": 29545775,
        "wbUserDetails": {
            "country": "BY",
            "hasPhoto": false,
            "name": "Жанна"
        },
        "nmId": 33478818,
        "text": "Упаковано всё замечательно. В кофе никакого привкуса не чувствуется.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-13T17:12:49Z",
        "updatedDate": "2023-08-16T10:09:44.713217849Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:44:00Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8d5bbbaf-9b3a-4b64-87da-34fcc65962d4_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8d5bbbaf-9b3a-4b64-87da-34fcc65962d4_ms.jpg"
            }
        ],
        "photo": [
            78814243
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 525
    },
    {
        "id": "OiTowYQBBvkor_vv4W7M",
        "globalUserId": "",
        "wbUserId": 58199556,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Светлана",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Все цело и быстро. Продуктом пользуюсь уже не один год. Мой любимый и самый полезный заменитель сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-29T05:42:55Z",
        "updatedDate": "2023-08-16T10:09:43.679175338Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-29T05:43:04Z",
            "metadata": null,
            "text": "Спасибо за доверие нашему бренду!",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 58199556,
                "helpfulness": "plus",
                "time": "2022-11-29T05:43:40.202782840Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/cc2f8d38-2b77-4ee2-a536-2ec6535a0db1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/cc2f8d38-2b77-4ee2-a536-2ec6535a0db1_ms.jpg"
            }
        ],
        "photo": [
            47789793
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 524
    },
    {
        "id": "-ytF9IUBbskpjQp2VhS-",
        "globalUserId": "54484139",
        "wbUserId": 54484139,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Валентина"
        },
        "nmId": 33478818,
        "text": "Пришло быстро, упаковка целая, даже не мятая, срок хороший.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-27T17:27:43Z",
        "updatedDate": "2023-08-16T10:09:45.121730476Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:00:21Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/460f57db-2acf-4844-80c5-2a248934efea_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/460f57db-2acf-4844-80c5-2a248934efea_ms.jpg"
            }
        ],
        "photo": [
            81043520
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 523
    },
    {
        "id": "_kWTx4oBqdFzcemqPysp",
        "globalUserId": "14486347",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Мария"
        },
        "nmId": 33478818,
        "text": "Беру не первый раз. Довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T14:23:51Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 522
    },
    {
        "id": "2eS0t4cBN75J-nvtmTY3",
        "globalUserId": "29831171",
        "wbUserId": 29831171,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Лида",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Всё отлично 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-25T09:17:59Z",
        "updatedDate": "2023-08-16T10:09:48.543161607Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-25T13:24:43Z",
            "createDate": "2023-04-25T13:24:43Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2f3f9cd9-29ba-4d89-8221-b068f1e8bd27_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2f3f9cd9-29ba-4d89-8221-b068f1e8bd27_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/c77b0f6d-0ca1-4ec3-ba78-55fdd133ba44_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c77b0f6d-0ca1-4ec3-ba78-55fdd133ba44_ms.jpg"
            }
        ],
        "photo": [
            98184948,
            98184949
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 521
    },
    {
        "id": "hCRoVIYBk9xzjhYZhXfR",
        "globalUserId": "44643680",
        "wbUserId": 44643680,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Марина"
        },
        "nmId": 138785478,
        "text": "Соответствует описанию, самое главное 0 калорий.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-15T09:29:42Z",
        "updatedDate": "2023-08-16T10:09:45.708988019Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-15T09:54:02Z",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5cf4eabe-193e-45f1-831b-6dddb7544b8c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5cf4eabe-193e-45f1-831b-6dddb7544b8c_ms.jpg"
            }
        ],
        "photo": [
            86069790
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 520
    },
    {
        "id": "Lx8CSoYBv9_TjX_m8j4s",
        "globalUserId": "8177944",
        "wbUserId": 8177944,
        "wbUserDetails": {
            "country": "RU",
            "name": "Алёна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Пришёл в идеальном состоянии, спасибо огромное!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-13T09:02:32Z",
        "updatedDate": "2023-08-16T10:09:45.624822069Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-13T09:03:31Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "8177944",
                "wbUserId": 8177944,
                "helpfulness": "plus",
                "time": "2023-02-13T09:08:22Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/39f8e28a-0145-47f7-9327-34dc519d950c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/39f8e28a-0145-47f7-9327-34dc519d950c_ms.jpg"
            }
        ],
        "photo": [
            85719868
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 519
    },
    {
        "id": "DeJn8YgBU_f9vkXPLrfQ",
        "globalUserId": "53383622",
        "wbUserId": 53383622,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Спасибо🙏💕",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-25T07:14:11Z",
        "updatedDate": "2023-08-16T10:09:50.305766503Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T11:36:27Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/863e3dac-4303-4baa-9e67-221b2c1ece41_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/863e3dac-4303-4baa-9e67-221b2c1ece41_ms.jpg"
            }
        ],
        "photo": [
            108387585
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 518
    },
    {
        "id": "xSbJx4oBl47NwnZ1NGPj",
        "globalUserId": "25641835",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Оленька",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Упаковано хорошо все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T15:22:47Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 517
    },
    {
        "id": "U5zlgoYBBhsOOdMGf6xt",
        "globalUserId": "28323713",
        "wbUserId": 28323713,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алексей",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Упакован так себе, в остальном вроде хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-24T10:08:44Z",
        "updatedDate": "2023-08-16T10:09:46.049000263Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-02T08:44:58Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e882a00f-1c35-4ef7-8d48-58abd218de55_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e882a00f-1c35-4ef7-8d48-58abd218de55_ms.jpg"
            }
        ],
        "photo": [
            87649390
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 516
    },
    {
        "id": "-uLjtYoBuX2t2Z2-x6xJ",
        "globalUserId": "97257046",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Саят",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Очень удобные просто прелесть",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T03:58:38Z",
        "updatedDate": "2023-09-21T04:28:36Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "13771066",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-09-21T04:28:36Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 515
    },
    {
        "id": "SrSWmYoBnnhI58-2F8T_",
        "globalUserId": "24791202",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Кристина",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Упоковки все были порванные, к продукту нет притензии , но доставлена в ужасном виде",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 3,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T16:04:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 514
    },
    {
        "id": "pWcdZ4oBYE-HFOZ5XTLW",
        "globalUserId": "68607774",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Зулетта"
        },
        "nmId": 31468103,
        "text": "Всё доставили в срок. Классный сах.зам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-05T20:51:32Z",
        "updatedDate": "2023-09-06T07:52:19Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T07:52:19Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 513
    },
    {
        "id": "nteQmIoBuX2t2Z2-RQeb",
        "globalUserId": "50325146",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Нравится,не первый раз заказываю",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T11:18:26Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 512
    },
    {
        "id": "STkEvIoBU2re2XP7kAeB",
        "globalUserId": "10843768",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 62622901,
        "text": "Давно берем , отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T08:32:10Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 511
    },
    {
        "id": "oHcouIoBfuChzMpwrL_W",
        "globalUserId": "48810612",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Мой любимый сахарозаменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T14:33:08Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 510
    },
    {
        "id": "4ujrtYUBc4LImbbGnafe",
        "globalUserId": "66276673",
        "wbUserId": 66276673,
        "wbUserDetails": {
            "name": "Мария",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "В данный момент пью кофе с ним, люблю❤️ А это Фрося, ей 5 лет😻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-15T14:53:16Z",
        "updatedDate": "2023-08-16T10:09:44.757328150Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-25T12:31:57Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "22123771",
                "wbUserId": 22123771,
                "helpfulness": "plus",
                "time": "2023-01-15T14:54:36Z"
            },
            {
                "globalUserId": "31730534",
                "wbUserId": 31730534,
                "helpfulness": "plus",
                "time": "2023-01-15T16:36:26Z"
            },
            {
                "globalUserId": "58655826",
                "wbUserId": 58655826,
                "helpfulness": "plus",
                "time": "2023-01-16T06:53:10Z"
            },
            {
                "globalUserId": "19944910",
                "wbUserId": 19944910,
                "helpfulness": "plus",
                "time": "2023-01-16T07:46:46Z"
            },
            {
                "globalUserId": "56819255",
                "wbUserId": 56819255,
                "helpfulness": "plus",
                "time": "2023-01-16T13:00:07Z"
            },
            {
                "globalUserId": "21907490",
                "wbUserId": 21907490,
                "helpfulness": "plus",
                "time": "2023-01-16T14:23:58Z"
            },
            {
                "globalUserId": "44132679",
                "wbUserId": 44132679,
                "helpfulness": "plus",
                "time": "2023-01-17T05:04:11Z"
            },
            {
                "globalUserId": "24182564",
                "wbUserId": 24182564,
                "helpfulness": "plus",
                "time": "2023-01-19T18:07:52Z"
            },
            {
                "globalUserId": "9031574",
                "wbUserId": 9031574,
                "helpfulness": "minus",
                "time": "2023-01-20T19:13:41Z"
            },
            {
                "globalUserId": "92290869",
                "wbUserId": 92290869,
                "helpfulness": "plus",
                "time": "2023-01-20T19:31:14Z"
            },
            {
                "globalUserId": "22047018",
                "wbUserId": 22047018,
                "helpfulness": "plus",
                "time": "2023-01-21T21:27:59Z"
            },
            {
                "globalUserId": "23739773",
                "wbUserId": 23739773,
                "helpfulness": "plus",
                "time": "2023-01-22T08:57:37Z"
            },
            {
                "globalUserId": "54705268",
                "wbUserId": 54705268,
                "helpfulness": "plus",
                "time": "2023-01-26T09:39:50Z"
            },
            {
                "globalUserId": "28722566",
                "wbUserId": 28722566,
                "helpfulness": "plus",
                "time": "2023-01-31T20:10:38Z"
            },
            {
                "globalUserId": "444236",
                "wbUserId": 444236,
                "helpfulness": "minus",
                "time": "2023-02-01T15:08:06Z"
            },
            {
                "globalUserId": "6863796",
                "wbUserId": 6863796,
                "helpfulness": "plus",
                "time": "2023-02-02T17:54:51Z"
            },
            {
                "globalUserId": "21836387",
                "wbUserId": 21836387,
                "helpfulness": "plus",
                "time": "2023-02-08T03:01:11Z"
            },
            {
                "globalUserId": "21673467",
                "wbUserId": 21673467,
                "helpfulness": "plus",
                "time": "2023-02-08T20:38:04Z"
            },
            {
                "globalUserId": "36429788",
                "wbUserId": 36429788,
                "helpfulness": "plus",
                "time": "2023-02-13T23:12:32Z"
            },
            {
                "globalUserId": "20405000",
                "wbUserId": 20405000,
                "helpfulness": "plus",
                "time": "2023-04-03T02:24:24Z"
            },
            {
                "globalUserId": "88520472",
                "wbUserId": 88520472,
                "helpfulness": "plus",
                "time": "2023-04-10T19:51:15Z"
            },
            {
                "globalUserId": "29760446",
                "wbUserId": 29760446,
                "helpfulness": "plus",
                "time": "2023-06-05T09:42:08Z"
            },
            {
                "globalUserId": "11972453",
                "wbUserId": 11972453,
                "helpfulness": "plus",
                "time": "2023-06-13T16:39:29Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a3b3d626-86b3-475b-93c6-705df336547a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a3b3d626-86b3-475b-93c6-705df336547a_ms.jpg"
            }
        ],
        "photo": [
            79135240
        ],
        "video": null,
        "votes": {
            "pluses": 21,
            "minuses": 2
        },
        "rank": 509
    },
    {
        "id": "ZyG9cIQBeiioYr22Tb3W",
        "globalUserId": "",
        "wbUserId": 40657901,
        "wbUserDetails": {
            "name": "Виктория",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Вкусно, без посторонних привкусов. Главное чтобы сахар мой не повышался. Ровно 60 пакетиков, как и заявлено.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-13T11:26:05Z",
        "updatedDate": "2023-08-16T10:09:43.293687350Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2022-11-14T08:59:29Z",
            "createDate": "2022-11-14T08:55:26Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c255a760-77f8-4353-a16e-8e8c97eb00d8_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c255a760-77f8-4353-a16e-8e8c97eb00d8_ms.jpg"
            }
        ],
        "photo": [
            33669435
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 508
    },
    {
        "id": "4N7asIcBqd2abVcykWL3",
        "globalUserId": "81873817",
        "wbUserId": 81873817,
        "wbUserDetails": {
            "name": "Роман",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хороший такой аппарат",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-24T01:22:07Z",
        "updatedDate": "2023-08-16T10:09:48.485179077Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-24T10:21:22Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "19966997",
                "wbUserId": 19966997,
                "helpfulness": "plus",
                "time": "2023-04-24T04:09:51Z"
            },
            {
                "globalUserId": "18630207",
                "wbUserId": 18630207,
                "helpfulness": "plus",
                "time": "2023-04-26T14:01:18Z"
            },
            {
                "globalUserId": "1855125",
                "wbUserId": 1855125,
                "helpfulness": "minus",
                "time": "2023-04-26T18:07:28Z"
            },
            {
                "globalUserId": "17434918",
                "wbUserId": 17434918,
                "helpfulness": "minus",
                "time": "2023-04-27T17:35:53Z"
            },
            {
                "globalUserId": "9790710",
                "wbUserId": 9790710,
                "helpfulness": "plus",
                "time": "2023-05-01T16:47:16Z"
            },
            {
                "globalUserId": "34215564",
                "wbUserId": 34215564,
                "helpfulness": "minus",
                "time": "2023-05-07T09:10:49Z"
            },
            {
                "globalUserId": "28190082",
                "wbUserId": 28190082,
                "helpfulness": "minus",
                "time": "2023-06-16T08:11:30Z"
            },
            {
                "globalUserId": "60562283",
                "wbUserId": 60562283,
                "helpfulness": "plus",
                "time": "2023-06-18T08:22:45Z"
            },
            {
                "globalUserId": "61410609",
                "wbUserId": 61410609,
                "helpfulness": "plus",
                "time": "2023-06-22T14:22:13Z"
            },
            {
                "globalUserId": "52507321",
                "wbUserId": 52507321,
                "helpfulness": "plus",
                "time": "2023-07-08T22:26:17Z"
            },
            {
                "globalUserId": "47158774",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-07-21T13:43:07Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9ce6ff82-1866-47d6-ba75-6d8e8cf0e078_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9ce6ff82-1866-47d6-ba75-6d8e8cf0e078_ms.jpg"
            }
        ],
        "photo": [
            97964369
        ],
        "video": null,
        "votes": {
            "pluses": 7,
            "minuses": 4
        },
        "rank": 507
    },
    {
        "id": "r8DnRYYBQOOSTxXoY3ax",
        "globalUserId": "43982888",
        "wbUserId": 43982888,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Всё классно очень сладенький",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-12T13:53:58Z",
        "updatedDate": "2023-08-16T10:09:45.608021769Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-13T08:26:13Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d1230cd4-921f-435b-9cac-3341df0c0f56_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d1230cd4-921f-435b-9cac-3341df0c0f56_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a3516bc3-464a-4d0f-9847-455430f21b40_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a3516bc3-464a-4d0f-9847-455430f21b40_ms.jpg"
            }
        ],
        "photo": [
            85608883,
            85608885
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 506
    },
    {
        "id": "JRttQYYBk9xzjhYZKoYE",
        "globalUserId": "15598364",
        "wbUserId": 15598364,
        "wbUserDetails": {
            "name": "Лариса",
            "country": "KZ",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Любимый сахзам, все отлично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-11T17:01:59Z",
        "updatedDate": "2023-08-16T10:09:45.576092826Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-13T10:38:48Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "15598364",
                "wbUserId": 15598364,
                "helpfulness": "plus",
                "time": "2023-02-13T11:30:27Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7ee8a7a7-59c0-455d-9532-7f2328e6a6da_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7ee8a7a7-59c0-455d-9532-7f2328e6a6da_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a8dd18ee-a09a-4678-8753-5cccfab80f3b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a8dd18ee-a09a-4678-8753-5cccfab80f3b_ms.jpg"
            }
        ],
        "photo": [
            85482738,
            85482740
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 505
    },
    {
        "id": "Rpom_4cBYE-HFOZ5lnqn",
        "globalUserId": "25181326",
        "wbUserId": 25181326,
        "wbUserDetails": {
            "country": "ru",
            "name": "Регина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Не сказала бы, что заменяет две ложки сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-09T06:15:32Z",
        "updatedDate": "2023-08-16T10:09:48.949030972Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-17T06:21:24Z",
            "createDate": "2023-05-17T06:21:24Z",
            "editable": false,
            "employeeId": 0,
            "text": "Рекомендуем встряхивать сахарозаменитель перед употреблением, это способствует равномерному распределению компонентов и достижению указанной сладости!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "24823220",
                "wbUserId": 24823220,
                "helpfulness": "plus",
                "time": "2023-05-10T12:23:15Z"
            },
            {
                "globalUserId": "15612287",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-05-10T14:03:36Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2aa5e1c8-a7df-41cf-b912-f7de3558db01_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2aa5e1c8-a7df-41cf-b912-f7de3558db01_ms.jpg"
            }
        ],
        "photo": [
            100520443
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 1
        },
        "rank": 504
    },
    {
        "id": "qb3IfYoBlM6cTg_RsKBJ",
        "globalUserId": "16100973",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Лейсан",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Мне нравится. Очень заменяет сахар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T06:30:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 503
    },
    {
        "id": "6l-IjYoBUYwa0QEfc3iN",
        "globalUserId": "7089425",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Светлана"
        },
        "nmId": 62622901,
        "text": "Обожаю его😍заказываю второй раз👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T07:54:05Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 502
    },
    {
        "id": "c9eRRYoBUdF-4nlzV_P5",
        "globalUserId": "17534701",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Покупаю давно, хороший заменитель сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T08:31:08Z",
        "updatedDate": "2023-09-06T08:53:08Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:53:08Z",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 501
    },
    {
        "id": "DTOLkIgBAtKOitoByJCq",
        "globalUserId": "32752888",
        "wbUserId": 32752888,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-06T11:51:00Z",
        "updatedDate": "2023-08-16T10:09:49.772899132Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-06T12:44:17Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4422e0b7-a016-42a2-a061-c2e61d58d1ae_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4422e0b7-a016-42a2-a061-c2e61d58d1ae_ms.jpg"
            }
        ],
        "photo": [
            105152262
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 500
    },
    {
        "id": "1NOHkYoBuX2t2Z2-L8-r",
        "globalUserId": "63036078",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Заказываю постоянно, рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T02:31:10Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 499
    },
    {
        "id": "94bks4oBYE-HFOZ5bqRw",
        "globalUserId": "27386333",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Виктория Данилова"
        },
        "nmId": 138785478,
        "text": "Всегда беру, мне нравиться!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-20T18:40:07Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 498
    },
    {
        "id": "FIDqx4QBTTPU1bwQJUeb",
        "globalUserId": "",
        "wbUserId": 91578214,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "BY",
            "name": "Людмила"
        },
        "nmId": 33478818,
        "text": "Спасибо продавцу, сладкий,без всяких привкусов,удобные сапёры,одного на кружку достаточно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-30T09:42:01Z",
        "updatedDate": "2023-08-16T10:09:43.721739758Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-05T11:55:14Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/e59d6321-b8d3-49d0-992f-328570a28ab8_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e59d6321-b8d3-49d0-992f-328570a28ab8_ms.jpg"
            }
        ],
        "photo": [
            48871135
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 497
    },
    {
        "id": "DKGsqIoBXuPSsPE16zTt",
        "globalUserId": "86833602",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Оксана",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Беру постоянно, все отлично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T14:23:39Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 496
    },
    {
        "id": "3t0Ta4oBwM8MKVouN1Xt",
        "globalUserId": "84990128",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Соник",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Не советую тем,у кого вечные проблемы с желудком.\nДелает только хуже.\nА так сладкий на вкус",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T15:18:56Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 495
    },
    {
        "id": "-mfBoIoBUYwa0QEfq4nK",
        "globalUserId": "21159608",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Эмилия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "беру не первый раз. нравится.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T01:29:22Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 494
    },
    {
        "id": "-GrxJIoBqVKcxP1mZB9E",
        "globalUserId": "30676897",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Илона"
        },
        "nmId": 138785478,
        "text": "Упаковка пришла целая. Сахзам удачный: в меру сладкий, двум чайным ложкам сахара (без горки) на вкус соответствует. За 3 месяца похудела на нем на 3 кг, заменив сахар на данный сахзам, частично отказавшись от мучного и уменьшив потребление шоколада.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-24T00:28:34Z",
        "updatedDate": "2023-08-26T06:18:25Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-24T06:24:24Z",
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "23916772",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-26T06:18:25Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 493
    },
    {
        "id": "pOj5u4oBqd2abVcySFDD",
        "globalUserId": "54581005",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Юлия",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всегда его беру , нравится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T08:19:51Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 492
    },
    {
        "id": "Bphkk4oBXuPSsPE1SGb2",
        "globalUserId": "64724997",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Вс отлично доставлено. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T11:12:18Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 491
    },
    {
        "id": "FbHCkooBnnhI58-2qqa2",
        "globalUserId": "32320075",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Любовь",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Беру постоянно.На ПП самое то.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T08:15:46Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 490
    },
    {
        "id": "q9UgEn8Bjrh6pdv-L4GH",
        "globalUserId": "",
        "wbUserId": 37988254,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Дарья"
        },
        "nmId": 33478818,
        "text": "Впервые попробовала сахарозаменитель, скажу вам, что это прям супер , мне хватает одного пакетика на 1 стакан чая 👍🏽 Все пришло в целости! Ещё и презент в виде зефирки , приятно и вкусно 😀 ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-19T13:16:13Z",
        "updatedDate": "2023-08-16T10:09:39.443090Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T07:44:05Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/3d1ed540-b98b-4e53-97c7-ae89e369571b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3d1ed540-b98b-4e53-97c7-ae89e369571b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/e4b0271d-a8cd-49ab-a927-cd45eb0e3b08_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e4b0271d-a8cd-49ab-a927-cd45eb0e3b08_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2c54b1ca-092b-4208-a00c-59d9ee0a121f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2c54b1ca-092b-4208-a00c-59d9ee0a121f_ms.jpg"
            }
        ],
        "photo": [
            62369635,
            62369652,
            62369669
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 489
    },
    {
        "id": "MrksVIoBKNxVyjcC0QU7",
        "globalUserId": "16686678",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Уже 5 лет использую, про сахар забыла",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T04:35:38Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 488
    },
    {
        "id": "mg5jR4gBKNxVyjcCIgxY",
        "globalUserId": "53448517",
        "wbUserId": 53448517,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Всё хорошо 👌🏼",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-23T06:54:19Z",
        "updatedDate": "2023-08-16T10:09:49.389115384Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-23T07:04:21Z",
            "createDate": "2023-05-23T07:04:21Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/20b40190-0494-4a95-89d5-637ad33340ad_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/20b40190-0494-4a95-89d5-637ad33340ad_ms.jpg"
            }
        ],
        "photo": [
            102888132
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 487
    },
    {
        "id": "OmXBl4oBpT2rITcGfcs5",
        "globalUserId": "31718894",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Юлия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Самый лучший сахарозаменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T07:32:35Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 486
    },
    {
        "id": "XuaEk4oBZx8DR9RADkwG",
        "globalUserId": "43612435",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Рекомендую , лучший  сах /зам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T11:47:00Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 485
    },
    {
        "id": "0_PgWYoBBgFQoSo0aWnp",
        "globalUserId": "31867985",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Артем",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Коробка мятая,  а так все нормально.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T07:09:54Z",
        "updatedDate": "2023-09-05T06:57:54Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T06:57:54Z",
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 484
    },
    {
        "id": "gMV5ZooBqd2abVcywKxt",
        "globalUserId": "103606284",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Мне нравится этот сахарзаменитель.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-05T17:52:50Z",
        "updatedDate": "2023-09-06T07:47:43Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T07:47:43Z",
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 483
    },
    {
        "id": "1rMOPYoBuX2t2Z2-zhlf",
        "globalUserId": "37240370",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Отличный товар все пришло целым спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T16:51:35Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 482
    },
    {
        "id": "V9zZooABdKRAqqwZdwJz",
        "globalUserId": "",
        "wbUserId": 56764122,
        "wbUserDetails": {
            "country": "RU",
            "name": "Лариса",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Упаковка целая, саше 60 шт специально пересчитала)) и на вкус сладкий. Вообщем пользуюсь давно, а здесь заказываю в первый раз, не разочарована, а тем более за такую цену. Спасибо продавцу 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-08T08:46:42Z",
        "updatedDate": "2023-08-16T10:09:40.418386082Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-12T12:05:18Z",
            "metadata": null,
            "text": "Спасибо за доверие нашему бренду!",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 56764122,
                "helpfulness": "plus",
                "time": "2022-05-12T12:49:32Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/71a58b66-4313-48f5-af6e-62f399531c49_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/71a58b66-4313-48f5-af6e-62f399531c49_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6e668e5c-fd3e-4956-b8a9-7d2aef93929f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6e668e5c-fd3e-4956-b8a9-7d2aef93929f_ms.jpg"
            }
        ],
        "photo": [
            57284676,
            57284692
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 481
    },
    {
        "id": "UN2VrYoBKNxVyjcC30ab",
        "globalUserId": "38629517",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Екатерина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Понравилось. Сладко в меру",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T13:16:35Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 480
    },
    {
        "id": "6G9oo4oBfuChzMpwajHr",
        "globalUserId": "52406417",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Армине"
        },
        "nmId": 31468103,
        "text": "Заказываю уже третий раз  👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T13:50:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 479
    },
    {
        "id": "-sdbk4oBlM6cTg_R1Jjc",
        "globalUserId": "46774207",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Рекомендую,беру не первый раз",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T11:03:04Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 478
    },
    {
        "id": "_FwnhooBUYwa0QEfZljc",
        "globalUserId": "46586292",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Валентина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Коробка мятая, но не критично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T21:30:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 477
    },
    {
        "id": "ilPwtIQBJjG5B9ArpXOi",
        "globalUserId": "",
        "wbUserId": 30569921,
        "wbUserDetails": {
            "country": "RU",
            "name": "Наиля",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Еще не пробовала, но думаю хорошая вещь для худеющих",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-26T17:16:20Z",
        "updatedDate": "2023-08-16T10:09:43.592597644Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-29T06:35:30Z",
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/d15e6e37-de83-4a86-bf3c-4697cfa621bb_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d15e6e37-de83-4a86-bf3c-4697cfa621bb_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/f9d361ad-8363-4dcc-9199-7616c4a7e6d5_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f9d361ad-8363-4dcc-9199-7616c4a7e6d5_ms.jpg"
            }
        ],
        "photo": [
            45687799,
            45687800
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 476
    },
    {
        "id": "ap7xoYoBXuPSsPE1_A77",
        "globalUserId": "43890601",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Удобно, что по 1гр пакетики",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T07:01:45Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 475
    },
    {
        "id": "4zQEp4IBSRNhp5RIbsP6",
        "globalUserId": "",
        "wbUserId": 20335756,
        "wbUserDetails": {
            "name": "Илюза",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Коробка была в плёнке, немного мятая  коробка, содержимое 60шт., все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-16T14:17:40Z",
        "updatedDate": "2023-08-16T10:09:41.718471833Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-17T08:04:41Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 55429745,
                "helpfulness": "plus",
                "time": "2022-09-21T13:24:36.329998914Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/3d74fd74-dde0-4847-a2bd-2187e147ddcd_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3d74fd74-dde0-4847-a2bd-2187e147ddcd_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/1da9bace-cbe0-4106-a9b2-31f47d6de5ad_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1da9bace-cbe0-4106-a9b2-31f47d6de5ad_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/f7eb16b6-0011-4010-bf78-9fccd656b573_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f7eb16b6-0011-4010-bf78-9fccd656b573_ms.jpg"
            }
        ],
        "photo": [
            71785166,
            71785186,
            71785205
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 474
    },
    {
        "id": "2fQ-w4MBKfYiBPKgQa0a",
        "globalUserId": "",
        "wbUserId": 42513946,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Алина"
        },
        "nmId": 33478818,
        "text": "Пришёл немного помятый, но мне всё равно, беру себе) Самое главное цена 🔥",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-10T18:53:00Z",
        "updatedDate": "2023-08-16T10:09:42.531587211Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-11T11:18:48Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/06d988ec-c096-4650-ac24-cca774a5c172_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/06d988ec-c096-4650-ac24-cca774a5c172_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/a90d5d55-52f7-4d0b-94c8-26f9e3ee470b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a90d5d55-52f7-4d0b-94c8-26f9e3ee470b_ms.jpg"
            }
        ],
        "photo": [
            52691223,
            52691238
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 473
    },
    {
        "id": "xoTvY4oBXuPSsPE1a-67",
        "globalUserId": "9650510",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Мария"
        },
        "nmId": 31468103,
        "text": "Прекрасная замена обычному сахару",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-05T06:02:30Z",
        "updatedDate": "2023-09-05T06:04:11Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T06:04:11Z",
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 472
    },
    {
        "id": "BsfShIoBCaW1RxrSo_7j",
        "globalUserId": "98139142",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Я часто заказываю этот товар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T15:18:32Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 471
    },
    {
        "id": "R8q2eooBuX2t2Z2-7RNk",
        "globalUserId": "57341855",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень сладкий. 1саше на 0,5 л.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T16:12:03Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 470
    },
    {
        "id": "ZqxUxooBXuPSsPE1Vo7f",
        "globalUserId": "5036811",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Анастасия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Упакован хорошо!Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T08:35:31Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 469
    },
    {
        "id": "TnPKoYoBUvtK502_wnI1",
        "globalUserId": "69628061",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Классный заменитель сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T06:18:54Z",
        "updatedDate": "2023-09-17T08:31:20Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "67079066",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-17T08:31:20Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 468
    },
    {
        "id": "bRKyW4oBU2re2XP7mEdz",
        "globalUserId": "27590037",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Валерия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Беру только его, претензий нет 👍🏼",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T15:39:06Z",
        "updatedDate": "2023-09-04T04:54:31Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T04:54:31Z",
            "metadata": null,
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 467
    },
    {
        "id": "z0czooYBk9xzjhYZBHKt",
        "globalUserId": "18129947",
        "wbUserId": 18129947,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Пришло все целое, будем пробовать",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-02T12:01:38Z",
        "updatedDate": "2023-08-16T10:09:46.247414615Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-02T12:08:03Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ba369d9b-fb54-4f8f-9fdc-2f72769bab0b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ba369d9b-fb54-4f8f-9fdc-2f72769bab0b_ms.jpg"
            }
        ],
        "photo": [
            88736718
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 466
    },
    {
        "id": "r-UqPIMBbunnhI3gFNwl",
        "globalUserId": "",
        "wbUserId": 53020662,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Лучший махрам из всех существующих. Пришла помятая коробка - тут вопрос к Вайлдберриз",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-14T13:22:14Z",
        "updatedDate": "2023-08-16T10:09:42.076111433Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T08:55:32Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 53020662,
                "helpfulness": "plus",
                "time": "2022-09-15T15:20:35.015286052Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/28a7e967-163b-4ca0-8183-ba40f0663ff4_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/28a7e967-163b-4ca0-8183-ba40f0663ff4_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/aa598f55-0979-4f79-a7d2-bb60a854cf53_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/aa598f55-0979-4f79-a7d2-bb60a854cf53_ms.jpg"
            }
        ],
        "photo": [
            67668364,
            67668378
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 465
    },
    {
        "id": "8RcfVooBqdFzcemq09M-",
        "globalUserId": "9689474",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший, никакого привкуса нет!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T13:40:41Z",
        "updatedDate": "2023-09-05T08:18:57Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T08:18:57Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 464
    },
    {
        "id": "VmSqGYcB8a_hUs_ow8Pn",
        "globalUserId": "12132295",
        "wbUserId": 12132295,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Лариса",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Отлично, спасибо продавцу",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-25T16:47:14Z",
        "updatedDate": "2023-08-16T10:09:47.199878419Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:54:23Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/781a73e8-c6b3-4889-ac44-3e6ecaf55f49_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/781a73e8-c6b3-4889-ac44-3e6ecaf55f49_ms.jpg"
            }
        ],
        "photo": [
            93053939
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 463
    },
    {
        "id": "r2B9c4oBUvtK502_izhh",
        "globalUserId": "95213183",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ольга"
        },
        "nmId": 138785478,
        "text": "Качество на высоте, рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T06:32:02Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 462
    },
    {
        "id": "7Oa7M4QBBvkor_vv4a-8",
        "globalUserId": "",
        "wbUserId": 16374520,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Татьяна"
        },
        "nmId": 62622901,
        "text": "Упаковано хорошо, будем надеется на честность, пересчитывать не стала. Довольна покупкой! Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-01T15:07:41Z",
        "updatedDate": "2023-08-16T10:09:43.008447684Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-02T11:12:26Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/b3daf03b-4ca3-4de6-b791-5bd0cd327c52_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/b3daf03b-4ca3-4de6-b791-5bd0cd327c52_ms.jpg"
            }
        ],
        "photo": [
            20058375
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 461
    },
    {
        "id": "OAZvvYAB3nxHumYkXbHR",
        "globalUserId": "",
        "wbUserId": 37809668,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Крис"
        },
        "nmId": 33478818,
        "text": "Покупаю только его.Но в магните цены на него ОГО.Здесь тоже конечно цены выросли.Но что поделать,буду брать ещё.Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-13T12:40:33Z",
        "updatedDate": "2023-08-16T10:09:40.496056540Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-13T12:50:03Z",
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/44052edf-bfb5-4ce1-888c-3ce56584cd2a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/44052edf-bfb5-4ce1-888c-3ce56584cd2a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2a8798ef-fa28-4f99-9d04-53cbd028cd9c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2a8798ef-fa28-4f99-9d04-53cbd028cd9c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/12b2c095-ceee-46fd-b809-9f47c61651ee_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/12b2c095-ceee-46fd-b809-9f47c61651ee_ms.jpg"
            }
        ],
        "photo": [
            51968111,
            51968130,
            51968146
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 460
    },
    {
        "id": "T2l1lYoBfuChzMpwebRo",
        "globalUserId": "11451508",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Тамара",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший заменитель сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T20:50:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 459
    },
    {
        "id": "aMnOf4oBKNxVyjcCA6F-",
        "globalUserId": "29787826",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Товар соответствует описанию",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T15:55:22Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 458
    },
    {
        "id": "42dtoYoBUYwa0QEfUK9j",
        "globalUserId": "19014169",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Марина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру второй раз, нрваится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T04:36:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 457
    },
    {
        "id": "jzJbrIoBU2re2XP7GK9D",
        "globalUserId": "24228547",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Яна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший сахар заменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T07:32:46Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 456
    },
    {
        "id": "_9gnR4oBUdF-4nlzWuAK",
        "globalUserId": "71912013",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Тамара",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "делю один пакет на два раза, очень",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T15:54:36Z",
        "updatedDate": "2023-09-06T08:44:34Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:44:34Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "19467769",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-02T07:03:03Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 455
    },
    {
        "id": "8qepqoUB6z8QKrUHLgn7",
        "globalUserId": "12899136",
        "wbUserId": 12899136,
        "wbUserDetails": {
            "name": "Таьяна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Пришёл вовремя, упакован хорошо, ничего не помялись.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-13T10:24:52Z",
        "updatedDate": "2023-08-16T10:09:44.690511882Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:51:09Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/60c190cc-98d7-4f58-a6b0-41ffd88f328e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/60c190cc-98d7-4f58-a6b0-41ffd88f328e_ms.jpg"
            }
        ],
        "photo": [
            78730382
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 454
    },
    {
        "id": "eUydpoEB73bXwc5ENXTG",
        "globalUserId": "",
        "wbUserId": 24823307,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "На такой стакан кофе ушло пол пакетика,сладко.Не горчит,нет постороннего привкуса,как будто положила сахар.Все отлично,мне нравится.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-27T19:22:08Z",
        "updatedDate": "2023-08-16T10:09:41.160271003Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-28T13:43:09Z",
            "metadata": null,
            "editable": true,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/2154ae70-4aae-4931-b54c-838158139101_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2154ae70-4aae-4931-b54c-838158139101_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/f7067b9d-179b-40ca-b346-5f0220cc26a0_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f7067b9d-179b-40ca-b346-5f0220cc26a0_ms.jpg"
            }
        ],
        "photo": [
            62060572,
            62060593
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 453
    },
    {
        "id": "qJUURocBv9_TjX_mSYjO",
        "globalUserId": "32059416",
        "wbUserId": 32059416,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Юлия"
        },
        "nmId": 138785478,
        "text": "все норм, пришел целый",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-03T07:45:47Z",
        "updatedDate": "2023-08-16T10:09:47.615277994Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-03T09:07:09Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/867af673-70f4-4bec-a6be-7f703b4c2907_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/867af673-70f4-4bec-a6be-7f703b4c2907_ms.jpg"
            }
        ],
        "photo": [
            94504210
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 452
    },
    {
        "id": "1uGIiIoBZx8DR9RA_xd0",
        "globalUserId": "14624303",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "вера",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хороший сахзам. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T08:36:34Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 451
    },
    {
        "id": "DOv7jooBwM8MKVoujfg6",
        "globalUserId": "12792339",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "by",
            "hasPhoto": false,
            "name": "Светлана"
        },
        "nmId": 138785478,
        "text": "Всё соответствует описанию",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T14:39:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 450
    },
    {
        "id": "FNOCkIoBuX2t2Z2-968z",
        "globalUserId": "94028815",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Юрий",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Товар хороший рекомендую !",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T21:46:57Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 449
    },
    {
        "id": "1YPRyooBUvtK502_nb9g",
        "globalUserId": "25151996",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Розалия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Как всегда все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T05:30:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 448
    },
    {
        "id": "8RJPU4kBUvtK502_e-3o",
        "globalUserId": "9484320",
        "wbUserId": 9484320,
        "wbUserDetails": {
            "country": "ru",
            "name": "Милонова",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Единственный сахарозаменитель который мне очень нравится по вкусу. Не горчит. Не имеет привкусов. Единственное соотношение 1 саще-2 чайные ложки сахара считаю не совсем верным. С обычным сахаром пью кофе с тремя чайными ложками или 2,5, а с сахарозаменителем с двумя пакетиками. Если  делать выпечку, то класть его нужно больше чем обычно указывается в рецептах чтобы было сладко. Но вкус лучший из всех конечно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-14T07:31:05Z",
        "updatedDate": "2023-08-16T10:09:51.015423581Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T07:33:12Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "93263145",
                "wbUserId": 93263145,
                "helpfulness": "minus",
                "time": "2023-07-15T12:00:39Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 447
    },
    {
        "id": "WB-WxYYB6z8QKrUHK9_C",
        "globalUserId": "19598106",
        "wbUserId": 19598106,
        "wbUserDetails": {
            "name": "Евгений",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Упаковано хорошо! Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-09T08:56:39Z",
        "updatedDate": "2023-08-16T10:09:46.482670340Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T08:53:35Z",
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/c5055a8f-74bb-4259-81d2-28cf9dec59b2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/c5055a8f-74bb-4259-81d2-28cf9dec59b2_ms.jpg"
            }
        ],
        "photo": [
            89997079
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 446
    },
    {
        "id": "h8KO_n4Bs0nmRRaKaJ_s",
        "globalUserId": "",
        "wbUserId": 43919650,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Очень хороший сахарозаменитель. Беру не первый раз, очень удобный, можно легко брать с собой в дорогу. Особо было приятно получить подарочек 🎁 очень вкусный зефир 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-15T18:04:12Z",
        "updatedDate": "2023-08-16T10:09:39.369464567Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-16T07:10:48Z",
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "text": "Очень рады, что вы остаётесь с нами, благодарим за покупку! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/412f452a-52a1-445c-ba03-3a9830634ce1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/412f452a-52a1-445c-ba03-3a9830634ce1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/01cf203a-0752-491e-b92f-f8582015bd70_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/01cf203a-0752-491e-b92f-f8582015bd70_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6771fb5d-c65b-44fc-a606-e828eb3a24ce_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6771fb5d-c65b-44fc-a606-e828eb3a24ce_ms.jpg"
            }
        ],
        "photo": [
            48091796,
            48091813,
            48091832
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 445
    },
    {
        "id": "Vao52YYBheedNkNu8lqT",
        "globalUserId": "21964831",
        "wbUserId": 21964831,
        "wbUserDetails": {
            "country": "RU",
            "name": "Мария",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Спасибо, все очень хорошо 👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-13T04:28:19Z",
        "updatedDate": "2023-08-16T10:09:46.605457378Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-13T11:27:27Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/be0984b1-4e95-47d5-9f87-c506b7ba920c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/be0984b1-4e95-47d5-9f87-c506b7ba920c_ms.jpg"
            }
        ],
        "photo": [
            90725078
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 444
    },
    {
        "id": "FrpcY4oBCaW1RxrS8tRt",
        "globalUserId": "31678511",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Настенька"
        },
        "nmId": 138785478,
        "text": "Коробка пришла в помятом состоянии, но на качество продукта это не повлияло.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-05T03:22:30Z",
        "updatedDate": "2023-09-05T05:03:46Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T05:03:46Z",
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 443
    },
    {
        "id": "dAfdK4cB5HDhYKpHccvV",
        "globalUserId": "8949097",
        "wbUserId": 8949097,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Венера"
        },
        "nmId": 33478818,
        "text": "Все отлично упаковано.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-29T05:35:46Z",
        "updatedDate": "2023-08-16T10:09:47.386679280Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-29T05:49:00Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/72f42003-88d3-4842-91db-f2687c7aef68_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/72f42003-88d3-4842-91db-f2687c7aef68_ms.jpg"
            }
        ],
        "photo": [
            93643694
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 442
    },
    {
        "id": "ptM0iYoBqd2abVcyBioM",
        "globalUserId": "23125206",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Екатерина"
        },
        "nmId": 138785478,
        "text": "Хороший заменитель сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T11:43:23Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 441
    },
    {
        "id": "Idi2lIoBqd2abVcyOqZk",
        "globalUserId": "33585701",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Виктория",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Рекомендую. Лучше сахара",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T17:21:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 440
    },
    {
        "id": "ZGYMnoEBIzqgZI3qogV7",
        "globalUserId": "",
        "wbUserId": 5437932,
        "wbUserDetails": {
            "name": "Людмила",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Действительно ничем не отличается от сахара по вкусу,супер! Я в процессе похудения и поэтому решила отказаться от сахара,но отказаться от кофе не могу)) спасибо! Очень выручили😙❤ теперь кофе только с ним!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-26T03:27:16Z",
        "updatedDate": "2023-08-16T10:09:41.151066671Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-29T12:47:31Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/50e6c061-84ee-4267-8a77-c46d80f150d6_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/50e6c061-84ee-4267-8a77-c46d80f150d6_ms.jpg"
            }
        ],
        "photo": [
            66387202
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 439
    },
    {
        "id": "-wJtKIgBKNxVyjcClvaT",
        "globalUserId": "33002849",
        "wbUserId": 33002849,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алена"
        },
        "nmId": 33478818,
        "text": "Всё отлично, упаковка в идеале. \nХотела сказать по поводу вкусовых качеств - возможно для тех, кто давно употребляет сахарозаменители, отличия от сахара нет. Но оно есть и очень чувствуется. Во-первых, фит парад сильно слаще в напитках, я насыпала 1/2 пакетика в капучино и не смогла его пить, это мега-сладко, видимо чайные ложки у того кто оценивал сладость огромные и с горкой\nВо-вторых, сильно чувствуется во рту сладкое послевкусие от стевии, из-за которого я не смогла её употреблять. \nВ-третьих, в выпечке всё вообще наоборот, делала творожные вафли, насыпала 3г как в рецепте, их не чувствуется от слова совсем, пока ешь, а когда уже проглотил, на языке остаётся этот неприятно сладкий налёт, о котором написала выше. \nТе, кто как я, всю жизнь ел обычный сахар, имейте в виду, что не всё так просто) но оценку качественному продукту, конечно, снижать за это не буду",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-17T06:37:31Z",
        "updatedDate": "2023-08-16T10:09:49.176762836Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-18T08:52:10Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "21801447",
                "wbUserId": 21801447,
                "helpfulness": "plus",
                "time": "2023-05-17T12:49:25Z"
            },
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-17T17:45:04Z"
            },
            {
                "globalUserId": "5007893",
                "wbUserId": 5007893,
                "helpfulness": "minus",
                "time": "2023-05-18T05:36:39Z"
            },
            {
                "globalUserId": "33002849",
                "wbUserId": 33002849,
                "helpfulness": "plus",
                "time": "2023-05-25T11:51:55Z"
            },
            {
                "globalUserId": "39343426",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-28T09:30:22Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 3
        },
        "rank": 438
    },
    {
        "id": "L1-ZhoQBbunnhI3gKwmC",
        "globalUserId": "",
        "wbUserId": 68323972,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Отличный продукт , буду брать ещё. Всё цело , упакован хорошо . Мне понравился",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-17T17:18:15Z",
        "updatedDate": "2023-08-16T10:09:43.381391973Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-18T09:32:45Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/3ce2d05e-645c-4c76-8ffc-823baec0a94c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3ce2d05e-645c-4c76-8ffc-823baec0a94c_ms.jpg"
            }
        ],
        "photo": [
            36488995
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 437
    },
    {
        "id": "5TKxIYMB65w5GC7jpvS3",
        "globalUserId": "",
        "wbUserId": 22384254,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Дарья",
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Самый вкусны скхлразмннитель. Пакетики удобно брать с собой и добавлять в кофе ☕️ Советую к покупке! Вышло дешевле, чем в магазине",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-09T10:00:34Z",
        "updatedDate": "2023-08-16T10:09:41.995522332Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-07T10:07:53Z",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2b3e3676-200b-4b38-b438-fc736c2cffc9_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2b3e3676-200b-4b38-b438-fc736c2cffc9_ms.jpg"
            }
        ],
        "photo": [
            72081217
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 436
    },
    {
        "id": "t_SfjooBUdF-4nlzmbbg",
        "globalUserId": "5455612",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Прекрасно всё упаковано!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T12:58:59Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 435
    },
    {
        "id": "kXvr-4QBIq9DQExlz2XP",
        "globalUserId": "",
        "wbUserId": 13645942,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Надежда"
        },
        "nmId": 33478818,
        "text": "Очень приятно что маленький подарочек положили. Спасибо большое.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-10T12:04:06Z",
        "updatedDate": "2023-08-16T10:09:44.073745943Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-14T08:16:01Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a79d37ed-61ad-435e-918f-ed62effebe01_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a79d37ed-61ad-435e-918f-ed62effebe01_ms.jpg"
            }
        ],
        "photo": [
            60596592
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 434
    },
    {
        "id": "aX68N4UB6z8QKrUHrNMN",
        "globalUserId": "",
        "wbUserId": 9973434,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алла",
            "country": "BY"
        },
        "nmId": 33478818,
        "text": "Брала в первый раз. Очень понравился, буду заказывать ещё)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-22T02:49:49Z",
        "updatedDate": "2023-08-16T10:09:44.330592668Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-22T05:16:02Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/9c7ebdc5-9475-45a8-83ce-0f0a631489e4_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/9c7ebdc5-9475-45a8-83ce-0f0a631489e4_ms.jpg"
            }
        ],
        "photo": [
            72293419
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 433
    },
    {
        "id": "2pNcAn8B9I0UHoorARLz",
        "globalUserId": "",
        "wbUserId": 32036307,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Любовь"
        },
        "nmId": 33478818,
        "text": "Покупала ранее фитпарад синего цвета (стевия, что ли🤔), мне не понравился, чувствовала привкус и горчинку. Отдала подруге. Этот заказывала с опаской. Но нет. Этот мне понравился. Действительно сладкий и нет странного послевкусия. Порадовал небольшой подарочек-зефирка☺️спасибо большое! Пытаюсь вовсе слезть с сахара. Думаю, что теперь это дело пойдёт активнее😄",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-16T11:47:38Z",
        "updatedDate": "2023-08-16T10:09:39.381687214Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-18T10:57:48Z",
            "metadata": null,
            "text": "Очень рады, что товар пришёлся вам по вкусу! ",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/041cbab4-9e00-442a-b147-73d7f22153ec_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/041cbab4-9e00-442a-b147-73d7f22153ec_ms.jpg"
            }
        ],
        "photo": [
            50743825
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 432
    },
    {
        "id": "GM5eBokBqdFzcemqU_Nv",
        "globalUserId": "19691252",
        "wbUserId": 19691252,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Оксана"
        },
        "nmId": 138785478,
        "text": "Очень хороший заменитель сахара 👍🏼благодаря ему перестала пить кофе, потому в кофе он мне совсем невкусный, а пить несладкий кофе совсем не люблю 😁 в чае по вкусу как обычный Сахар. Но самое интересное, что предыдущая упаковка была менее сладкая. На кружку приходилось класть 2 пакетика. А крайняя упаковка, которую получила пару дней назад, достаточно 1 пакетика на кружку 350 мл. Отличный заменитель, не мучают инсулиновые качели и по вкусу полное удовлетворение 👍🏼👍🏼👍🏼",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-29T08:56:32Z",
        "updatedDate": "2023-08-16T10:09:50.461349653Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T08:58:29Z",
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 431
    },
    {
        "id": "6F8W24IBjTOCcEakE4IY",
        "globalUserId": "",
        "wbUserId": 16898315,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Людмила",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Добавляла в чай и в цикорий с молоком. Нормальный подсластитель. Только пришла коробочка совсем помятая, не упакована, а просто как в магазине с полочки взята без всякой упаковки . Вот так складывается негатив о WB.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-26T16:57:13Z",
        "updatedDate": "2023-08-16T10:09:41.835823055Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-29T08:45:30Z",
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 43831233,
                "helpfulness": "plus",
                "time": "2022-08-28T14:34:55.500726054Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 22767349,
                "helpfulness": "plus",
                "time": "2022-08-31T21:30:08.193314978Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/d4d170ed-dd12-4dc7-9955-b0f582f48fd7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d4d170ed-dd12-4dc7-9955-b0f582f48fd7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/b38e3099-4b8c-4898-8a56-2a1898ca6989_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b38e3099-4b8c-4898-8a56-2a1898ca6989_ms.jpg"
            }
        ],
        "photo": [
            75267448,
            75267459
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 430
    },
    {
        "id": "FvRPvIoBAtKOitoBaTBa",
        "globalUserId": "43002946",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Как всегда на высоте",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T09:53:55Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 429
    },
    {
        "id": "xGgpoooBUYwa0QEffRUf",
        "globalUserId": "96004429",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Татьяна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший подсластитель.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T08:02:23Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 428
    },
    {
        "id": "pBnKW4oBqdFzcemqEf4b",
        "globalUserId": "57560718",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "За такую цену, просто супер!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T16:04:44Z",
        "updatedDate": "2023-09-04T12:42:55Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T12:42:55Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 427
    },
    {
        "id": "u06EXIoBpT2rITcG2iAB",
        "globalUserId": "63701043",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Екатерина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Все отлично, беру его всегда",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T19:28:45Z",
        "updatedDate": "2023-09-04T04:52:23Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T04:52:23Z",
            "text": "Благодарим за выбор нашего продукта!",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 426
    },
    {
        "id": "bArkooYBfnpWqlBe_JxM",
        "globalUserId": "69967647",
        "wbUserId": 69967647,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Рябова"
        },
        "nmId": 33478818,
        "text": "Все пришло целое, спасибо ☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-02T15:16:01Z",
        "updatedDate": "2023-08-16T10:09:46.250406681Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-03T11:39:23Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/96e620df-7a15-4244-8fd6-9488616781c5_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/96e620df-7a15-4244-8fd6-9488616781c5_ms.jpg"
            }
        ],
        "photo": [
            88776032
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 425
    },
    {
        "id": "d3KKb4YBheedNkNuRvYJ",
        "globalUserId": "28450757",
        "wbUserId": 28450757,
        "wbUserDetails": {
            "name": "Ландыш",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Рекомендую! За 199 руб,отлично!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-20T15:56:18Z",
        "updatedDate": "2023-08-16T10:09:45.921368086Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-22T07:24:11Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/879ccbb4-57a1-4b32-a039-bf6e4a7efd83_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/879ccbb4-57a1-4b32-a039-bf6e4a7efd83_ms.jpg"
            }
        ],
        "photo": [
            87027816
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 424
    },
    {
        "id": "6tHKYn8BOaYV4f1xGjcG",
        "globalUserId": "",
        "wbUserId": 23621519,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Мне нравится этот сахарозаменитель,беру всегда только его, цена здесь радует 👍🏽за зефирку отдельное Спасибо, было очень приятно ее получить в подарок ☺️ коробка пришла целая,быстро,срок отличный.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-07T05:11:26Z",
        "updatedDate": "2023-08-16T10:09:39.660286182Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-09T08:41:22Z",
            "text": "Благодарим за высокую оценку нашего товара!",
            "metadata": null,
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/ba34a53a-531b-4265-b8ce-d3241a5aafa1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ba34a53a-531b-4265-b8ce-d3241a5aafa1_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/0bb2d9f2-15f7-4910-95f5-ef34ee245d12_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0bb2d9f2-15f7-4910-95f5-ef34ee245d12_ms.jpg"
            }
        ],
        "photo": [
            49355424,
            49355440
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 423
    },
    {
        "id": "FsB2S4cBJ99b87NGLN58",
        "globalUserId": "13186510",
        "wbUserId": 13186510,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Виктория",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-04T08:50:49Z",
        "updatedDate": "2023-08-16T10:09:47.684833856Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T08:53:58Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5af1baf0-470a-4376-8750-d284acced4d8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5af1baf0-470a-4376-8750-d284acced4d8_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/87215c5e-6894-43d5-820f-d1b323412472_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/87215c5e-6894-43d5-820f-d1b323412472_ms.jpg"
            }
        ],
        "photo": [
            94696452,
            94696453
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 422
    },
    {
        "id": "379Ei4ABaOM9ENKQWLYi",
        "globalUserId": "",
        "wbUserId": 14267546,
        "wbUserDetails": {
            "name": "Аза",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 31468103,
        "text": "Все хорошо напечатано,коробки целые,количество Саше соответствует.Очень довольна товаром.Спасибо большое.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-03T18:52:32Z",
        "updatedDate": "2023-08-16T10:09:40.343052096Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-19T06:00:18Z",
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/6dec0dab-6340-4f26-bcda-de52529f6132_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/6dec0dab-6340-4f26-bcda-de52529f6132_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/4762590a-594a-4d7b-a44f-1690b4b822a6_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4762590a-594a-4d7b-a44f-1690b4b822a6_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/f9fbdbd4-9844-4a6b-84d7-f07498d80c20_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f9fbdbd4-9844-4a6b-84d7-f07498d80c20_ms.jpg"
            }
        ],
        "photo": [
            60411338,
            60411354,
            60411374
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 421
    },
    {
        "id": "-qtJhYoBnnhI58-2679D",
        "globalUserId": "34231764",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Инга",
            "country": "RU"
        },
        "nmId": 62622901,
        "text": "Всё чётко, хороший фит.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T17:28:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 420
    },
    {
        "id": "PuKPw4oBCaW1RxrS0gNw",
        "globalUserId": "31183247",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Все пришло отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T19:41:37Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 419
    },
    {
        "id": "UCdQr4UBheedNkNunkJ3",
        "globalUserId": "91131738",
        "wbUserId": 91131738,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 33478818,
        "text": "Хороший заменитель сахара, в меру сладкий 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-14T08:06:14Z",
        "updatedDate": "2023-08-16T10:09:44.728363741Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-26T07:58:52Z",
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/f253e969-a607-4ea0-8cb0-234e0ab6140e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f253e969-a607-4ea0-8cb0-234e0ab6140e_ms.jpg"
            }
        ],
        "photo": [
            78874058
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 418
    },
    {
        "id": "oIqaf4oBqVKcxP1mId6T",
        "globalUserId": "41624324",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Всегда покупаю. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T14:58:42Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 417
    },
    {
        "id": "uvrjsooBwM8MKVouRtZ8",
        "globalUserId": "105253230",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 31468103,
        "text": "Отличное,рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-20T13:59:14Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 416
    },
    {
        "id": "_SkULIIBjTOCcEak9_Or",
        "globalUserId": "",
        "wbUserId": 26723564,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Пришло без упаковки, но не мятый, в идеальном состоянии. Спасибо большое, покупкой довольна))))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-23T17:22:27Z",
        "updatedDate": "2023-08-16T10:09:41.467811895Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-09T10:17:56Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 5803747,
                "helpfulness": "plus",
                "time": "2022-07-24T15:14:10Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/0ac07b5b-98c4-45e6-9d20-dab5e7d7f908_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0ac07b5b-98c4-45e6-9d20-dab5e7d7f908_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/a97d4e8b-9f34-4d97-ab92-99319639407a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a97d4e8b-9f34-4d97-ab92-99319639407a_ms.jpg"
            }
        ],
        "photo": [
            50852490,
            50852509
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 415
    },
    {
        "id": "Lrhno4oBnnhI58-2Vpa-",
        "globalUserId": "36316873",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Виктория",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Самый любимый сахзам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T13:49:33Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 414
    },
    {
        "id": "iIE1DYYB5HDhYKpHFSqI",
        "globalUserId": "57011581",
        "wbUserId": 57011581,
        "wbUserDetails": {
            "name": "Анна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё отлично , срок годности хороший",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-01T13:40:28Z",
        "updatedDate": "2023-08-16T10:09:45.252749013Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-02T05:57:05Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ce6f4bc4-45ad-4296-a7d6-b7a28c902a1f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ce6f4bc4-45ad-4296-a7d6-b7a28c902a1f_ms.jpg"
            }
        ],
        "photo": [
            81812204
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 413
    },
    {
        "id": "OMsTUIoBZx8DR9RAP4e9",
        "globalUserId": "64246363",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Качество хорошее. Не горчит",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T09:29:13Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 412
    },
    {
        "id": "PdR-oooBCaW1RxrSR4yQ",
        "globalUserId": "53044994",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Людмила",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все ок, всегда беру.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T09:34:59Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 411
    },
    {
        "id": "v18hToEBjeZkdKc5PXDi",
        "globalUserId": "",
        "wbUserId": 8724435,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ксения",
            "country": "RU"
        },
        "nmId": 62622901,
        "text": "Самый лучший сахзам!!🥰🥰 Спасибо большое, положили чуть больше пакетиков.Но главное не меньше, спасибо огромное!!! 😌☺☺☺",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-10T15:00:10Z",
        "updatedDate": "2023-08-16T10:09:40.962958565Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-16T07:47:59Z",
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/8d70f478-4088-4820-a801-181702981456_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/8d70f478-4088-4820-a801-181702981456_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/eb22fbc1-6a31-4721-b183-06a2e18718bc_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/eb22fbc1-6a31-4721-b183-06a2e18718bc_ms.jpg"
            }
        ],
        "photo": [
            57901380,
            57901399
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 410
    },
    {
        "id": "i2yd034BZBLx1OSDK6s_",
        "globalUserId": "",
        "wbUserId": 24352714,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Александра"
        },
        "nmId": 33478818,
        "text": "Отличный заменитель сахара, к производителю и к качеству продукта претензий нет. Единственное что не понравилось, так это доставка, т.к. упаковка была вся помята (хорошо, что внутри всё саше были целые).",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-07T09:56:39Z",
        "updatedDate": "2023-08-16T10:09:39.225681205Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-08T10:31:25Z",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "text": "Нам очень жаль, что при обработке заказа случаются такие казусы, но к сожалению мы не можем контролировать работу интернет-площадки, а так же процессы сборки и доставки ваших заказов, т.к. этим занимается сам маркетплейс!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/0217642b-fd67-4876-a300-477cf31d836f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0217642b-fd67-4876-a300-477cf31d836f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/0097070c-32bf-4bfe-aae3-33615ed70a78_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0097070c-32bf-4bfe-aae3-33615ed70a78_ms.jpg"
            }
        ],
        "photo": [
            52788485,
            52788500
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 409
    },
    {
        "id": "aeuSN4MBTTPU1bwQ7BsG",
        "globalUserId": "",
        "wbUserId": 40381258,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Елена"
        },
        "nmId": 31468103,
        "text": "Хороший сахарозаменитель, только упаковки пришли помятые и пыльные.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-13T15:58:39Z",
        "updatedDate": "2023-08-16T10:09:42.067009391Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-09-14T05:14:35Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b8b4a7f3-1498-43ca-9d6d-a95ec23590f0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b8b4a7f3-1498-43ca-9d6d-a95ec23590f0_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/caab1152-c5ac-453f-8a3c-4aab4a94915f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/caab1152-c5ac-453f-8a3c-4aab4a94915f_ms.jpg"
            }
        ],
        "photo": [
            69289200,
            69289214
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 408
    },
    {
        "id": "ZZL_SokBwM8MKVouGT3r",
        "globalUserId": "20817943",
        "wbUserId": 20817943,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Сахзам понравился, действительно без привкуса химозы, чисто сладкий вкус и все. Раньше когда отказалась от сахара, пила несладкий чай но требовалось что то к чаю. А теперь и \"к чаю\" покупать нет нужды, т.к. чай уже сдадкий!! и моя потребность в сладком удовлетворена сполна. Рекомендую тем, кто хочет скинуть вес и следит за питанием",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T16:46:20Z",
        "updatedDate": "2023-08-16T10:09:50.946325233Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T07:20:15Z",
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "editable": true,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 407
    },
    {
        "id": "O1yFSIcB6z8QKrUHkIAo",
        "globalUserId": "19947103",
        "wbUserId": 19947103,
        "wbUserDetails": {
            "name": "Нина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Дата прям свежая😋",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-03T19:08:45Z",
        "updatedDate": "2023-08-16T10:09:47.660889142Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-04T05:16:19Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "26975413",
                "wbUserId": 26975413,
                "helpfulness": "plus",
                "time": "2023-04-05T01:51:15Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/86208504-aa99-4da1-9cad-5702d4bf146a_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/86208504-aa99-4da1-9cad-5702d4bf146a_ms.jpg"
            }
        ],
        "photo": [
            94640307
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 406
    },
    {
        "id": "S2n4F4oBXuPSsPE1ucPF",
        "globalUserId": "3566410",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "EEE",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Обычно брала в магазинах после отзывов на вб о подделках. Использую много лет, поэтому могу понять сахзам ок или нет\nКоробка пришла без повреждений, по вкусу все в порядке 👌🏽",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-21T12:01:31Z",
        "updatedDate": "2023-08-22T13:24:00Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-22T13:24:00Z",
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за доверие нашему бренду!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 405
    },
    {
        "id": "PFn1O4oBCYLDkq7NjYxF",
        "globalUserId": "20788458",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Анастасия",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Беру раз 5, все прекрасно )",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T11:44:23Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 404
    },
    {
        "id": "x9ejXYEBEUCUNtmVT9Wv",
        "globalUserId": "",
        "wbUserId": 46733205,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 62622901,
        "text": "Беру не первый раз. Все отлично. Замена обычному сахару на все 100%. Спасибо ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-13T15:16:32Z",
        "updatedDate": "2023-08-16T10:09:40.996235679Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-15T10:54:46Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/cc1ad1e1-7a82-41cd-b2dc-99bc1d8a7d39_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/cc1ad1e1-7a82-41cd-b2dc-99bc1d8a7d39_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/54eb5263-4fda-4d75-8e71-0fc21b0e892e_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/54eb5263-4fda-4d75-8e71-0fc21b0e892e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/fe651ee8-4ee2-4638-bd6c-10be61f30616_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/fe651ee8-4ee2-4638-bd6c-10be61f30616_ms.jpg"
            }
        ],
        "photo": [
            58325001,
            58325014,
            58325028
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 403
    },
    {
        "id": "FX-TVIoBXuPSsPE1mC7i",
        "globalUserId": "42221275",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Богатырева Ирина Васильевна"
        },
        "nmId": 33478818,
        "text": "Отличный сахарозаменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T06:27:54Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 402
    },
    {
        "id": "qezqOoYBtPujtG5sQjh-",
        "globalUserId": "96707000",
        "wbUserId": 96707000,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё пришло, хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-10T10:41:16Z",
        "updatedDate": "2023-08-16T10:09:45.511036967Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-10T10:58:40Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/52b3bd45-fa4b-4de7-8bed-bbb3482c19a3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/52b3bd45-fa4b-4de7-8bed-bbb3482c19a3_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/f653281e-74d2-4b7f-a30f-fba90c78c24e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f653281e-74d2-4b7f-a30f-fba90c78c24e_ms.jpg"
            }
        ],
        "photo": [
            85247811,
            85247812
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 401
    },
    {
        "id": "DyZsE4cBMBFuPskoeKaD",
        "globalUserId": "59037144",
        "wbUserId": 59037144,
        "wbUserDetails": {
            "name": "Вера",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-24T11:41:29Z",
        "updatedDate": "2023-08-16T10:09:47.134159374Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T10:55:30Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7cb8eec8-5cff-4b33-88ab-2d3c9321db03_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7cb8eec8-5cff-4b33-88ab-2d3c9321db03_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/a486a66e-ce30-4d1b-bc01-1e0e6c094f9d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/a486a66e-ce30-4d1b-bc01-1e0e6c094f9d_ms.jpg"
            }
        ],
        "photo": [
            92814786,
            92814787
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 400
    },
    {
        "id": "P7-JXooBuX2t2Z2-DVRp",
        "globalUserId": "60483857",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Барно"
        },
        "nmId": 138785478,
        "text": "Очень довольна не горчит",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-04T04:52:35Z",
        "updatedDate": "2023-09-04T12:40:34Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T12:40:34Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 399
    },
    {
        "id": "whrHPYEBUcbpNqImSIGV",
        "globalUserId": "",
        "wbUserId": 32391724,
        "wbUserDetails": {
            "name": "Олеся",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Смесь пришла в целом виде. Упаковано было хорошо. Упаковка целая. Все на месте.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-07T10:47:58Z",
        "updatedDate": "2023-08-16T10:09:40.913405862Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-07T12:52:58Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 32391724,
                "helpfulness": "plus",
                "time": "2022-06-07T13:35:13Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/91134bff-0e22-4273-8120-f2a79f8b258c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/91134bff-0e22-4273-8120-f2a79f8b258c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/f51bcbef-768b-4740-8cf9-c143e182ca58_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f51bcbef-768b-4740-8cf9-c143e182ca58_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/9ca43600-5b12-486a-914c-3770dddd69f0_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/9ca43600-5b12-486a-914c-3770dddd69f0_ms.jpg"
            }
        ],
        "photo": [
            68486867,
            68486882,
            68486904
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 398
    },
    {
        "id": "ddSDO4oBUdF-4nlzik3_",
        "globalUserId": "31005549",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Отличный сахзам, без горечи",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T09:39:51Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 397
    },
    {
        "id": "agn9RYoBU2re2XP7gv76",
        "globalUserId": "16140821",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "приобретаю всегда, советую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T10:29:17Z",
        "updatedDate": "2023-08-30T10:57:24Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-30T10:57:24Z",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за доверие нашему бренду!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 396
    },
    {
        "id": "5_yjgoUBJ99b87NGKoX3",
        "globalUserId": "",
        "wbUserId": 751191,
        "wbUserDetails": {
            "name": "Инна",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Все как обычно. Пришёл хорошо упакованным.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-05T15:53:30Z",
        "updatedDate": "2023-08-16T10:09:44.548033009Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-10T09:54:05Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/d5ae161e-80d4-47ad-884b-4eef42b465fc_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d5ae161e-80d4-47ad-884b-4eef42b465fc_ms.jpg"
            }
        ],
        "photo": [
            77465990
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 395
    },
    {
        "id": "Z69SYIMBAN9IxNcczUqa",
        "globalUserId": "",
        "wbUserId": 37640811,
        "wbUserDetails": {
            "country": "RU",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Товар соответствует описанию. Хорошо запакованы. Правда коробка мятая. А содержимое всё сохранено.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-21T13:53:02Z",
        "updatedDate": "2023-08-16T10:09:42.187331787Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T07:09:25Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/7e10e6f2-e0be-43c7-84a5-62b5d79aa194_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/7e10e6f2-e0be-43c7-84a5-62b5d79aa194_ms.jpg"
            }
        ],
        "photo": [
            72123774
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 394
    },
    {
        "id": "_gAmaYYBtPujtG5stnc4",
        "globalUserId": "59666945",
        "wbUserId": 59666945,
        "wbUserDetails": {
            "name": "Регина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Приехал в хорошем состоянии",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-19T10:09:50Z",
        "updatedDate": "2023-08-16T10:09:45.866894496Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-20T10:52:40Z",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9278fbd5-c43c-4423-9b50-f418f657ea88_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9278fbd5-c43c-4423-9b50-f418f657ea88_ms.jpg"
            }
        ],
        "photo": [
            86779660
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 393
    },
    {
        "id": "dSDZiYMBCjydpYzr82Dh",
        "globalUserId": "",
        "wbUserId": 4693063,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Виктория"
        },
        "nmId": 31468103,
        "text": "Моя любимая сладость! Доставка быстрая, упаковки не повреждены! Всё супер! Огромное спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-29T15:25:05Z",
        "updatedDate": "2023-08-16T10:09:42.322152832Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-03T13:36:25Z",
            "metadata": null,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/5818b72a-e692-4e7f-9d12-e356be9b3e2f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/5818b72a-e692-4e7f-9d12-e356be9b3e2f_ms.jpg"
            }
        ],
        "photo": [
            71334093
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 392
    },
    {
        "id": "j738wYYBBhsOOdMGa0J5",
        "globalUserId": "62259109",
        "wbUserId": 62259109,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ульяна",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Пришел хорошо упакован",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-08T16:09:51Z",
        "updatedDate": "2023-08-16T10:09:46.461249642Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-09T07:48:38Z",
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e62931b7-3f53-4adb-ba25-17fc38205280_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e62931b7-3f53-4adb-ba25-17fc38205280_ms.jpg"
            }
        ],
        "photo": [
            89913426
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 391
    },
    {
        "id": "Md2Bt4oBCaW1RxrSZFCa",
        "globalUserId": "31422138",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Качество хорошее.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T11:30:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 390
    },
    {
        "id": "8Z6FrYoBqVKcxP1mXpd9",
        "globalUserId": "58358085",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Отличный фитпарад",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T12:58:33Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 389
    },
    {
        "id": "ri4joooBU2re2XP7Pluz",
        "globalUserId": "36307083",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Покупаю постоянно👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T07:55:33Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 388
    },
    {
        "id": "trY-nooBnnhI58-2eYpb",
        "globalUserId": "36546025",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Дмитрий",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Обычный как всегда",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T13:46:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 387
    },
    {
        "id": "KnB3OYoBqVKcxP1mqqxb",
        "globalUserId": "29259439",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Мария",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "👍использую выпечке и в чай",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T00:07:38Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 386
    },
    {
        "id": "yhTbw4kBpT2rITcGitPM",
        "globalUserId": "89206550",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Благодарю изготовителя, после отказа от сахара и кондитерских изделий с сахаром, ушло 5 кг лишнего веса. Всю выпечку сейчас только использую \"Фит парад\" и цельнозерновую или рисовую муку, заказываю тоже с Weldberis.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-05T04:01:32Z",
        "updatedDate": "2023-08-26T06:23:40Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "23916772",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-26T06:23:40Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 385
    },
    {
        "id": "6M4iqoQBXuaGXcNpW62b",
        "globalUserId": "",
        "wbUserId": 96436049,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Андрей Герцен"
        },
        "nmId": 33478818,
        "text": "Всё как заявлено, спасибо продавцу)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-24T14:54:49Z",
        "updatedDate": "2023-08-16T10:09:43.535862375Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-25T06:06:22Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/5bce515b-818f-4428-90c2-ac7220624f7c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/5bce515b-818f-4428-90c2-ac7220624f7c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/5f973408-d566-446f-873c-784b747d9972_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/5f973408-d566-446f-873c-784b747d9972_ms.jpg"
            }
        ],
        "photo": [
            44329797,
            44329798
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 384
    },
    {
        "id": "CYsUt4cBWg4f_fchlnVG",
        "globalUserId": "59703513",
        "wbUserId": 59703513,
        "wbUserDetails": {
            "name": "Анастасия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Понравилось👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-25T06:23:12Z",
        "updatedDate": "2023-08-16T10:09:48.537143170Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-25T06:42:23Z",
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "59703513",
                "wbUserId": 59703513,
                "helpfulness": "plus",
                "time": "2023-04-25T07:04:25Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/24e207f0-b487-46f3-b05c-4f6e9a0b5d72_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/24e207f0-b487-46f3-b05c-4f6e9a0b5d72_ms.jpg"
            }
        ],
        "photo": [
            98157912
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 383
    },
    {
        "id": "366TiooBnnhI58-2tCnm",
        "globalUserId": "41937499",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Мадина"
        },
        "nmId": 31468103,
        "text": "Довольна покупкой.👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T18:07:31Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 382
    },
    {
        "id": "WZQyi4oBXuPSsPE1Ycth",
        "globalUserId": "52266229",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Аксана",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Всегда заказываю 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T21:00:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 381
    },
    {
        "id": "wsRdaooBuX2t2Z2-TwVo",
        "globalUserId": "60576800",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Дарина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "хорошая замена сахару",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T12:00:15Z",
        "updatedDate": "2023-09-06T12:35:16Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T12:35:16Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 380
    },
    {
        "id": "UpnLNYIBTTPU1bwQEseN",
        "globalUserId": "",
        "wbUserId": 21118510,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Фит парадом пользуюсь давно. В такой упаковке беру первый раз. Удобно дозировано",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-25T14:37:57Z",
        "updatedDate": "2023-08-16T10:09:41.495107625Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-09T08:16:37Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/787cc7f5-b16f-4b0b-b5a7-29aad0d3870e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/787cc7f5-b16f-4b0b-b5a7-29aad0d3870e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/57bdc563-2c5a-4a37-a16f-c85283e47384_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/57bdc563-2c5a-4a37-a16f-c85283e47384_ms.jpg"
            }
        ],
        "photo": [
            61900291,
            61900311
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 379
    },
    {
        "id": "oPeYq4EBuVlYHVlwX3fz",
        "globalUserId": "",
        "wbUserId": 18481532,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "Сам заменитель ещё не пробовала. Коробка только пришла мятая, но внутри ничего не повредилось, пакетики целые, влага не попала.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-28T18:34:58Z",
        "updatedDate": "2023-08-16T10:09:41.169302737Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-29T12:08:19Z",
            "metadata": null,
            "editable": true,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a3924a6f-c4e7-401a-afbc-48baf274dc0c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a3924a6f-c4e7-401a-afbc-48baf274dc0c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/212cbc38-2052-4347-8dd3-febcd4ecb859_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/212cbc38-2052-4347-8dd3-febcd4ecb859_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/613f8ff2-7f0a-4ad8-b31d-9399e6c979ea_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/613f8ff2-7f0a-4ad8-b31d-9399e6c979ea_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/c904e24c-2570-4461-a813-632cc5f30224_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c904e24c-2570-4461-a813-632cc5f30224_ms.jpg"
            }
        ],
        "photo": [
            61564271,
            61564286,
            61564301,
            61564316
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 378
    },
    {
        "id": "dFDZZ4oBUYwa0QEfwGBq",
        "globalUserId": "34173993",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Мария",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Нравится этот сах зам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T00:17:18Z",
        "updatedDate": "2023-09-06T07:45:33Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T07:45:33Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 377
    },
    {
        "id": "NfkFr4oBwM8MKVouR29g",
        "globalUserId": "86652299",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "любимый сахзам 💋",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T19:57:53Z",
        "updatedDate": "2023-09-20T02:18:21Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "39347781",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-09-20T02:18:21Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 376
    },
    {
        "id": "vWqY34YBk9xzjhYZaP8t",
        "globalUserId": "33107853",
        "wbUserId": 33107853,
        "wbUserDetails": {
            "name": "Лилия",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Товар пришёл быстро",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-14T10:09:13Z",
        "updatedDate": "2023-08-16T10:09:46.659581855Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-14T13:04:37Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/11aa6ebd-f0e8-4e14-9617-177ad5693c5c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/11aa6ebd-f0e8-4e14-9617-177ad5693c5c_ms.jpg"
            }
        ],
        "photo": [
            90969403
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 375
    },
    {
        "id": "pNEPnocBKNxVyjcCVzoc",
        "globalUserId": "33849679",
        "wbUserId": 33849679,
        "wbUserDetails": {
            "country": "ru",
            "name": "Прибыткова Валентина",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Все огонечек",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-20T09:46:58Z",
        "updatedDate": "2023-08-16T10:09:48.365683684Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-20T11:35:47Z",
            "createDate": "2023-04-20T11:35:47Z",
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/16d94228-4c6c-4781-9414-16da7af4aebd_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/16d94228-4c6c-4781-9414-16da7af4aebd_ms.jpg"
            }
        ],
        "photo": [
            97343050
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 374
    },
    {
        "id": "TKXIDIoBqd2abVcyGwJS",
        "globalUserId": "38370790",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ирина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "ФитПарадом пользуюсь уже много лет, но в магазинах стало сложно найти, да и цены... Хорошо, что можно заказать здесь.  Да ещё и 90 саше. Проверила и правда 90.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-19T07:52:35Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 373
    },
    {
        "id": "u3Q4vIoBpT2rITcGbZ9c",
        "globalUserId": "44737054",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру постоянно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T09:28:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 372
    },
    {
        "id": "A4xHo4YBheedNkNuVQWL",
        "globalUserId": "50395139",
        "wbUserId": 50395139,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 31468103,
        "text": "Безумно в первый  раз",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-02T17:03:27Z",
        "updatedDate": "2023-08-16T10:09:46.257977973Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-03T11:37:39Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/85297d7e-bd02-4168-bc42-d362d204403f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/85297d7e-bd02-4168-bc42-d362d204403f_ms.jpg"
            }
        ],
        "photo": [
            88799775
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 371
    },
    {
        "id": "_SEYzIoBBgFQoSo06VlQ",
        "globalUserId": "48960120",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Людмила",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Беру постоянно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T11:28:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 370
    },
    {
        "id": "z5cxy4oBmorVKbOQv4Xf",
        "globalUserId": "26204917",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Оксана",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Беру регулярно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T07:15:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 369
    },
    {
        "id": "d4DEB38BZBLx1OSDqidA",
        "globalUserId": "",
        "wbUserId": 32668183,
        "wbUserDetails": {
            "name": "Евгения",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Отличный продукт и по составу (сукралоза, экстракт стевии, эритрит), и по форме (порошок) : очень удобно подслушивать творог, кондитерские изделия👍 брала за 140₽",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-17T13:00:03Z",
        "updatedDate": "2023-08-16T10:09:39.400514136Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-18T10:45:43Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/285b7986-fef0-4400-8d0c-9232ac4b4afa_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/285b7986-fef0-4400-8d0c-9232ac4b4afa_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/cddc6b88-9685-471d-8f44-623ce8e65dbf_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/cddc6b88-9685-471d-8f44-623ce8e65dbf_ms.jpg"
            }
        ],
        "photo": [
            51752249,
            51752268
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 368
    },
    {
        "id": "xE-V-H8B9g1jf2Z4myvC",
        "globalUserId": "",
        "wbUserId": 59467081,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Татьяна"
        },
        "nmId": 33478818,
        "text": "Это самый вкусный сахзам который я пробовала,постороннего привкуса нет совсем,как будто чай с сахаром пьёшь.Буду теперь только его брать.Посылка пришла очень быстро,коробка слегка помята,но это не критично.Всем советую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-05T07:17:08Z",
        "updatedDate": "2023-08-16T10:09:39.909084500Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-05T08:17:49Z",
            "editable": true,
            "employeeId": 0,
            "text": "Очень рады, что товар пришёлся вам по вкусу ",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/ba8d9030-da37-423f-bc8d-e77aa40879b6_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ba8d9030-da37-423f-bc8d-e77aa40879b6_ms.jpg"
            }
        ],
        "photo": [
            56118245
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 367
    },
    {
        "id": "Lzspw4oBU2re2XP71uDp",
        "globalUserId": "55311570",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Гюлбажи"
        },
        "nmId": 33478818,
        "text": "Очень приятный",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T17:50:14Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 366
    },
    {
        "id": "2MXef4oBCaW1RxrS08ZX",
        "globalUserId": "7987469",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Надежда"
        },
        "nmId": 33478818,
        "text": "Очень удобная вещь",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T16:13:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 365
    },
    {
        "id": "fmzBj4oBUvtK502_-nZr",
        "globalUserId": "61611071",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Галина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Отличное качество",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T18:16:09Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 364
    },
    {
        "id": "l8OEb4oBKNxVyjcCAovX",
        "globalUserId": "5513672",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Удобно пользоваться",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T12:00:37Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 363
    },
    {
        "id": "gWZYpYIBE-Uw035eVRaY",
        "globalUserId": "",
        "wbUserId": 22129935,
        "wbUserDetails": {
            "name": "Саби",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Все отлично как всегда) заказываю раз в месяц стабильно ) хочу передать привет ГСДшицам 😁 девочки держимся 🐣",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-16T06:30:05Z",
        "updatedDate": "2023-08-16T10:09:41.712408922Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-16T08:49:44Z",
            "editable": true,
            "supplierId": 237660,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/533cd65b-d217-4fc2-8dd2-35827630a2a9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/533cd65b-d217-4fc2-8dd2-35827630a2a9_ms.jpg"
            }
        ],
        "photo": [
            67729397
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 362
    },
    {
        "id": "ae36BH0BTchSwNhtR8Fi",
        "globalUserId": "",
        "wbUserId": 47806046,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ксения",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Коробка пришла немного помятой, но это не важно. Товар цел и я рада. Ради интереса пересчитала пакетики, ровно 60 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2021-11-09T13:54:10Z",
        "updatedDate": "2023-08-16T10:09:38.687716136Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "0001-01-01T00:00:00Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/613b7350-6329-457b-b882-c05c3f8d3a96_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/613b7350-6329-457b-b882-c05c3f8d3a96_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/b25324be-b0d0-474c-850c-137c0088edfc_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b25324be-b0d0-474c-850c-137c0088edfc_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/24ebcbf9-1bf1-4900-bc42-aa79cbc72fa7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/24ebcbf9-1bf1-4900-bc42-aa79cbc72fa7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/a3e51ca7-68eb-48f4-be8d-6dcccbca82de_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a3e51ca7-68eb-48f4-be8d-6dcccbca82de_ms.jpg"
            }
        ],
        "photo": [
            39332595,
            39332610,
            39332625,
            39332640
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 361
    },
    {
        "id": "WqLHIYABaOM9ENKQudG_",
        "globalUserId": "",
        "wbUserId": 47096638,
        "wbUserDetails": {
            "name": "Алия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сахарозаменитель хороший, но шёл оооочень долго,  плюс коробка вся мятая, как будто ее пинали до пункта выдачи. Снимать за это звезду не буду, так как понимаю, что не вина производителя, а службы доставки. ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-13T07:16:18Z",
        "updatedDate": "2023-08-16T10:09:40.043394952Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T07:36:53Z",
            "employeeId": 0,
            "text": "Благодарим, что вы с пониманием относитесь к этой ситуации, однако всё равно жаль, что товар был доставлен в таком виде( ",
            "metadata": null,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/5548e66a-2bdd-45c1-9506-1160e1c8b6a8_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/5548e66a-2bdd-45c1-9506-1160e1c8b6a8_ms.jpg"
            }
        ],
        "photo": [
            66299945
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 360
    },
    {
        "id": "uXSqwYoBUYwa0QEfrenf",
        "globalUserId": "25480321",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Марина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T10:51:43Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 359
    },
    {
        "id": "5OuvRIoBBgFQoSo04lwV",
        "globalUserId": "59050551",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Как всегда отлично!!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T04:24:52Z",
        "updatedDate": "2023-09-06T08:58:36Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:58:36Z",
            "supplierId": 237660,
            "text": "Спасибо за доверие нашему бренду!",
            "metadata": null,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 358
    },
    {
        "id": "Ej0l3IYBmBixw9tzyGnn",
        "globalUserId": "19496436",
        "wbUserId": 19496436,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ольга"
        },
        "nmId": 33478818,
        "text": "Минус одна звёзда,тк в упаковке 59 саше,а не 60",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-13T18:05:09Z",
        "updatedDate": "2023-08-16T10:09:46.643029869Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-14T05:41:32Z",
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "33066332",
                "wbUserId": 33066332,
                "helpfulness": "minus",
                "time": "2023-03-13T23:28:23Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/292be41b-dea9-4591-8b56-462e50f97765_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/292be41b-dea9-4591-8b56-462e50f97765_ms.jpg"
            }
        ],
        "photo": [
            90886332
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 357
    },
    {
        "id": "ndWqhoQBwEHULCRVyJ3b",
        "globalUserId": "",
        "wbUserId": 81329162,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Вячеслав"
        },
        "nmId": 33478818,
        "text": "Доволен доставкой",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-17T17:37:30Z",
        "updatedDate": "2023-08-16T10:09:43.382887929Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-18T09:32:14Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/887e17a4-f90d-4fbe-80fb-0361fd42f793_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/887e17a4-f90d-4fbe-80fb-0361fd42f793_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6f974608-3bcb-4d49-8904-f3fa80376df7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6f974608-3bcb-4d49-8904-f3fa80376df7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/0b6d0d3c-7bb3-497e-9fea-3bf1307e6370_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0b6d0d3c-7bb3-497e-9fea-3bf1307e6370_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/11f20260-1942-49ba-bb01-8947ecb5d891_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/11f20260-1942-49ba-bb01-8947ecb5d891_ms.jpg"
            }
        ],
        "photo": [
            36492706,
            36492707,
            36492709,
            36492710
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 356
    },
    {
        "id": "zMOcwYIBTTPU1bwQ7uIA",
        "globalUserId": "",
        "wbUserId": 17383365,
        "wbUserDetails": {
            "country": "BY",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Брала,для мамы, так как цена устроила, взяла сразу 2 упаковки, целое не разочаровалась. Рекомендую....",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-21T18:14:22Z",
        "updatedDate": "2023-08-16T10:09:41.771671963Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-22T07:03:11Z",
            "editable": true,
            "metadata": null,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/9296d750-6659-4105-94a7-3b778bf083eb_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/9296d750-6659-4105-94a7-3b778bf083eb_ms.jpg"
            }
        ],
        "photo": [
            60165422
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 355
    },
    {
        "id": "71rre4QBbunnhI3ghT1L",
        "globalUserId": "",
        "wbUserId": 9357702,
        "wbUserDetails": {
            "name": "светлана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Коробочка целая , будем покупать! Сладкий вкусный вкус!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-15T15:32:23Z",
        "updatedDate": "2023-08-16T10:09:43.336057952Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-18T09:40:46Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/5f2729fe-a892-42ab-ab06-cb908b964b35_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/5f2729fe-a892-42ab-ab06-cb908b964b35_ms.jpg"
            }
        ],
        "photo": [
            34637184
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 354
    },
    {
        "id": "IG0CXIcBT-IFIyX_cXoh",
        "globalUserId": "62267439",
        "wbUserId": 62267439,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Все нормально",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-07T13:57:57Z",
        "updatedDate": "2023-08-16T10:09:47.841281904Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-12T08:06:30Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2bdf61ee-3534-49ba-b848-13e58ba9b865_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2bdf61ee-3534-49ba-b848-13e58ba9b865_ms.jpg"
            }
        ],
        "photo": [
            95282821
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 353
    },
    {
        "id": "qF61bX8B8cSqyKR8rzb4",
        "globalUserId": "",
        "wbUserId": 19739883,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "Всё пришло целое. Очень меня удивила зефирка внутри и брашура с зефирками, сколько брала ни чего ни разу не попадала, буду пробовать. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-09T08:04:57Z",
        "updatedDate": "2023-08-16T10:09:39.673956536Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-09T11:24:17Z",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим вас за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 13041401,
                "helpfulness": "plus",
                "time": "2022-03-09T11:10:29Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 8868459,
                "helpfulness": "plus",
                "time": "2022-04-21T16:19:12Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/802f209e-c42e-44b8-b1d4-30d1dff3473a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/802f209e-c42e-44b8-b1d4-30d1dff3473a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/515aeb7c-e35b-4c4e-81a5-4ec5ec4fbf26_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/515aeb7c-e35b-4c4e-81a5-4ec5ec4fbf26_ms.jpg"
            }
        ],
        "photo": [
            60309689,
            60309708
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 352
    },
    {
        "id": "std-0oQBeiioYr22AExV",
        "globalUserId": "",
        "wbUserId": 32930617,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Надежда"
        },
        "nmId": 33478818,
        "text": "Коробка сильно помята, но количество саше верное",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-02T10:59:43Z",
        "updatedDate": "2023-08-16T10:09:43.800527919Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-05T11:07:19Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов."
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/2798e801-b284-49ce-9023-075791e7a477_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2798e801-b284-49ce-9023-075791e7a477_ms.jpg"
            }
        ],
        "photo": [
            51773163
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 351
    },
    {
        "id": "AOMr2IQBfdxlb2FD5WKF",
        "globalUserId": "",
        "wbUserId": 88351488,
        "wbUserDetails": {
            "name": "Сергей",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Пришло в картонной коробке, не мято, не порвано",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-03T13:27:46Z",
        "updatedDate": "2023-08-16T10:09:43.854004671Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-05T06:58:42Z",
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a3b518e9-08d8-4e46-8305-9b10ff1eaa30_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a3b518e9-08d8-4e46-8305-9b10ff1eaa30_ms.jpg"
            }
        ],
        "photo": [
            53352596
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 350
    },
    {
        "id": "Isu1-IUBT-IFIyX_MYrn",
        "globalUserId": "54376376",
        "wbUserId": 54376376,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Беру не первый раз! Сладкий!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-28T14:08:22Z",
        "updatedDate": "2023-08-16T10:09:45.150943548Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-30T13:48:24Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/7e645036-1073-403d-89e1-7f61ec2bac54_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/7e645036-1073-403d-89e1-7f61ec2bac54_ms.jpg"
            }
        ],
        "photo": [
            81169968
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 349
    },
    {
        "id": "y0Efq30Bs0nmRRaKkzuJ",
        "globalUserId": "",
        "wbUserId": 8757679,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Белозерцева"
        },
        "nmId": 33478818,
        "text": "Отличный сахарозаменитель! Раньше пила чай с двумя-тремя ложками сахара, теперь вполне хватает одного саше на кружку. Спасибо! Но жаль, что коробка была помята очень сильно, будто ее пинали :(",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2021-12-11T20:11:52Z",
        "updatedDate": "2023-08-16T10:09:38.858006795Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2021-12-13T05:29:00.617319334Z",
            "metadata": null,
            "text": "Нам очень жаль, что при обработке заказа случаются такие казусы, но к сожалению мы не можем контролировать работу интернет-площадки, а так же процессы сборки и доставки ваших заказов, т.к. этим занимается сам маркетплейс!",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/bdc039fe-6c27-41ef-b3be-849ee8cfa7f9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/bdc039fe-6c27-41ef-b3be-849ee8cfa7f9_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/944e1f76-802e-471a-85e7-b6950bb3982d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/944e1f76-802e-471a-85e7-b6950bb3982d_ms.jpg"
            }
        ],
        "photo": [
            39404109,
            39404124
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 348
    },
    {
        "id": "RhohJYIBdo9ArRfAPz6V",
        "globalUserId": "",
        "wbUserId": 24227620,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 62622901,
        "text": "Пришел хорошо упакован, в коробке. Нигде не рваный. Пакетики не считали))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-22T08:58:32Z",
        "updatedDate": "2023-08-16T10:09:41.449677299Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-09T08:37:48Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/71de96f8-3294-470a-bec7-7714fa07c0ec_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/71de96f8-3294-470a-bec7-7714fa07c0ec_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/07e54502-6942-44b8-922a-6b0f9b3daf8e_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/07e54502-6942-44b8-922a-6b0f9b3daf8e_ms.jpg"
            }
        ],
        "photo": [
            61900180,
            61900202
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 347
    },
    {
        "id": "K29Yo4oBfuChzMpwjyhu",
        "globalUserId": "22432564",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Светлана",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Отличный сахзам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T13:33:25Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 346
    },
    {
        "id": "AAM7cYYBtPujtG5sfusy",
        "globalUserId": "19483497",
        "wbUserId": 19483497,
        "wbUserDetails": {
            "name": "Дарья",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Коробки пришла открытая и рваная,на вкус еще не пробывали",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-20T23:49:30Z",
        "updatedDate": "2023-08-16T10:09:45.932092191Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-02-22T07:15:00Z",
            "metadata": null,
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/693d3896-585d-4626-a7f2-61f19ddec4b1_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/693d3896-585d-4626-a7f2-61f19ddec4b1_ms.jpg"
            }
        ],
        "photo": [
            87082067
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 345
    },
    {
        "id": "njmwjYIBdo9ArRfAI3bI",
        "globalUserId": "",
        "wbUserId": 34280762,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Галина"
        },
        "nmId": 33478818,
        "text": "Товар хорошего качества.Количество соответствует. Упаковка плохая, товарного вида нет, коробку получила всю помятую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-11T16:15:05Z",
        "updatedDate": "2023-08-16T10:09:41.666884638Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-12T05:24:06Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/ad811b93-7c2c-4a54-93e2-8259f44b439b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ad811b93-7c2c-4a54-93e2-8259f44b439b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/0943f996-5979-44ee-8b9a-11fd02d6a44e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0943f996-5979-44ee-8b9a-11fd02d6a44e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/561bbb3c-fa53-4337-9341-bfd2cca46d2f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/561bbb3c-fa53-4337-9341-bfd2cca46d2f_ms.jpg"
            }
        ],
        "photo": [
            52390904,
            52390919,
            52390936
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 344
    },
    {
        "id": "PPdhUooBl47NwnZ1lK5D",
        "globalUserId": "7420477",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Инна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный заменитель.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T20:14:01Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 343
    },
    {
        "id": "ofSpCYoBU2re2XP7T-Bn",
        "globalUserId": "45591698",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Вика",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Я не знаю хороший ли это сахара заменитель потому что сразу же его выбросила. А выбросила потому что заказала из за рпп, но потом решила лечиться",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-18T17:20:05Z",
        "updatedDate": "2023-08-26T06:19:47Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "28561230",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-19T00:01:01Z"
            },
            {
                "globalUserId": "46480931",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-19T09:41:41Z"
            },
            {
                "globalUserId": "23916772",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-26T06:19:47Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 3
        },
        "rank": 342
    },
    {
        "id": "0u_0-oQBTZEOO9f5nv7D",
        "globalUserId": "",
        "wbUserId": 15068331,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "К сожалению, отказ из-за того, что коробка была очень помятой. Перезакажу,надеюсь всё придёт в хорошем состоянии.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-10T07:34:06Z",
        "updatedDate": "2023-08-16T10:09:44.064624797Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-14T08:57:41Z",
            "editable": false,
            "employeeId": 0,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 32106041,
                "helpfulness": "minus",
                "time": "2022-12-10T08:22:01.609104913Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 6127865,
                "helpfulness": "minus",
                "time": "2022-12-10T11:13:07.990245460Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/9f087301-afe4-4ca5-8aac-f1306a38a76e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/9f087301-afe4-4ca5-8aac-f1306a38a76e_ms.jpg"
            }
        ],
        "photo": [
            60311147
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 2
        },
        "rank": 341
    },
    {
        "id": "gat_w4oBXuPSsPE1MMop",
        "globalUserId": "42156752",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Сакинат"
        },
        "nmId": 138785478,
        "text": "Любимый сахар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T19:23:27Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 340
    },
    {
        "id": "xz9INIkBuWYO9ah2QYmR",
        "globalUserId": "37227524",
        "wbUserId": 37227524,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель,без привкусов,послевкусий,абсолютная польза!!!хотелось бы приобрести пачку ещё большего объема,например  500 пакетиков на 1 упаковку,по приятной цене,специально заказала 90,брала раньше 60,мне на месяц не хватало,а 500 штук решило бы проблему на полгода)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-08T06:54:58Z",
        "updatedDate": "2023-08-16T10:09:50.760694318Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T11:25:29Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Есть фасовка 1000 штук. Спасибо за ваш отзыв!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 339
    },
    {
        "id": "nfhZzYABUcbpNqIm-zoK",
        "globalUserId": "",
        "wbUserId": 11576096,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Людмила"
        },
        "nmId": 33478818,
        "text": "Быстрая доставка . без доп упаковки,но пришло всё целое .на вкус сладко,без привкуса  специфического",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-16T14:51:07Z",
        "updatedDate": "2023-08-16T10:09:40.546088429Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-18T08:33:03Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 28985879,
                "helpfulness": "minus",
                "time": "2022-05-16T15:11:35Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/0e345d9e-8496-49df-a619-e31cba7e877f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0e345d9e-8496-49df-a619-e31cba7e877f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6d32d7b1-2e2a-485b-8cfc-2e15c472fe67_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6d32d7b1-2e2a-485b-8cfc-2e15c472fe67_ms.jpg"
            }
        ],
        "photo": [
            60126760,
            60126776
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 338
    },
    {
        "id": "HHYEPIoBXuPSsPE1IywK",
        "globalUserId": "34243083",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Наталья"
        },
        "nmId": 33478818,
        "text": "Классный сахзам!!!!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T12:00:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 337
    },
    {
        "id": "kz_Dd4EB73bXwc5ES_Gf",
        "globalUserId": "",
        "wbUserId": 29369179,
        "wbUserDetails": {
            "name": "Зульфия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Первый раз заказываю. Все понравилось.  Коробка чуть-чуть помята  , не критично,  на качество продукта не влияет . Саше герметично закрыто.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-18T17:01:36Z",
        "updatedDate": "2023-08-16T10:09:41.067507835Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-20T13:46:31Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 10293401,
                "helpfulness": "plus",
                "time": "2022-06-19T05:48:42Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/2bc4064f-0222-4de7-ad80-16eea3a40c4e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2bc4064f-0222-4de7-ad80-16eea3a40c4e_ms.jpg"
            }
        ],
        "photo": [
            74388084
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 336
    },
    {
        "id": "w7WP6X8BZBLx1OSDlWtP",
        "globalUserId": "",
        "wbUserId": 11139038,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "мне понравилось.от магазинного который в таблетках очень отличается по вкусу.у этого нет привкуса,как у тех.буду брать еще",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-02T09:16:15Z",
        "updatedDate": "2023-08-16T10:09:39.856811382Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-08T11:14:00Z",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара! ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/09a8e1e9-cd62-433d-b047-2f672dfa6346_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/09a8e1e9-cd62-433d-b047-2f672dfa6346_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/c9484f78-8be1-462f-ae02-f96967936e19_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c9484f78-8be1-462f-ae02-f96967936e19_ms.jpg"
            }
        ],
        "photo": [
            59901217,
            59901232
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 335
    },
    {
        "id": "A1GVU4EBIzqgZI3qCH8m",
        "globalUserId": "",
        "wbUserId": 44111005,
        "wbUserDetails": {
            "name": "Елена",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Уже не в первый раз заказываю. Мой любимый сахзам. Упакован хорошо, количество в норме.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-11T16:24:44Z",
        "updatedDate": "2023-08-16T10:09:40.981185932Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-15T13:52:30Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 10786375,
                "helpfulness": "plus",
                "time": "2022-06-12T14:45:54Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/11f6848c-5c50-4db2-b3a2-356ab9474f84_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/11f6848c-5c50-4db2-b3a2-356ab9474f84_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/383e58ef-f68b-4f5e-9399-ac3606b1e917_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/383e58ef-f68b-4f5e-9399-ac3606b1e917_ms.jpg"
            }
        ],
        "photo": [
            59155979,
            59155993
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 334
    },
    {
        "id": "9uUGvIoBuX2t2Z2-CkM0",
        "globalUserId": "58907021",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталия",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Мне нравится.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-22T08:33:47Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 333
    },
    {
        "id": "m9ZPbooBZx8DR9RAXauP",
        "globalUserId": "15064260",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Очень сладкий, даже когда ложишь один пакетик.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T06:23:30Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 332
    },
    {
        "id": "Dq1lUooBlM6cTg_ROeOf",
        "globalUserId": "56045561",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Люция",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру не первый раз👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T20:18:00Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 331
    },
    {
        "id": "oVk3GYAB9g1jf2Z4VBSW",
        "globalUserId": "",
        "wbUserId": 27466916,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Андрей"
        },
        "nmId": 62622901,
        "text": "На чашку 400 гр.нужно 5 пакетов и то не та сладость как от 4 ложек сахара.Но на вкус неплохо,нет посторонней горечи как от стевии .Если действительно не вредный ,то можно брать ,хоть и дороговат как по мне.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-11T15:21:38Z",
        "updatedDate": "2023-08-16T10:09:40.009927427Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-12T09:35:56Z",
            "metadata": null,
            "editable": true,
            "text": "Не вредный, за исключением возможной индивидуальной непереносимости. Товар состоит из природных компонентов. Спасибо за покупку!  ",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/a71e712c-2545-4fa4-893a-f57ce17b67b9_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/a71e712c-2545-4fa4-893a-f57ce17b67b9_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/2ea53796-5fb2-4ca1-a174-54220deea004_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/2ea53796-5fb2-4ca1-a174-54220deea004_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/9d22e9af-1fd9-41a5-99c5-d70aea86a1a9_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/9d22e9af-1fd9-41a5-99c5-d70aea86a1a9_ms.jpg"
            }
        ],
        "photo": [
            48953604,
            48953619,
            48953636
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 330
    },
    {
        "id": "ilwJg4oBpT2rITcGB0lp",
        "globalUserId": "11538380",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный сахзам!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T06:58:42Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 329
    },
    {
        "id": "NnJZIYoBuWYO9ah2VQKZ",
        "globalUserId": "37780193",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Сах. зам супер😱💕 Почти не отличается от сахара обычного, прям спасение для организма и фигуры. Спасибо производителю и продавцу🔥",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-23T07:43:37Z",
        "updatedDate": "2023-08-23T08:43:16Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-23T08:43:16Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 328
    },
    {
        "id": "ZLxdTIoBqd2abVcyYReP",
        "globalUserId": "28055916",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Галина"
        },
        "nmId": 33478818,
        "text": "Пользуюсь постоянно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T16:11:43Z",
        "updatedDate": "2023-09-06T08:23:15Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:23:15Z",
            "employeeId": 0,
            "metadata": null,
            "editable": true,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 327
    },
    {
        "id": "fNHNiooBuX2t2Z2-FxfG",
        "globalUserId": "28973083",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Патимат",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Покупаю 2й раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T19:10:11Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 326
    },
    {
        "id": "4LDkfn0BGcs-85_32EjJ",
        "globalUserId": "",
        "wbUserId": 27582431,
        "wbUserDetails": {
            "name": "Лилия",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Большое спасибо, всегда беру этот сахарозаменитель, и не по одному пачке, как говорится прозапас. Товар хорошо упакован, быстрая доставка. Хорошая цена, советую. Продавцу спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2021-12-03T06:04:26Z",
        "updatedDate": "2023-08-16T10:09:38.825928010Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "0001-01-01T00:00:00Z",
            "text": "Спасибо за покупку!",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8bcd2a21-f9af-4f31-9d3e-362d8657060d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8bcd2a21-f9af-4f31-9d3e-362d8657060d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2d6d36bd-1c97-4dcf-9ef0-67d39c591779_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2d6d36bd-1c97-4dcf-9ef0-67d39c591779_ms.jpg"
            }
        ],
        "photo": [
            39903164,
            39903186
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 325
    },
    {
        "id": "gHWnwooBUYwa0QEfJ2AB",
        "globalUserId": "12002809",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Юлия",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришло целое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T15:27:29Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 324
    },
    {
        "id": "g7r8SYoBqd2abVcyAMhN",
        "globalUserId": "5371122",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Удобно использовать",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T05:06:07Z",
        "updatedDate": "2023-09-06T08:40:04Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:40:04Z",
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 323
    },
    {
        "id": "zbTyRooBKNxVyjcCUmZn",
        "globalUserId": "8070509",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Постоянно его беру.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-30T14:56:41Z",
        "updatedDate": "2023-09-06T08:44:17Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:44:17Z",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за доверие нашему бренду!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 322
    },
    {
        "id": "yP8SZooBl47NwnZ1nAGM",
        "globalUserId": "53130305",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Евгения",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Благодарю удобно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-05T16:00:10Z",
        "updatedDate": "2023-09-06T07:53:22Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T07:53:22Z",
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 321
    },
    {
        "id": "BNZVnYoBKNxVyjcCBGhP",
        "globalUserId": "105518465",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Людмила",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T09:31:49Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 320
    },
    {
        "id": "uWj3ooMBXuaGXcNpXrfe",
        "globalUserId": "",
        "wbUserId": 31024449,
        "wbUserDetails": {
            "country": "RU",
            "name": "Лидия",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Всё хорошо упаковано, это радует. Хороший сахзам. Спасибо продавцу!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-04T12:27:44Z",
        "updatedDate": "2023-08-16T10:09:42.406827151Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-04T12:46:56Z",
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/9f02aee5-c3f1-4ddd-afae-a579dd3fbfbb_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/9f02aee5-c3f1-4ddd-afae-a579dd3fbfbb_ms.jpg"
            }
        ],
        "photo": [
            68514084
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 319
    },
    {
        "id": "zfqyUYEBaOM9ENKQY2cy",
        "globalUserId": "",
        "wbUserId": 37799828,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Отличный зам. сахара. Пользуюсь им давно. Заказывала 2 упаковки. Рекомендую. Все пришла в отличном состоянии. Спасибо поставщикам и wB",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-11T07:37:34Z",
        "updatedDate": "2023-08-16T10:09:40.970576285Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-16T07:54:08Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/6719b656-19dd-465a-a057-40980c9bcbb8_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6719b656-19dd-465a-a057-40980c9bcbb8_ms.jpg"
            }
        ],
        "photo": [
            74292410
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 318
    },
    {
        "id": "EJaXmIoBqVKcxP1muwGC",
        "globalUserId": "50325146",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T11:26:35Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 317
    },
    {
        "id": "SUPg8YABjeZkdKc5smpN",
        "globalUserId": "",
        "wbUserId": 61996915,
        "wbUserDetails": {
            "country": "RU",
            "name": "Оксана",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Доставка быстрая! Первый раз заказала. Очень понравилось, как по мне пол саше на чашку кофе 👍. Целый пакетик для меня много, притарно. Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-23T17:04:35Z",
        "updatedDate": "2023-08-16T10:09:40.672986836Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-07T06:16:21Z",
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/98522880-9204-433e-b7ab-74926b8aabfb_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/98522880-9204-433e-b7ab-74926b8aabfb_ms.jpg"
            }
        ],
        "photo": [
            65571377
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 316
    },
    {
        "id": "wTz3L4oBUYwa0QEfxFLK",
        "globalUserId": "4561123",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Покупаю не первый раз,отлично! Нет постороннего привкуса,отличная замена сахару и экономно и без вреда! Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-26T03:51:22Z",
        "updatedDate": "2023-08-28T05:57:33Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "15107465",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-28T05:57:33Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 315
    },
    {
        "id": "lf-oJooBU2re2XP7nIiQ",
        "globalUserId": "122199917",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный сахзам!! Удобно, что расфасован сразу в саше!! Покупаю не первый раз здесь!! Еще буду заказывать!! 👍👍👍💯💯🌹🌹🌹❤❤❤",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-24T08:28:19Z",
        "updatedDate": "2023-08-24T11:08:24Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-24T11:08:24Z",
            "supplierId": 237660,
            "text": "Спасибо за доверие нашему бренду!",
            "employeeId": 0,
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 314
    },
    {
        "id": "7jYIIIoBUYwa0QEfKJVS",
        "globalUserId": "11649221",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Анжелика",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Очень хороший сахарозаменитель, без неприятных привкусов, как обычный сахар на вкус. Добавляю в кофе, чай, выпечку, каши.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-23T01:35:20Z",
        "updatedDate": "2023-08-23T05:39:12Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-23T05:39:12Z",
            "editable": true,
            "metadata": null,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 313
    },
    {
        "id": "HGdwNX8BC4F6na4y1U1Z",
        "globalUserId": "",
        "wbUserId": 23389505,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "Сахзам полностью соответствует описанию, дата изготовления 14.02.22!!Все пришло быстро и хорошо упаковано )) внутри попалась какая-то сладкая штучка, похожая на зефир ( мелочь , а приятно ) рекомендую к покупке ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-26T09:51:01Z",
        "updatedDate": "2023-08-16T10:09:39.552536492Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-10T12:06:13Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Cпасибо! Нам очень важна ваша рекомендация!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/1ce3a042-5efb-477d-b6c8-e5e4c33de91e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1ce3a042-5efb-477d-b6c8-e5e4c33de91e_ms.jpg"
            }
        ],
        "photo": [
            48110608
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 312
    },
    {
        "id": "JcxfjooBCaW1RxrSSEme",
        "globalUserId": "91347427",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алсу"
        },
        "nmId": 138785478,
        "text": "вкусно топитоп",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-13T11:48:44Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 311
    },
    {
        "id": "RjiSE4oBfuChzMpwRFux",
        "globalUserId": "96477112",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "by",
            "name": "Виктория"
        },
        "nmId": 138785478,
        "text": "Отлично. Раньше в кофе сыграла 2 ложки сахара теперь один пакетик и супер заменяет. Упаковано замечательно. Рекомендую 👍👍👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-20T15:31:08Z",
        "updatedDate": "2023-08-21T08:21:41Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-21T08:21:41Z",
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660,
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 310
    },
    {
        "id": "NPeMt38Bs0nmRRaKYOwe",
        "globalUserId": "",
        "wbUserId": 26244180,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Лилия"
        },
        "nmId": 62622901,
        "text": "Около двух лет \"сидим\" на этом сахарозаменителе. Эта расфасовка в 1000 штук самая экономичная и удобная, жаль,что редко бывает. Вот сейчас не успела заказать,как опять товара нет в наличии.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-23T16:11:42Z",
        "updatedDate": "2023-08-16T10:09:39.794012185Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-08T12:48:49Z",
            "text": "Она не то чтобы редко бывает, а её активно раскупают, потому что как вы верно заметили - получается очень экономично)) ",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 60039837,
                "helpfulness": "plus",
                "time": "2022-04-13T12:03:33Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/7599fe02-5dd9-4215-aafe-3a091f446008_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/7599fe02-5dd9-4215-aafe-3a091f446008_ms.jpg"
            }
        ],
        "photo": [
            56872957
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 309
    },
    {
        "id": "C3cAWoMBA7K10CNUu5MR",
        "globalUserId": "",
        "wbUserId": 11949850,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Екатерина"
        },
        "nmId": 33478818,
        "text": "Упакован, не вскрыто, коробка помята, но это не критично, мне для себя.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-20T08:25:40Z",
        "updatedDate": "2023-08-16T10:09:42.164570692Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T07:21:44Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/ff2f446d-46d4-4524-b58a-f71397928011_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ff2f446d-46d4-4524-b58a-f71397928011_ms.jpg"
            }
        ],
        "photo": [
            75798085
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 308
    },
    {
        "id": "O3-KCYABXFbbY20L9WEQ",
        "globalUserId": "",
        "wbUserId": 54701946,
        "wbUserDetails": {
            "name": "Сергей",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 62622901,
        "text": "Этот сахарозаменитель покупаю около 6 лет, на Валдберис заказали первый раз. Пришёл быстро. Упакован отлично. Выгодно брать в такой упаковке. В пачке было 1012 штук. Спасибо)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-08T14:18:18Z",
        "updatedDate": "2023-08-16T10:09:39.965764052Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T08:07:40Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за покупку "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/26872a96-43d1-403d-b47e-9f5eef756f5c_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/26872a96-43d1-403d-b47e-9f5eef756f5c_ms.jpg"
            }
        ],
        "photo": [
            48511646
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 307
    },
    {
        "id": "fyv0y4oB0v3RIh2Jdhm2",
        "globalUserId": "10200466",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Вера",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный! 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-25T10:48:31Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 306
    },
    {
        "id": "Ka6MBYUB7nVCbHkC_2qf",
        "globalUserId": "",
        "wbUserId": 49627774,
        "wbUserDetails": {
            "name": "Анастасия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Все пришло целое, быстрая доставка 📦🔥",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-12T08:56:44Z",
        "updatedDate": "2023-08-16T10:09:44.116602194Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-13T10:33:19Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/4c495967-acb6-448f-8786-86d19ff644cf_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/4c495967-acb6-448f-8786-86d19ff644cf_ms.jpg"
            }
        ],
        "photo": [
            63222349
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 305
    },
    {
        "id": "e93sc4MBI2nds0aGVSBI",
        "globalUserId": "",
        "wbUserId": 49003529,
        "wbUserDetails": {
            "name": "Альбина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Заказала 2 уп,обе пришли мятые и рваные. Такое чувство,что их пиннали до пункта выдачи. Внутри вроде целые",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-25T09:13:31Z",
        "updatedDate": "2023-08-16T10:09:42.243610460Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T06:07:46Z",
            "metadata": null,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/da23b2ca-3096-43d1-b9e6-3e567180d36a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/da23b2ca-3096-43d1-b9e6-3e567180d36a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/3c14d758-4ba2-4aef-9664-ce45d8d415fb_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3c14d758-4ba2-4aef-9664-ce45d8d415fb_ms.jpg"
            }
        ],
        "photo": [
            49544499,
            49544517
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 304
    },
    {
        "id": "5IsAdYoBXuPSsPE1F5Da",
        "globalUserId": "10153063",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Надежда",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный товар!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T13:34:15Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 303
    },
    {
        "id": "kJ0CAIQBwEHULCRVFYWo",
        "globalUserId": "",
        "wbUserId": 47858120,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Самый лучший сах.зам.,без всяких ненужных привкусов!👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-22T14:04:07Z",
        "updatedDate": "2023-08-16T10:09:42.821898780Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-02T11:52:49Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 7860283,
                "helpfulness": "plus",
                "time": "2022-12-02T15:14:50.083833260Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 55172097,
                "helpfulness": "plus",
                "time": "2022-12-07T05:22:03.143265051Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/80a0ea11-c5ba-4160-9df2-c9c7c5b103af_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/80a0ea11-c5ba-4160-9df2-c9c7c5b103af_ms.jpg"
            }
        ],
        "photo": [
            7143679
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 302
    },
    {
        "id": "4fsnVIoB0v3RIh2JYBZx",
        "globalUserId": "13826174",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Советую к покупки",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T04:29:41Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 301
    },
    {
        "id": "CBa4L38BpGVGCfJ_48SK",
        "globalUserId": "",
        "wbUserId": 53871076,
        "wbUserDetails": {
            "name": "Анастасия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Мне понравился заменитель сахара, как для меня на кружку с чаем одного пакетика хватает. Буду заказывать. Продавцу спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-25T07:11:59Z",
        "updatedDate": "2023-08-16T10:09:39.540439227Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-25T08:54:04Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Очень рады, что вы остаётесь с нами, благодарим за покупку! ",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/440fde75-ed57-4c8a-a141-ec46b19caee7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/440fde75-ed57-4c8a-a141-ec46b19caee7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/c1e1fd14-749c-4e54-8770-52e7cc4bc4ac_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c1e1fd14-749c-4e54-8770-52e7cc4bc4ac_ms.jpg"
            }
        ],
        "photo": [
            56537662,
            56537677
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 300
    },
    {
        "id": "t2t_c4oBYE-HFOZ5T4k9",
        "globalUserId": "23777240",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Надежда"
        },
        "nmId": 33478818,
        "text": "Сахар как сахар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T06:33:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 299
    },
    {
        "id": "YWuAHYoBXuPSsPE1rvlE",
        "globalUserId": "58303949",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Анна",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-22T13:48:07Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/1a898da0-af0b-47c8-a2c0-8404c42d383b_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1a898da0-af0b-47c8-a2c0-8404c42d383b_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8a501234-a0db-498e-ba97-dbb721261c9f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8a501234-a0db-498e-ba97-dbb721261c9f_ms.jpg"
            }
        ],
        "photo": [
            120755667,
            120755670
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 298
    },
    {
        "id": "KgyKgYoB0v3RIh2J2UTu",
        "globalUserId": "40189438",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Элеонора"
        },
        "nmId": 138785478,
        "text": "Хороший товар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T00:01:15Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 297
    },
    {
        "id": "KZOGS4ABuVlYHVlwgx6N",
        "globalUserId": "",
        "wbUserId": 36266513,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Екатерина"
        },
        "nmId": 33478818,
        "text": "Отличный продукт! Покупаю постоянно! Коробка в слюде, не помятая, саше не считала, так как привыкла доверять производителю! Благодарю WB за оперативную работу!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-21T09:49:08Z",
        "updatedDate": "2023-08-16T10:09:40.162029578Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-22T05:10:01Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку нашего товара! Рады, что вы остаётесь с нами)) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a5e1d2f0-6af3-46b8-b7c1-846f528a50ec_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a5e1d2f0-6af3-46b8-b7c1-846f528a50ec_ms.jpg"
            }
        ],
        "photo": [
            59673802
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 296
    },
    {
        "id": "GbbVL4UBk9xzjhYZdtjV",
        "globalUserId": "",
        "wbUserId": 30371302,
        "wbUserDetails": {
            "country": "BY",
            "hasPhoto": false,
            "name": "Анжела"
        },
        "nmId": 33478818,
        "text": "Всё что ожидалось, то и получила!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-20T13:59:56Z",
        "updatedDate": "2023-08-16T10:09:44.295557558Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-22T05:35:38Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/14c54f98-f1c4-4d68-9d72-ecb2299a5edf_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/14c54f98-f1c4-4d68-9d72-ecb2299a5edf_ms.jpg"
            }
        ],
        "photo": [
            70104864
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 295
    },
    {
        "id": "mwfifYoBl47NwnZ1h3tP",
        "globalUserId": "52295467",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Один пакет на два раза. Очень сладкий!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T06:58:32Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 294
    },
    {
        "id": "-EsnGYoBYE-HFOZ5wRNF",
        "globalUserId": "52965134",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алексей",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "конечно он не такой сладкий как написано на коробке но слаще сахара, 2 пакетика как 1 чайная ложка сахара примерно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-21T17:32:30Z",
        "updatedDate": "2023-08-22T12:46:37Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-22T12:46:37Z",
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "14715982",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-22T04:51:02Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 293
    },
    {
        "id": "EL1ImIkBl47NwnZ15y9f",
        "globalUserId": "4893140",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Светлана",
            "country": "by"
        },
        "nmId": 138785478,
        "text": "всегда покупаю FitParad N7\nприходит хорошо упакован в кортонной коробке, с хорошим сроком годности.\nвидно продавец ответственно относится к своей работе, уважает покупателя",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T16:57:42Z",
        "updatedDate": "2023-08-16T10:09:51.596621304Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T10:26:30Z",
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 292
    },
    {
        "id": "tYxC6IEBZ4onbIiBwwwk",
        "globalUserId": "",
        "wbUserId": 41809860,
        "wbUserDetails": {
            "name": "Марина",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Понравился. Но очень сладкий, одного пакетика хватает на 2 раза",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-10T13:18:18Z",
        "updatedDate": "2023-08-16T10:09:41.325514500Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-07-11T13:00:49Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 1552169,
                "helpfulness": "plus",
                "time": "2022-07-11T05:20:36Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 41953867,
                "helpfulness": "plus",
                "time": "2022-07-15T19:48:01Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a4a6df1f-fd2f-49e5-b5d0-dd97edf15d0e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a4a6df1f-fd2f-49e5-b5d0-dd97edf15d0e_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/5c6253bd-cf16-4d7e-b968-7a3e2abae908_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/5c6253bd-cf16-4d7e-b968-7a3e2abae908_ms.jpg"
            }
        ],
        "photo": [
            54635148,
            54635167
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 291
    },
    {
        "id": "1L-8lokB0v3RIh2J_v-f",
        "globalUserId": "53122913",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Пришло все упаковано аккуратно. Очень сладкий, на кружку хватает одного пакетика, даже иногда ясыплю пол пакетика. Очень удобно в пакетиках, спасибо производителю и продавцу!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T09:45:16Z",
        "updatedDate": "2023-08-16T10:09:51.565293729Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T12:52:25Z",
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 290
    },
    {
        "id": "PJPbRIUBtPujtG5srAn8",
        "globalUserId": "",
        "wbUserId": 44590375,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Пришло всё хорошо упакованное 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-24T15:58:45Z",
        "updatedDate": "2023-08-16T10:09:44.372963724Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-27T06:14:12Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/b5727c0d-babb-4744-a963-85a3b83d9937_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/b5727c0d-babb-4744-a963-85a3b83d9937_ms.jpg"
            }
        ],
        "photo": [
            73364744
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 289
    },
    {
        "id": "lyVQWoMBOCC9xHZPj8gn",
        "globalUserId": "",
        "wbUserId": 10855342,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ирина",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Хороший заменитель, всегда его покупаю , тут вышло очень выгодно 👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-20T09:52:52Z",
        "updatedDate": "2023-08-16T10:09:42.166101813Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T07:21:35Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3e35e830-7397-4022-ab5d-988ffc16cba0_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3e35e830-7397-4022-ab5d-988ffc16cba0_ms.jpg"
            }
        ],
        "photo": [
            71876944
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 288
    },
    {
        "id": "YZSv04kBAtKOitoBs3aB",
        "globalUserId": "46480931",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель, кладу 1 пакетик на кружку чая/кофе 240 мл, по вкусу отлично. Рекомендую. Хочу попробовать класть его в пп выпечку)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-08T05:47:35Z",
        "updatedDate": "2023-08-16T10:09:52.353884663Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "48057729",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-08T10:19:36Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 287
    },
    {
        "id": "2xnKtooBBgFQoSo05BBv",
        "globalUserId": "113691135",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Понравилось",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-21T08:11:05Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 286
    },
    {
        "id": "D8509YkBBgFQoSo0Fae1",
        "globalUserId": "6117114",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Самый лучший сах.зам.Беру не первый раз,очень удобно что именно в виде сахара,а не таблеток,добавлять удобно в чай и выпечку",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-14T19:09:33Z",
        "updatedDate": "2023-08-16T10:09:54.157002790Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 285
    },
    {
        "id": "G5qeUIIBbunnhI3glvuM",
        "globalUserId": "",
        "wbUserId": 17016326,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Назгуль",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Все норм  раньше тоже брала его ,мне нравится ,советую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-30T19:39:06Z",
        "updatedDate": "2023-08-16T10:09:41.540547618Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-03T08:59:58Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/718a6734-185f-4545-bad7-59c42288c2b5_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/718a6734-185f-4545-bad7-59c42288c2b5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2200c2cd-738d-4074-8aee-ebe9c19a7b22_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2200c2cd-738d-4074-8aee-ebe9c19a7b22_ms.jpg"
            }
        ],
        "photo": [
            68790783,
            68790799
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 284
    },
    {
        "id": "6AbHcooB0v3RIh2JTZgO",
        "globalUserId": "21657652",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Надежда",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Покупаю всегда",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T03:12:59Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 283
    },
    {
        "id": "pNTsb4oBAtKOitoBwN6n",
        "globalUserId": "24051047",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наргиза",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T13:55:01Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 282
    },
    {
        "id": "j4-gKoIBbunnhI3gZIUQ",
        "globalUserId": "",
        "wbUserId": 66031542,
        "wbUserDetails": {
            "name": "Любовь",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Все хорошо, кроме доставки! Это ужас. Можно пожалуйста при сборке груза, лёгкие предметы класть на верх а не в самы низ! Тогда и проблем не будет.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-23T10:35:30Z",
        "updatedDate": "2023-08-16T10:09:41.461757511Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-09T10:18:21Z",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов."
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 48612535,
                "helpfulness": "plus",
                "time": "2022-07-30T11:44:30Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/230ffb65-531b-4cc5-a1f8-4618fb9b3a04_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/230ffb65-531b-4cc5-a1f8-4618fb9b3a04_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/384b3045-9bbb-4298-bf8a-fba54fd00086_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/384b3045-9bbb-4298-bf8a-fba54fd00086_ms.jpg"
            }
        ],
        "photo": [
            53455786,
            53455803
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 281
    },
    {
        "id": "jwQZEH0B3RV12wxfTpoR",
        "globalUserId": "",
        "wbUserId": 16053690,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Очень люблю этот заменитель, особенно удобно в саше, можно кинуть в сумку с собой. \nДоставка WB убивает конечно 🤦 но мы храним их в контейнере поэтому не критично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2021-11-11T17:43:48Z",
        "updatedDate": "2023-08-16T10:09:38.710852129Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "0001-01-01T00:00:00Z",
            "text": "Нам очень жаль, что при обработке заказа случаются такие казусы, но к сожалению мы не можем контролировать работу интернет-площадки, а так же процессы сборки и доставки ваших заказов, т.к. этим занимается сам маркетплейс!",
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 19638120,
                "helpfulness": "plus",
                "time": "2022-03-02T13:42:44Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/40fed43f-300c-4e7b-bdad-29a3ef4e09d9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/40fed43f-300c-4e7b-bdad-29a3ef4e09d9_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/81ed5bfb-247b-4b32-a0dd-0594c9d9911a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/81ed5bfb-247b-4b32-a0dd-0594c9d9911a_ms.jpg"
            }
        ],
        "photo": [
            37594027,
            37594045
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 280
    },
    {
        "id": "wADyy4kBJytgTHhBznH8",
        "globalUserId": "26797866",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Не знаю какой должен быть вкус, но между сахаром и этим сахарозаменителем в чае разницы не заметила. Если так и должно быть, то это здорово.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-06T17:43:55Z",
        "updatedDate": "2023-08-16T10:09:52.191733548Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T05:41:24Z",
            "employeeId": 0,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 279
    },
    {
        "id": "GoKYF4ABXFbbY20LiOwY",
        "globalUserId": "",
        "wbUserId": 9342569,
        "wbUserDetails": {
            "name": "Виолетта",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 62622901,
        "text": "Отличный сах зам. Покупала за 1855. В коробке было 1002 пакетика. ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-11T07:46:29Z",
        "updatedDate": "2023-08-16T10:09:40.005379434Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T05:57:10Z",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку! ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/3b48d4d1-5d69-4943-b73f-afbd3f32fb83_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/3b48d4d1-5d69-4943-b73f-afbd3f32fb83_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/fbd5eb47-b7da-4d40-b016-95a04a4cbe16_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/fbd5eb47-b7da-4d40-b016-95a04a4cbe16_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/4743/47435533/ca4eb7e3-d5db-4a21-a908-8927a55cc064_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/ca4eb7e3-d5db-4a21-a908-8927a55cc064_ms.jpg"
            }
        ],
        "photo": [
            50523568,
            50523584,
            50523599
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 278
    },
    {
        "id": "BJJ_TokBwM8MKVou1XqY",
        "globalUserId": "50930607",
        "wbUserId": 50930607,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 62622901,
        "text": "Много лет покупаю этот заменитель. На мой взгляд это лучшая альтернатива сахару. Радует, что появилась такая экономичная упаковка. Получается почти в 2 раза дешевле, чем в коробочках. Спасибо производителю.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-13T09:05:48Z",
        "updatedDate": "2023-08-16T10:09:50.979201772Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T05:38:28Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 277
    },
    {
        "id": "YwJIgIoBBgFQoSo0-io9",
        "globalUserId": "31408182",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Лучший сахзам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T18:09:41Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 276
    },
    {
        "id": "p3n7gYoBmorVKbOQFzNf",
        "globalUserId": "26233796",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Юлия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Все отлично !",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-11T02:03:51Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 275
    },
    {
        "id": "nffrSYoB0v3RIh2J6Gb0",
        "globalUserId": "94241542",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Оля",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Довольна заказом",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T04:48:32Z",
        "updatedDate": "2023-09-06T08:39:58Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:39:58Z",
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 274
    },
    {
        "id": "Cz2wUIcBQOOSTxXoTGit",
        "globalUserId": "8520958",
        "wbUserId": 8520958,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо ☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-05T09:12:24Z",
        "updatedDate": "2023-08-16T10:09:47.734359454Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-04-05T09:28:31Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/146a47fa-d526-429b-9642-4eedfae1dcb3_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/146a47fa-d526-429b-9642-4eedfae1dcb3_ms.jpg"
            }
        ],
        "photo": [
            94877120
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 273
    },
    {
        "id": "Ax_3aooBqdFzcemqMLY-",
        "globalUserId": "45305373",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный парад",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T14:48:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 272
    },
    {
        "id": "kR5FrIoB0v3RIh2Ji53P",
        "globalUserId": "32559260",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Лилия",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T07:09:14Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 271
    },
    {
        "id": "0HWyQIABT-hvrnNLtLiN",
        "globalUserId": "",
        "wbUserId": 8228612,
        "wbUserDetails": {
            "name": "Виктория",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Заказывала впервые,  пришло буквально через 3 дня. Упаковка целая в защитной плёнке. WB 5 балов и сахарозаменитель очень понравился тоже на 5 балов.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-19T07:21:34Z",
        "updatedDate": "2023-08-16T10:09:40.133067818Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-19T13:53:32Z",
            "metadata": null,
            "text": "Благодарим вас за выбор нашего товара ",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c2e0b99c-0575-4541-b768-1e882bb95f94_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c2e0b99c-0575-4541-b768-1e882bb95f94_ms.jpg"
            }
        ],
        "photo": [
            49509430
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 270
    },
    {
        "id": "1YulIocBc4LImbbGG7bF",
        "globalUserId": "55287316",
        "wbUserId": 55287316,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Отличный !!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-27T10:37:39Z",
        "updatedDate": "2023-08-16T10:09:47.281189984Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-28T12:53:34Z",
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/681f5b0c-f2f1-4fa8-9392-b5e4fa25f64e_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/681f5b0c-f2f1-4fa8-9392-b5e4fa25f64e_ms.jpg"
            }
        ],
        "photo": [
            93330305
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 269
    },
    {
        "id": "2taXA4YBmBixw9tz7zZ0",
        "globalUserId": "4916513",
        "wbUserId": 4916513,
        "wbUserDetails": {
            "name": "Зульфия",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Все отлично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-30T16:52:14Z",
        "updatedDate": "2023-08-16T10:09:45.210089214Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T05:30:16Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/0ac365e8-c81c-4d07-b231-d2d7e59d1549_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/0ac365e8-c81c-4d07-b231-d2d7e59d1549_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/1ba0a21d-d63f-49f1-b918-5931b3e2beb7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/1ba0a21d-d63f-49f1-b918-5931b3e2beb7_ms.jpg"
            }
        ],
        "photo": [
            81523866,
            81523867
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 268
    },
    {
        "id": "hYn1pooBmorVKbOQwSlR",
        "globalUserId": "88994326",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Альбина"
        },
        "nmId": 31468103,
        "text": "Одна коркоробкабла повреждена.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T06:23:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 267
    },
    {
        "id": "LEOwQIoBpT2rITcG7EEH",
        "globalUserId": "51749354",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Много пакетиков нужно добавлять. Не удобно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-29T09:47:31Z",
        "updatedDate": "2023-08-31T11:07:29Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "43631141",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-31T11:07:29Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 266
    },
    {
        "id": "HS67pokBmorVKbOQTZ1L",
        "globalUserId": "91118154",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Зара",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Нужны peбятa для paбoты, нyжнo nuсaть oтзывы u вce! Бeз onытa, бeз влoжeнuй. 30 oтзывoв-2000. 100 oтзывoв-10.000. Пuсaть тeлerpam:\nalevtina_wildberries",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-30T12:17:20Z",
        "updatedDate": "2023-08-16T10:09:51.734178486Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-31T08:57:36Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "88367373",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-31T01:12:53Z"
            },
            {
                "globalUserId": "21897005",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-02T06:48:09Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 2
        },
        "rank": 265
    },
    {
        "id": "1JuyU4kBUdF-4nlzE_ZL",
        "globalUserId": "30377138",
        "wbUserId": 30377138,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алина"
        },
        "nmId": 31468103,
        "text": "Это лучший сахарозаменитель из всех что я пробовала, на вкус ничем не отличается от обычного сахара \nРаньше покупала в магните, в этот раз решила заказать здесь, все пришло целое \nСпасибо продавцу)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-14T09:18:47Z",
        "updatedDate": "2023-08-16T10:09:51.021335641Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:20:51Z",
            "text": "Спасибо за Ваш отзыв, мы очень рады что Вам понравилась продукция FitParad!",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 264
    },
    {
        "id": "pJKCSokBwM8MKVou0itF",
        "globalUserId": "62206165",
        "wbUserId": 62206165,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Лариса",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Супер! Сладкий, не отличить от сахара! Одного саше лично мне хватает на кружку 350 мл. И что самое главное, нет этого противного послевкусия, как от других заменителей! Однозначно буду заказывать еще!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T14:30:35Z",
        "updatedDate": "2023-08-16T10:09:50.940353089Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T07:36:13Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "16104734",
                "wbUserId": 16104734,
                "helpfulness": "plus",
                "time": "2023-07-14T12:09:36Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 263
    },
    {
        "id": "OCB55IkBpT2rITcGWBDt",
        "globalUserId": "13800541",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Людмила",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Купила первый раз,у меня инсулинорезистентность,сахар нельзя,мед тоже.Продукт понравился,но послевкусие конечно не очень..",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T12:01:45Z",
        "updatedDate": "2023-08-16T10:09:53.277378441Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 262
    },
    {
        "id": "zrMd-okBwM8MKVouVbmd",
        "globalUserId": "3383430",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "by",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный продукт, это первый заменитель сахара который попробовала, полностью меня устроил, буду покупать и дальше",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-15T16:52:54Z",
        "updatedDate": "2023-08-16T10:09:54.424897778Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 261
    },
    {
        "id": "5J5Q3okBZx8DR9RAlKAx",
        "globalUserId": "22325965",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 31468103,
        "text": "Очень довольна ценой относительно магазинов, \nУпаковано всё отлично 3 пачки спаены, всё аккуратно, спасибо продавцу\nРекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-10T07:19:30Z",
        "updatedDate": "2023-08-16T10:09:52.854575492Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 260
    },
    {
        "id": "sIELuokBKNxVyjcCxMWr",
        "globalUserId": "50268820",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Товар пришёл хорошо упакованный, в коробке. Как и заявлено, внутри саше. Цена дешевле, чем в магазине за 60 шт. Спасибо продавцу! Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-03T06:18:01Z",
        "updatedDate": "2023-08-16T10:09:51.920443835Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T07:38:07Z",
            "editable": true,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 259
    },
    {
        "id": "ZEQKXoUB5HDhYKpHsqDe",
        "globalUserId": "",
        "wbUserId": 69259056,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Назакет"
        },
        "nmId": 33478818,
        "text": "Хорошо упакованы",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-29T13:20:38Z",
        "updatedDate": "2023-08-16T10:09:44.447062959Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-11T06:26:13Z",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/f7022a17-c045-4678-800b-c337011954ff_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f7022a17-c045-4678-800b-c337011954ff_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/4999e4bb-0ff6-4f38-a64a-8da46bd2248e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/4999e4bb-0ff6-4f38-a64a-8da46bd2248e_ms.jpg"
            }
        ],
        "photo": [
            76498120,
            76498121
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 258
    },
    {
        "id": "MbYImYkBBgFQoSo0V5uq",
        "globalUserId": "26867120",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Нино",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Спасибо  за товар и доставку!! Подходит по всем параметрам ! для меня  без привкусов и тд ,у меня диаб 2 типа ,глюкометр показывает волшебные показатели ))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T20:26:48Z",
        "updatedDate": "2023-08-16T10:09:51.613194399Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T10:25:18Z",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 257
    },
    {
        "id": "X0lpJn8B8cSqyKR8FBo6",
        "globalUserId": "",
        "wbUserId": 13408142,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Настя"
        },
        "nmId": 33478818,
        "text": "Отличный  ахарозам. Без лишних привкусов. Этого я и искала. Всё аккуратно, упаковка целая. Продавцу спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-23T11:48:14Z",
        "updatedDate": "2023-08-16T10:09:39.517763275Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-24T05:26:34Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c228bb45-8c8c-430a-a005-18e258e30cf2_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c228bb45-8c8c-430a-a005-18e258e30cf2_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6813a437-f0a6-4aa2-a5ae-66b509e73ab9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6813a437-f0a6-4aa2-a5ae-66b509e73ab9_ms.jpg"
            }
        ],
        "photo": [
            57080205,
            57080221
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 256
    },
    {
        "id": "60N90oABkjR8f6ggIj7F",
        "globalUserId": "",
        "wbUserId": 13361790,
        "wbUserDetails": {
            "country": "RU",
            "name": "Надежда",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Отличный заменитель сахара, без привкусов, 1 пакетик = 1 ч.л сахара, 0 калорий.  Быстрая доставка, цена ниже чем в магазине👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-17T14:47:37Z",
        "updatedDate": "2023-08-16T10:09:40.564395153Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-18T07:46:30Z",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/dd251ac5-dae3-45a5-9062-406db11e149f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/dd251ac5-dae3-45a5-9062-406db11e149f_ms.jpg"
            }
        ],
        "photo": [
            51759209
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 255
    },
    {
        "id": "JmzzTokBlM6cTg_RKlqZ",
        "globalUserId": "59246359",
        "wbUserId": 59246359,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Марзет"
        },
        "nmId": 138785478,
        "text": "У меня сах диабет, единственный сахзаменитель который мне понравился, без всякого постороннего привкуса и послевкусия👍как обычный сахар👍Советую всем, кто как и я давно ищет нормальный заменитель!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-13T11:11:46Z",
        "updatedDate": "2023-08-16T10:09:50.983725203Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T05:31:48Z",
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "47694106",
                "wbUserId": 47694106,
                "helpfulness": "plus",
                "time": "2023-07-13T18:32:34Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 254
    },
    {
        "id": "Ll1TmIMB_coMhyPZiO43",
        "globalUserId": "",
        "wbUserId": 84471817,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Юлия"
        },
        "nmId": 62622901,
        "text": "Всё отлично, упаковано в коробку и пакет, всё аккуратно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-02T10:52:34Z",
        "updatedDate": "2023-08-16T10:09:42.374901052Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-03T11:11:40Z",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/c31bc929-fc79-46f7-8e4c-de77b09e3a8b_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/c31bc929-fc79-46f7-8e4c-de77b09e3a8b_ms.jpg"
            }
        ],
        "photo": [
            72167217
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 253
    },
    {
        "id": "x_jpjYkBJytgTHhBYbmG",
        "globalUserId": "27482556",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Виктория"
        },
        "nmId": 138785478,
        "text": "отличный) 90 саше на месяц хватило,привкус есть у всех сахзамов,но как и писали - со временем не чувствуешь его,лично мне не принципиально,главное не химозный",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-25T16:37:10Z",
        "updatedDate": "2023-08-16T10:09:51.474278942Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T07:12:25Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": true,
            "text": "Благодарим Вас за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 252
    },
    {
        "id": "X5VaBIkBUdF-4nlzp2Jq",
        "globalUserId": "59838801",
        "wbUserId": 59838801,
        "wbUserDetails": {
            "name": "Софья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Половины пакетика хватает на большую кружку) действительно как сахар) только не колорийный) в следующий раз закажу в банке. Очень понравилось, что на коробке написали « что от него инсулин не поднимается». Будем надеяться что так и будет",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-28T23:33:17Z",
        "updatedDate": "2023-08-16T10:09:50.455256296Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T05:12:16Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 251
    },
    {
        "id": "PpTsMYMBE-Uw035e4RSE",
        "globalUserId": "",
        "wbUserId": 48894355,
        "wbUserDetails": {
            "country": "RU",
            "name": "Алексей",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сахар классный пользуюсь не первый раз,но пришол весь помятый.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-12T13:39:11Z",
        "updatedDate": "2023-08-16T10:09:42.051913450Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-09-13T06:12:24Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/03dc44b0-03d9-446d-affc-b299f0aeb4f7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/03dc44b0-03d9-446d-affc-b299f0aeb4f7_ms.jpg"
            }
        ],
        "photo": [
            75623871
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 250
    },
    {
        "id": "sqdN_YkBAtKOitoBCXW2",
        "globalUserId": "8077553",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Понравился. Нет неприятного послевкусия в кофе, каше. А вот  как повёл себя в чае не понравилось, химией отдает",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-16T07:43:52Z",
        "updatedDate": "2023-08-16T10:09:54.523642835Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-16T08:44:08Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 249
    },
    {
        "id": "IVbsV4oBUvtK502_9BmR",
        "globalUserId": "16063154",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Беру постоянно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T22:04:21Z",
        "updatedDate": "2023-09-05T08:13:42Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T08:13:42Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 248
    },
    {
        "id": "ta_rV4oBlM6cTg_RrdUu",
        "globalUserId": "16063154",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру постоянно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T22:02:58Z",
        "updatedDate": "2023-09-05T08:13:38Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T08:13:38Z",
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 247
    },
    {
        "id": "rjODBYcBQBfka8TNlxi1",
        "globalUserId": "42533110",
        "wbUserId": 42533110,
        "wbUserDetails": {
            "country": "RU",
            "name": "Надежда",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Спасибо🤗👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-21T18:52:03Z",
        "updatedDate": "2023-08-16T10:09:47.024176633Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T05:56:54Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/51f22261-4839-4557-a070-03d2d17f09b7_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/51f22261-4839-4557-a070-03d2d17f09b7_ms.jpg"
            }
        ],
        "photo": [
            92373843
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 246
    },
    {
        "id": "Xl02iYoBUYwa0QEfsZKk",
        "globalUserId": "37724301",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ирина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Не очень, натуральный вкуснее))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T11:46:18Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 245
    },
    {
        "id": "8cCrHIIBOCC9xHZPvjD5",
        "globalUserId": "",
        "wbUserId": 48223006,
        "wbUserDetails": {
            "country": "RU",
            "name": "Татьяна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный заменитель, сладкий, на большую 300грамовую кружку кофе хватает два пакетика.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-20T17:33:13Z",
        "updatedDate": "2023-08-16T10:09:41.429892686Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-09T13:25:55Z",
            "editable": true,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 42686472,
                "helpfulness": "plus",
                "time": "2022-07-23T06:05:10Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/898dc9c4-d99c-441a-b5e1-2a66728dffa7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/898dc9c4-d99c-441a-b5e1-2a66728dffa7_ms.jpg"
            }
        ],
        "photo": [
            74722470
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 244
    },
    {
        "id": "ikU5p4kBXuPSsPE1ycEl",
        "globalUserId": "5857249",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "ИРИНА"
        },
        "nmId": 138785478,
        "text": "Нужны peбятa для paбoты, нyжнo nuсaть oтзывы u вce! Бeз onытa, бeз влoжeнuй. 30 oтзывoв-2000. 100 oтзывoв-10.000. Пuсaть тeлerpam:\nalevtina_wbr",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-30T14:35:29Z",
        "updatedDate": "2023-08-16T10:09:51.753702879Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:38:17Z",
            "supplierId": 237660,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "editable": true,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "42227502",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-30T15:56:07Z"
            },
            {
                "globalUserId": "17444472",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-30T16:12:16Z"
            },
            {
                "globalUserId": "5857249",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-31T11:33:17Z"
            },
            {
                "globalUserId": "21591932",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-31T17:44:38Z"
            },
            {
                "globalUserId": "21897005",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-02T06:47:51Z"
            },
            {
                "globalUserId": "626258",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-02T15:16:42Z"
            },
            {
                "globalUserId": "58577998",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-08-03T08:33:23Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 6
        },
        "rank": 243
    },
    {
        "id": "CpURmIoBqVKcxP1mhbWl",
        "globalUserId": "20300604",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Алёна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Товар супер",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T09:00:00Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 242
    },
    {
        "id": "GzJ-mIoBqdFzcemqWnHs",
        "globalUserId": "73369816",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Надежда",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все хорошо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T10:58:52Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 241
    },
    {
        "id": "S4I0MIkBAtKOitoBlIgK",
        "globalUserId": "19091090",
        "wbUserId": 19091090,
        "wbUserDetails": {
            "country": "ru",
            "name": "Полина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень порадовала доставка, прислали хорошо упакованный товар, в отдельной коробке. Пробую этот сахарозаменитель не первый раз. На чашку чая мне хватает одного пакетика, везде добавляю его. Мне нравится!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-07T11:54:59Z",
        "updatedDate": "2023-08-16T10:09:50.727709995Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T11:00:40Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "text": "Добрый день, спасибо что уделили время и написали отзыв."
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "60527116",
                "wbUserId": 60527116,
                "helpfulness": "plus",
                "time": "2023-07-07T20:33:20Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 240
    },
    {
        "id": "FMnLA4cBTZEOO9f5su8D",
        "globalUserId": "17371765",
        "wbUserId": 17371765,
        "wbUserDetails": {
            "country": "KZ",
            "hasPhoto": false,
            "name": "Анна"
        },
        "nmId": 33478818,
        "text": "Упаковано 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-21T10:51:34Z",
        "updatedDate": "2023-08-16T10:09:46.991705734Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-23T06:11:38Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "17371765",
                "wbUserId": 17371765,
                "helpfulness": "plus",
                "time": "2023-03-23T06:14:09Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/af516eee-ad4e-47f7-8d04-c12234a95172_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/af516eee-ad4e-47f7-8d04-c12234a95172_ms.jpg"
            }
        ],
        "photo": [
            92273178
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 239
    },
    {
        "id": "DVuwRYEBJHRxs3OTuIsc",
        "globalUserId": "",
        "wbUserId": 43611607,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Заменитель хорош) самый лучший! Спасибо  за выгодное предложение)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-08T23:40:18Z",
        "updatedDate": "2023-08-16T10:09:40.939299022Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-09T05:17:37Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/ffe84038-d4bf-40f1-837f-25372e77be48_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ffe84038-d4bf-40f1-837f-25372e77be48_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/d87ff110-6a17-40d2-85ab-98e42d1bb137_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d87ff110-6a17-40d2-85ab-98e42d1bb137_ms.jpg"
            }
        ],
        "photo": [
            69087366,
            69087381
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 238
    },
    {
        "id": "Lrl5g4MBAN9IxNcckd3z",
        "globalUserId": "",
        "wbUserId": 26882807,
        "wbUserDetails": {
            "name": "Элла",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Сахар хороший, упакован хорошо, покупаем не первый раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-28T09:42:06Z",
        "updatedDate": "2023-08-16T10:09:42.301026624Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-04T07:51:18Z",
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/de1503e1-7322-4cfd-bf70-ddbe90bcf948_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/de1503e1-7322-4cfd-bf70-ddbe90bcf948_ms.jpg"
            }
        ],
        "photo": [
            49577235
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 237
    },
    {
        "id": "vV_CU4oBYE-HFOZ5Z57e",
        "globalUserId": "16965511",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T02:39:24Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 236
    },
    {
        "id": "oEu_PIoBUvtK502_WOyP",
        "globalUserId": "73227226",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Татьяна",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "👍Беру постоянно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T15:24:48Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 235
    },
    {
        "id": "EHJsrIoBfuChzMpwQsu9",
        "globalUserId": "6219352",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Мария"
        },
        "nmId": 33478818,
        "text": "Все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T07:51:31Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 234
    },
    {
        "id": "uqf5K4oBCaW1RxrST4i-",
        "globalUserId": "42316585",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Екатерина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хорошая альтернатива сахару. Главное не переборщить, иначе появляется немного приторный вкус",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-25T09:14:34Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 233
    },
    {
        "id": "Natuw34Bs0nmRRaKArXy",
        "globalUserId": "",
        "wbUserId": 8544491,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "Заказываю не первый раз. Коробка как обычно немного помята, но это не суть. Товаром довольна. Отличный сахзам!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-04T06:31:13Z",
        "updatedDate": "2023-08-16T10:09:39.178080444Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-08T10:49:45Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/cbcdf237-6380-4c06-a032-9c37bff71042_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/cbcdf237-6380-4c06-a032-9c37bff71042_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/14a47301-c65f-41ae-b96c-a72371cdd7f5_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/14a47301-c65f-41ae-b96c-a72371cdd7f5_ms.jpg"
            }
        ],
        "photo": [
            52988762,
            52988777
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 232
    },
    {
        "id": "22oEqIoBUYwa0QEfpLE2",
        "globalUserId": "65460848",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 138785478,
        "text": "Понравился",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-18T11:19:51Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 231
    },
    {
        "id": "as5go4oBlM6cTg_Riix4",
        "globalUserId": "21320606",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Юлия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Прикольный",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T13:42:08Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 230
    },
    {
        "id": "kehICYABOaYV4f1xdts3",
        "globalUserId": "",
        "wbUserId": 21468944,
        "wbUserDetails": {
            "name": "Марчела",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Довольна товаром.Высыпала 1 пакетик на чашку 250 мл.,и по мне так оочень сладко,как будто насыпала 3 ч.л. сахара,постороннего привкуса нет.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-08T13:06:25Z",
        "updatedDate": "2023-08-16T10:09:39.964240590Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T07:27:53Z",
            "supplierId": 237660,
            "editable": true,
            "text": "Спасибо за покупку",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/38c3f59c-4995-42ef-a92c-198729d6f9c9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/38c3f59c-4995-42ef-a92c-198729d6f9c9_ms.jpg"
            }
        ],
        "photo": [
            49762258
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 229
    },
    {
        "id": "XKDd4IkBZx8DR9RA4G-Y",
        "globalUserId": "118143652",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Зарема"
        },
        "nmId": 138785478,
        "text": "Сахар хороший, но в пачке 10гр.мало.Но всё равно хорошо что есть. А так как обычный сахар на вкус, чуть дороговато.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-10T19:13:05Z",
        "updatedDate": "2023-08-16T10:09:53.065733478Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "6751895",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-13T17:45:33Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 228
    },
    {
        "id": "gj7sIooBfuChzMpwflI7",
        "globalUserId": "95865527",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру всегда. Есть разного вкуса. Все очень прикольные. В кофе здорово звучит ФИТ с амаретто.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-23T15:03:59Z",
        "updatedDate": "2023-08-24T06:28:30Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-24T06:28:30Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 227
    },
    {
        "id": "3C-iDIoBUYwa0QEfoUsy",
        "globalUserId": "49601586",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Лилия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Доставка в срок.Брала впервые для выпечки.Вкус понравился,очень удобно,что в пакетиках))Рекомендую!👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-19T07:11:39Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 226
    },
    {
        "id": "2fTqSYoBl47NwnZ17USl",
        "globalUserId": "94241542",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Оля"
        },
        "nmId": 138785478,
        "text": "Очень довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-31T04:47:28Z",
        "updatedDate": "2023-09-06T08:39:55Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:39:55Z",
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 225
    },
    {
        "id": "jOBzdIoBwM8MKVouTbIC",
        "globalUserId": "22009966",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Все отлично.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-08T11:00:28Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 224
    },
    {
        "id": "zy0MoIoBU2re2XP7EMSj",
        "globalUserId": "24311974",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T22:11:00Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 223
    },
    {
        "id": "4hHVpokBpT2rITcGQ1ZE",
        "globalUserId": "16978503",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Юлия",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Фитпарад как обычно не разочаровал. Товары этого бренда в моей практике всегда досталяются быстро, упакованы надëжно, качество хорошее.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-30T12:45:42Z",
        "updatedDate": "2023-08-16T10:09:51.741650219Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-31T08:53:31Z",
            "metadata": null,
            "text": "Благодарим вас за то, что остаётесь с нами!",
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 222
    },
    {
        "id": "fJW1J4QBXuaGXcNpNJOg",
        "globalUserId": "",
        "wbUserId": 72176405,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Кристина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Отличный заменитель сахара. Советую всем!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-30T07:04:57Z",
        "updatedDate": "2023-08-16T10:09:42.969172305Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-31T10:01:09Z",
            "editable": false,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/6bb8e8c5-a308-44dd-8d11-2fd06655cbf1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6bb8e8c5-a308-44dd-8d11-2fd06655cbf1_ms.jpg"
            }
        ],
        "photo": [
            17272029
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 221
    },
    {
        "id": "8Orz64QBTZEOO9f5rxU5",
        "globalUserId": "",
        "wbUserId": 54993723,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Спасибо за качественный товар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-07T09:38:46Z",
        "updatedDate": "2023-08-16T10:09:43.984053197Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-07T10:28:06Z",
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/698af2c0-3d69-4575-b38b-9e35fb4b90ba_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/698af2c0-3d69-4575-b38b-9e35fb4b90ba_ms.jpg"
            }
        ],
        "photo": [
            56190028
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 220
    },
    {
        "id": "xPQqCIMBCjydpYzrPiWc",
        "globalUserId": "",
        "wbUserId": 22715087,
        "wbUserDetails": {
            "name": "Асият",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Заказала по хорошим отзывам, думаю самое то, вместо сахара 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-04T11:02:09Z",
        "updatedDate": "2023-08-16T10:09:41.930400532Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-09-05T08:32:32Z",
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/67aee8a4-c6e3-4eaf-aa73-693912025c9f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/67aee8a4-c6e3-4eaf-aa73-693912025c9f_ms.jpg"
            }
        ],
        "photo": [
            70712424
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 219
    },
    {
        "id": "iZSiSIUBtPujtG5svxgy",
        "globalUserId": "56007659",
        "wbUserId": 56007659,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Класс  очень хороший крем",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-25T09:35:03Z",
        "updatedDate": "2023-08-16T10:09:44.380569346Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-26T12:44:19Z",
            "employeeId": 0,
            "text": "Мы рады, что крем Вам понравился, надеемся и наш сахарозаменитель тоже!) ",
            "supplierId": 237660,
            "metadata": null,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "10084815",
                "wbUserId": 10084815,
                "helpfulness": "plus",
                "time": "2022-12-27T15:32:22Z"
            },
            {
                "globalUserId": "45472709",
                "wbUserId": 45472709,
                "helpfulness": "plus",
                "time": "2022-12-28T21:53:32Z"
            },
            {
                "globalUserId": "87877747",
                "wbUserId": 87877747,
                "helpfulness": "plus",
                "time": "2023-01-06T21:57:19Z"
            },
            {
                "globalUserId": "33030417",
                "wbUserId": 33030417,
                "helpfulness": "minus",
                "time": "2023-01-07T14:43:23Z"
            },
            {
                "globalUserId": "56140721",
                "wbUserId": 56140721,
                "helpfulness": "plus",
                "time": "2023-01-07T15:07:10Z"
            },
            {
                "globalUserId": "6788221",
                "wbUserId": 6788221,
                "helpfulness": "plus",
                "time": "2023-01-09T06:20:45Z"
            },
            {
                "globalUserId": "10695900",
                "wbUserId": 10695900,
                "helpfulness": "plus",
                "time": "2023-01-10T08:40:01Z"
            },
            {
                "globalUserId": "20935988",
                "wbUserId": 20935988,
                "helpfulness": "plus",
                "time": "2023-01-13T09:16:06Z"
            },
            {
                "globalUserId": "19606913",
                "wbUserId": 19606913,
                "helpfulness": "plus",
                "time": "2023-01-20T02:42:52Z"
            },
            {
                "globalUserId": "92290869",
                "wbUserId": 92290869,
                "helpfulness": "plus",
                "time": "2023-01-20T19:40:28Z"
            },
            {
                "globalUserId": "32488993",
                "wbUserId": 32488993,
                "helpfulness": "plus",
                "time": "2023-01-21T19:56:01Z"
            },
            {
                "globalUserId": "28439932",
                "wbUserId": 28439932,
                "helpfulness": "plus",
                "time": "2023-01-22T02:40:51Z"
            },
            {
                "globalUserId": "54721361",
                "wbUserId": 54721361,
                "helpfulness": "plus",
                "time": "2023-01-28T17:21:32Z"
            },
            {
                "globalUserId": "9768319",
                "wbUserId": 9768319,
                "helpfulness": "plus",
                "time": "2023-02-03T12:32:29Z"
            },
            {
                "globalUserId": "31730534",
                "wbUserId": 31730534,
                "helpfulness": "plus",
                "time": "2023-02-03T15:28:24Z"
            },
            {
                "globalUserId": "39934466",
                "wbUserId": 39934466,
                "helpfulness": "plus",
                "time": "2023-02-04T23:44:13Z"
            },
            {
                "globalUserId": "36429788",
                "wbUserId": 36429788,
                "helpfulness": "plus",
                "time": "2023-02-13T23:12:52Z"
            },
            {
                "globalUserId": "101230138",
                "wbUserId": 101230138,
                "helpfulness": "plus",
                "time": "2023-02-25T20:19:11Z"
            },
            {
                "globalUserId": "37349270",
                "wbUserId": 37349270,
                "helpfulness": "plus",
                "time": "2023-04-28T04:34:19Z"
            },
            {
                "globalUserId": "29760446",
                "wbUserId": 29760446,
                "helpfulness": "minus",
                "time": "2023-06-05T09:42:16Z"
            },
            {
                "globalUserId": "11972453",
                "wbUserId": 11972453,
                "helpfulness": "plus",
                "time": "2023-06-13T16:39:37Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/7f5552bc-306a-45c8-836e-551eaec2a292_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/7f5552bc-306a-45c8-836e-551eaec2a292_ms.jpg"
            }
        ],
        "photo": [
            73458618
        ],
        "video": null,
        "votes": {
            "pluses": 19,
            "minuses": 2
        },
        "rank": 218
    },
    {
        "id": "eEsXWX8BiZL1N_qNAapS",
        "globalUserId": "",
        "wbUserId": 50223146,
        "wbUserDetails": {
            "country": "RU",
            "name": "Вера",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный товар беру не первый раз мне нравится. 7",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-05T07:59:14Z",
        "updatedDate": "2023-08-16T10:09:39.633024019Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-09T09:48:09Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 20016840,
                "helpfulness": "plus",
                "time": "2022-03-12T18:04:01Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/e5a57993-2878-4df8-908b-2394bfe98b63_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e5a57993-2878-4df8-908b-2394bfe98b63_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/42727b0f-e4d9-4e8f-b72e-97e98d10d4f7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/42727b0f-e4d9-4e8f-b72e-97e98d10d4f7_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/1a35d6cb-5c54-4635-8cf0-f7552fb3e6d6_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1a35d6cb-5c54-4635-8cf0-f7552fb3e6d6_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/a5650c4d-51e8-4835-8121-be4bdd9e17b2_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a5650c4d-51e8-4835-8121-be4bdd9e17b2_ms.jpg"
            }
        ],
        "photo": [
            49943231,
            49943246,
            49943262,
            49943277
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 217
    },
    {
        "id": "LCKDu34B8cSqyKR8u19s",
        "globalUserId": "",
        "wbUserId": 13878907,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё отлично. Спасибо.  Была приятно удивлена открыв коробку и обнаружив там маленькую баночку варенья😃.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-02T17:37:59Z",
        "updatedDate": "2023-08-16T10:09:39.159861024Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-08T12:46:49Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 17946377,
                "helpfulness": "plus",
                "time": "2022-02-08T17:22:10Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 19480690,
                "helpfulness": "plus",
                "time": "2022-02-13T12:10:47Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 47716250,
                "helpfulness": "plus",
                "time": "2022-03-23T02:36:50Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/ff67cd8d-9470-40c9-8cfa-a338f5ea8959_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ff67cd8d-9470-40c9-8cfa-a338f5ea8959_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2c1847f3-988f-4643-af8a-a6624ab369eb_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2c1847f3-988f-4643-af8a-a6624ab369eb_ms.jpg"
            }
        ],
        "photo": [
            49067980,
            49067999
        ],
        "video": null,
        "votes": {
            "pluses": 3,
            "minuses": 0
        },
        "rank": 216
    },
    {
        "id": "Y3NGSoMBA7K10CNUuTF6",
        "globalUserId": "",
        "wbUserId": 26568474,
        "wbUserDetails": {
            "name": "Виолетта",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Дешевле, чем в магазине.В следующий раз закажу 1000шт.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-17T07:08:12Z",
        "updatedDate": "2023-08-16T10:09:42.132380487Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T08:02:17Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/2d612e71-122b-406e-aef3-6eff6543dad2_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/2d612e71-122b-406e-aef3-6eff6543dad2_ms.jpg"
            }
        ],
        "photo": [
            75729875
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 215
    },
    {
        "id": "2d9bDooBl47NwnZ18UV5",
        "globalUserId": "21403164",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "Отличный сахарощаменитель. Одного пакетика хватает на чашку чая. Очень сладкий. Слаще сахара.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-19T15:13:41Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 214
    },
    {
        "id": "Scr8EX8Bs0nmRRaKSAkk",
        "globalUserId": "",
        "wbUserId": 56273154,
        "wbUserDetails": {
            "country": "RU",
            "name": "Макка",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "покупала Для Мамы, и Мама довольна♥️отдельно спасибо за зефирчик:)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-19T12:37:00Z",
        "updatedDate": "2023-08-16T10:09:39.441573743Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T07:19:03Z",
            "text": "Очень рады, что товар пришёлся вам по вкусу! ",
            "editable": true,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c233cffd-bcf1-40f0-8b9f-2207a84d9670_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c233cffd-bcf1-40f0-8b9f-2207a84d9670_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/d3b59d23-f4e5-4a6f-9c2c-d5b92007ce31_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d3b59d23-f4e5-4a6f-9c2c-d5b92007ce31_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/f987959b-48ae-4641-8e7e-deaaff963897_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f987959b-48ae-4641-8e7e-deaaff963897_ms.jpg"
            }
        ],
        "photo": [
            56777728,
            56777743,
            56777758
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 213
    },
    {
        "id": "1Xw5xYoBfuChzMpw0laP",
        "globalUserId": "8378196",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Галина"
        },
        "nmId": 33478818,
        "text": "вкусный.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-24T03:26:56Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 212
    },
    {
        "id": "WMwl8IkBBgFQoSo0hTuy",
        "globalUserId": "43794230",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Яна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отлично, пришло все целое. Хороший заменитель сахара. Добавляю в чай, кофе, творог, выпечку. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-13T18:25:38Z",
        "updatedDate": "2023-08-16T10:09:53.884242334Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T08:18:10Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 211
    },
    {
        "id": "G-tO9IkBU2re2XP7CtS4",
        "globalUserId": "27367983",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елизавета"
        },
        "nmId": 62622901,
        "text": "На упаковке со стиками информация, что её вес 1кг(1000шт×1,0 г) .Получается, что это не чистый вес самого сахарозаменителя, а  вместе с бумажными стиками, т.е вес брутто?А нетто соответственно  меньше на вес бумаги.Не поленилась, пересчитала - 967 штук.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-14T13:48:23Z",
        "updatedDate": "2023-08-16T10:09:54.082863146Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "8586849",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-16T07:30:18Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 210
    },
    {
        "id": "uNaaCIoBBgFQoSo0E5RG",
        "globalUserId": "85385398",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "by",
            "hasPhoto": false,
            "name": "Мария"
        },
        "nmId": 138785478,
        "text": "Товар хороший, но вот поражена упаковкой, настолько всё аккуратно и добросовестно,, спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-18T12:23:50Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 209
    },
    {
        "id": "ekIXmIYBv9_TjX_mezdY",
        "globalUserId": "22386094",
        "wbUserId": 22386094,
        "wbUserDetails": {
            "country": "RU",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Пришло целое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-02-28T12:55:21Z",
        "updatedDate": "2023-08-16T10:09:46.188338597Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-01T07:14:07Z",
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/57a982fc-7222-4525-aeb1-edd8e54e107f_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/57a982fc-7222-4525-aeb1-edd8e54e107f_ms.jpg"
            }
        ],
        "photo": [
            88388573
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 208
    },
    {
        "id": "j_VkqH8Bs0nmRRaKBiM3",
        "globalUserId": "",
        "wbUserId": 13528838,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Наталья"
        },
        "nmId": 33478818,
        "text": "Все в порядке коробка была чуть мятая. Зато положили в подарок зефир очень приятно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-20T17:33:20Z",
        "updatedDate": "2023-08-16T10:09:39.770287848Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T11:57:21Z",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "text": "Благодарим вас за отзыв! ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/29f4c496-5150-4360-b379-ead529747346_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/29f4c496-5150-4360-b379-ead529747346_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/d84f6d40-5ff1-40c1-b7aa-7a413e026cd7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d84f6d40-5ff1-40c1-b7aa-7a413e026cd7_ms.jpg"
            }
        ],
        "photo": [
            59817238,
            59817252
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 207
    },
    {
        "id": "9tTIbooBAtKOitoBR0hO",
        "globalUserId": "32649607",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T08:35:34Z",
        "updatedDate": "2023-09-07T10:09:09Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-07T10:09:09Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 206
    },
    {
        "id": "lMQ4a4oBuX2t2Z2-rnM8",
        "globalUserId": "25416528",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ольга"
        },
        "nmId": 138785478,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T15:59:51Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 205
    },
    {
        "id": "tNwLqogBmorVKbOQVELF",
        "globalUserId": "56342868",
        "wbUserId": 56342868,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Взяла в саше так как часто выезжаю из дома и в дороге пользоваться очень удобно, а так вообще беру в зиппакете по 0,5 кг. Но прямо убедительная просьба к производителю  сделать ваш продукт по дешевле,  я думаю не только мне а многим и другим все таки дороговато",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-11T10:40:49Z",
        "updatedDate": "2023-08-16T10:09:49.921965310Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T08:49:23Z",
            "createDate": "2023-06-20T08:49:23Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 204
    },
    {
        "id": "cPlEl4kBJytgTHhBNqhn",
        "globalUserId": "33724760",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Алёна"
        },
        "nmId": 138785478,
        "text": "Покупаю первый раз для похудения, понравились\nОдин пакет на 250 стакан кофе и сладкий как будто 2 ложки сахара.\nПокупкой довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T12:12:57Z",
        "updatedDate": "2023-08-16T10:09:51.575808896Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T12:44:57Z",
            "editable": true,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "42352080",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-07-29T02:38:55Z"
            },
            {
                "globalUserId": "96559574",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-07-29T08:53:56Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 203
    },
    {
        "id": "JN2jf4oBZx8DR9RAtFDE",
        "globalUserId": "37064091",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Лариса"
        },
        "nmId": 138785478,
        "text": "Все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-10T15:09:10Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 202
    },
    {
        "id": "UprsEn8BZrF3AGZLK0Fo",
        "globalUserId": "",
        "wbUserId": 27742287,
        "wbUserDetails": {
            "country": "RU",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Мне очень понравились пакетики ,с собой удобно, реально очень сладкие, для диеты самое оно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-19T16:59:01Z",
        "updatedDate": "2023-08-16T10:09:39.449121017Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T07:54:29Z",
            "text": "Благодарим вас за выбор нашего товара!",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/04af1ba6-ad27-466b-a148-b91ed8a8705f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/04af1ba6-ad27-466b-a148-b91ed8a8705f_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/bdd4d09a-f66d-4af3-8f4c-dfc93658a976_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/bdd4d09a-f66d-4af3-8f4c-dfc93658a976_ms.jpg"
            }
        ],
        "photo": [
            62381691,
            62381708
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 201
    },
    {
        "id": "a49TOooBnnhI58-28gD6",
        "globalUserId": "32937202",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Снежана",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хороший товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-28T04:08:15Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 200
    },
    {
        "id": "de5wOIoBl47NwnZ1fxpb",
        "globalUserId": "23004712",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Регина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Лучший сахзам",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-27T19:20:11Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 199
    },
    {
        "id": "iFDET4gBl47NwnZ1XKKS",
        "globalUserId": "21965974",
        "wbUserId": 21965974,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ольга"
        },
        "nmId": 33478818,
        "text": "Заказываю только этот сахзам потому что нравится его вкус. Нет излишней приторности и горечи характерной для таблеток. Мне удобен пакетированный вариант фитпарада. Один пакетик равен 2 ч.л. обычного сахара. С правильным сбалансированным питанием и сахзамом преодолела свою тотальную тягу к сладкому и похудела после 4 родов",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-24T21:57:29Z",
        "updatedDate": "2023-08-16T10:09:49.433205126Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-25T08:22:56Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "12820760",
                "wbUserId": 12820760,
                "helpfulness": "plus",
                "time": "2023-05-25T22:01:00Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 198
    },
    {
        "id": "Mkwetn8BhU8klzxNe8Bc",
        "globalUserId": "",
        "wbUserId": 31316194,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анастасия",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё пришло целое, не мятое. Жаль что не было подарочка внутри, очень хотелось.)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-23T09:32:04Z",
        "updatedDate": "2023-08-16T10:09:39.789449957Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T11:33:04Z",
            "text": "Здравствуйте, подарочек - временная акция, но уверены, что не последняя) ",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 38425967,
                "helpfulness": "minus",
                "time": "2022-03-23T14:38:32Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c38de26c-1a76-4a73-b0e2-30b069d0c970_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c38de26c-1a76-4a73-b0e2-30b069d0c970_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/2d30d66f-e406-4c20-bb17-de951e079861_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2d30d66f-e406-4c20-bb17-de951e079861_ms.jpg"
            }
        ],
        "photo": [
            49763027,
            49763042
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 197
    },
    {
        "id": "OywyxYkBCYLDkq7N-d6Y",
        "globalUserId": "49046736",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Дарья"
        },
        "nmId": 138785478,
        "text": "быстрая доставка , хорошая доступная цена , ещё не пробовала, взяла с целью употреблять на интервалом голодании",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-05T10:16:40Z",
        "updatedDate": "2023-08-16T10:09:52.079026349Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T12:09:38Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 196
    },
    {
        "id": "_llYCX8BC4F6na4ykDHW",
        "globalUserId": "",
        "wbUserId": 24014578,
        "wbUserDetails": {
            "name": "Айгуль",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Упаковка целая!Всё хорошо!И маленькая приятность ждала внутри-зефиринка))Спасибо большое!)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-17T20:21:13Z",
        "updatedDate": "2023-08-16T10:09:39.406574531Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-18T11:10:23Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/32feb862-3384-4cbf-a179-cb4418d4cd0c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/32feb862-3384-4cbf-a179-cb4418d4cd0c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6cd31629-75fa-4e29-ab23-508a0b348700_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6cd31629-75fa-4e29-ab23-508a0b348700_ms.jpg"
            }
        ],
        "photo": [
            52735762,
            52735777
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 195
    },
    {
        "id": "bQxtAokBUvtK502_sUIR",
        "globalUserId": "10067651",
        "wbUserId": 10067651,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анна"
        },
        "nmId": 138785478,
        "text": "Чай и кофе с сахаром не люблю. А вот в выпечку или для запеканки, например, отлично подходит этот заменитель. Сладкий, экономный, без привкуса. Пользуюсь, чтобы снизить калорийность блюд. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-28T14:34:51Z",
        "updatedDate": "2023-08-16T10:09:50.433940159Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T05:17:30Z",
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 194
    },
    {
        "id": "zLJ4MooBqd2abVcyrJ1W",
        "globalUserId": "51184585",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Алена",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Хороший товар",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-26T15:31:24Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 193
    },
    {
        "id": "i2Xvl4oBpT2rITcGOuaX",
        "globalUserId": "26807085",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Рекоендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-15T08:22:32Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 192
    },
    {
        "id": "Lcm3uYQB65w5GC7jgcmg",
        "globalUserId": "",
        "wbUserId": 13635610,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Хороший, сладкий.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-27T15:32:02Z",
        "updatedDate": "2023-08-16T10:09:43.629009974Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-28T05:22:24Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/3b9337bd-573a-4a66-81cb-59f7d20b588d_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/3b9337bd-573a-4a66-81cb-59f7d20b588d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2344/23441056/8d5e7343-2950-498b-8c6d-ef6326476243_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/8d5e7343-2950-498b-8c6d-ef6326476243_ms.jpg"
            }
        ],
        "photo": [
            46887261,
            46887262
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 191
    },
    {
        "id": "LCz86okBUvtK502_kPsS",
        "globalUserId": "103929511",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Хороший заменитель, с одного пакетика чай слегка сладкий. Мне хватает этого. Беру не первый раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-12T18:22:48Z",
        "updatedDate": "2023-08-16T10:09:53.611410635Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 190
    },
    {
        "id": "mbl0sokBBgFQoSo0KFVn",
        "globalUserId": "97006353",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "kz",
            "hasPhoto": false,
            "name": "Мадина"
        },
        "nmId": 138785478,
        "text": "Хорошие сахарозаменители , довольна , вкус ничем не отличается от простого сахара , зато калорий ноль ! Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-01T18:55:04Z",
        "updatedDate": "2023-08-16T10:09:51.866307295Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T08:00:03Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 189
    },
    {
        "id": "PQUi94cBZx8DR9RAN62B",
        "globalUserId": "56023460",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Честно, даже не ожидала, что мне понравится! Очень качественный продукт. На вкус , прям то , что нужно, по вкусу очень похож на обычный сахар , без каких то посторонних запахов и привкусов.  Пили и чай и кофе , просто находка для меня!!!! Кто хочет избавиться от вредной привычки (имею в виду от сахара), или кто хочет иногда побаловать себя сладеньким , само то !!!! Рекомендую , однозначно!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-07T16:53:48Z",
        "updatedDate": "2023-08-16T10:09:48.907272793Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-18T08:57:19Z",
            "createDate": "2023-05-18T08:57:19Z",
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:44:17Z"
            },
            {
                "globalUserId": "15612287",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-05-10T14:04:21Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 1
        },
        "rank": 188
    },
    {
        "id": "l91yrYoBKNxVyjcCyDPW",
        "globalUserId": "19810808",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Классный",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-19T12:38:16Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 187
    },
    {
        "id": "Is54X4oBAtKOitoBhniv",
        "globalUserId": "24133453",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Валентина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-04T09:14:09Z",
        "updatedDate": "2023-09-04T12:39:17Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-04T12:39:17Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 186
    },
    {
        "id": "_AgQSIkBpT2rITcGQjqg",
        "globalUserId": "34111254",
        "wbUserId": 34111254,
        "wbUserDetails": {
            "country": "ru",
            "name": "Юлия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Очень удобно брать с собой несколько пакетиков сахзама. Всегда есть штук 5 в сумке. А дома фитпарад 7, 500 гр, только в сахарнице. Обложилась со всех сторон😁",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T03:06:13Z",
        "updatedDate": "2023-08-16T10:09:50.923756279Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-12T05:54:11Z",
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 185
    },
    {
        "id": "c64qQ4oBCaW1RxrSI-IE",
        "globalUserId": "26432846",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Юлия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Самый лучший",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-29T21:19:10Z",
        "updatedDate": "2023-09-06T09:00:05Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T09:00:05Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": true,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 184
    },
    {
        "id": "n0qzVooBUYwa0QEfXRtD",
        "globalUserId": "25415769",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Варвара",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Всегда беру",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T16:21:50Z",
        "updatedDate": "2023-09-06T03:27:53Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T08:16:09Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "5011037",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-09-06T03:27:53Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 183
    },
    {
        "id": "bKcV84gBBgFQoSo0Xs9m",
        "globalUserId": "36001059",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "kz",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо за качественный товар и быструю доставку. Одного пакетика хватает на большой стакан чая. Чай в меру сладкий, такой как я люблю. Без неприятного послевкусия. Все супер. Буду заказывать ещё.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-25T15:04:04Z",
        "updatedDate": "2023-08-16T10:09:50.324895389Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T08:59:25Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 182
    },
    {
        "id": "x3_xpokBKNxVyjcCbrte",
        "globalUserId": "14674335",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Екатерина",
            "country": "ru"
        },
        "nmId": 62622901,
        "text": "отличный товар! \nхватит мне точно надолго, 1 пакетик на 2 раза попить кофе, достатлчно сладко, целый уже приторно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-30T13:16:28Z",
        "updatedDate": "2023-08-16T10:09:51.743160233Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-31T08:53:13Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 181
    },
    {
        "id": "q-pPpYoBAtKOitoBLehl",
        "globalUserId": "93206532",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сладкий.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T22:42:24Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 180
    },
    {
        "id": "2-ap6YkBU2re2XP7J_5Q",
        "globalUserId": "7508171",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Берём не в первый раз деду-диабетику. Перестал втихушку пить кофе с сахаром, значит вкусно)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-12T12:12:04Z",
        "updatedDate": "2023-08-16T10:09:53.532266864Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 179
    },
    {
        "id": "TjoL9ogBuWYO9ah22V9V",
        "globalUserId": "7528134",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Лязат",
            "country": "kz",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Действительно хороший сахарозаменитель брала по отзывам блогеров. 1 саше хватает на 1 стакан чая в меру сладкий. Сразу же заказала еще 90 шт. Хороший товар , быстрая доставка спасибо продавцу.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-26T04:52:32Z",
        "updatedDate": "2023-08-16T10:09:50.337441604Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-26T10:25:28Z",
            "createDate": "2023-06-26T10:25:28Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 178
    },
    {
        "id": "d0Y7IH8B8cSqyKR8a40u",
        "globalUserId": "",
        "wbUserId": 61630053,
        "wbUserDetails": {
            "country": "RU",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё понравилось,упаковка целая в коробочку положили зефирку, очень вкусная спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-22T07:00:39Z",
        "updatedDate": "2023-08-16T10:09:39.495087084Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T07:06:22Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true,
            "text": "Очень рады, что товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/1d5d5edc-92bf-4a65-bbb9-428444804c64_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1d5d5edc-92bf-4a65-bbb9-428444804c64_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/8f936291-6227-449f-9b68-773f4c9d6fd3_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8f936291-6227-449f-9b68-773f4c9d6fd3_ms.jpg"
            }
        ],
        "photo": [
            50250731,
            50250748
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 177
    },
    {
        "id": "lFtuL4oBmorVKbOQC11k",
        "globalUserId": "88591280",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хороший продукт, приятный на вкус. Если не знаешь от сахара не отличить.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-26T01:20:56Z",
        "updatedDate": "2023-08-26T13:04:30Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "107324468",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-26T13:04:30Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 176
    },
    {
        "id": "GAinCH8BpGVGCfJ_6KOI",
        "globalUserId": "",
        "wbUserId": 33277037,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Эльвира"
        },
        "nmId": 33478818,
        "text": "Товар пришёл быстро! Коробка целая! Был приятный бонус- зефирка)) \nНа мой вкус самый лучший фитпарад  именно номер 7! Покупкой довольна. ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-17T17:08:15Z",
        "updatedDate": "2023-08-16T10:09:39.405064655Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-18T10:20:42Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/b81722e3-3862-4460-8d45-45a088b7bb64_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b81722e3-3862-4460-8d45-45a088b7bb64_ms.jpg"
            }
        ],
        "photo": [
            51184307
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 175
    },
    {
        "id": "LNJN8IkBl47NwnZ1Svja",
        "globalUserId": "27577008",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Екатерина"
        },
        "nmId": 33478818,
        "text": "Покупала себе, по совету  друзей, еще есть, но мало. Пользовалась в основном в напитках.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-13T19:09:05Z",
        "updatedDate": "2023-08-16T10:09:53.891727495Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T08:05:16Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 174
    },
    {
        "id": "_9Cu84UBtPujtG5siJ4-",
        "globalUserId": "44512154",
        "wbUserId": 44512154,
        "wbUserDetails": {
            "name": "Елена",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Очень нравится.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-27T14:43:00Z",
        "updatedDate": "2023-08-16T10:09:45.118671799Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-31T07:02:15Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень рады, что наш товар пришёлся вам по вкусу! ",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/404f6e23-332e-4521-8776-b70150bbed14_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/404f6e23-332e-4521-8776-b70150bbed14_ms.jpg"
            }
        ],
        "photo": [
            81010200
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 173
    },
    {
        "id": "MoKaq4kBuX2t2Z2-675r",
        "globalUserId": "43293814",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Гоарик"
        },
        "nmId": 138785478,
        "text": "всё пришло отлично упакованное. гораздо дешевле чем продаётся в специализированных магазинах спасибо огромное.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-31T11:00:04Z",
        "updatedDate": "2023-08-16T10:09:51.795871001Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:38:10Z",
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "99564997",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-01T12:36:18Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 172
    },
    {
        "id": "qoQERokBAtKOitoBqHVS",
        "globalUserId": "53499225",
        "wbUserId": 53499225,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Марина"
        },
        "nmId": 31468103,
        "text": "Покупаю постоянно в ближайших магазинах, тут сразу 3 вышло немного дешевле ну и в магазине у нас  по 60саше , а тут сразу 90) теперь на долго хваьит",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-11T17:34:18Z",
        "updatedDate": "2023-08-16T10:09:50.913166101Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-12T06:54:26Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 171
    },
    {
        "id": "7-7CzYUBv9_TjX_mJRTq",
        "globalUserId": "17683033",
        "wbUserId": 17683033,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Алена"
        },
        "nmId": 33478818,
        "text": "Пришло все целое",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-20T05:58:51Z",
        "updatedDate": "2023-08-16T10:09:44.890974334Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-20T09:55:06Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/ceaf8ae5-8960-4b51-953a-1b2236b6722c_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/ceaf8ae5-8960-4b51-953a-1b2236b6722c_ms.jpg"
            }
        ],
        "photo": [
            79906599
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 170
    },
    {
        "id": "UeeLEIEBaOM9ENKQ6QLA",
        "globalUserId": "",
        "wbUserId": 12582626,
        "wbUserDetails": {
            "name": "Надежда",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Все пришло в целости и сохраности, покупкой довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-29T16:00:13Z",
        "updatedDate": "2023-08-16T10:09:40.767475890Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-02T10:15:45Z",
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/757c79f5-e99e-423a-a58d-02fd8f16a230_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/757c79f5-e99e-423a-a58d-02fd8f16a230_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/7c23c738-e641-411a-a7da-e9d08885827b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/7c23c738-e641-411a-a7da-e9d08885827b_ms.jpg"
            }
        ],
        "photo": [
            61452607,
            61452622
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 169
    },
    {
        "id": "5nRXoIkBlM6cTg_RUio6",
        "globalUserId": "27503587",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Людмила",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный заменитель. Покупаю только номер 7. \nФасовку в саше беру для поездок и с собой в сумочку. Очень удобно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-29T06:30:25Z",
        "updatedDate": "2023-08-16T10:09:51.674616277Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:42:31Z",
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 168
    },
    {
        "id": "pzEd-4cBl47NwnZ1jMds",
        "globalUserId": "53943624",
        "wbUserId": 53943624,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Марина"
        },
        "nmId": 138785478,
        "text": "Отличный подсластитель и качественная замена сахару! Спасибо производителю за эту возможность питаться без ограничений, делая низкокалорийные десерты, выпечку, напитки и т.д. Уже пекла кулич, заменяя им сахар, тиромису ( добавляла вместо 150 гр сахара 13 саше Fit), получается без всякого привкуса , как если бы это была одна стевия. Что бы я без него делала?!😃👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-08T11:27:10Z",
        "updatedDate": "2023-08-16T10:09:48.927941182Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-05-17T06:26:25Z",
            "createDate": "2023-05-17T06:26:25Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-10T13:44:03Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 167
    },
    {
        "id": "_dGR_YkBBgFQoSo0R-lT",
        "globalUserId": "49111580",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру уже второй раз. Мне очень нравится. В следующий раз попробую с другими вкусами",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-16T08:58:24Z",
        "updatedDate": "2023-08-16T10:09:54.541595549Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 166
    },
    {
        "id": "InyG7YgBAtKOitoBwFVf",
        "globalUserId": "18350612",
        "wbUserId": 18350612,
        "wbUserDetails": {
            "country": "ru",
            "name": "Нина",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Самый лучший сахарозаменитель, уже лет 10 покупаю. Очень выгодно 1000 шт брать, надолго хватает и цена очень приятная в пересчете на коробочку из Магнита. Спасибо, обязательно закажу ещё.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-24T13:10:11Z",
        "updatedDate": "2023-08-16T10:09:50.289064322Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T11:50:26Z",
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 165
    },
    {
        "id": "HdiVlIoBqd2abVcyIZQ8",
        "globalUserId": "62437113",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Джой",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Amazing!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-14T16:45:16Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 164
    },
    {
        "id": "k1FZ_n8B9g1jf2Z43lVP",
        "globalUserId": "",
        "wbUserId": 55086659,
        "wbUserDetails": {
            "country": "RU",
            "name": "Юлия",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Посылка пришла целая, не мятая. Быстро. Спасибо продавцу за оперативность. На вкус не знаю, ещё не пробовала.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-06T10:09:36Z",
        "updatedDate": "2023-08-16T10:09:39.927463602Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-06T11:23:39Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за положительный отзыв! ",
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8c507726-de4f-4877-a9bb-3c2dbe0f1e53_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8c507726-de4f-4877-a9bb-3c2dbe0f1e53_ms.jpg"
            }
        ],
        "photo": [
            49529537
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 163
    },
    {
        "id": "cFY8eYoBUYwa0QEf-Ms7",
        "globalUserId": "12307079",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Юлия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T09:19:13Z",
        "updatedDate": "2023-09-10T08:43:41Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "103646894",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-09-10T08:43:41Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 162
    },
    {
        "id": "_0OZpn4BqvV4YH0egMwM",
        "globalUserId": "",
        "wbUserId": 44273662,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 33478818,
        "text": "Коробка пришла памятая! Но это не критично, взяла для себя, этот сахзам беру всегда 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-01-29T16:09:44Z",
        "updatedDate": "2023-08-16T10:09:39.100166673Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-08T12:44:24Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/73e559bf-23cc-40f6-b2de-1cde6ff7c8c6_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/73e559bf-23cc-40f6-b2de-1cde6ff7c8c6_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6f1b4524-a6f5-41f5-984d-2bff98596c51_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6f1b4524-a6f5-41f5-984d-2bff98596c51_ms.jpg"
            }
        ],
        "photo": [
            41716947,
            41716963
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 161
    },
    {
        "id": "mSBCxoYB6z8QKrUHapVO",
        "globalUserId": "37875765",
        "wbUserId": 37875765,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Все круто",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-09T12:04:47Z",
        "updatedDate": "2023-08-16T10:09:46.496323757Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-15T08:50:03Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/e582dded-4547-4e7f-93e6-5d401526dbd8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/e582dded-4547-4e7f-93e6-5d401526dbd8_ms.jpg"
            }
        ],
        "photo": [
            90034491
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 160
    },
    {
        "id": "0g7kh4kBpT2rITcG1hd1",
        "globalUserId": "65022545",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Валентина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Заказываю не первый раз. Это единственный сахорозаменитель который мне понравился. Нет такой горчинки как у других.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-24T12:34:29Z",
        "updatedDate": "2023-08-16T10:09:51.410281535Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T08:56:44Z",
            "editable": true,
            "employeeId": 0,
            "text": "Благодарим Вас за выбор нашего товара!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 159
    },
    {
        "id": "Mj9C6oABJHRxs3OTsTxp",
        "globalUserId": "",
        "wbUserId": 19750038,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Евгения"
        },
        "nmId": 33478818,
        "text": "Шёл долго, и упаковка помята, поэтому убрала 1 звезду \nА в целом пакетики не повреждены, все целые и по количествукак заявлено. Спасибо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-22T05:34:38Z",
        "updatedDate": "2023-08-16T10:09:40.634624341Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-23T13:52:53Z",
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 18382511,
                "helpfulness": "minus",
                "time": "2022-05-23T12:29:17Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/1ef8c84e-a279-4340-8fa9-2650e89090ad_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1ef8c84e-a279-4340-8fa9-2650e89090ad_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6bb141e4-b7bb-4ffc-b79b-2e9148641ea4_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6bb141e4-b7bb-4ffc-b79b-2e9148641ea4_ms.jpg"
            }
        ],
        "photo": [
            59086219,
            59086234
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 158
    },
    {
        "id": "sT83-okBYE-HFOZ5aiI3",
        "globalUserId": "23522763",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Ирина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший сахарозаменитель, нет послевкусия, и в кофе ☕️, и в кашу, и в выпечку 👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-15T17:21:23Z",
        "updatedDate": "2023-08-16T10:09:54.430937223Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 157
    },
    {
        "id": "RQEyCIkBUYwa0QEf8B0V",
        "globalUserId": "42494260",
        "wbUserId": 42494260,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ульяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Самый любимый мой сахзам, продавцу спасибо за дополнительную упаковку, в прошлый раз пришел просто в пакете, вся коробка помята, в это раз упакован в доп коробку! Супер",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-29T17:28:23Z",
        "updatedDate": "2023-08-16T10:09:50.471834269Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T05:39:32Z",
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "1183878",
                "wbUserId": 1183878,
                "helpfulness": "plus",
                "time": "2023-07-02T08:04:04Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 156
    },
    {
        "id": "2M1TVYoBZx8DR9RAPGA7",
        "globalUserId": "42356169",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Алена"
        },
        "nmId": 138785478,
        "text": "Все хорошо",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T09:57:13Z",
        "updatedDate": "2023-09-05T12:44:21Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T12:44:21Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 155
    },
    {
        "id": "JZEtx4kBAtKOitoBH-2g",
        "globalUserId": "14813197",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Прекрасный сахарозаменитель, по вкусу даже лучше сахара. Добавляю в кофе и в сырники, нравится",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-05T19:29:31Z",
        "updatedDate": "2023-08-16T10:09:52.119485501Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T08:24:11Z",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 154
    },
    {
        "id": "VlLQu4EB73bXwc5EpHax",
        "globalUserId": "",
        "wbUserId": 38867921,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Анастасия"
        },
        "nmId": 62622901,
        "text": "Очень понравился вкус)))! Положили 1029 штук) Спасибо за подарочек ☺️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-01T22:10:21Z",
        "updatedDate": "2023-08-16T10:09:41.231643869Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-07-05T12:00:33Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/6494ddb6-6570-42de-8c09-523ccb10dabb_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/6494ddb6-6570-42de-8c09-523ccb10dabb_ms.jpg"
            }
        ],
        "photo": [
            66694101
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 153
    },
    {
        "id": "O-gSBYABOaYV4f1xQg6N",
        "globalUserId": "",
        "wbUserId": 13998900,
        "wbUserDetails": {
            "country": "RU",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "К самому продукту претензии нет , но качество доставки просто ужас . Коробка порвана, понята . Буд-то ее ногами , как мяч закидывали в машину для транспортировки .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-07T17:28:44Z",
        "updatedDate": "2023-08-16T10:09:39.951920222Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T07:49:55Z",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "text": "Жаль, что товар пришёл в таком виде. Сборкой и доставкой товара занимается не поставщик, а сам маркетплейс. ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 33605689,
                "helpfulness": "minus",
                "time": "2022-04-14T09:18:36Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 52892906,
                "helpfulness": "minus",
                "time": "2022-06-10T01:33:22Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/51869179-af34-4aee-810b-8bce1ccc71fc_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/51869179-af34-4aee-810b-8bce1ccc71fc_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/e1dc5723-102a-4f55-bd00-a0b37aa9bd86_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/e1dc5723-102a-4f55-bd00-a0b37aa9bd86_ms.jpg"
            }
        ],
        "photo": [
            60676107,
            60676123
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 2
        },
        "rank": 152
    },
    {
        "id": "a1nvcIEBIzqgZI3qUPJk",
        "globalUserId": "",
        "wbUserId": 40642055,
        "wbUserDetails": {
            "name": "Виктория",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Заместитель отличный,без постороннего привкуса,в меру  сладкий. Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-17T09:12:20Z",
        "updatedDate": "2023-08-16T10:09:41.046225220Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-22T07:29:42Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 40642055,
                "helpfulness": "plus",
                "time": "2022-06-21T11:44:31Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/3c2740e0-30f0-4201-bc2c-a3683d5b9d9d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3c2740e0-30f0-4201-bc2c-a3683d5b9d9d_ms.jpg"
            }
        ],
        "photo": [
            57385159
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 151
    },
    {
        "id": "xlq72YEB73bXwc5EPOQn",
        "globalUserId": "",
        "wbUserId": 11740283,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Марина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Спасибо за вкуснейший сахзам) 10й номер оказался для меня вкуснее)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-07T17:35:35Z",
        "updatedDate": "2023-08-16T10:09:41.296870064Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-07-13T09:08:33Z",
            "editable": true,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 10453713,
                "helpfulness": "minus",
                "time": "2022-07-22T07:51:05Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/60c68966-1fba-4450-86b9-59f7cd3f5a61_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/60c68966-1fba-4450-86b9-59f7cd3f5a61_ms.jpg"
            }
        ],
        "photo": [
            67598302
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 150
    },
    {
        "id": "OqkOioQBeLZkh6kUzjkI",
        "globalUserId": "",
        "wbUserId": 6529410,
        "wbUserDetails": {
            "country": "RU",
            "name": "екатерина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Снимаю звезду за упаковку, пришла помятая!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-18T09:25:36Z",
        "updatedDate": "2023-08-16T10:09:43.388897656Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-18T09:26:42Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/94f5300c-bf2e-41b5-9fb3-07085ee2dd5c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/94f5300c-bf2e-41b5-9fb3-07085ee2dd5c_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/99bb576c-47cf-4e7f-97e9-0e61daca1b2d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/99bb576c-47cf-4e7f-97e9-0e61daca1b2d_ms.jpg"
            }
        ],
        "photo": [
            37281071,
            37281082
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 149
    },
    {
        "id": "GL61RIUBv9_TjX_mup_J",
        "globalUserId": "",
        "wbUserId": 25202446,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Валерия",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Приходят как из одного места ! Упаковывайте их лучше",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-24T15:17:19Z",
        "updatedDate": "2023-08-16T10:09:44.371406629Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-27T06:16:23Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Боимся, что вопрос в качестве доставки, а не упаковке."
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 56140721,
                "helpfulness": "plus",
                "time": "2023-01-07T15:07:40Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/73f76a19-d45f-4ab2-8ccb-f5d31c190ebf_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/73f76a19-d45f-4ab2-8ccb-f5d31c190ebf_ms.jpg"
            }
        ],
        "photo": [
            73356281
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 148
    },
    {
        "id": "eN5yrIkBqdFzcemqPiEg",
        "globalUserId": "72353511",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Саша",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "все пришло вовремя , даже на день раньше , упаковано хорошо , качество как всегда , рекомендую . 👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-31T14:55:15Z",
        "updatedDate": "2023-08-16T10:09:51.804957983Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:35:28Z",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 147
    },
    {
        "id": "hd-Zn4QBwEHULCRVdmEd",
        "globalUserId": "",
        "wbUserId": 37941336,
        "wbUserDetails": {
            "name": "Юлия",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Сладко и вкусно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-22T13:49:05Z",
        "updatedDate": "2023-08-16T10:09:43.470428477Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-29T10:14:37Z",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/464cea9f-e6c1-43a5-9a5d-5a497690f943_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/464cea9f-e6c1-43a5-9a5d-5a497690f943_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/b1a58a5a-230e-480e-b486-5edd4ccdb09b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/b1a58a5a-230e-480e-b486-5edd4ccdb09b_ms.jpg"
            }
        ],
        "photo": [
            42653109,
            42653111
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 146
    },
    {
        "id": "DGsDSIkBlM6cTg_RBLZL",
        "globalUserId": "94839761",
        "wbUserId": 94839761,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": ""
        },
        "nmId": 138785478,
        "text": "Я сладкоежка и мне очень сложно отказаться от сахара. Попробовала чай с этими пакетиками, думала я справлюсь. Спасибо. Упаковка хорошая.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T02:51:44Z",
        "updatedDate": "2023-08-16T10:09:50.922271688Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-12T05:54:31Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": false,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 145
    },
    {
        "id": "BIQmu4kBuX2t2Z2-e316",
        "globalUserId": "34498028",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Анастасия"
        },
        "nmId": 138785478,
        "text": "П о л у ч а й   о т  ЗООО  в  д е н ь \nна о т з ы в а х. Т е г р а м м\ne s e n i a _ w b o z o n",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-03T11:26:49Z",
        "updatedDate": "2023-08-16T10:09:51.932357321Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:31:50Z",
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 144
    },
    {
        "id": "Fv2eo4oBUdF-4nlz5H_U",
        "globalUserId": "41342960",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хороший",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-17T14:50:14Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 143
    },
    {
        "id": "L5Xj2IUBMBFuPskoTfOY",
        "globalUserId": "25466709",
        "wbUserId": 25466709,
        "wbUserDetails": {
            "name": "Анна",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Ожидала большего. Химозное послевкусие.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-22T09:50:53Z",
        "updatedDate": "2023-08-16T10:09:44.938039138Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-23T08:37:02Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Вероятно у вас повышенная чувствительность к компонентам в составе. "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/f6e4dec2-a225-4cdc-99b4-d5f93950ad45_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/f6e4dec2-a225-4cdc-99b4-d5f93950ad45_ms.jpg"
            }
        ],
        "photo": [
            80264939
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 142
    },
    {
        "id": "e2j_nYoBpT2rITcG6mwp",
        "globalUserId": "89692907",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Светлана"
        },
        "nmId": 138785478,
        "text": "Отоично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-16T12:38:29Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 141
    },
    {
        "id": "0dDu44kB0v3RIh2JqQ9p",
        "globalUserId": "48643164",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Кристина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Если пить чай с лимоном то вообще нет ни какой разницы как будто обычный  сахар !👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T09:30:16Z",
        "updatedDate": "2023-08-16T10:09:53.226172447Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 140
    },
    {
        "id": "kVGJ1IkBuWYO9ah2F42C",
        "globalUserId": "27364677",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Беру уже несколько раз. Один пакетик заменяет ложку сахара. Без постороннего привкуса.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-08T09:45:02Z",
        "updatedDate": "2023-08-16T10:09:52.387040325Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-08T10:01:29Z",
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 139
    },
    {
        "id": "qSVfSYkBmorVKbOQq6E-",
        "globalUserId": "106920239",
        "wbUserId": 106920239,
        "wbUserDetails": {
            "name": "",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Доставка меньше суток, Упаковано в две коробки, привкуса вобще не почувствовал, расход меньше чем ожидал. Всё отлично👌. Пять звёзд.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T09:12:34Z",
        "updatedDate": "2023-08-16T10:09:50.931280632Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:36:50Z",
            "text": "Спасибо за доверие нашему бренду!",
            "editable": false,
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 138
    },
    {
        "id": "nwxJA4kBUvtK502_lFyO",
        "globalUserId": "8647804",
        "wbUserId": 8647804,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Очень хороший заменитель, одной порции хватает на чашку 250 гр чая с лимоном или кофе. Привкуса неприятного нет. Перепробовала много сахзамов, это самый лучший",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-28T18:35:01Z",
        "updatedDate": "2023-08-16T10:09:50.444580417Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T05:13:37Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "8107394",
                "wbUserId": 8107394,
                "helpfulness": "plus",
                "time": "2023-06-28T18:53:26Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 137
    },
    {
        "id": "qIcSJ4kBZx8DR9RAgHnl",
        "globalUserId": "28566531",
        "wbUserId": 28566531,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Олег"
        },
        "nmId": 138785478,
        "text": "Заказ пришел вовремя. Отличная замена сахару, очень вкусно. Добавляю в кофе и чай. Дети тоже с удовольствием используют. Заказала ещё упаковку.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-05T17:21:11Z",
        "updatedDate": "2023-08-16T10:09:50.669012847Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-06T05:32:46Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 136
    },
    {
        "id": "yfUwwYoBAtKOitoBJurG",
        "globalUserId": "59229212",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Светлана",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "👍👍👍👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-23T08:37:53Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 135
    },
    {
        "id": "tl43mYMB_coMhyPZqEcE",
        "globalUserId": "",
        "wbUserId": 40973389,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Любовь"
        },
        "nmId": 33478818,
        "text": "Качество отлично , упаковка помятая ( поэтому 4 звезды )",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 4,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-02T15:01:45Z",
        "updatedDate": "2023-08-16T10:09:42.380990996Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-03T09:34:08Z",
            "metadata": null,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/30696b8e-ac00-467d-b2bc-7e4361f226ba_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/30696b8e-ac00-467d-b2bc-7e4361f226ba_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/9799f126-a4b3-4d59-a11b-a230f37462d4_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/9799f126-a4b3-4d59-a11b-a230f37462d4_ms.jpg"
            }
        ],
        "photo": [
            49602956,
            49602969
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 134
    },
    {
        "id": "pvdlBH8BIBf9BdlroZS9",
        "globalUserId": "",
        "wbUserId": 24553572,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Олеся"
        },
        "nmId": 33478818,
        "text": "Беру уже не первый раз, все очень нравится. Спасибо! Хотелось бы чтобы у вас в продаже снова появились саше по 1000 шт.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-16T21:17:23Z",
        "updatedDate": "2023-08-16T10:09:39.392411317Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-18T10:49:10Z",
            "employeeId": 0,
            "text": "Спасибо за покупку! Фасовка по 1000 шт. скоро будет доступна по следующему артикулу: 62622901",
            "editable": true,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/3ada0952-47a1-4141-a7b0-83b4eac46f2c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3ada0952-47a1-4141-a7b0-83b4eac46f2c_ms.jpg"
            }
        ],
        "photo": [
            48861980
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 133
    },
    {
        "id": "QeNXOIAB73bXwc5Er5RR",
        "globalUserId": "",
        "wbUserId": 27449640,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Коробка была помята, но это не страшно, буду пробовать. ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-17T16:25:12Z",
        "updatedDate": "2023-08-16T10:09:40.108231289Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-20T08:45:17Z",
            "employeeId": 0,
            "metadata": null,
            "editable": true,
            "text": "Спасибо за выбор нашего товара ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/de0c133f-43ed-492d-b228-b294ed3af617_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/de0c133f-43ed-492d-b228-b294ed3af617_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/208d07ea-1148-4287-91cf-277807831a31_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/208d07ea-1148-4287-91cf-277807831a31_ms.jpg"
            }
        ],
        "photo": [
            49165150,
            49165164
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 132
    },
    {
        "id": "UHlbT4kBuX2t2Z2-ydCA",
        "globalUserId": "19878958",
        "wbUserId": 19878958,
        "wbUserDetails": {
            "country": "ru",
            "name": "Александр",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный сахзам! К нему как и ко всем похожим сахзамам нужно привыкнуть. Этот лучший из всех, которые я пробывал. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-13T13:06:03Z",
        "updatedDate": "2023-08-16T10:09:50.992802666Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T05:29:15Z",
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "47694106",
                "wbUserId": 47694106,
                "helpfulness": "minus",
                "time": "2023-07-13T18:32:17Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 131
    },
    {
        "id": "NTPW8IABIzqgZI3qWbaW",
        "globalUserId": "",
        "wbUserId": 21906280,
        "wbUserDetails": {
            "name": "Марина",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Спасибо за быструю доставку, всё качественно упакованно. Рекомендую к покупке.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-23T12:13:40Z",
        "updatedDate": "2023-08-16T10:09:40.663986297Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-23T13:29:19Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим вас за рекомендацию нашего товара!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/4ee529f0-5259-498c-899e-d0adfb1f7133_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/4ee529f0-5259-498c-899e-d0adfb1f7133_ms.jpg"
            }
        ],
        "photo": [
            51917698
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 130
    },
    {
        "id": "eIOFwYkBKNxVyjcCAACr",
        "globalUserId": "7251676",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Юлия",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный сахарозаменитель, даже очень сладкий. Одного пакетика на чашку кофе многовато.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-04T17:07:46Z",
        "updatedDate": "2023-08-16T10:09:52.028897896Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "10605667",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-05T10:14:08Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 129
    },
    {
        "id": "hzSPFX8BiZL1N_qNXctU",
        "globalUserId": "",
        "wbUserId": 32716339,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё пришло хорошо упаковано) и ещё положили какой-то подарочек 🌺🌺🌺👍👍👍👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-20T05:16:31Z",
        "updatedDate": "2023-08-16T10:09:39.456704383Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T07:58:24Z",
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "text": "Благодарим за высокую оценку нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/46bc5990-2911-4d94-a591-da11f69a7e48_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/46bc5990-2911-4d94-a591-da11f69a7e48_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6543070b-ca3b-4dcd-aca4-e951a0c0b2af_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6543070b-ca3b-4dcd-aca4-e951a0c0b2af_ms.jpg"
            }
        ],
        "photo": [
            51616237,
            51616253
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 128
    },
    {
        "id": "mOz_5IkBqdFzcemqQ5CG",
        "globalUserId": "65717309",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Вкусный и полезный. Понравился подсластитель.  Спасибо поставщику и Валдберис",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T14:28:01Z",
        "updatedDate": "2023-08-16T10:09:53.305742411Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 127
    },
    {
        "id": "2J9VIn8B9I0UHoordo80",
        "globalUserId": "",
        "wbUserId": 25502042,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Александра"
        },
        "nmId": 33478818,
        "text": "Пришёл НЕ мятый . На вкус сладенький . Все норм ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-22T16:48:19Z",
        "updatedDate": "2023-08-16T10:09:39.504137291Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-24T05:39:57Z",
            "metadata": null,
            "editable": true,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/698b7540-0e33-44ff-935b-78274304b75a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/698b7540-0e33-44ff-935b-78274304b75a_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/52cbccae-b702-4936-b1ae-daff69e87798_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/52cbccae-b702-4936-b1ae-daff69e87798_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/97ed79bb-92b1-4a52-a9a6-73f57c4b9c82_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/97ed79bb-92b1-4a52-a9a6-73f57c4b9c82_ms.jpg"
            }
        ],
        "photo": [
            50568987,
            50569004,
            50569020
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 126
    },
    {
        "id": "b8U3nYgBUvtK502_xpDs",
        "globalUserId": "62628484",
        "wbUserId": 62628484,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Всё супер. Пришёл заказ быстро, упаковано хорошо, дата изготовления май 2023, то есть  со сроком годности всё отлично. По вкусу точно такой же как я покупала в магазине, только здесь дешевле. 👍спасибо 😊",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-08T22:54:18Z",
        "updatedDate": "2023-08-16T10:09:49.847542333Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-09T11:23:43Z",
            "createDate": "2023-06-09T11:23:43Z",
            "text": "Спасибо за доверие нашему бренду!",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 125
    },
    {
        "id": "ClIzbooBUYwa0QEfsrDg",
        "globalUserId": "22936114",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Алина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Класс!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-07T05:53:17Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 124
    },
    {
        "id": "VYCJrIkBKNxVyjcCW2LC",
        "globalUserId": "56121575",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Зухра",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Спасибо продавцу,  все пришло в срок  и хорошо упаковано в картонную коробочку.  Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-31T15:20:30Z",
        "updatedDate": "2023-08-16T10:09:51.807980369Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T10:13:21Z",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 123
    },
    {
        "id": "lFeSfYgBNO444I63HquQ",
        "globalUserId": "18542575",
        "wbUserId": 18542575,
        "wbUserDetails": {
            "name": "Марина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Раньше пила кофе и три ложки сахара. Потом перешла на сироп , купила фит. На кружку достаточно один пакетик. Сладко. Не привычно, но буду привыкать и по вкусу более менее нравится. Пока для меня лучше сироп топинамбура.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-02T19:25:08Z",
        "updatedDate": "2023-08-16T10:09:49.682882036Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T10:48:58Z",
            "createDate": "2023-06-20T10:48:58Z",
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 122
    },
    {
        "id": "0un6iYoBwM8MKVoufZ-U",
        "globalUserId": "16417382",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Наталья"
        },
        "nmId": 33478818,
        "text": "Отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-12T15:20:09Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 121
    },
    {
        "id": "X4f7vYEBkjR8f6ggPVmn",
        "globalUserId": "",
        "wbUserId": 62607236,
        "wbUserDetails": {
            "country": "RU",
            "name": "Инна",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Товар пришёл очень быстро, коробочка целая.Спасибо продавцу👍🏻",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-02T08:16:07Z",
        "updatedDate": "2023-08-16T10:09:41.234713864Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-07-06T12:52:52Z",
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/59dddfeb-ad63-4cfc-927f-8c0f64e6bdf1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/59dddfeb-ad63-4cfc-927f-8c0f64e6bdf1_ms.jpg"
            }
        ],
        "photo": [
            59569912
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 120
    },
    {
        "id": "enZlQX8BqvV4YH0eingO",
        "globalUserId": "",
        "wbUserId": 9186978,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Оксана",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Люблю этот сахарозаменитель. Всегда его беру. Очень удобный. В это раз положили какую-то печенюшку. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-28T17:34:07Z",
        "updatedDate": "2023-08-16T10:09:39.593370254Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2022-03-10T11:22:32Z",
            "createDate": "2022-03-10T11:22:30Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Очень рады, что товар пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 24601204,
                "helpfulness": "minus",
                "time": "2022-03-13T04:51:27Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/c7918fab-1a62-41c3-a90e-19e42769d560_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c7918fab-1a62-41c3-a90e-19e42769d560_ms.jpg"
            }
        ],
        "photo": [
            59269810
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 119
    },
    {
        "id": "6_MPW4gBI9q8Yb8j3jFa",
        "globalUserId": "61543433",
        "wbUserId": 61543433,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Раньше покупала в аптеке,  но там гораздо дороже. Заказ пришёл в коробке, все целое. Качество хорошее, ничем не отличается от аптеки. Использую в чай, кофе, творог и т.д, вкус приятный. Не горчит как другие заменители сахара. Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-27T02:35:47Z",
        "updatedDate": "2023-08-16T10:09:49.501938787Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-29T13:26:42Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 118
    },
    {
        "id": "GZY58YkBuX2t2Z2-u8fN",
        "globalUserId": "38274478",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Лера",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "очень хороший,и хватает на долго! брала в апреле закончился в конце июля",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-13T23:27:20Z",
        "updatedDate": "2023-08-16T10:09:53.909804850Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T09:00:59Z",
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 117
    },
    {
        "id": "buSMn4AB_0QEDAONRWfJ",
        "globalUserId": "",
        "wbUserId": 8868459,
        "wbUserDetails": {
            "country": "RU",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хороший, двух пакетиков хватает на кружку 240 мл. Коробка целая, приемлемая цена",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-07T17:23:31Z",
        "updatedDate": "2023-08-16T10:09:40.403160367Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-13T12:57:36Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 44851330,
                "helpfulness": "plus",
                "time": "2022-05-08T15:26:34Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/6b35f488-398b-4d14-95ea-0f915e1e7ded_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6b35f488-398b-4d14-95ea-0f915e1e7ded_ms.jpg"
            }
        ],
        "photo": [
            51581273
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 116
    },
    {
        "id": "eHdO-YkBnnhI58-2B36R",
        "globalUserId": "96740934",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Яна",
            "country": "by"
        },
        "nmId": 138785478,
        "text": "",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-15T13:06:28Z",
        "updatedDate": "2023-08-16T10:09:54.370901498Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-15T13:20:26Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/d92f7094-781f-4035-a8bc-62154a0f50c8_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/d92f7094-781f-4035-a8bc-62154a0f50c8_ms.jpg"
            }
        ],
        "photo": [
            118929706
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 115
    },
    {
        "id": "xyi-YIMBOCC9xHZPCHhH",
        "globalUserId": "",
        "wbUserId": 30494029,
        "wbUserDetails": {
            "name": "Елена",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Сроки в норме,только упаковка помята",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-09-21T15:50:10Z",
        "updatedDate": "2023-08-16T10:09:42.190403997Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-05T07:07:48Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов.",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/78b1a6f3-543a-4f75-afe6-78e42da9a380_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/78b1a6f3-543a-4f75-afe6-78e42da9a380_ms.jpg"
            }
        ],
        "photo": [
            49558208
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 114
    },
    {
        "id": "Tf8K4IMBI2nds0aGSULx",
        "globalUserId": "",
        "wbUserId": 27235322,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ангелина"
        },
        "nmId": 33478818,
        "text": "Коробка в отвратительном состоянии",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-10-16T09:05:14Z",
        "updatedDate": "2023-08-16T10:09:42.663414834Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-10-17T07:16:00Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов."
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 41307940,
                "helpfulness": "minus",
                "time": "2022-10-17T09:09:09.785920499Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 14987247,
                "helpfulness": "minus",
                "time": "2022-10-23T13:40:13.910294511Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/1c1c1e12-c4a6-46ec-9ecf-9eb2ddeb648d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1c1c1e12-c4a6-46ec-9ecf-9eb2ddeb648d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/4ef47f4c-8582-4a22-be28-0592651c65f5_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/4ef47f4c-8582-4a22-be28-0592651c65f5_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/3603c314-e4e6-4265-ac43-6b127dfd2216_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/3603c314-e4e6-4265-ac43-6b127dfd2216_ms.jpg"
            }
        ],
        "photo": [
            5581181,
            5581182,
            5581183
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 2
        },
        "rank": 113
    },
    {
        "id": "mrJ9bokBBgFQoSo0n2by",
        "globalUserId": "80446857",
        "wbUserId": 80446857,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Любовь"
        },
        "nmId": 138785478,
        "text": "Товар пришел целый, не мятый, в дополнительно картонной коробке. Сам продукт понравился, для ПП рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-19T14:11:14Z",
        "updatedDate": "2023-08-16T10:09:51.212860705Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T08:52:24Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 112
    },
    {
        "id": "GuP6K4oBBgFQoSo04b7R",
        "globalUserId": "95465107",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Оптимальный сбалансированный набор заменителей. Понравился.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-25T09:16:17Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 111
    },
    {
        "id": "kcX4J4oBwM8MKVou6ndH",
        "globalUserId": "85518578",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Зульфия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "На вкус точно как сахар даже лучше \n90 штук хватит на долго",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-24T14:35:39Z",
        "updatedDate": "2023-09-06T12:39:19Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T12:39:19Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 110
    },
    {
        "id": "cqvvgIUBIq9DQExlabO2",
        "globalUserId": "",
        "wbUserId": 21937591,
        "wbUserDetails": {
            "country": "RU",
            "name": "Евгений",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Отличный товар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-01-05T07:57:32Z",
        "updatedDate": "2023-08-16T10:09:44.540478689Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-01-10T09:58:55Z",
            "supplierId": 237660,
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/d56dbc42-cb46-4642-b57b-80f99c5058aa_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d56dbc42-cb46-4642-b57b-80f99c5058aa_ms.jpg"
            }
        ],
        "photo": [
            77374888
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 109
    },
    {
        "id": "O6xELIkBBgFQoSo0l4cb",
        "globalUserId": "22625117",
        "wbUserId": 22625117,
        "wbUserDetails": {
            "name": "Евгения",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Всё хорошо, всё целое.👍Первый раз заказала этот товар через wb, \nтак как в магазинах с полок они пропали.😄Спасибо, что выручаете.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-06T17:34:00Z",
        "updatedDate": "2023-08-16T10:09:50.706591241Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-07T06:04:05Z",
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 108
    },
    {
        "id": "2n3gzYIBAN9IxNcc4irN",
        "globalUserId": "",
        "wbUserId": 41788894,
        "wbUserDetails": {
            "country": "RU",
            "name": "Елизавета",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всё отлично, пришёл вовремя) упаковка целая",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-24T03:24:03Z",
        "updatedDate": "2023-08-16T10:09:41.800725203Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-24T05:19:47Z",
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/fac3574a-ce2d-4b75-88c7-bc2ebe85aefd_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/fac3574a-ce2d-4b75-88c7-bc2ebe85aefd_ms.jpg"
            }
        ],
        "photo": [
            60333036
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 107
    },
    {
        "id": "wF4ib38B8cSqyKR81fsT",
        "globalUserId": "",
        "wbUserId": 20176163,
        "wbUserDetails": {
            "country": "RU",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Нравится сах.зам заказала 2 коробки, так как больших нет, одна пришла сильно мятая но звезд не снимаю",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-09T14:43:47Z",
        "updatedDate": "2023-08-16T10:09:39.681563844Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-10T05:50:12Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим вас за выбор нашего товара!",
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 22363541,
                "helpfulness": "plus",
                "time": "2022-03-30T15:02:18Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/55639125-21e6-469b-9c99-e1ad3d57e184_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/55639125-21e6-469b-9c99-e1ad3d57e184_ms.jpg"
            }
        ],
        "photo": [
            50135128
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 106
    },
    {
        "id": "0FCOOoIBAN9IxNccRmFC",
        "globalUserId": "",
        "wbUserId": 16100357,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Татьяна"
        },
        "nmId": 33478818,
        "text": "В принципе всё хорошо, но коробку как будто пинали!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-26T12:49:38Z",
        "updatedDate": "2023-08-16T10:09:41.504170058Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-05T11:38:53Z",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Очень жаль, что товар пришёл к вам в таком виде, но мы не занимаемся доставкой до покупателя. Этим занимается сам WB и они не читают ваших отзывов. Советуем обращаться в службу поддержки клиентов."
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/379a97cc-2600-4e57-9130-47e2cfa76f32_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/379a97cc-2600-4e57-9130-47e2cfa76f32_ms.jpg"
            }
        ],
        "photo": [
            68765528
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 105
    },
    {
        "id": "fhYQ9ogBYE-HFOZ5LmfU",
        "globalUserId": "43632143",
        "wbUserId": 43632143,
        "wbUserDetails": {
            "country": "ru",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Хорошо упаковано хорошо,все 3 упаковки в запаянном пакете и в кортонной коробке.По кол - ву во всех трёх по 90 штук,пересчитали👍🏻Спасибо продавцу и WB)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-26T04:57:16Z",
        "updatedDate": "2023-08-16T10:09:50.338946885Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-26T10:25:16Z",
            "createDate": "2023-06-26T10:25:16Z",
            "employeeId": 0,
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 104
    },
    {
        "id": "TERVJooBUvtK502_LZla",
        "globalUserId": "12507791",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Майя",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Прекрасная замена сахару. Здорово, что можно купить на WB)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-24T06:57:11Z",
        "updatedDate": "2023-08-24T11:12:17Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-24T11:12:17Z",
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 103
    },
    {
        "id": "snbyeYoBmorVKbOQVEt_",
        "globalUserId": "13566308",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-09T12:37:19Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 102
    },
    {
        "id": "u5NYVYkBwM8MKVou8Coo",
        "globalUserId": "14489558",
        "wbUserId": 14489558,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Такое ощущение, в другой упаковке 60 шт. были слаще. Мне хватало пол пакетика, а этот использую полный пакетик.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-14T17:00:39Z",
        "updatedDate": "2023-08-16T10:09:51.030365734Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:13:52Z",
            "editable": true,
            "metadata": null,
            "text": "Благодарим за отзыв!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "18459701",
                "wbUserId": 18459701,
                "helpfulness": "plus",
                "time": "2023-07-15T16:31:13Z"
            },
            {
                "globalUserId": "54422806",
                "wbUserId": 54422806,
                "helpfulness": "plus",
                "time": "2023-07-18T04:32:51Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 101
    },
    {
        "id": "kLfuAooBwM8MKVouekNV",
        "globalUserId": "8217358",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру второй раз на 200мл сыплю половину  пакетика. Мне нравиться.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T09:58:18Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 100
    },
    {
        "id": "nMwg5IgBqdFzcemqHxzC",
        "globalUserId": "17833725",
        "wbUserId": 17833725,
        "wbUserDetails": {
            "country": "ru",
            "name": "Евгений",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Благодарю вас вас! Все правильно и своевременно! Продавец отличный покупкой очень доволен отправление быстрое упаковка очень надёжная! С уважением к вам!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-22T17:21:31Z",
        "updatedDate": "2023-08-16T10:09:50.237413988Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-07T07:50:15Z",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 99
    },
    {
        "id": "FrzYaIEBACg9Pibik_mV",
        "globalUserId": "",
        "wbUserId": 53297476,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Ирина"
        },
        "nmId": 33478818,
        "text": "Фитпарад отличный, всегда его заказываю, спасибо магазину и wb",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-15T19:30:32Z",
        "updatedDate": "2023-08-16T10:09:41.028123939Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-16T07:40:28Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "editable": true,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/191302fb-7fa8-4e46-8785-ad7ab3ec236f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/191302fb-7fa8-4e46-8785-ad7ab3ec236f_ms.jpg"
            }
        ],
        "photo": [
            56276800
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 98
    },
    {
        "id": "FYlO44kBCaW1RxrSnqEE",
        "globalUserId": "9265620",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Яна",
            "country": "ru"
        },
        "nmId": 62622901,
        "text": "Упакованы в пакет и в коробку. Все целое. Количество не перечитывала)))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T06:35:28Z",
        "updatedDate": "2023-08-16T10:09:53.160281699Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 97
    },
    {
        "id": "ZkihkYkBuWYO9ah2AyZJ",
        "globalUserId": "43555933",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "супер, до этого брала другой сахарозаменитель, решила попробовать Фитпарад, очень понравился",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-26T09:56:36Z",
        "updatedDate": "2023-08-16T10:09:51.510261600Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T07:08:36Z",
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "text": "Добрый день, спасибо что уделили время и написали отзыв.",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 96
    },
    {
        "id": "Ox7DY4EB_0QEDAON0ne6",
        "globalUserId": "",
        "wbUserId": 12192000,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Нана"
        },
        "nmId": 33478818,
        "text": "Первый раз заказывала с лимоном, попробую теперь этот вкус!!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-06-14T19:49:46Z",
        "updatedDate": "2023-08-16T10:09:41.014353511Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-15T10:52:24Z",
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 42013105,
                "helpfulness": "minus",
                "time": "2022-06-15T10:11:13Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 10293401,
                "helpfulness": "plus",
                "time": "2022-06-19T05:50:01Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/63349e3e-daef-4b15-b2dd-bcd2fb7516d8_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/63349e3e-daef-4b15-b2dd-bcd2fb7516d8_ms.jpg"
            }
        ],
        "photo": [
            68283620
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 1
        },
        "rank": 95
    },
    {
        "id": "4nj6wIkBlM6cTg_RwwEh",
        "globalUserId": "61754815",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Юлия",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "доставка, упаковка отлично, будем пробовать, у меня это первый опыт с сахзамом",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-04T14:36:47Z",
        "updatedDate": "2023-08-18T08:33:02Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-18T08:33:02Z",
            "metadata": null,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 94
    },
    {
        "id": "njnf7okBYE-HFOZ58_NM",
        "globalUserId": "33438147",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Нина Ивановна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Сахарозаменитель хороший, но обычно предпочитаю в большой упаковке.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-13T12:30:02Z",
        "updatedDate": "2023-08-16T10:09:53.800521200Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T12:47:41Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 93
    },
    {
        "id": "1ADdf4QBMx-JN032Hz8O",
        "globalUserId": "",
        "wbUserId": 37374269,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Светлана"
        },
        "nmId": 33478818,
        "text": "Всё отлично 👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-16T09:55:08Z",
        "updatedDate": "2023-08-16T10:09:43.354314620Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-16T13:18:11Z",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/5a3fdedc-171f-4c34-a1f9-f5956824f665_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/5a3fdedc-171f-4c34-a1f9-f5956824f665_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/324bbd96-963d-4f5e-a2be-719a91df088e_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/324bbd96-963d-4f5e-a2be-719a91df088e_ms.jpg"
            }
        ],
        "photo": [
            35605036,
            35605038
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 92
    },
    {
        "id": "xn9h1IgBZx8DR9RAIfLL",
        "globalUserId": "95490381",
        "wbUserId": 95490381,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Сахар для диабетиков, в магазине дороже и не всегда бывает. Один пакетик заменяет 2 чайных ложки сахара, но сахар не поднимает. Теперь буду заказывать здесь",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-19T15:58:35Z",
        "updatedDate": "2023-08-16T10:09:50.157620326Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T06:51:56Z",
            "createDate": "2023-06-20T06:51:56Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 91
    },
    {
        "id": "cjCD74gBqVKcxP1mDp_6",
        "globalUserId": "69944101",
        "wbUserId": 69944101,
        "wbUserDetails": {
            "name": "Мищенко",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Лучший сах.зам. из всех,не отличить от сахара. Добавляю в кофе, чай,выпечку. Не думала что так хорош будет, приехал в целой не помятой коробочке.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-24T22:25:24Z",
        "updatedDate": "2023-08-16T10:09:50.301214639Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T11:45:15Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 90
    },
    {
        "id": "_7sNG38BOaYV4f1xNfpH",
        "globalUserId": "",
        "wbUserId": 25166563,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Пришло все целое, окуратно упаковано, внутри приятный подарок",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-21T06:52:04Z",
        "updatedDate": "2023-08-16T10:09:39.479792115Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-21T12:02:45Z",
            "editable": true,
            "text": "Благодарим вас за выбор нашего товара!",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/1e8d5f67-8010-4f74-9dff-ad05fb24816d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/1e8d5f67-8010-4f74-9dff-ad05fb24816d_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/26c84fef-1b87-42a7-bc68-597456b2253b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/26c84fef-1b87-42a7-bc68-597456b2253b_ms.jpg"
            }
        ],
        "photo": [
            54635719,
            54635740
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 89
    },
    {
        "id": "v3zOv34BZrF3AGZLDIhT",
        "globalUserId": "",
        "wbUserId": 54505953,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Алена"
        },
        "nmId": 33478818,
        "text": "Упаковка целая, не мятая. Внутри подарок, очень приятно. Спасибо.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-03T13:37:38Z",
        "updatedDate": "2023-08-16T10:09:39.170467200Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-08T11:42:42Z",
            "supplierId": 237660,
            "text": "Благодарим вас за выбор нашего товара!",
            "metadata": null,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 174731,
                "helpfulness": "plus",
                "time": "2022-02-05T22:28:24Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 8544491,
                "helpfulness": "plus",
                "time": "2022-02-08T11:39:57Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/bac946e7-3b2b-4d7c-9b02-2e451b0992cf_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/bac946e7-3b2b-4d7c-9b02-2e451b0992cf_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/6d78139b-2bce-40bc-b63e-be12e462867d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6d78139b-2bce-40bc-b63e-be12e462867d_ms.jpg"
            }
        ],
        "photo": [
            52510433,
            52510448
        ],
        "video": null,
        "votes": {
            "pluses": 2,
            "minuses": 0
        },
        "rank": 88
    },
    {
        "id": "5_qexX8Bs0nmRRaKLV2q",
        "globalUserId": "",
        "wbUserId": 67565325,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Виктория"
        },
        "nmId": 33478818,
        "text": "Заказываю уже второй раз. На этот раз коробка не пострадала,как это было в первом случае",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-26T09:45:33Z",
        "updatedDate": "2023-08-16T10:09:39.810970894Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-07T08:28:27Z",
            "text": "Хорошо, что товар службой доставки был доставлен в целости и сохранности! ",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/33d37ab2-9fe6-45f6-8c86-dae21f51171a_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/33d37ab2-9fe6-45f6-8c86-dae21f51171a_ms.jpg"
            }
        ],
        "photo": [
            49904001
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 87
    },
    {
        "id": "EzS4-okBUvtK502_pQoA",
        "globalUserId": "121236099",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Варвара",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Пришло целое, коробка немного помята, с хорошим сроком годности",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-15T19:42:32Z",
        "updatedDate": "2023-08-16T10:09:54.462357117Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-16T08:54:08Z",
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 86
    },
    {
        "id": "nfoXbIoBBgFQoSo0F7G1",
        "globalUserId": "23777240",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Надежда"
        },
        "nmId": 138785478,
        "text": "Все ок.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-06T20:02:47Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 85
    },
    {
        "id": "Wk3DxIgBqd2abVcyk62D",
        "globalUserId": "40102608",
        "wbUserId": 40102608,
        "wbUserDetails": {
            "country": "ru",
            "name": "Екатерина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Фит парад давно покупаю, для диабетика 2 типа самое то, тк не сладкий чай и кофе я пить так и не научилась.\nЭтот вариант удобен в дорогу, с собой в сумочке и тд",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-16T15:12:12Z",
        "updatedDate": "2023-08-16T10:09:50.070302179Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T08:05:03Z",
            "createDate": "2023-06-20T08:05:03Z",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "metadata": null,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 84
    },
    {
        "id": "DjzVdIkBqVKcxP1mPhwh",
        "globalUserId": "53154695",
        "wbUserId": 53154695,
        "wbUserDetails": {
            "name": "Кристина",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "лучший сахарозаменитель,на вкус как сахар,растворяется за секунду,без противного химозного вкуса",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-20T19:44:39Z",
        "updatedDate": "2023-08-16T10:09:51.267108982Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T10:55:51Z",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": false,
            "text": "Благодарим за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 83
    },
    {
        "id": "3hBxsokBUYwa0QEfgtKx",
        "globalUserId": "93277275",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Любовь",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "много лет пользуюсь этим фитпарадом, поэтому заказала, цена ниже чем в магазине",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-01T18:52:11Z",
        "updatedDate": "2023-08-16T10:09:51.864785076Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T08:00:06Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 82
    },
    {
        "id": "MPwtsokBJytgTHhBSp1T",
        "globalUserId": "14142607",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Спасибо продавцу за быструю доставку. Исключила полностью сахар, приятный вкус.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-01T17:37:40Z",
        "updatedDate": "2023-08-16T10:09:51.861786416Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T08:05:29Z",
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 81
    },
    {
        "id": "CSIZVX8BpGVGCfJ_S5Nm",
        "globalUserId": "",
        "wbUserId": 44425710,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Отличный сахарозаменитель, мы только им и пользуемся, ни каких не приятных привкусов и запахов)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-04T13:23:15Z",
        "updatedDate": "2023-08-16T10:09:39.625453828Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-09T13:18:12Z",
            "text": "Благодарим вас за выбор нашего товара!",
            "employeeId": 0,
            "editable": true,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8abc3ed4-86cf-4177-a675-e6395cc03fe1_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8abc3ed4-86cf-4177-a675-e6395cc03fe1_ms.jpg"
            }
        ],
        "photo": [
            49409981
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 80
    },
    {
        "id": "f6ZI3okBwM8MKVoucjdR",
        "globalUserId": "41714531",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Татьяна",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Упакован хорошо, замечательный сахарозаменитель. Беру не в первый раз",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-10T07:10:37Z",
        "updatedDate": "2023-08-16T10:09:52.850163112Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-10T07:18:35Z",
            "editable": true,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 79
    },
    {
        "id": "BlyM5YQBfnpWqlBe3jOF",
        "globalUserId": "",
        "wbUserId": 96131945,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Галина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Хороший заменитель",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-06T03:48:45Z",
        "updatedDate": "2023-08-16T10:09:43.936717753Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-07T10:30:29Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/858b1301-d958-4fca-8aee-4c5e603bc04b_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/858b1301-d958-4fca-8aee-4c5e603bc04b_ms.jpg"
            }
        ],
        "photo": [
            55046737
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 78
    },
    {
        "id": "U4NC_IgBZx8DR9RAYPaf",
        "globalUserId": "10356809",
        "wbUserId": 10356809,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "Очень удобно вместо сахара заменителя. Без запаха и один  пакетик- одна норма.И в каши добавляю и в чай и кофе. Очень удобно. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-27T09:49:49Z",
        "updatedDate": "2023-08-16T10:09:50.383639657Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-29T08:54:10Z",
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 77
    },
    {
        "id": "9lakUIkBnnhI58-2D_Vw",
        "globalUserId": "15062611",
        "wbUserId": 15062611,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ольга"
        },
        "nmId": 138785478,
        "text": "быстрая доставка, упаковка целая, дата изготовления июнь 2023, постоянно покупаю,очень нравится,рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-13T19:04:36Z",
        "updatedDate": "2023-08-16T10:09:51.007889228Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T04:59:13Z",
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 76
    },
    {
        "id": "fT_NFoEBIzqgZI3qt795",
        "globalUserId": "",
        "wbUserId": 22654523,
        "wbUserDetails": {
            "name": "Ольга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Пользуюсь таким сазхамом несколько лет, никогда не подводил. 🤤🤤🤤",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-30T21:09:49Z",
        "updatedDate": "2023-08-16T10:09:40.793345913Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-06-02T06:14:36Z",
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/d90b8f66-fa32-43cb-b5d8-56775e2e53f7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/d90b8f66-fa32-43cb-b5d8-56775e2e53f7_ms.jpg"
            }
        ],
        "photo": [
            60348560
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 75
    },
    {
        "id": "KscbEIkBU2re2XP7FPxC",
        "globalUserId": "18797323",
        "wbUserId": 18797323,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Беру впервые, очень довольна. Нет привеуса , как у других заменителей сахара, один пакетик заменяет 2 ч.л. сахара. Буду брать еще",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-01T06:19:17Z",
        "updatedDate": "2023-08-16T10:09:50.514417665Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-07T06:29:14Z",
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "74239569",
                "wbUserId": 74239569,
                "helpfulness": "plus",
                "time": "2023-07-01T17:42:13Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 74
    },
    {
        "id": "Pxbr74YBQOOSTxXoir-B",
        "globalUserId": "12589675",
        "wbUserId": 12589675,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Лилия"
        },
        "nmId": 138785478,
        "text": "Пришел быстро , без брака . По вкусу, чувствуется странный привкус , но не такой как на стевии в таблетках , ктр вообще отвратительный на вкус. Взяла на пробу ,так как более экономичный ,чем эритрит, хотя тот действительно без вкуса постороннего , но цена космическая стала . Наверное попробую ещё со вкусом карамели . Удобно ,что растворяется, по сладости одного Саше достаточно . В подарок положили три шт по 0.5 черной смородины сахзам ,( неплохой,для вкуса достаточно одного саше ). Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-03-17T14:13:56Z",
        "updatedDate": "2023-08-16T10:09:46.827463235Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-03-24T06:45:58Z",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 73
    },
    {
        "id": "yZrp_4kBKNxVyjcCM-Hp",
        "globalUserId": "87120493",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Карина",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Хороший сахарозаменитель, сладкий. Доставили быстро! Спасибо)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-16T19:53:41Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 72
    },
    {
        "id": "jWQyjIgBl47NwnZ1ehLT",
        "globalUserId": "101980381",
        "wbUserId": 101980381,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Евгения"
        },
        "nmId": 138785478,
        "text": "отличная вещь! сами пьем и родственникам заказали. удобные пакетики , растворяются мгновенно и привкуса почти нет никакого, сластит будь здоров, одного саше на большую чашку выше крыши",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-05T15:34:59Z",
        "updatedDate": "2023-08-16T10:09:49.751662559Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-06T10:41:54Z",
            "editable": false,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "19375929",
                "wbUserId": 19375929,
                "helpfulness": "minus",
                "time": "2023-06-07T12:16:07Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 71
    },
    {
        "id": "E78pAIoBUdF-4nlzFEs2",
        "globalUserId": "19264320",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "Светлана",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Прекрасно упаковано. Доставлено в срок. Спасибо. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-16T21:03:27Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 70
    },
    {
        "id": "AldNUokBnnhI58-2ewUI",
        "globalUserId": "33622150",
        "wbUserId": 33622150,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все целое)) хорошо запаковано , из всех сахзамов, фитпарад единственный без всяких привкусов) спасибо ❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-14T02:49:17Z",
        "updatedDate": "2023-08-16T10:09:51.010926916Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T04:57:50Z",
            "metadata": null,
            "editable": true,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 69
    },
    {
        "id": "7CLKE4EB73bXwc5E71q5",
        "globalUserId": "",
        "wbUserId": 48669920,
        "wbUserDetails": {
            "name": "Илга",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Чуть помялся пока доехал, но всё равно заказ забрали! Благодарю",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-30T07:07:50Z",
        "updatedDate": "2023-08-16T10:09:40.776686598Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-30T13:52:18Z",
            "supplierId": 237660,
            "editable": true,
            "metadata": null,
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 17571015,
                "helpfulness": "plus",
                "time": "2022-05-30T16:19:07Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/2f428a3d-260e-4afa-86e4-42497247df8d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2f428a3d-260e-4afa-86e4-42497247df8d_ms.jpg"
            }
        ],
        "photo": [
            58873999
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 68
    },
    {
        "id": "cIrTSYkBZx8DR9RASW0a",
        "globalUserId": "10889771",
        "wbUserId": 10889771,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Лариса"
        },
        "nmId": 33478818,
        "text": "Отличный товар. Советую. Упакован хорошо, доставка быстрая. Благодарю производителя и валдбирис  за товар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T11:18:51Z",
        "updatedDate": "2023-08-16T10:09:50.932784338Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:37:36Z",
            "text": "Спасибо за доверие нашему бренду!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 67
    },
    {
        "id": "-CsmAooBUYwa0QEfvjL_",
        "globalUserId": "21857227",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Александр"
        },
        "nmId": 138785478,
        "text": "Заказываю не первый раз, для организма полезней, чем сахар.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T06:20:08Z",
        "updatedDate": "2023-08-17T06:54:51Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-17T06:54:51Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 66
    },
    {
        "id": "_6L05IkBZx8DR9RAJ3QL",
        "globalUserId": "120517997",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Юлия"
        },
        "nmId": 138785478,
        "text": "Отличный подсластитель, все пришло отлично упаковано, я довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T14:15:53Z",
        "updatedDate": "2023-08-16T10:09:53.304259072Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 65
    },
    {
        "id": "Lx8W44kBpT2rITcGmgPo",
        "globalUserId": "4653314",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Первый опыт, считаю что удачный Цена, качество.Буду покупать еще.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T05:34:17Z",
        "updatedDate": "2023-08-16T10:09:53.137825702Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 64
    },
    {
        "id": "jYIBXIoBXuPSsPE100P2",
        "globalUserId": "23488308",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Феликс"
        },
        "nmId": 138785478,
        "text": "Отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-03T17:05:38Z",
        "updatedDate": "2023-09-04T12:44:58Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-09-04T12:44:58Z",
            "createDate": "2023-09-04T12:43:43Z",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за покупку!",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 63
    },
    {
        "id": "HFfnI4oBmorVKbOQ4oli",
        "globalUserId": "16323231",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все пришло хорошо упаковано.Очееь удобно в поездках.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-23T19:38:34Z",
        "updatedDate": "2023-08-24T06:26:51Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-24T06:26:51Z",
            "employeeId": 0,
            "editable": true,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 62
    },
    {
        "id": "-DhauogBqd2abVcy-gR1",
        "globalUserId": "21528104",
        "wbUserId": 21528104,
        "wbUserDetails": {
            "name": "Елена",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Отличный. Долго не могла подобрать заменитель сахара, у всех привкус и горечь. Этот приятно удивил. Быстро привыкла. Больше сахар дома не держу. Спасибо!)))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-14T14:41:44Z",
        "updatedDate": "2023-08-16T10:09:50.013418599Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-15T08:53:00Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "86868826",
                "wbUserId": 86868826,
                "helpfulness": "plus",
                "time": "2023-06-15T18:24:52Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 61
    },
    {
        "id": "oWVzr4QBY6K0WqDrgAO7",
        "globalUserId": "",
        "wbUserId": 19739088,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Олеся"
        },
        "nmId": 33478818,
        "text": "Все отлично",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-25T15:41:33Z",
        "updatedDate": "2023-08-16T10:09:43.562268083Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-29T09:49:54Z",
            "text": "Спасибо за покупку!",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/cecfa414-9cb3-4d94-b588-254f6aacf235_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/cecfa414-9cb3-4d94-b588-254f6aacf235_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/ea05aae0-487e-42c8-a35e-65843e3c2835_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/ea05aae0-487e-42c8-a35e-65843e3c2835_ms.jpg"
            }
        ],
        "photo": [
            44516995,
            44516996
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 60
    },
    {
        "id": "cj_R64QBQOOSTxXofE1M",
        "globalUserId": "",
        "wbUserId": 19160116,
        "wbUserDetails": {
            "country": "RU",
            "name": "Варсеник",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Супер понравилось",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-12-07T09:01:25Z",
        "updatedDate": "2023-08-16T10:09:43.979449243Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-12-07T10:30:14Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/739adf1c-91b4-4f9c-bfe9-3a74e51076d9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/739adf1c-91b4-4f9c-bfe9-3a74e51076d9_ms.jpg"
            }
        ],
        "photo": [
            56151206
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 59
    },
    {
        "id": "TFj4Z4kBnnhI58-2s_RL",
        "globalUserId": "63653655",
        "wbUserId": 63653655,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Понравился заменитель, брала впервые. До этого был опыт неудачных сахарозаменителей. Этот хорош",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-18T07:48:19Z",
        "updatedDate": "2023-08-16T10:09:51.142922435Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-18T08:47:37Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 58
    },
    {
        "id": "trFK9YkBwM8MKVoui6P6",
        "globalUserId": "7669282",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Наталья",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Лучший заменитель, без химозного привкуса. Покупаю его давно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-14T18:24:11Z",
        "updatedDate": "2023-08-17T01:29:12Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-15T05:47:47Z",
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "12700132",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-17T01:29:12Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 57
    },
    {
        "id": "t4p6TokBZx8DR9RAu9Sj",
        "globalUserId": "16104734",
        "wbUserId": 16104734,
        "wbUserDetails": {
            "name": "Ольга",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 31468103,
        "text": "Соответствует описанию,пришел хорошо упакованный.Добавляла в кофе,ни какого постороннего привкуса нет.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-13T09:00:14Z",
        "updatedDate": "2023-08-16T10:09:50.977678802Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T05:38:42Z",
            "employeeId": 0,
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 56
    },
    {
        "id": "Gxuja4ABJHRxs3OTMPnd",
        "globalUserId": "",
        "wbUserId": 57013597,
        "wbUserDetails": {
            "name": "Александра",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Товар пришёл в хорошем состоянии, не мятый, хорошо упакован. спасибо)))",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-27T15:28:18Z",
        "updatedDate": "2023-08-16T10:09:40.248597556Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-29T11:28:48Z",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Спасибо за покупку!",
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/635f9a9a-cb5c-422c-8e64-0256cd18fa1d_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/635f9a9a-cb5c-422c-8e64-0256cd18fa1d_ms.jpg"
            }
        ],
        "photo": [
            48406983
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 55
    },
    {
        "id": "5t7zA4oB0v3RIh2JGopI",
        "globalUserId": "38420017",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Милана",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший сахарозаменитель беру уже не первый раз, советую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T14:42:58Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 54
    },
    {
        "id": "sDNhE4oBpT2rITcGK7Eg",
        "globalUserId": "5309654",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Беру постоянно родителям, иногда и сама им пользуюсь.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-20T14:37:30Z",
        "updatedDate": "2023-08-22T13:26:37Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-22T13:26:37Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 53
    },
    {
        "id": "jbaSl4kBBgFQoSo0FmNB",
        "globalUserId": "39454051",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Елена",
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Очень нравится  сахарозаменитель этого производителя.Без постороннего привкуса.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-27T13:38:01Z",
        "updatedDate": "2023-08-16T10:09:51.583272408Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T10:32:13Z",
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 52
    },
    {
        "id": "MLgxbYkBl47NwnZ1uuKQ",
        "globalUserId": "46472458",
        "wbUserId": 46472458,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": ""
        },
        "nmId": 138785478,
        "text": "Очень удобно добавлять дозировано. Сладкий чай с одного пакетика. Всё нравиться! Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-19T08:08:43Z",
        "updatedDate": "2023-08-16T10:09:51.190391922Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T08:56:13Z",
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 51
    },
    {
        "id": "ldsUz4kBU2re2XP7zHDR",
        "globalUserId": "49713070",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "hasPhoto": false,
            "country": "RU"
        },
        "nmId": 138785478,
        "text": "Упаковка целая, срок годности нормальный. Хороший сахарозаменитель.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-07T08:19:54Z",
        "updatedDate": "2023-08-16T10:09:52.245636373Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T08:22:10Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "metadata": null,
            "supplierId": 237660,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 50
    },
    {
        "id": "zspeVYoBAtKOitoB776I",
        "globalUserId": "32816892",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "гуд всё",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-02T10:10:00Z",
        "updatedDate": "2023-09-05T12:44:44Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-05T12:44:44Z",
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "employeeId": 0,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 49
    },
    {
        "id": "VwyfhokBUYwa0QEfZwR-",
        "globalUserId": "84611515",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Екатерина"
        },
        "nmId": 138785478,
        "text": "свекрови понравился очень удобно маленькие Саше сабой можно взять будем ещё заказать",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-24T06:39:01Z",
        "updatedDate": "2023-08-16T10:09:51.388971749Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-24T11:13:49Z",
            "metadata": null,
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0,
            "text": "Спасибо за доверие нашему бренду!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 48
    },
    {
        "id": "sxPsk4kBfuChzMpwIPLy",
        "globalUserId": "4270457",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "RU",
            "name": "Галина",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Много лет пользуемся именно этим заменителем сахара. Очень довольны. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-26T20:37:53Z",
        "updatedDate": "2023-08-16T10:09:51.550554966Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T06:48:47Z",
            "employeeId": 0,
            "editable": false,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 47
    },
    {
        "id": "lD8nIIgBl47NwnZ1xCjj",
        "globalUserId": "7799145",
        "wbUserId": 7799145,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Ольга",
            "country": "ru"
        },
        "nmId": 33478818,
        "text": "Упаковка была помята, несмотря на дополнительную коробку, и внутри пересыпался немного порошок. Не поняла - откуда рассыпался порошок, т.к. внутри все пакетики на месте и на вид целые. Срок годности в порядке. Купила на пробу.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-05-15T16:04:17Z",
        "updatedDate": "2023-08-16T10:09:49.158084217Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-05-16T12:27:00Z",
            "text": "Благодарим за выбор нашего товара!",
            "employeeId": 0,
            "editable": false,
            "metadata": null,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "54010598",
                "wbUserId": 54010598,
                "helpfulness": "minus",
                "time": "2023-05-17T17:44:43Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 46
    },
    {
        "id": "XpczgokBwM8MKVouqi0n",
        "globalUserId": "55103745",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Упаковано отлично, коробка целая, всё норм. Состав безопасный, 0 калорий. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-23T10:02:51Z",
        "updatedDate": "2023-08-16T10:09:51.354219164Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T08:59:15Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим Вас за выбор нашего товара!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 45
    },
    {
        "id": "-rQLyoEBH6ARqmmlaDNi",
        "globalUserId": "",
        "wbUserId": 18713847,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Ольга"
        },
        "nmId": 33478818,
        "text": "Качество хорошее. Вкус приятный. Пришёл помятый. ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-07-04T16:29:13Z",
        "updatedDate": "2023-08-16T10:09:41.263303140Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-07-05T06:19:11Z",
            "editable": true,
            "metadata": null,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/258510d6-d6ed-4d75-b368-7ded06aad3a7_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/258510d6-d6ed-4d75-b368-7ded06aad3a7_ms.jpg"
            }
        ],
        "photo": [
            62601519
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 44
    },
    {
        "id": "UvrynIkBJytgTHhB12Au",
        "globalUserId": "87668517",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Елена"
        },
        "nmId": 138785478,
        "text": "упаковано хорошо,а вот доставка очень долгая была,задержали на несколько дней",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-28T14:41:48Z",
        "updatedDate": "2023-08-16T10:09:51.650622635Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:53:09Z",
            "metadata": null,
            "editable": false,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! "
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 43
    },
    {
        "id": "sj54C38B8cSqyKR86a-P",
        "globalUserId": "",
        "wbUserId": 19774500,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Зарина",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Саше высокого качества, в употреблении приятные. Внутри был сюрприз-зефирка, очень вкусная. ",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-18T06:15:47Z",
        "updatedDate": "2023-08-16T10:09:39.415710772Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-18T10:41:30Z",
            "employeeId": 0,
            "text": "Очень рады, что вы остаётесь с нами, благодарим за покупку! ",
            "supplierId": 237660,
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/0c382348-2e8e-4120-8149-98853bc59740_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/0c382348-2e8e-4120-8149-98853bc59740_ms.jpg"
            }
        ],
        "photo": [
            52793241
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 42
    },
    {
        "id": "rZyMtYkBwM8MKVouDZy3",
        "globalUserId": "15429944",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Мария",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Заказываю не первый раз.Саше очень удобно в использовании.Вкус приятный.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-02T09:20:02Z",
        "updatedDate": "2023-08-16T10:09:51.884383372Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-02T13:24:59Z",
            "supplierId": 237660,
            "metadata": null,
            "employeeId": 0,
            "editable": true,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 41
    },
    {
        "id": "aZVJ9IkBKNxVyjcC07nF",
        "globalUserId": "16326149",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Мария",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Блин, все равно послевкусие есть, надо привыкнуть наверное",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-14T13:43:46Z",
        "updatedDate": "2023-08-16T10:09:54.079891495Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 40
    },
    {
        "id": "aBT6m4kBfuChzMpwssSp",
        "globalUserId": "19217535",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Анастасия",
            "hasPhoto": false,
            "country": "by"
        },
        "nmId": 138785478,
        "text": "Не первый раз заказываю этот сахзам, отлично заменяет сахар и надолго хватает!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-28T10:10:46Z",
        "updatedDate": "2023-08-16T10:09:51.634183722Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T08:57:14Z",
            "metadata": null,
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 39
    },
    {
        "id": "T4lJ44kBCaW1RxrSCJy9",
        "globalUserId": "25451459",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Елена",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Как всегда на высоте, постоянно покупаю этот сахарозаменитель.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T06:29:22Z",
        "updatedDate": "2023-08-16T10:09:53.155826832Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 38
    },
    {
        "id": "LDwRC4kBuWYO9ah2si8o",
        "globalUserId": "14783276",
        "wbUserId": 14783276,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "RU",
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Хороший сахзам, понравился на данный момент больше всего (а испробовано много). Вкус кофе такой же как с обычным сахаром",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-30T06:50:56Z",
        "updatedDate": "2023-08-16T10:09:50.482321547Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T08:03:05Z",
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 37
    },
    {
        "id": "xlwpFn8BC4F6na4yCdTr",
        "globalUserId": "",
        "wbUserId": 49495494,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Анастасия",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Беру не в первый раз,очень хороший сахзам!) В этот раз ещё и подарочек положили,благодарю:)",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-20T08:04:22Z",
        "updatedDate": "2023-08-16T10:09:39.462849586Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T05:53:28Z",
            "text": "Благодарим вас за выбор нашего товара!",
            "metadata": null,
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/8ccc4c92-1239-4233-9412-e23627d033fd_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/8ccc4c92-1239-4233-9412-e23627d033fd_ms.jpg"
            }
        ],
        "photo": [
            50309541
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 36
    },
    {
        "id": "4wx-VYABACg9PibijuMq",
        "globalUserId": "",
        "wbUserId": 45552401,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Дошло быстро и в хорошем качестве.Спасибо и дешевле чем в магазинах!!!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-04-23T08:16:38Z",
        "updatedDate": "2023-08-16T10:09:40.189580796Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-26T08:48:06Z",
            "metadata": null,
            "text": "Спасибо за доверие нашему бренду!",
            "supplierId": 237660,
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/09baf78f-e071-4a9e-bb25-3b0fb5f985c9_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/09baf78f-e071-4a9e-bb25-3b0fb5f985c9_ms.jpg"
            }
        ],
        "photo": [
            50865741
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 35
    },
    {
        "id": "Fn_FjokBuX2t2Z2-IMz_",
        "globalUserId": "113012741",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Класная штука, сладко и нет не каких посторонних привкусов. Сладко как с сахаром",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-25T20:37:11Z",
        "updatedDate": "2023-08-16T10:09:51.489218219Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-27T07:07:33Z",
            "text": "Добрый день, спасибо что уделили время и написали отзыв.",
            "metadata": null,
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 34
    },
    {
        "id": "69u0T4oBUdF-4nlzuP_p",
        "globalUserId": "93186010",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Алим Кармов",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Полезно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-09-01T07:45:58Z",
        "updatedDate": "2023-09-06T08:16:58Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-09-06T08:16:58Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 33
    },
    {
        "id": "JCnZw4kBYE-HFOZ5zuSs",
        "globalUserId": "57611787",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Вера",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Купила в первый раз, понравилось. Одного пакета на кружечку как раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-05T03:59:39Z",
        "updatedDate": "2023-08-16T10:09:52.051444520Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 32
    },
    {
        "id": "kBhav4kBfuChzMpw9aoX",
        "globalUserId": "23884313",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Марина"
        },
        "nmId": 138785478,
        "text": "Одного пакетика хватает на  большую чашку.нет послевкусия, рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-04T07:02:36Z",
        "updatedDate": "2023-08-16T10:09:51.983020743Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-04T07:22:31Z",
            "editable": true,
            "supplierId": 237660,
            "employeeId": 0,
            "text": "Благодарим Вас за рекомендацию нашего продукта! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор продуктов на любой вкус! :) ",
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "13770445",
                "wbUserId": 0,
                "helpfulness": "plus",
                "time": "2023-08-04T15:27:24Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 31
    },
    {
        "id": "EcmBpYcBlM6cTg_ROZV6",
        "globalUserId": "37272629",
        "wbUserId": 37272629,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Юля"
        },
        "nmId": 33478818,
        "text": "Товаром довольна👍🏻 Пользуясь случаем, хочу сказать, вдруг кому-то актуально. Ищу менеджера на удаленную работу. ЗП от 15000 рублей. Официально. Можно без опыта, всему необходимому научу. Подойдет для мамочек в декрете, для студентов и для любого, кому нужна подработка. Писать WhatsApp 8.937.47.327.79",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-04-21T20:28:42Z",
        "updatedDate": "2023-08-16T10:09:48.415578572Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-04-26T08:34:26Z",
            "createDate": "2023-04-26T08:34:26Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": false,
            "text": "Благодарим за выбор нашего товара!"
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "1855125",
                "wbUserId": 1855125,
                "helpfulness": "minus",
                "time": "2023-04-26T18:07:52Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 1
        },
        "rank": 30
    },
    {
        "id": "2nxk8ogBAtKOitoBE6XL",
        "globalUserId": "17701564",
        "wbUserId": 17701564,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Ольга"
        },
        "nmId": 138785478,
        "text": "Один пакетик примерно равен по вкусу одной чайной ложке сахара. По мне вкус хороший, не искаженный. Можно добавлять в выпечку👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-25T11:50:25Z",
        "updatedDate": "2023-08-16T10:09:50.317898033Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T11:25:17Z",
            "editable": false,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 29
    },
    {
        "id": "kaIoXIQB65w5GC7jueWN",
        "globalUserId": "28898058",
        "wbUserId": 28898058,
        "wbUserDetails": {
            "country": "RU",
            "name": "Анастасия",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Не вкусно!!! Сладкий, но очень химозный горький привкус",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 2,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-11-09T11:31:23Z",
        "updatedDate": "2023-08-16T10:09:43.198388982Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-11-09T12:00:25Z",
            "employeeId": 0,
            "metadata": null,
            "supplierId": 237660,
            "editable": false,
            "text": "У вас повышенная чувствительность к компонентам в составе. Никакой химии в наших сахарозаменителях конечно же нет. "
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "55284523",
                "wbUserId": 55284523,
                "helpfulness": "plus",
                "time": "2022-11-10T20:54:08.162989077Z"
            },
            {
                "globalUserId": "7860283",
                "wbUserId": 7860283,
                "helpfulness": "minus",
                "time": "2022-12-02T15:14:00.327822168Z"
            },
            {
                "globalUserId": "92290869",
                "wbUserId": 92290869,
                "helpfulness": "minus",
                "time": "2023-01-20T19:43:33Z"
            },
            {
                "globalUserId": "39343426",
                "wbUserId": 0,
                "helpfulness": "minus",
                "time": "2023-07-28T09:31:31Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/a42d8a63-9ab6-4c16-a406-c188f45a3d43_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/a42d8a63-9ab6-4c16-a406-c188f45a3d43_ms.jpg"
            }
        ],
        "photo": [
            29839126
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 3
        },
        "rank": 28
    },
    {
        "id": "K4_oMIkBwM8MKVou4vfl",
        "globalUserId": "37955067",
        "wbUserId": 37955067,
        "wbUserDetails": {
            "name": "Евгения",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Мне понравился. Нет постороннего  привкуса. №7  - одного саше на хороший бокал чая - сладости достаточно.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-07T15:11:56Z",
        "updatedDate": "2023-08-16T10:09:50.735151869Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-21T11:01:16Z",
            "text": "Добрый день, спасибо что уделили время и написали отзыв.",
            "editable": false,
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "63019118",
                "wbUserId": 63019118,
                "helpfulness": "plus",
                "time": "2023-07-11T05:02:00Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 27
    },
    {
        "id": "XZMS-IkBCaW1RxrSTEDt",
        "globalUserId": "21559488",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Валентина"
        },
        "nmId": 31468103,
        "text": "Посылка пришла в срок) хорошо упакована) отличный сахзам👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-15T07:21:36Z",
        "updatedDate": "2023-08-16T10:09:54.273604283Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 26
    },
    {
        "id": "6sIw2IgBU2re2XP7e8dU",
        "globalUserId": "33879612",
        "wbUserId": 33879612,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Вероника",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Отличный заменитель очень похож на на натуральный сахар👍🏻👍🏻Только подскажите чем различаются коробочки по цвету есть синие есть зеленые?",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-20T09:43:56Z",
        "updatedDate": "2023-08-16T10:09:50.172946466Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T10:30:20Z",
            "createDate": "2023-06-20T10:30:20Z",
            "metadata": null,
            "text": "Здравствуйте, составом, сладостью, всё есть в описании ;)",
            "supplierId": 237660,
            "editable": false,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 25
    },
    {
        "id": "RfwAeYIB65w5GC7jWNcj",
        "globalUserId": "",
        "wbUserId": 31666292,
        "wbUserDetails": {
            "name": "Светлана",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Хорошо упакован ,покупаю не первый раз.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-07T15:50:41Z",
        "updatedDate": "2023-08-16T10:09:41.619625208Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-08T12:01:07Z",
            "metadata": null,
            "editable": true,
            "employeeId": 0,
            "text": "Благодарим за выбор нашего товара!",
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/dc7835e4-2185-4394-b562-3cead4cf4b38_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/dc7835e4-2185-4394-b562-3cead4cf4b38_ms.jpg"
            }
        ],
        "photo": [
            50802928
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 24
    },
    {
        "id": "dcni238BZrF3AGZLmUTQ",
        "globalUserId": "",
        "wbUserId": 52452179,
        "wbUserDetails": {
            "name": "Кадрия",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Всегда беру фит парад. Мне нравится. Коробочку помяли конечно. Но не страшно",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-30T17:30:45Z",
        "updatedDate": "2023-08-16T10:09:39.830968974Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-13T12:46:11Z",
            "metadata": null,
            "text": "Благодарим за выбор нашего товара ",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/49bf62c6-5ca6-4204-8eaf-3dc1036a8516_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/49bf62c6-5ca6-4204-8eaf-3dc1036a8516_ms.jpg"
            }
        ],
        "photo": [
            65998541
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 23
    },
    {
        "id": "rtRWS4kBqdFzcemqnsAN",
        "globalUserId": "51301774",
        "wbUserId": 51301774,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Нина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Заказываю не первый раз, так как сахар не употребляю.меня очень устраивает, можно и собой взять.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T18:21:55Z",
        "updatedDate": "2023-08-16T10:09:50.956723397Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-14T06:51:49Z",
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "editable": false
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 22
    },
    {
        "id": "h3KcyoIBE-Uw035e8Ta0",
        "globalUserId": "",
        "wbUserId": 16529460,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Мариета",
            "country": "RU"
        },
        "nmId": 33478818,
        "text": "Как заменитель сахара очень хороший",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2022-08-23T12:10:58Z",
        "updatedDate": "2023-08-16T10:09:41.791588392Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-08-24T05:27:29Z",
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "metadata": null,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/f912c44a-3e6b-45f5-9560-8bad8919fbc5_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/f912c44a-3e6b-45f5-9560-8bad8919fbc5_ms.jpg"
            }
        ],
        "photo": [
            60651620
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 21
    },
    {
        "id": "uofGyokBuX2t2Z2-exCT",
        "globalUserId": "57820652",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Наталья"
        },
        "nmId": 138785478,
        "text": "Я на Кето диете и этот сахарозаменитель мне в помощь🥰 благодарю🌺",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-06T12:15:53Z",
        "updatedDate": "2023-08-16T10:09:52.167748629Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-07T08:00:30Z",
            "employeeId": 0,
            "text": "Благодарим за выбор нашего продукта!",
            "editable": false,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 20
    },
    {
        "id": "T5FlBIoBlM6cTg_RhOQW",
        "globalUserId": "90633541",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "name": "Ольга",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Самый лучший,  без какого либо привкуса, как у других",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T16:47:56Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 19
    },
    {
        "id": "arGHB4oBZx8DR9RAwW8k",
        "globalUserId": "19251154",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Рамела",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Сахар отличный, не горчит,много раз беру этот номер7.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-18T07:24:12Z",
        "updatedDate": "2023-08-18T07:55:09Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-18T07:55:09Z",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": true,
            "text": "Спасибо за покупку!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 18
    },
    {
        "id": "nYo1h4kBAtKOitoBF1PO",
        "globalUserId": "24027342",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Антонина"
        },
        "nmId": 138785478,
        "text": "Беру не первый раз, покупкой довольна. По вкусу, практически как сахар. Спасибо!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-24T09:22:31Z",
        "updatedDate": "2023-08-16T10:09:51.399811390Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-24T11:11:13Z",
            "editable": false,
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "text": "Спасибо за доверие нашему бренду!"
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 17
    },
    {
        "id": "bD7YK4kBuWYO9ah2KuJT",
        "globalUserId": "17577377",
        "wbUserId": 17577377,
        "wbUserDetails": {
            "name": "Наталья",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший сахзам. В таком варианте удобно брать с собой. Все в целостности и сохранности. Хорошо упакован.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-06T15:35:34Z",
        "updatedDate": "2023-08-16T10:09:50.697535852Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-07T06:05:02Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": false,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "17577377",
                "wbUserId": 17577377,
                "helpfulness": "plus",
                "time": "2023-07-07T06:59:01Z"
            }
        ],
        "video": null,
        "votes": {
            "pluses": 1,
            "minuses": 0
        },
        "rank": 16
    },
    {
        "id": "8rFD7YgB0v3RIh2JxiIe",
        "globalUserId": "101592290",
        "wbUserId": 101592290,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Зарета",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Современное и очень качественное средство для здорового питания  и для коррекции веса. Бонусом - упаковка в плотный картон 👍",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-24T11:57:02Z",
        "updatedDate": "2023-08-16T10:09:50.282954602Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-06-30T11:52:09Z",
            "editable": false,
            "text": "Очень рады, что наш продукт пришёлся вам по вкусу! ",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 15
    },
    {
        "id": "GClIrYkBCYLDkq7NFOSK",
        "globalUserId": "17233921",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Татьяна",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Постоянно пользуюсь этим сахзамом. Такая форма удобна в использовании.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-31T18:48:50Z",
        "updatedDate": "2023-08-16T10:09:51.819883119Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-03T08:55:58Z",
            "editable": true,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "supplierId": 237660,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 14
    },
    {
        "id": "-WM8oYgBBgFQoSo0_qYx",
        "globalUserId": "2354698",
        "wbUserId": 2354698,
        "wbUserDetails": {
            "country": "RU",
            "name": "Наталья",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Покупаю не в первый раз. Очень хороший заменитель. Без привкуса. Как и заявлено, 1 пакетик равен примерно 2 чайным ложкам (без горки) сахара. Рекомендую.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-09T17:38:29Z",
        "updatedDate": "2023-08-16T10:09:49.868822197Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T10:05:03Z",
            "createDate": "2023-06-20T10:05:03Z",
            "text": "Благодарим Вас за рекомендацию нашего товара! Рекомендуем заглянуть на страничку магазина и присмотреть что-нибудь ещё, ведь у нас огромный выбор товаров на любой вкус! :) ",
            "metadata": null,
            "employeeId": 0,
            "editable": false,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 13
    },
    {
        "id": "O07B4okBqVKcxP1mL4pt",
        "globalUserId": "13419123",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Оксана",
            "hasPhoto": false,
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Понравился этот заменитель сахара, не горчит!!! Рекомендую!",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-11T04:00:59Z",
        "updatedDate": "2023-08-16T10:09:53.113700512Z",
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 12
    },
    {
        "id": "6Tgsr4ABkjR8f6gggWqq",
        "globalUserId": "",
        "wbUserId": 62420592,
        "wbUserDetails": {
            "country": "RU",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "60 пакетиков) беру только его)сладкий",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-10T18:12:50Z",
        "updatedDate": "2023-08-16T10:09:40.451978740Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-12T11:54:37Z",
            "metadata": null,
            "text": "Спасибо за покупку!",
            "supplierId": 237660,
            "editable": true,
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/6488b35a-0ab9-43a4-a60f-574def605d58_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/6488b35a-0ab9-43a4-a60f-574def605d58_ms.jpg"
            },
            {
                "fullSizeUri": "feedbacks/2517/25171667/c8a10d23-cfd1-4a51-b836-6527924f381f_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/c8a10d23-cfd1-4a51-b836-6527924f381f_ms.jpg"
            }
        ],
        "photo": [
            51309108,
            51309123
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 11
    },
    {
        "id": "SBIrwYkBUYwa0QEf5Kqt",
        "globalUserId": "36648152",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Ольга"
        },
        "nmId": 138785478,
        "text": "Хорошая упаковка, все пришло целое, вкусовые качества устраивают.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-04T15:30:27Z",
        "updatedDate": "2023-08-18T08:32:44Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-18T08:32:44Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 10
    },
    {
        "id": "XN2hAooB0v3RIh2JPsAL",
        "globalUserId": "85305116",
        "wbUserId": 0,
        "wbUserDetails": {
            "hasPhoto": false,
            "name": "Мадина",
            "country": "ru"
        },
        "nmId": 138785478,
        "text": "Все как в описании ,пришло все целое товаром довольна",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-17T08:33:56Z",
        "updatedDate": null,
        "answer": null,
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 9
    },
    {
        "id": "L_3yFX8BIBf9BdlrU0ve",
        "globalUserId": "",
        "wbUserId": 24856705,
        "wbUserDetails": {
            "country": "RU",
            "name": "Елена",
            "hasPhoto": false
        },
        "nmId": 33478818,
        "text": "Быстрая доставка. Спасибо большое. Ещё подарочек , зефирка ПП 🤗 Рекомендую к покупке .",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-20T07:04:36Z",
        "updatedDate": "2023-08-16T10:09:39.458246136Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-02-22T07:13:50Z",
            "metadata": null,
            "editable": true,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/2ec0345f-2abb-45a4-886e-7fdcef891c33_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/2ec0345f-2abb-45a4-886e-7fdcef891c33_ms.jpg"
            }
        ],
        "photo": [
            48864044
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 8
    },
    {
        "id": "c0_6O38B8cSqyKR82vSB",
        "globalUserId": "",
        "wbUserId": 15848184,
        "wbUserDetails": {
            "country": "RU",
            "hasPhoto": false,
            "name": "Анастасия"
        },
        "nmId": 33478818,
        "text": "Очень люблю этот сахарозаменитель :) Спасибо продавцу, в подарок положили зефирку❤️",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-02-27T16:19:29Z",
        "updatedDate": "2023-08-16T10:09:39.572243103Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-03-01T09:43:58Z",
            "employeeId": 0,
            "metadata": null,
            "text": "Благодарим за высокую оценку нашего товара!",
            "editable": true,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2517/25171667/271f5b6b-a90e-43f2-9d31-3d4ff0ddf15c_fs.jpg",
                "minSizeUri": "feedbacks/2517/25171667/271f5b6b-a90e-43f2-9d31-3d4ff0ddf15c_ms.jpg"
            }
        ],
        "photo": [
            55203567
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 7
    },
    {
        "id": "PamOGIoBqd2abVcyDIKX",
        "globalUserId": "19572897",
        "wbUserId": 0,
        "wbUserDetails": {
            "name": "Надежда",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Хороший заменитель, на кружку два сашше хватает.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-21T14:44:37Z",
        "updatedDate": "2023-08-22T12:48:45Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-22T12:48:45Z",
            "editable": true,
            "supplierId": 237660,
            "metadata": null,
            "text": "Благодарим за выбор нашего продукта!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 6
    },
    {
        "id": "k5pG84kBqd2abVcy7nv-",
        "globalUserId": "16779719",
        "wbUserId": 0,
        "wbUserDetails": {
            "country": "ru",
            "hasPhoto": false,
            "name": "Татьяна"
        },
        "nmId": 138785478,
        "text": "Удобные Саше. Для одного раза хватает. Добавляю в кофе.",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-08-14T09:01:00Z",
        "updatedDate": "2023-08-16T10:09:54.009282575Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-08-14T10:06:01Z",
            "employeeId": 0,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "editable": true,
            "metadata": null
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 5
    },
    {
        "id": "InJvSokBCaW1RxrSUuuW",
        "globalUserId": "22647177",
        "wbUserId": 22647177,
        "wbUserDetails": {
            "hasPhoto": false,
            "country": "ru",
            "name": "Стелла"
        },
        "nmId": 138785478,
        "text": "только этот сахзам и использую, в чай и в выпечку, удобно, что в саше. нет привкуса никакого",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-07-12T14:09:17Z",
        "updatedDate": "2023-08-16T10:09:50.937366397Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2023-07-20T12:28:56Z",
            "editable": true,
            "text": "Благодарим за отзыв!",
            "metadata": null,
            "employeeId": 0,
            "supplierId": 237660
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 4
    },
    {
        "id": "FLRU54ABEUCUNtmVTMgS",
        "globalUserId": "",
        "wbUserId": 41011823,
        "wbUserDetails": {
            "name": "Мадина",
            "country": "RU",
            "hasPhoto": false
        },
        "nmId": 62622901,
        "text": "Запаслась сахаром на долго😅 Упаковка была целой👌 Спасибо🔥",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-05-21T15:55:02Z",
        "updatedDate": "2023-08-16T10:09:40.628571295Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-05-23T13:34:59Z",
            "text": "Спасибо за покупку!",
            "employeeId": 0,
            "supplierId": 237660,
            "metadata": null,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": [
            {
                "globalUserId": "",
                "wbUserId": 10682014,
                "helpfulness": "plus",
                "time": "2022-05-21T17:06:12Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 41011823,
                "helpfulness": "plus",
                "time": "2022-05-24T17:13:17Z"
            },
            {
                "globalUserId": "",
                "wbUserId": 59394989,
                "helpfulness": "plus",
                "time": "2022-06-11T06:19:37Z"
            }
        ],
        "photos": [
            {
                "fullSizeUri": "feedbacks/4743/47435533/942a5583-ab9e-45c5-a1d3-7ccf551cf97e_fs.jpg",
                "minSizeUri": "feedbacks/4743/47435533/942a5583-ab9e-45c5-a1d3-7ccf551cf97e_ms.jpg"
            }
        ],
        "photo": [
            59329792
        ],
        "video": null,
        "votes": {
            "pluses": 3,
            "minuses": 0
        },
        "rank": 3
    },
    {
        "id": "yaIsp38BOItVDAWmEJAJ",
        "globalUserId": "",
        "wbUserId": 3943752,
        "wbUserDetails": {
            "country": "RU",
            "name": "Марина",
            "hasPhoto": false
        },
        "nmId": 31468103,
        "text": "Хорошо упаковано. Быстрая доставка. Всё  целости и сохранности.  Рекомендую",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "full",
        "matchingDescription": "full",
        "productValuation": 5,
        "color": "",
        "size": "0",
        "createdDate": "2022-03-20T11:52:36Z",
        "updatedDate": "2023-08-16T10:09:39.767217064Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "0001-01-01T00:00:00Z",
            "createDate": "2022-04-14T06:26:41Z",
            "metadata": null,
            "supplierId": 237660,
            "text": "Благодарим вас за рекогмендацию! ",
            "employeeId": 0,
            "editable": true
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "photos": [
            {
                "fullSizeUri": "feedbacks/2344/23441056/77972347-56e5-40d2-ad20-26205ddb8837_fs.jpg",
                "minSizeUri": "feedbacks/2344/23441056/77972347-56e5-40d2-ad20-26205ddb8837_ms.jpg"
            }
        ],
        "photo": [
            49636718
        ],
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 2
    },
    {
        "id": "63NfvogBl47NwnZ1Owxq",
        "globalUserId": "16965972",
        "wbUserId": 16965972,
        "wbUserDetails": {
            "name": "Анна",
            "country": "ru",
            "hasPhoto": false
        },
        "nmId": 138785478,
        "text": "Упаковка пришла жуть какая мятая и рванная, но благо на сам продукт это не повлияло, потому звезду не снимаю.  А в остальном все хорошо 👌",
        "pros": "",
        "cons": "",
        "matchingSize": "",
        "matchingPhoto": "",
        "matchingDescription": "",
        "productValuation": 5,
        "color": "colorless",
        "size": "0",
        "createdDate": "2023-06-15T09:24:52Z",
        "updatedDate": "2023-08-16T10:09:50.038000904Z",
        "answer": {
            "state": "wbRu",
            "lastUpdate": "2023-06-20T08:45:09Z",
            "createDate": "2023-06-20T08:45:09Z",
            "editable": false,
            "metadata": null,
            "supplierId": 237660,
            "text": "Спасибо за покупку!",
            "employeeId": 0
        },
        "metadata": null,
        "feedbackHelpfulness": null,
        "video": null,
        "votes": {
            "pluses": 0,
            "minuses": 0
        },
        "rank": 1
    }
]